In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')


In [9]:
data.drop("Unnamed: 0", axis=1).head()[:2][:2]

,links,characters,name,position 1,position 2,experince 1,experince 2,field of studies 1,field of studies 2,degree 1,degree 2,industry,skills,influencer,country,summary
0,https://nl.linkedin.com/in/mieke-zonneveld-58a...,Encourager (Is),Mieke Zonneveld,Docent Nederlands,Auteur,9.0,9.0,Opleiding leraar Nederlands in de eerste graad,Nederlandse taal- en letterkunde,Master,Master of Arts - MA,Primary/Secondary Education,[],False,Netherlands,NaN
1,https://nl.linkedin.com/in/jiska-de-ligt-2a7527b,Counselor (Si),Jiska de Ligt,HR Manager,bestuurslid (secretaris),1.0,1.0,Comparative Literature,English Language and Literature/Letters,Research master,NaN,Information Technology & Services,"['Academic Writing', 'English Literature', 'Li...",False,Netherlands,Bij Infi ben ik verantwoordelijk voor het verz...


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Load the data
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')

# Convert non-string values to strings in the feature columns
text_features = data[['position 1', 'position 2', 'field of studies 1', 'field of studies 2', 'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country']].copy()

# Handle non-string values in each column
for column in text_features.columns:
    text_features[column] = text_features[column].astype(str)

# Combine all text features into a single string column
text_data = text_features.apply(lambda x: ' '.join(x), axis=1).tolist()
labels = data['characters'].tolist()

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label_mapping[label] for label in labels])

# Update label mapping to start from 0
label_mapping = {label: idx for label, idx in label_mapping.items()}
num_classes = len(label_mapping)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
print(f"max_sequence------>>>>{max_sequence_length}")
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert the data to NumPy arrays
X_text = np.array(padded_sequences)

# Split the data into training and testing sets
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42)

# Textual input branch
text_input = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_sequence_length)(text_input)
lstm_layer = LSTM(100)(embedding_layer)
output_layer = Dense(num_classes, activation='softmax')(lstm_layer)

# Define the model
model = Model(inputs=text_input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_text, y_train, validation_data=(X_test_text, y_test),
          epochs=10, batch_size=32)

# Save the model
model.save('model.h5')


In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Load the data
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')
cols=['position 1', 'position 2', 'field of studies 1','experince 1', 'experince 2', 'field of studies 2',
       'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country']

for col in cols:
  data[col] = data[col].fillna("Unknown")
# Convert non-string values to strings in the feature columns
text_features = data[['position 1', 'position 2', "experince 1","experince 2", 
                      'field of studies 1', 'field of studies 2', 
                      'degree 1', 'degree 2', 'industry', 'skills',
                      'influencer', 'country', 'summary']].copy()

# Handle non-string values in each column
for column in text_features.columns:
    text_features[column] = text_features[column].astype(str)

# Combine all text features into a single string column
text_data = text_features.apply(lambda x: ' '.join(x), axis=1).tolist()
labels = data['characters'].tolist()

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label_mapping[label] for label in labels])

# Update label mapping to start from 0
label_mapping = {label: idx for label, idx in label_mapping.items()}
num_classes = len(label_mapping)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
print(max_sequence_length)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert the data to NumPy arrays
X_text = np.array(padded_sequences)

# Split the data into training and testing sets
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.1, random_state=42)

# Textual input branch
text_input = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_sequence_length)(text_input)
lstm_layer = LSTM(100)(embedding_layer)
output_layer = Dense(num_classes, activation='softmax')(lstm_layer)

# Define the model
model = Model(inputs=text_input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_text, y_train, validation_data=(X_test_text, y_test),
          epochs=50, batch_size=32)

# Save the model
model.save('model2_test_size=0.1.h5')


523
Epoch 1/50
115/115 [==============================] - 65s 527ms/step - loss: 2.5251 - accuracy: 0.1548 - val_loss: 2.4070 - val_accuracy: 0.1966
Epoch 2/50
115/115 [==============================] - 59s 513ms/step - loss: 2.2792 - accuracy: 0.2105 - val_loss: 2.3431 - val_accuracy: 0.2015
Epoch 3/50
115/115 [==============================] - 59s 513ms/step - loss: 2.0298 - accuracy: 0.2768 - val_loss: 2.4085 - val_accuracy: 0.2064
Epoch 4/50
115/115 [==============================] - 59s 512ms/step - loss: 1.7504 - accuracy: 0.3808 - val_loss: 2.4702 - val_accuracy: 0.1941
Epoch 5/50
115/115 [==============================] - 59s 509ms/step - loss: 1.3977 - accuracy: 0.5242 - val_loss: 2.5675 - val_accuracy: 0.2088
Epoch 6/50
115/115 [==============================] - 59s 511ms/step - loss: 1.0272 - accuracy: 0.6631 - val_loss: 2.7992 - val_accuracy: 0.2260
Epoch 7/50
115/115 [==============================] - 58s 507ms/step - loss: 0.7348 - accuracy: 0.7767 - val_loss: 3.1143 - va

In [5]:
import datetime
import requests
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

positions0=[]
positions1=[]
experinces0=[]
experinces1=[]
skills=[]
influencer=[]
country=[]
field_of_studies0=[]
field_of_studies1=[]
degrees0=[]
degrees1=[]
industries=[]
names=[]
summaries=[]
certifications=[]

link="https://www.linkedin.com/in/ismatulla-kuyliev-43286a199/"
id = link.split('/')[4]
url = "https://api.iscraper.io/v2/profile-details"

payload = {
  'profile_id': id,
}

headers = {
  'X-API-KEY': 'hVSqiv11cY1W5YUawXUDLBn0jb4G5W44',
}

response = requests.post(url, json=payload, headers=headers)
res=response.json()
try:
  skills.append(res['skills'])
except:
  skills.append("Unknown")
position0=""
try:
  position0=res['position_groups'][0]['profile_positions'][0]['title']
  positions0.append(position0)
except:
  positions0.append("Unknown")

position1=""
try:
  position1=res['position_groups'][1]['profile_positions'][0]['title']
  positions1.append(position1)
except:
  positions1.append("Unknown")

experince0=0
try:
  experince0=res['position_groups'][0]['date']['start']['year']-res['position_groups'][0]['date']['end']['year']
  experinces0.append(experince0)
except:
  try:
    experince0=datetime.date.today().year - res['position_groups'][0]['date']['start']['year']
    experinces0.append(experince0)
  except:
    experinces0.append("Unknown")

experince1=0
try:
  experince1=res['position_groups'][1]['date']['start']['year']-res['position_groups'][1]['date']['end']['year']
  experinces1.append(experince1.abs())
except:
  try:
    experince1=datetime.date.today().year - res['position_groups'][1]['date']['start']['year']
    experinces1.append(experince1.abs())
  except:
    experinces1.append("Unknown")
      
try:
  influencer.append(res['influencer'])
except:
  influencer.append("Unknown")
try:
  country.append(res['location']['country'])
except:
  country.append("Unknown")
field_of_study0=""
try:
  field_of_study0=res['education'][0]['field_of_study']
  field_of_studies0.append(field_of_study0)
except:
  field_of_study0="Unknown"
  field_of_studies0.append(field_of_study0)

field_of_study1=""
try:
  field_of_study1=res['education'][1]['field_of_study']
  field_of_studies1.append(field_of_study1)
except:
  field_of_study1="Unknown"
  field_of_studies1.append(field_of_study1)

degree_name0=''
try:
  degree_name0=res['education'][0]['degree_name']
  degrees0.append(degree_name0)
except:
  degree_name0="Unknown"
  degrees0.append(degree_name0)

degree_name1=''
try:
  degree_name1=res['education'][1]['degree_name']
  degrees1.append(degree_name1)
except:
  degree_name1="Unknown"
  degrees1.append(degree_name1)

try:
  industries.append(res['industry'])
except:
  industries.append("Unknown")

try:
  if res['summary']:
    summaries.append(res['summary'])
  else:
    summaries.append("Unknown")
except:
  summaries.append("Unknown")

# Load the saved model
model = load_model('model.h5')
a= [
    positions0[0],
    positions1[0],
    str(experinces0[0]),
    str(experinces1[0]),
    field_of_studies0[0],
    field_of_studies1[0],
    degrees0[0],
    degrees1[0],
    industries[0],
    skills[0],
    summaries[0]
]
new_text_data=[]
for i in a:
    if i != "Unknown":
        new_text_data.append(i)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(new_text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(new_text_data)

# Pad sequences to have consistent length
max_sequence_length = 519
print(f"max_sequence------>>>>{max_sequence_length}")
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
# Preprocess the new text data
new_sequences = tokenizer.texts_to_sequences(new_text_data)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)

# Make predictions
predictions = model.predict(new_padded_sequences)

# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Map the predicted labels back to original labels
label_mapping={'Analyst (C)': 0, 'Questioner (CD)': 1, 'Captain (D)': 2, 'Encourager (Is)': 3, 'Stabilizer (SC)': 4, 
  'Supporter (S)': 5, 'Counselor (Si)': 6, 'Editor (Cs)': 7, 'Motivator (I)': 8, 'Driver (Di)': 9,
  'Planner (Sc)': 10, 'Skeptic (Cd)': 11, 'Architect (Dc)': 12, 'Driver (Di);': 13, 
  'Influencer (Id)': 14, 'Harmonizer (IS)': 15, 'Initiator (DI)': 16}
reverse_label_mapping = {idx: label for label, idx in label_mapping.items()}
predicted_labels = [reverse_label_mapping[label] for label in predicted_labels]

# Print the predictions
for text, label in zip(new_text_data, predicted_labels):
    print(f"Text: {text}\tPredicted Label: {label}")
print(max(set(predicted_labels), key = predicted_labels.count))

max_sequence------>>>>519
1/1 [==============================] - 1s 1s/step
Text: Frontend mentor 	Predicted Label: Supporter (S)
Text: Fullstack | javaScript developer	Predicted Label: Analyst (C)
Text: 0	Predicted Label: Supporter (S)
Text: Petroleum Engineering	Predicted Label: Analyst (C)
Text: Oil and Gas	Predicted Label: Captain (D)
Text: Master of Science - MS	Predicted Label: Analyst (C)
Text: Bachelor's degree	Predicted Label: Editor (Cs)
Text: Computer Software	Predicted Label: Supporter (S)
Text: ['Python (Programming Language)', 'Django', 'REST APIs', 'Problem Solving', 'Responsive Web Design', 'Front-End Development', 'Firebase', 'Web Development', 'Communication', 'Contract Negotiation', 'Construction', 'Management', 'Analytical Skills', 'Software as a Service (SaaS)', 'Translation', 'Technical Translation', 'English Translation', 'Russian Translation', 'Redux.js', 'Git']	Predicted Label: Analyst (C)
Text: I am a skilled Frontend Developer with over 1.5 years of experienc

In [15]:
predicted_labels

['Supporter (S)',
 'Motivator (I)',
 'Harmonizer (IS)',
 'Questioner (CD)',
 'Counselor (Si)',
 'Supporter (S)',
 'Harmonizer (IS)',
 'Supporter (S)',
 'Harmonizer (IS)',
 'Influencer (Id)']

In [ ]:
# new_text_data = [
#     positions0[0],
#     positions1[0],
#     str(experinces0[0]),
#     str(experinces1[0]),
#     field_of_studies0[0],
#     field_of_studies1[0],
#     degrees0[0],
#     degrees1[0],
#     industries[0],
#     skills[0],
#     # str(influencer[0]),
#     # country[0],
#     summaries[0]

# ]


In [59]:
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('model.h5')
a= [
    positions0[0],
    positions1[0],
    str(experinces0[0]),
    str(experinces1[0]),
    field_of_studies0[0],
    field_of_studies1[0],
    degrees0[0],
    degrees1[0],
    industries[0],
    skills[0],
    summaries[0]
]
new_text_data=[]
for i in a:
    if i != "Unknown" and i != None:
        new_text_data.append(i)

# Preprocess the new text data
new_sequences = tokenizer.texts_to_sequences(new_text_data)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)

# Make predictions
predictions = model.predict(new_padded_sequences)

# Get the predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Map the predicted labels back to original labels
reverse_label_mapping = {idx: label for label, idx in label_mapping.items()}
predicted_labels = [reverse_label_mapping[label] for label in predicted_labels]

# Print the predictions
for text, label in zip(new_text_data, predicted_labels):
    print(f"Text: {text}\tPredicted Label: {label}")
print(max(set(predicted_labels), key = predicted_labels.count))


1/1 [==============================] - 1s 1s/step
Text: Sustainability Partner to Boards - Director Marketing & Sales	Predicted Label: Encourager (Is)
Text: Founder	Predicted Label: Captain (D)
Text: 14	Predicted Label: Driver (Di);
Text: Marketing	Predicted Label: Motivator (I)
Text: Post-doctorate degree in Brand Management	Predicted Label: Architect (Dc)
Text: Environmental Services	Predicted Label: Captain (D)
Text: ['Leadership Development', 'Investments', 'Strategy', 'Leadership', 'Sustainability Consulting', 'Marketing Strategy', 'Business Strategy', 'Marketing', 'B2B', 'New Business Development', 'Business Development', 'Marketing Management', 'Entrepreneurship', 'Start-ups', 'Project Planning', 'Supply Chain', 'Management Consulting', 'Change Management', 'FMCG', 'Strategic Planning']	Predicted Label: Motivator (I)
Text: Committed to help leaders at all levels transition their organisations towards businesses that are a force for good... leaving no footprint.

Main fields of e

In [46]:
max(set(predicted_labels), key = predicted_labels.count)

'Editor (Cs)'

In [45]:
predicted_labels

['Editor (Cs)',
 'Supporter (S)',
 'Supporter (S)',
 'Editor (Cs)',
 'Editor (Cs)',
 'Captain (D)',
 'Editor (Cs)']

In [26]:
# positions0[0]
# positions1[0]
# str(experinces0[0])
# str(experinces1[0])
# field_of_studies0[0]
# field_of_studies1[0]
# degrees0[0]
# degrees1[0]
# industries[0]
# skills[0]
# str(influencer[0])
# country[0]
summaries[0]

In [27]:
summaries

[None]

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

# Load the data
data = pd.read_csv('https://raw.githubusercontent.com/fayyoz24/Bots_using_selenium/main/model_crystal/without%20certifications%20linkedin%20profiles.csv')
cols = ['position 1', 'position 2', 'field of studies 1', 'experince 1', 'experince 2', 'field of studies 2',
        'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country']

for col in cols:
    data[col] = data[col].fillna("Unknown")

# Convert non-string values to strings in the feature columns
text_features = data[
    ['position 1', 'position 2', "experince 1", "experince 2", 'field of studies 1', 'field of studies 2',
     'degree 1', 'degree 2', 'industry', 'skills', 'influencer', 'country', 'summary']].copy()

# Handle non-string values in each column
for column in text_features.columns:
    text_features[column] = text_features[column].astype(str)

# Combine all text features into a single string column
text_data = text_features.apply(lambda x: ' '.join(x), axis=1).tolist()
labels = data['characters'].tolist()

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(set(labels))}
y = np.array([label_mapping[label] for label in labels])

# Update label mapping to start from 0
label_mapping = {label: idx for label, idx in label_mapping.items()}
num_classes = len(label_mapping)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(text_data)

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
print(max_sequence_length)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert the data to NumPy arrays
X_text = np.array(padded_sequences)

# Split the data into training and testing sets
X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_text, y, test_size=0.1, random_state=42)

# Textual input branch
text_input = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(vocab_size, 100, input_length=max_sequence_length)(text_input)
lstm_layer = LSTM(100, kernel_regularizer=l2(0.01))(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)  # Add a dropout layer for regularization
output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

# Define the model
model = Model(inputs=text_input, outputs=output_layer)

# Compile the model
learning_rate = 0.001  # Decrease the learning
# Compile the model
learning_rate = 0.001  # Decrease the learning rate

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Apply early stopping to prevent overfitting
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train_text, y_train,
                    validation_data=(X_test_text, y_test),
                    epochs=50,
                    batch_size=32,
                    callbacks=[early_stopping])

# Save the model
model.save('model3.h5')


523


NameError: name 'tf' is not defined

In [ ]:
s=<body dir="ltr" class="render-mode-BIGPIPE nav-v2 ember-application boot-complete icons-loaded" data-t-link-to-event-attached="true">
  

  
    <!-- HUED-11420 -->
    <div id="artdeco-toasts__wormhole" role="region">    <section id="artdeco-toasts" class="artdeco-toasts" aria-label="Toast message">
      <header class="artdeco-toasts__header">
        <h2 class="artdeco-toasts__title">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">0 reports in total</font></font>
        </h2>
      </header>

<!---->
      <div class="artdeco-toasts_toasts">
<!---->      </div>
    </section>
</div>
    <!-- EMBER_CLI_FASTBOOT_BODY -->
<div id="app-boot-bg-loader" class="app-boot-bg-skeleton">
  <div class="top-bar"></div>
  <div class="content">
    <div class="initial-load-animation fade-load">
      <div class="linkedin-image display-flex justify-center">
        <svg xmlns="http://www.w3.org/2000/svg" class="loader__linkedin-logo" width="190" height="48" viewBox="0 0 190 48">
          <g>
            <g>
              <path d="M58,27.22V41H51V28.88c0-3.7-2.07-5.24-4-5.24a5,5,0,0,0-5.14,4.85,4.34,4.34,0,0,0,0,.51V41H35V18h6.6v3.25h.09c.69-1.41,3.64-3.75,7.66-3.75S58,20.22,58,27.22ZM24,41h7V18H24ZM27.5,6.45a4.05,4.05,0,1,0,4.1,4.05,4,4,0,0,0-4-4ZM190,3.5v41a3.5,3.5,0,0,1-3.5,3.5h-41a3.5,3.5,0,0,1-3.5-3.5V3.5A3.5,3.5,0,0,1,145.5,0h41A3.5,3.5,0,0,1,190,3.5ZM156,18h-7V41h7Zm.6-7.5a4.1,4.1,0,1,0-4.15,4.05h.05a4,4,0,0,0,4.1-3.9ZM183,27.22c0-7-4.63-9.72-8.65-9.72s-7,2.34-7.66,3.75h-.09V18H160V41h7V29c0-3.69,2.51-5.33,4.95-5.33,2,0,4.05,1.54,4.05,5.24V41h7ZM8,7H0V41H21V34H8ZM108,29.77v2H91a1.33,1.33,0,0,0,.11.43c.58,1.93,2.67,3.56,5.75,3.56A6.3,6.3,0,0,0,102,33.52l5.1,3.18a12.72,12.72,0,0,1-10.45,4.8C89.94,41.5,84,37.42,84,29.59S90,17.5,96.44,17.5,108,21.81,108,29.77ZM101,27c0-2.4-1.56-4.38-4.75-4.38-3,0-5.09,2-5.25,4.38ZM85.26,18H76.68l-7.54,9.37H69V7H62V41h7V30h.14l7.72,11h8.77L76.2,28.43ZM128,7h7V41h-6.6V38h-.09c-.88,1.52-3.24,3.49-7.4,3.49-5,0-10.91-3.63-10.91-12,0-7.53,5.1-11.95,10.82-11.95a9.55,9.55,0,0,1,7.09,3H128Zm.3,22.49a5.74,5.74,0,0,0-5.59-5.89h-.15A5.54,5.54,0,0,0,116.89,29c0,.15,0,.29,0,.44a5.61,5.61,0,0,0,5.26,5.94h.4A5.83,5.83,0,0,0,128.3,29.49Z" fill="currentColor"></path>
            </g>
          </g>
        </svg>
      </div>
      <div class="loading-bar">
        <div class="blue-bar"></div>
      </div>
    </div>
  </div>
</div>

    <script src="https://static.licdn.com/sc/h/9vghoawmkb5fvzav7oabel24l" data-fastboot-src="/assets/vendor-static.js" crossorigin=""></script>
    <script src="https://static.licdn.com/sc/h/cfg1twijroe2fxyw06xj1tl2b" data-fastboot-src="/assets/vendor.js" crossorigin=""></script>
    <!-- graphql-script-injection -->
    <script src="https://static.licdn.com/sc/h/a8i9w57wqoyi8fjxs9ulzk14e" data-fastboot-src="/assets/pdsc/register-metadata.js" crossorigin=""></script>
    <script src="https://static.licdn.com/sc/h/8wptp0fxde5h0i9gff7kb1m4x" data-fastboot-src="/assets/chunk.493.5b8b62fabf1d1503e941.js" crossorigin=""></script>
<script src="https://static.licdn.com/sc/h/g05vqk1gm77fzj80yree9bjy" data-fastboot-src="/assets/chunk.143.a1eb44166acaecd73a8c.js" crossorigin=""></script>
    <script data-embroider-ignore="" src="https://static.licdn.com/sc/h/gse0tl7twp7z0fxtw0sr0txg" data-fastboot-src="/assets/i18n/support_nl_NL.js" crossorigin=""></script>
    <script src="https://static.licdn.com/sc/h/a3j8ctylze6rj0llcay1oty82"></script><script src="https://static.licdn.com/sc/h/509jd8gtpgo1wyjjko55vkyn1"></script><script src="https://static.licdn.com/sc/h/9q2lwmy4bj3o3bnsnvdbbxr0s"></script><script src="https://static.licdn.com/sc/h/ax2u4g85ovs8qdwk52xeima18"></script><script src="https://static.licdn.com/sc/h/c61zqgindr3pmda3ezj15ozm7"></script>
    <script src="https://static.licdn.com/sc/h/524t2ryyko20rgy5udl7f30am" data-fastboot-src="/assets/voyager-web.js" crossorigin=""></script>
    <div id="artdeco-modal-outlet"></div>

<code style="display: none" id="bpr-guid-6231299">
  {"data":{"entityUrn":"urn:li:collectionResponse:4HlijJjSiEXmgxSNTaZbyKnZ88db1eTefgOnimBx6wM=","elements":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719291,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719291,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48919_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48919","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48918_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48919","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"Het plaatsen van een gratis vacature is nu nog eenvoudiger","locale":"nl_NL","key":"i18n_form_header_jdai_v2_free_eligible","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.parent","displayName":"Child Test For WOW Page - Form - Header","description":"Child Test Test different headers on our form in the WOW page. Test efficiency messaging above the fold.\nFree Eligible Control: Post a job for free. Variants: 1) Posting a free job just became easier 2) Post a free job in minutes\nFree Ineligible Control: Post a job now. Variants: 1) Posting a job just became easier 2) Post a job in minutes","lixTreatment":"VAR_t48919_PR_1","parentLixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48919","creatorDisplayName":"cferraro","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4710852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4710852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"mynetwork/templates/network-manager/people-follow/following","message":"","locale":"nl_NL","key":"i18n_following_empty_state_description","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"SH Positioning Experiment: Following List","description":"Our primary goal is to learn whether adding the secondary value props of engagement, social proof, creator similarity, and LinkedIn guidance to the headline in the Following List module for those following 0 members will increase the number of follow edges made by Career Accelerators.","lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","creatorDisplayName":"mswoopes","status":"INITIAL_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718003,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718003,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32055","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_subheader_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_subheader_copy","description":"GTA subheader copy experiment.","lixTreatment":"control","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32055","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533282,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533282,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26210","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/templates/components/widgets/people-you-may-know","message":"","locale":"nl_NL","key":"i18n_onboarding_pymk_page_header_phase_3","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Onboarding PYMK CA and Concise Copy Test","description":"Test career accelerator confidence and concise related copy on PYMK onboarding widget on web.","lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26210","creatorDisplayName":"whchan","status":"CUSTOM_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-details-engine/components/detail/job-view-layout","message":"Meer vacatures","locale":"nl_NL","key":"i18n_simjob_header","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"similar-jobs-to-more-jobs-job-details-engine-copy","description":"Ramp prior copy change for \"Similar Jobs\" to \"More jobs\"","lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","creatorDisplayName":"rorodrig","status":"PERMANENT_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719292,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719292,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48920_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48920","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48918_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48920","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"Het plaatsen van een vacature is nu nog eenvoudiger","locale":"nl_NL","key":"i18n_form_header_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.parent","displayName":"Child Test For WOW Page - Form - Header","description":"Child Test Test different headers on our form in the WOW page. Test efficiency messaging above the fold.\nFree Eligible Control: Post a job for free. Variants: 1) Posting a free job just became easier 2) Post a free job in minutes\nFree Ineligible Control: Post a job now. Variants: 1) Posting a job just became easier 2) Post a job in minutes","lixTreatment":"VAR_t48920_PR_1","parentLixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48920","creatorDisplayName":"cferraro","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718002,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718002,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48706_PR_1","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32054","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Maak connecties en blijf op de hoogte van uw professionele netwerk in de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_subheader_profile","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_subheader_copy_profile","description":"Customize GTA subheader copy.","lixTreatment":"VAR_t48706_PR_1","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32054","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718001,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718001,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Bekijk het volledige profiel van {:firstName} via de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_profile","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy_profile","description":"Customize the desktop GTA profile header copy.","lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717859,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717859,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48697_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32045","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Maak een connectie met recruiters in de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_jobs","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy_jobs","description":"Customize the GTA page header on desktop.","lixTreatment":"VAR_t48697_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32045","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"profile-creator-mode-shared/components/creator-tools-list/item","message":"Meer informatie","locale":"nl_NL","key":"i18n_available","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Profile Page - Creator Mode - Learn More - 8/14/2023","description":"Profile Page - Creator Mode - Learn More - 8/14/2023 ","lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","creatorDisplayName":"jenli","status":"PERMANENT_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717999,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717999,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48700_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32048","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Ontdek onderweg nieuwe bedrijven","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_company","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy_company","description":"Customize the GTA header copy for company joins.","lixTreatment":"VAR_t48700_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32048","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718000,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718000,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48701_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32049","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Geef uw zoektocht naar een baan een boost met de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy","description":"Customize the GTA header copy.","lixTreatment":"VAR_t48701_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32049","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin-pages-entrypoint/components/card","message":"","locale":"nl_NL","key":"i18n_org_page_notifications","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Pages-left-rail-activity-copy","description":"Testing copy for the left rail landing page from page notifications to page activity (to mirror activity tab)","lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","creatorDisplayName":"nalberts","status":"INITIAL_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin/components/admin/admin-nav","message":"","locale":"nl_NL","key":"i18n_org_activity_tab","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Pages-Activity-Tab-Copy-Test","description":"Testing different copy for the activity tab in Pages admin view ","lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","creatorDisplayName":"nalberts","status":"INITIAL_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533279,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533279,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26106","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/templates/components/widgets/photo-upload","message":"","locale":"nl_NL","key":"i18n_onboarding_photo_page_header_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Onboarding Add Photo Social Proof Copy Test","description":"Test social proof in onboarding photo upload on web.","lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26106","creatorDisplayName":"whchan","status":"CUSTOM_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231299">
  {"request":"/voyager/api/voyagerSegmentsDashChameleonConfig","status":200,"body":"bpr-guid-6231299","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231299"><code style="display: none" id="bpr-guid-6231303">
  {"data":{"canBrowseProfiles":false,"reactivationFeaturesEligible":false,"canViewJobAnalytics":false,"canViewWVMP":false,"premiumFreeTrialEligible":true,"canAccessLearningVideos":false,"premiumLearningUpgradeEligible":false,"canViewCompanyInsights":false,"$type":"com.linkedin.voyager.premium.FeatureAccess"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231303">
  {"request":"/voyager/api/premium/featureAccess?name\u003DreactivationFeaturesEligible","status":200,"body":"bpr-guid-6231303","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231303"><code style="display: none" id="bpr-guid-6231304">
  {"data":{"plainId":1096125424,"publicContactInfo":{"$type":"com.linkedin.voyager.identity.shared.PublicContactInfo"},"premiumSubscriber":false,"*miniProfile":"urn:li:fs_miniProfile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","$type":"com.linkedin.voyager.common.Me"},"included":[{"customPronoun":null,"memorialized":false,"lastName":"Usmon","dashEntityUrn":"urn:li:fsd_profile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","standardizedPronoun":null,"occupation":"--","objectUrn":"urn:li:member:1096125424","backgroundImage":null,"picture":null,"$type":"com.linkedin.voyager.identity.shared.MiniProfile","firstName":"Fayyoz","entityUrn":"urn:li:fs_miniProfile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","publicIdentifier":"fayyoz-usmon-a1410b267","trackingId":"L+RhQkkbRHWM2PJg5WP/lg=="}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231304">
  {"request":"/voyager/api/me","status":200,"body":"bpr-guid-6231304","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231304"><code style="display: none" id="bpr-guid-6231320">
  {"data":{"entityUrn":"urn:li:collectionResponse:QQzXT8CXeExC567LZS8tvEQXw4LEVc6X26d/Bet8ySE=","elements":[],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"meta":{"microSchema":{"version":"2","types":{"com.linkedin.deco.recipe.anonymous.Anon1080314395":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon489758292":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"contentSeriesUrn":{"type":"string"},"contentSeries":{"type":"com.linkedin.deco.recipe.anonymous.Anon1789236903","resolvedFrom":"contentSeriesUrn"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToContentSeriesSubscribeMetadata"},"com.linkedin.deco.recipe.anonymous.Anon377984030":{"fields":{"professionalEvent":{"type":"com.linkedin.deco.recipe.anonymous.Anon1213723597","resolvedFrom":"professionalEventUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.voyager.dash.deco.common.image.Company":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction":{"fields":{"subscribed":{"type":"boolean"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"subscriberCount":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.publishing.SubscribeAction"},"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason":{"fields":{"sourceType":{"type":"string"},"reasonObjects":{"type":{"array":"string"}},"reasonContext":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.EdgeBuildingCohortReason"},"com.linkedin.deco.recipe.anonymous.Anon1444411496":{"fields":{"countryUrn":{"type":"string"},"country":{"type":"com.linkedin.deco.recipe.anonymous.Anon1918694303","resolvedFrom":"countryUrn"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon270389304":{"fields":{"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"productsAccessible":{"type":"boolean"},"croppedCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"lcpStaffingOrganization":{"type":"boolean"},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"lcpTreatment":{"type":"boolean"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"relevantFollowersForViewerAndOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon2053503342","isInjection":true},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"originalCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"name":{"type":"string"},"lcpCustomer":{"type":"boolean"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1226238669":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon798864187":{"fields":{"influencer":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel":{"fields":{"actionDataModel":{"type":{"union":{"follow":"com.linkedin.voyager.dash.deco.feed.FollowingState","connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"actionDataModelUnion"},"actionDataModelUnion":{"type":{"union":{"connectionOrInvitation":"string","relationshipActionData":"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData","memberToEntityRelationship":"string","follow":"string"}}},"emphasisStyle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.StatefulButtonModel"},"com.linkedin.deco.recipe.anonymous.Anon1636615691":{"fields":{"transcripts":{"type":{"array":"string"}},"pagesPerResolution":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1525160909"}}},"baseType":"com.linkedin.documentcontent.DocumentPages"},"com.linkedin.deco.recipe.anonymous.Anon934606613":{"fields":{"type":{"type":"string"},"name":{"type":"string"},"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipOption"},"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.deco.recipe.anonymous.Anon739258202","largeTemplate":"com.linkedin.deco.recipe.anonymous.Anon2003298771","smallTemplate":"com.linkedin.deco.recipe.anonymous.Anon548743587"}}},"stickerLinkTemplateSize":{"type":"string"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"string"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"string"},"urn":{"type":"string"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"text":{"type":"string"},"fourthCornerYOffsetPercentage":{"type":"float"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.deco.recipe.anonymous.Anon1956213100":{"fields":{"productCategories":{"type":{"array":"string"}},"signatureProduct":{"type":"boolean"},"localizedName":{"type":"string"},"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon570745354"},"resolvedFrom":"organizationsUsingProductUrns"},"entityUrn":{"type":"string"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1122581856","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"universalName":{"type":"string"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.deco.recipe.anonymous.Anon1312950162":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1792705474"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions":{"fields":{"canCreateBroadcast":{"type":"boolean"},"canReadContentSuggestions":{"type":"boolean"},"canSeeProducts":{"type":"boolean"},"canUpdateOrganizationProfile":{"type":"boolean"},"canSponsorShare":{"type":"boolean"},"canEditPendingDirectSponsoredContentPosters":{"type":"boolean"},"canReadAdminFeedFollowingPill":{"type":"boolean"},"canUntagFromMention":{"type":"boolean"},"canReadOrganizationActivity":{"type":"boolean"},"canManageCareerPages":{"type":"boolean"},"canAssociateHashtag":{"type":"boolean"},"canDeleteShare":{"type":"boolean"},"canInviteMemberToFollow":{"type":"boolean"},"canEditPipelineBuilderAdministrators":{"type":"boolean"},"canReadBroadcastAnalytics":{"type":"boolean"},"canManageMessagingAccess":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canReadPipelineBuilderAdminPage":{"type":"boolean"},"canDeactivateOrganization":{"type":"boolean"},"canCreateOrganicShare":{"type":"boolean"},"canNotifyEmployeesOfShare":{"type":"boolean"},"canEditDarkShare":{"type":"boolean"},"canSeePostJobButton":{"type":"boolean"},"canEnableCommentsShare":{"type":"boolean"},"canReadOrganizationLeadsAnalytics":{"type":"boolean"},"canDeleteDarkShare":{"type":"boolean"},"canCreateJobPosting":{"type":"boolean"},"canEditLeadGenerationFormManagers":{"type":"boolean"},"canReadMessages":{"type":"boolean"},"canReadOrganizationTalentBrandAnalytics":{"type":"boolean"},"canEditDirectSponsoredContentPosters":{"type":"boolean"},"canPinShare":{"type":"boolean"},"canReadPipelineBuilderAdministrators":{"type":"boolean"},"canCreateLinkedInPagesProductFeedBack":{"type":"boolean"},"canReadEvents":{"type":"boolean"},"organizationRoles":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType"}},"canEditEvents":{"type":"boolean"},"canManageOrganizationalPageFollow":{"type":"boolean"},"canReadOrganizationUpdateAnalytics":{"type":"boolean"},"canCreateComment":{"type":"boolean"},"canRequestAdminAccess":{"type":"boolean"},"canEditAdministrators":{"type":"boolean"},"canCreateDarkShare":{"type":"boolean"},"canEmployeesInviteToFollow":{"type":"boolean"},"canSeeEmployeeExperienceAsMember":{"type":"boolean"},"canReadAdministrators":{"type":"boolean"},"canExportLeads":{"type":"boolean"},"canReadDirectSponsoredContentPosters":{"type":"boolean"},"canCreateShowcase":{"type":"boolean"},"canManageEmployeeExperienceSettings":{"type":"boolean"},"canReadLeadGenerationFormManagers":{"type":"boolean"},"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canEditCurators":{"type":"boolean"},"canEditProducts":{"type":"boolean"},"canManageVerifiedEmailDomains":{"type":"boolean"},"canReadPendingAdministrators":{"type":"boolean"},"canReadPendingDirectSponsoredContentPosters":{"type":"boolean"},"canEditPipelineBuilderAdminPage":{"type":"boolean"},"canReadTermsAndAgreements":{"type":"boolean"},"canReadOrganizationFollowerAnalytics":{"type":"boolean"},"canManageOrganizationCompetitors":{"type":"boolean"},"canSendMessages":{"type":"boolean"},"canCreateReaction":{"type":"boolean"},"canDisableCommentsShare":{"type":"boolean"},"canEditPendingAdministrators":{"type":"boolean"},"canNotifyEmployees":{"type":"boolean"},"canReadOrganizationPipelineBuilderAnalytics":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.voyager.dash.deco.common.video.Transcript":{"fields":{"captionFormat":{"type":"string"},"locale":{"type":"com.linkedin.voyager.dash.deco.common.Locale"},"lines":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.TranscriptLine"}},"isAutogenerated":{"type":"boolean"},"captionFile":{"type":"string"}},"baseType":"com.linkedin.videocontent.Transcript"},"com.linkedin.deco.recipe.anonymous.Anon1286825962":{"fields":{"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"pageType":{"type":"string"},"logoV2":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.deco.recipe.anonymous.Anon1358424483":{"fields":{"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextFieldComponent"},"com.linkedin.deco.recipe.anonymous.Anon632521550":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon523036483":{"fields":{"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.FollowersMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1949606970":{"fields":{"followerResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1313960689","resolvedFrom":"follower"},"followedAt":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"follower":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationFollower"},"com.linkedin.voyager.dash.deco.common.image.Group":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.deco.recipe.anonymous.Anon1918694303":{"fields":{"entityUrn":{"type":"string"},"defaultLocalizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow":{"fields":{"socialProofInsight":{"type":{"union":{"standard":"com.linkedin.deco.recipe.anonymous.Anon1350307109","compact":"com.linkedin.deco.recipe.anonymous.Anon853977300"}},"resolvedFrom":"socialProofInsightUnion"},"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions"},"salesNavigatorCompanyUrl":{"type":"string"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryUrns":{"type":{"array":"string"}},"industryV2Urns":{"type":{"array":"string"}},"socialProofInsightUnion":{"type":{"union":{"standard":"string","compact":"string"}}},"employeeCount":{"type":"long"},"associatedSignatureProductUrn":{"type":"string"},"url":{"type":"string"},"callToAction":{"type":"com.linkedin.voyager.dash.deco.launchpad.CallToAction"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"associatedSignatureProduct":{"type":"com.linkedin.voyager.dash.deco.organization.FullMemberProduct","resolvedFrom":"associatedSignatureProductUrn"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1213723597":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.deco.recipe.anonymous.Anon184130218":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon487759616":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"schoolName":{"type":"string"},"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon1621960601","resolvedFrom":"schoolUrn"},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Education"},"com.linkedin.deco.recipe.anonymous.Anon1251747613":{"fields":{"urn":{"type":"string"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.voyager.dash.deco.organization.FullMemberProduct":{"fields":{"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1432799903"},"resolvedFrom":"organizationsUsingProductUrns"},"productUserTitle":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1369245809"},"resolvedFrom":"productUserTitleUrns"},"companyUrn":{"type":"string"},"productUserTitleUrns":{"type":{"array":"string"}},"similarProducts":{"type":"com.linkedin.deco.recipe.anonymous.Anon975174582","isInjection":true},"entityUrn":{"type":"string"},"organizationalPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1196575529","resolvedFrom":"organizationalPageUrn"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"similarProductsFromSameOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon1914510571","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon270389304","resolvedFrom":"companyUrn"},"productCardCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon205248910","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"state":{"type":"string"},"universalName":{"type":"string"},"groupUrn":{"type":"string"},"localizedDescription":{"type":"string"},"localizedWebsite":{"type":"string"},"customizableCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon205248910","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon690575082","resolvedFrom":"groupUrn"},"hashtag":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon2070591469"},"resolvedFrom":"hashtagUrns"},"localizedName":{"type":"string"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1517647575","isInjection":true},"mediaSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon805140200"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"hashtagUrns":{"type":{"array":"string"}},"productCategories":{"type":{"array":"string"}},"memberFacingCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon205248910","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"standardizedSkillUrns":{"type":{"array":"string"}},"organizationalPageUrn":{"type":"string"},"autoPublished":{"type":"boolean"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"standardizedSkill":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon503562015"},"resolvedFrom":"standardizedSkillUrns"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions":{"fields":{"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canCreateBroadcast":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.deco.recipe.anonymous.Anon1869367056":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection":{"fields":{"secondaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"secondaryAction"},"primaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"primaryAction"},"trackingIdentifier":{"type":"bytes"},"primaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"secondaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"overflowActionsV2":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"overflowActionsResolutionResults":{"type":{"map":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}}},"resolvedFrom":"overflowActions"},"overflowActions":{"type":{"array":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}}},"secondaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"},"trackingId":{"type":"string"},"primaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActions"},"com.linkedin.deco.recipe.anonymous.Anon2127563472":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon514683669":{"fields":{"currentProgress":{"type":"int"},"threshold":{"type":"int"},"progressText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadProgressMeter"},"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.FullProductCategory":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"productCategoryPageUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.ProductCategory"},"com.linkedin.deco.recipe.anonymous.Anon1428941918":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon2116040434"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon4539795":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon2058972220"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.text.ProfileForMention":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution":{"fields":{"attribution":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon831664802":{"fields":{"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon2070591469":{"fields":{"actionTarget":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon2127563472","isInjection":true},"displayName":{"type":"string"},"trackingUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.voyager.dash.deco.common.Industry":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Industry"},"com.linkedin.deco.recipe.anonymous.Anon912817537":{"fields":{"memberDistance":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1154789573":{"fields":{"maxResponseLength":{"type":"int"},"response":{"type":"string"},"validationType":{"type":"string"},"minResponseLength":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextInputComponent"},"com.linkedin.deco.recipe.anonymous.Anon8381687":{"fields":{"multipleChoiceQuestionOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1507465610"}},"placeholderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.DropdownSelectComponent"},"com.linkedin.voyager.dash.deco.relationships.InvitationRaw":{"fields":{"genericInvitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon739258202":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream":{"fields":{"initialBitRate":{"type":"int"},"protocol":{"type":"string"},"mediaType":{"type":"string"},"masterPlaylists":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}}},"baseType":"com.linkedin.videocontent.AdaptiveStream"},"com.linkedin.deco.recipe.anonymous.Anon1529112441":{"fields":{"originalImageUrl":{"type":"string"},"originalHeight":{"type":"int"},"url":{"type":"string"},"originalWidth":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.voyager.dash.deco.common.video.StreamingLocation":{"fields":{"url":{"type":"string"},"expiresAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.StreamingLocation"},"com.linkedin.deco.recipe.anonymous.Anon1683511457":{"fields":{"requiredFieldMissingErrorText":{"type":"string"},"formComponentUnion":{"type":{"union":{"textFieldComponent":"com.linkedin.deco.recipe.anonymous.Anon1358424483","dropdownSelectComponent":"com.linkedin.deco.recipe.anonymous.Anon8381687","textInputComponent":"com.linkedin.deco.recipe.anonymous.Anon1154789573","checkboxComponent":"com.linkedin.deco.recipe.anonymous.Anon554507693"}}},"question":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"editable":{"type":"boolean"},"leadGenFormQuestionUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormQuestion"},"com.linkedin.voyager.dash.deco.launchpad.LaunchpadView":{"fields":{"launchpadCards":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1121002957"}},"dismissDialog":{"type":"com.linkedin.deco.recipe.anonymous.Anon1670704461"},"headerImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"focusedCardIndex":{"type":"int"},"progressMeter":{"type":"com.linkedin.deco.recipe.anonymous.Anon514683669"},"branding":{"type":"com.linkedin.deco.recipe.anonymous.Anon1963883771"},"dismissible":{"type":"boolean"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"theme":{"type":"string"},"legoTrackingToken":{"type":"string"},"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadView"},"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata":{"fields":{"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl"},"progressiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata"}},"liveStreamCreatedAt":{"type":"long"},"prevMedia":{"type":"string"},"transcripts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Transcript"}},"aspectRatio":{"type":"float"},"media":{"type":"string"},"adaptiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream"}},"liveStreamEndedAt":{"type":"long"},"duration":{"type":"long"},"provider":{"type":"string"},"entityUrn":{"type":"string"},"nextMedia":{"type":"string"},"thumbnails":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Thumbnail"}},"trackingId":{"type":"string"}},"baseType":"com.linkedin.videocontent.VideoPlayMetadata"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground":{"fields":{"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon2058972220":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon412783727","invitation":"com.linkedin.deco.recipe.anonymous.Anon831664802"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon561948584":{"fields":{"ctaText":{"type":"string"},"landingPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1586568086"},"document":{"type":"com.linkedin.deco.recipe.anonymous.Anon856218111"},"confirmationTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"urlViewingBehavior":{"type":"string"},"sponsoredUrlAttributes":{"type":"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes"},"confirmationDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenGatedContent"},"com.linkedin.deco.recipe.anonymous.Anon783596108":{"fields":{"ctaColor":{"type":"string"},"ctaType":{"type":"string"},"ctaStyle":{"type":"string"},"presentationStyle":{"type":"string"},"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctaTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"deeplinkUrl":{"type":"string"},"ctaActionUnion":{"type":{"union":{"redirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","launchpadApiRequest":"com.linkedin.restli.common.EmptyRecord","launchpadApiRequestAndRedirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","shareBox":"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction"}}},"legoActionCategory":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCta"},"com.linkedin.voyager.dash.deco.relationships.Invitation":{"fields":{"inviteeMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"inviteeMember"},"invitationType":{"type":"string"},"inviteeMember":{"type":"string"},"entityUrn":{"type":"string"},"invitationState":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.IndustryV2":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.IndustryV2"},"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow":{"fields":{"lastName":{"type":"string"},"profileTopEducation":{"type":"com.linkedin.deco.recipe.anonymous.Anon964860152","isInjection":true},"profilePositions":{"type":"com.linkedin.deco.recipe.anonymous.Anon1861880920","isInjection":true},"educationOnProfileTopCardShown":{"type":"boolean"},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1449981882","isInjection":true},"associatedHashtagsCopy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"firstName":{"type":"string"},"profileInsight":{"type":"com.linkedin.deco.recipe.anonymous.Anon1439603308","isInjection":true},"geoLocation":{"type":"com.linkedin.deco.recipe.anonymous.Anon708807917"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon89639392","isInjection":true},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"companyNameOnProfileTopCardShown":{"type":"boolean"},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1069349607":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon912817537"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.deco.recipe.anonymous.Anon205248910":{"fields":{"dashLeadGenFormUrn":{"type":"string"},"privacyPolicyUrl":{"type":"string"},"dashLeadGenForm":{"type":"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm","resolvedFrom":"dashLeadGenFormUrn"},"organizationProductCallToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon625674184"},"leadGenFormUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductLeadGenFormCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon554507693":{"fields":{"checked":{"type":"boolean"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.CheckboxComponent"},"com.linkedin.deco.recipe.anonymous.Anon1128782547":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon1509374374":{"fields":{"displayText":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.GenericCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon503562015":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.deco.recipe.anonymous.Anon708807917":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1444411496","resolvedFrom":"geoUrn"},"geoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"frameType":{"type":"string"},"a11yText":{"type":"string"},"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon858976209":{"fields":{"type":{"type":"string"},"epochAt":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.deco.recipe.anonymous.Anon1517647575":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1703899306"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction":{"fields":{"deeplinkUrl":{"type":"string"},"recruiterContext":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ViewProfileInExternalContextAction"},"com.linkedin.deco.recipe.anonymous.Anon1843097075":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon1286825962"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon89639392":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.voyager.dash.deco.common.video.TranscriptLine":{"fields":{"lineStartAt":{"type":"long"},"caption":{"type":"string"},"lineEndAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.TranscriptLine"},"com.linkedin.voyager.dash.deco.launchpad.CallToAction":{"fields":{"displayText":{"type":"string"},"visible":{"type":"boolean"},"type":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.CallToAction"},"com.linkedin.deco.recipe.anonymous.Anon625674184":{"fields":{"type":{"type":"string"},"callToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon1509374374"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductCallToAction"},"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.deco.recipe.anonymous.Anon1432799903":{"fields":{"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1031101697":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"invitationRelationshipForm":{"type":"com.linkedin.deco.recipe.anonymous.Anon918376989"},"inviterResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon1271139837":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon1313960689":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"insight":{"type":"com.linkedin.deco.recipe.anonymous.Anon137428918","isInjection":true},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1428941918","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"backgroundPicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon163061530":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.relationships.profile":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"creator":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon378816379":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon660720507","resolvedFrom":"companyUrn"},"companyUrn":{"type":"string"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToCompanyFollowMetadata"},"com.linkedin.deco.recipe.anonymous.Anon743120773":{"fields":{"authorProfileId":{"type":"string"},"targetUrn":{"type":"string"},"contentSource":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.semaphore.SemaphoreContext"},"com.linkedin.deco.recipe.anonymous.Anon805140200":{"fields":{"mediaContentUnion":{"type":{"union":{"embeddedVideo":"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec","url":"string","videoPlayMetadata":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"description":{"type":"string"},"mediaEntityUnion":{"type":{"union":{"digitalMediaAssetUrn":"string","articleUrn":"string"}}},"title":{"type":"string"},"mediaSource":{"type":"string"},"primary":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.MediaSection"},"com.linkedin.deco.recipe.anonymous.Anon964860152":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon487759616"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.NoConnection":{"fields":{"memberDistance":{"type":"string"},"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1575907873":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeNavigationContext":{"type":"com.linkedin.deco.recipe.anonymous.Anon199392175"},"displayText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"textStartIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeOptionType":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeOption"},"com.linkedin.deco.recipe.anonymous.Anon40833127":{"fields":{"metadataUnion":{"type":{"union":{"memberToContentSeriesSubscribe":"com.linkedin.deco.recipe.anonymous.Anon489758292"}}},"subscribeActionUrn":{"type":"string"},"subscribeAction":{"type":"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction","resolvedFrom":"subscribeActionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.SubscribeRelationship"},"com.linkedin.deco.recipe.anonymous.Anon2115336743":{"fields":{"displayText":{"type":"string"},"toggledDisplayText":{"type":"string"},"enabled":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductAddSkillCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon336652209":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"tempStatusExpiredAtUnion":{"type":{"union":{"customizedExpiredAt":"long","standardizedExpiration":"string"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.VectorArtifact":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"long"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.voyager.dash.deco.common.text.School":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon2066892616":{"fields":{"invitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon1350307109":{"fields":{"insightViewee":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"insightVieweeUrn"},"insightVieweeUrn":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"sharedConnectionDetailTarget"},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.launchpad.PageKey":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.deco.recipe.anonymous.Anon430310092":{"fields":{"entityUrn":{"type":"string"},"creator":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1586568086":{"fields":{"url":{"type":"string"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Link"},"com.linkedin.deco.recipe.anonymous.Anon1976702492":{"fields":{"iweWarned":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction":{"fields":{"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.ShareBoxAction"},"com.linkedin.deco.recipe.anonymous.Anon1439603308":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1122468175"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon437558093":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.deco.recipe.anonymous.Anon1069349607","isInjection":true},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1507465610":{"fields":{"type":{"type":"string"},"textOption":{"type":"string"},"selected":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.MultipleChoiceQuestionOptions"},"com.linkedin.deco.recipe.anonymous.Anon409680083":{"fields":{"type":{"type":"string"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.deco.recipe.anonymous.Anon1792705474":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon2066892616"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1369245809":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.StandardizedTitle"},"com.linkedin.deco.recipe.anonymous.Anon412783727":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon352881287","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976702492","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon533924488":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"sharedConnectionDetailTarget"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.deco.recipe.anonymous.Anon918376989":{"fields":{"title":{"type":"string"},"invitationRelationshipOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon934606613"}},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipForm"},"com.linkedin.voyager.dash.deco.common.Locale":{"fields":{"variant":{"type":"string"},"country":{"type":"string"},"language":{"type":"string"}},"baseType":"com.linkedin.common.Locale"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl":{"fields":{"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.voyager.dash.deco.common.video.Resolution":{"fields":{"width":{"type":"int"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.Resolution"},"com.linkedin.deco.recipe.anonymous.Anon853977300":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.organization.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1525160909":{"fields":{"width":{"type":"int"},"imageUrls":{"type":{"array":"string"}},"height":{"type":"int"}},"baseType":"com.linkedin.documentcontent.DocumentResolutionPages"},"com.linkedin.voyager.dash.deco.common.Link":{"fields":{"type":{"type":"string"},"rel":{"type":"string"},"href":{"type":"string"}},"baseType":"com.linkedin.restli.common.Link"},"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel":{"fields":{"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.voyager.dash.deco.common.image.School":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.voyager.dash.deco.common.VectorImage":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon881636240":{"fields":{"inviter":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.voyager.dash.deco.relationships.Connection":{"fields":{"createdAt":{"type":"long"},"connectedMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"connectedMember"},"connectedMember":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.voyager.dash.deco.organization.MiniCompany":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon198476569":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"vieweeFollowingViewer":{"type":"boolean"},"viewerUrn":{"type":"string"},"viewee":{"type":"com.linkedin.voyager.dash.deco.relationships.profile","resolvedFrom":"vieweeUrn"},"vieweeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToMemberFollowMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1703899306":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon2003298771":{"fields":{"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"footerText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"insightText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"backgroundImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.voyager.dash.deco.feed.FollowingState":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"},"followingType":{"type":"string"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon1914510571":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship":{"fields":{"invitationUrn":{"type":"string"},"entityUrn":{"type":"string"},"invitation":{"type":"com.linkedin.voyager.dash.deco.relationships.InvitationRaw","resolvedFrom":"invitationUrn"},"relationshipDataUnion":{"type":{"union":{"follow":"com.linkedin.deco.recipe.anonymous.Anon191738356","subscribe":"com.linkedin.deco.recipe.anonymous.Anon40833127"}}}},"baseType":"com.linkedin.voyager.dash.relationships.DirectionalEntityRelationship"},"com.linkedin.deco.recipe.anonymous.Anon1568806612":{"fields":{"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon929195650","resolvedFrom":"schoolUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.deco.recipe.anonymous.Anon1449981882":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1566936928"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon660720507":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"iweWarned":{"type":"boolean"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.image.ImageViewModel":{"fields":{"attributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.image.ImageAttribute"}},"actionTarget":{"type":"string"},"totalCount":{"type":"int"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.deco.recipe.anonymous.Anon648914460":{"fields":{"companyUrn":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.deco.recipe.anonymous.Anon1042323916":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.Hashtag":{"fields":{"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"actionTarget":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.voyager.dash.deco.common.FullPaging":{"fields":{"start":{"type":"int"},"count":{"type":"int"},"total":{"type":"int"},"links":{"type":{"array":"com.linkedin.voyager.dash.deco.common.Link"}}},"baseType":"com.linkedin.restli.common.CollectionMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1122468175":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1271139837","resolvedFrom":"geoUrn"},"keywords":{"type":"string"},"geoUrn":{"type":"string"},"savedSearchId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.JobSearchQuery"},"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent":{"fields":{"logoImage":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.voyager.dash.deco.common.video.Thumbnail":{"fields":{"resolution":{"type":"com.linkedin.voyager.dash.deco.common.video.Resolution"},"url":{"type":"string"}},"baseType":"com.linkedin.videocontent.Thumbnail"},"com.linkedin.voyager.dash.deco.common.DateRange":{"fields":{"start":{"type":"com.linkedin.voyager.dash.deco.common.Date"},"end":{"type":"com.linkedin.voyager.dash.deco.common.Date"}},"baseType":"com.linkedin.common.DateRange"},"com.linkedin.deco.recipe.anonymous.Anon301818032":{"fields":{"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"contentHorizontallyCentered":{"type":"boolean"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenBannerComponent"},"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec":{"fields":{"url":{"type":"string"},"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImage"}},"baseType":"com.linkedin.voyager.dash.organization.EmbeddedVideo"},"com.linkedin.deco.recipe.anonymous.Anon1670704461":{"fields":{"title":{"type":"string"},"subTitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadDismissDialog"},"com.linkedin.voyager.dash.deco.common.Date":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata":{"fields":{"streamingLocations":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}},"size":{"type":"long"},"bitRate":{"type":"int"},"width":{"type":"int"},"mediaType":{"type":"string"},"mimeType":{"type":"string"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.ProgressiveDownloadMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1196575529":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType":{"fields":{"localizedName":{"type":"string"},"role":{"type":"string"},"paidMediaRole":{"type":"boolean"},"localizedDescription":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationRoleType"},"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2":{"fields":{"textDirection":{"type":"string"},"text":{"type":"string"},"attributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.deco.recipe.anonymous.Anon1789236903":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.publishing.ContentSeries"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm":{"fields":{"formSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1025867760"}},"submitted":{"type":"boolean"},"entityUrn":{"type":"string"},"privacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"postSubmissionContent":{"type":"com.linkedin.deco.recipe.anonymous.Anon561948584"},"lastSubmittedAt":{"type":"long"},"trackingMetadata":{"type":"string"},"header":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"banner":{"type":"com.linkedin.deco.recipe.anonymous.Anon301818032"},"submitButtonText":{"type":"string"},"customPrivacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"consentSection":{"type":"com.linkedin.deco.recipe.anonymous.Anon1025867760"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenForm"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1975396536":{"fields":{"saved":{"type":"boolean"},"salesListNavigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.SalesNavigatorSaveState"},"com.linkedin.deco.recipe.anonymous.Anon929195650":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon1299693827":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon352881287":{"fields":{"entityUrn":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1760283115":{"fields":{"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon690575082":{"fields":{"viewerGroupMembership":{"type":"com.linkedin.deco.recipe.anonymous.Anon632521550","resolvedFrom":"viewerGroupMembershipUrn"},"displayNotificationSubscriptionLevelOptions":{"type":"boolean"},"groupPostNotificationsEdgeSetting":{"type":"com.linkedin.deco.recipe.anonymous.Anon735489647","resolvedFrom":"groupPostNotificationsEdgeSettingUrn"},"groupPostNotificationsEdgeSettingUrn":{"type":"string"},"postApprovalEnabled":{"type":"boolean"},"backendGroupUrn":{"type":"string"},"memberCount":{"type":"int"},"owners":{"type":"com.linkedin.deco.recipe.anonymous.Anon1299693827","isInjection":true},"memberConnectionsFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"memberConnections":{"type":"com.linkedin.deco.recipe.anonymous.Anon721197842","isInjection":true},"membersFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"directJoinEnabled":{"type":"boolean"},"entityUrn":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"globalNewPostNotificationSettingOn":{"type":"boolean"},"viewerGroupMembershipUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.common.text.LearningCourse":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.deco.recipe.anonymous.Anon50101142":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1320789737"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"profileUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.deco.recipe.anonymous.Anon1566936928":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1122581856":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.Profile"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.image.ImageAttribute":{"fields":{"detailData":{"type":{"union":{"profilePictureWithoutFrame":"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame","profilePicture":"com.linkedin.voyager.dash.deco.common.image.Profile","profilePictureWithRingStatus":"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus","companyLogo":"com.linkedin.voyager.dash.deco.common.image.Company","professionalEventLogo":"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent","organizationalPageLogo":"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage","schoolLogo":"com.linkedin.voyager.dash.deco.common.image.School","groupLogo":"com.linkedin.voyager.dash.deco.common.image.Group"}},"resolvedFrom":"detailDataUnion"},"tintColor":{"type":"string"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"string","systemImage":"string","nonEntityGroupLogo":"com.linkedin.deco.recipe.anonymous.Anon1436383648","organizationalPageLogo":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution","nonEntityProfessionalEventLogo":"com.linkedin.deco.recipe.anonymous.Anon377984030","profilePicture":"string","imageUrl":"com.linkedin.deco.recipe.anonymous.Anon1529112441","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.deco.recipe.anonymous.Anon1568806612","nonEntityCompanyLogo":"com.linkedin.deco.recipe.anonymous.Anon648914460","schoolLogo":"string","groupLogo":"string","ghostImage":"string","nonEntityProfilePicture":"com.linkedin.deco.recipe.anonymous.Anon50101142"}}},"tapTargets":{"type":{"array":"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity"}},"scalingType":{"type":"string"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.deco.recipe.anonymous.Anon2053503342":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon523036483"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1949606970"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1080314395","isInjection":true},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"a11yText":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.voyager.dash.deco.common.text.Company":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon462345189":{"fields":{"controlName":{"type":"string"},"viewingBehavior":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.deco.recipe.anonymous.Anon956797500":{"fields":{"profileActionUnion":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"profileAction":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"profileActionUnion"},"trackingId":{"type":"bytes"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActionInfo"},"com.linkedin.deco.recipe.anonymous.Anon1121002957":{"fields":{"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"cardType":{"type":"string"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"completed":{"type":"boolean"},"legoTrackingToken":{"type":"string"},"cardTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctas":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon783596108"}},"animate":{"type":"boolean"},"videoMetadata":{"type":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata"},"cardLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"attributeUnion":{"type":{"union":{"followOrganization":"string","launchpad":"string","jobSearchQuery":"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery","companyJYMBII":"string","generalJYMBII":{"array":"string"},"viewExperienceEntity":"string","followPostCreator":"string","fifWidgetGroupId":"string","connectWithEntity":"string","entityActivityFeed":"string","edgeBuilding":{"array":"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason"},"jobAlertJYMBII":{"array":"string"},"organizationActivityFeed":"string"}}},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"overlayImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"attribute":{"type":{"union":{"followOrganization":"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow","companyJYMBII":"com.linkedin.deco.recipe.anonymous.Anon184130218","generalJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"connectWithEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","viewExperienceEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","entityActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon587401631","followPostCreator":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","jobAlertJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"organizationActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon1927941263"}},"resolvedFrom":"attributeUnion"},"cardSubtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"cardStyle":{"type":"string"},"entitySize":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCard"},"com.linkedin.deco.recipe.anonymous.Anon735489647":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.notifications.edgesetting.EdgeSetting"},"com.linkedin.deco.recipe.anonymous.Anon1927941263":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon587401631":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1621960601":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon199392175":{"fields":{"paidInMail":{"type":"boolean"},"recipient":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon587401631"},"resolvedFrom":"recipientUrns"},"messageRequestContextUrn":{"type":"string"},"recipientUrns":{"type":{"array":"string"}},"targetUrl":{"type":"string"},"existingConversation":{"type":"com.linkedin.deco.recipe.anonymous.Anon807183080","resolvedFrom":"existingConversationUrn"},"existingConversationUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeNavigationContext"},"com.linkedin.deco.recipe.anonymous.Anon1320789737":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon721197842":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1085906135":{"fields":{"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyName":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon184130218","resolvedFrom":"companyUrn"},"title":{"type":"string"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1963883771":{"fields":{"accentType":{"type":"string"},"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadBranding"},"com.linkedin.deco.recipe.anonymous.Anon807183080":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.Conversation"},"com.linkedin.deco.recipe.anonymous.Anon570745354":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.image.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.JobPosting":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2":{"fields":{"start":{"type":"int"},"length":{"type":"int"},"detailData":{"type":{"union":{"jobPostingName":"com.linkedin.voyager.dash.deco.common.text.JobPosting","profileFamiliarName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName","groupName":"com.linkedin.deco.recipe.anonymous.Anon163061530","profileFullName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName","learningCourseName":"com.linkedin.voyager.dash.deco.common.text.LearningCourse","companyName":"com.linkedin.voyager.dash.deco.common.text.Company","profileMention":"com.linkedin.voyager.dash.deco.common.text.ProfileForMention","schoolName":"com.linkedin.voyager.dash.deco.common.text.School","organizationalPageName":"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage","hashtag":"com.linkedin.voyager.dash.deco.common.text.Hashtag"}},"resolvedFrom":"detailDataUnion"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"string","color":"string","companyName":"string","icon":"string","systemImage":"string","epoch":"com.linkedin.deco.recipe.anonymous.Anon858976209","organizationalPageName":"string","textLink":"com.linkedin.deco.recipe.anonymous.Anon462345189","listItemStyle":"com.linkedin.deco.recipe.anonymous.Anon409680083","hyperlinkOpenExternally":"string","listStyle":"string","groupName":"string","profileFullName":"string","stringFieldReference":"com.linkedin.deco.recipe.anonymous.Anon1251747613","learningCourseName":"string","profileMention":"string","style":"string","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.deco.recipe.anonymous.Anon975174582":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1213132949":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1843097075"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData":{"fields":{"relationshipsTrackingId":{"type":"bytes"},"relationshipData":{"type":{"union":{"connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"relationshipDataUnion"},"relationshipDataUnion":{"type":{"union":{"connectionOrInvitation":"string","memberToEntityRelationship":"string"}}},"moduleKey":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.relationships.RelationshipActionData"},"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper":{"fields":{"memberRelationshipUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","resolvedFrom":"memberRelationshipUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationshipWrapper"},"com.linkedin.deco.recipe.anonymous.Anon137428918":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon533924488"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon191738356":{"fields":{"metadataUnion":{"type":{"union":{"memberToMemberFollow":"com.linkedin.deco.recipe.anonymous.Anon198476569","memberToCompanyFollow":"com.linkedin.deco.recipe.anonymous.Anon378816379"}}},"followingStateUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.FollowRelationship"},"com.linkedin.voyager.dash.deco.launchpad.RedirectAction":{"fields":{"presentationStyle":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.RedirectAction"},"com.linkedin.deco.recipe.anonymous.Anon2116040434":{"fields":{"company":{"type":"com.linkedin.voyager.dash.deco.organization.MiniCompany","resolvedFrom":"companyUrn"},"title":{"type":"string"},"multiLocaleTitle":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon856218111":{"fields":{"transcribedDocumentUrl":{"type":"string"},"urn":{"type":"string"},"manifestUrl":{"type":"string"},"scanRequiredForDownload":{"type":"boolean"},"totalPageCount":{"type":"int"},"manifestUrlExpiresAt":{"type":"long"},"title":{"type":"string"},"coverPages":{"type":"com.linkedin.deco.recipe.anonymous.Anon1636615691"},"transcribedDocumentUrlExpiresAt":{"type":"long"}},"baseType":"com.linkedin.documentcontent.Document"},"com.linkedin.deco.recipe.anonymous.Anon1025867760":{"fields":{"questions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1683511457"}}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormSection"},"com.linkedin.deco.recipe.anonymous.Anon1436383648":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.deco.recipe.anonymous.Anon548743587":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.deco.recipe.anonymous.Anon1861880920":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1085906135"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1585981083":{"fields":{"title":{"type":"string"},"description":{"type":"string"},"connectionsUsingProductTypeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ConnectionsUsingProductMetadata"},"com.linkedin.deco.recipe.anonymous.Anon2134644859":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"}}}},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231320">
  {"request":"/voyager/api/voyagerLaunchpadDashLaunchpadViews?decorationId\u003Dcom.linkedin.voyager.dash.deco.launchpad.LaunchpadView-69\u0026launchpadContext\u003DTAKEOVER\u0026q\u003Dcontext","status":200,"body":"bpr-guid-6231320","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231320"><code style="display: none" id="bpr-guid-6231324">
  {"data":{"data":{"feedDashGlobalNavs":{"logo":{"controlName":"nav_inbug","logo":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_LINKEDIN_BUG_COLOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":"LinkedIn","$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"$recipeTypes":["com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"],"hoverText":null,"$type":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"$recipeTypes":["com.linkedin.eae2ed6e463973c8e0d55baecd814363"],"spotlight":{"navContent":{"*premiumUpsellSlotUrn":"urn:li:fsd_premiumUpsellSlot:NAV_SPOTLIGHT","navElement":null},"type":"PREMIUM_UPSELL","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"primaryItems":[{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_homepage","showIconBorder":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","text":"Home","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"HOME","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_mynetwork","showIconBorder":false,"actionTarget":"https://www.linkedin.com/mynetwork/","text":"Mijn netwerk","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MY_NETWORK","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_jobs","showIconBorder":false,"actionTarget":"https://www.linkedin.com/jobs/","text":"Vacatures","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOBS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_messaging","showIconBorder":false,"actionTarget":"https://www.linkedin.com/messaging/","text":"Berichten","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MESSAGING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_notifications","showIconBorder":false,"actionTarget":"https://www.linkedin.com/notifications/","text":"Meldingen","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"NOTIFICATIONS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"meMenu":{"viewProfileText":"Profiel weergeven","*profile":"urn:li:fsd_profile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","meGroups":[{"meItems":[{"*premiumUpsellSlotUrn":"urn:li:fsd_premiumUpsellSlot:NAV_ME","meItem":null},{"meItem":{"controlName":"nav_settings_account_manage_account","actionTarget":"https://www.linkedin.com/mypreferences/d/","text":{"textDirection":"USER_LOCALE","text":"Instellingen en privacy","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_quick_help","actionTarget":"https://www.linkedin.com/help/linkedin/","text":{"textDirection":"USER_LOCALE","text":"Hulp","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":true,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_language","actionTarget":"https://www.linkedin.com/mypreferences/d/language/","text":{"textDirection":"USER_LOCALE","text":"Taal","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Account","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"},{"meItems":[{"meItem":{"controlName":"recent_activity_nav_all","actionTarget":"https://www.linkedin.com/in/fayyoz-usmon-a1410b267/recent-activity/","text":{"textDirection":"USER_LOCALE","text":"Bijdragen en activiteit","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_manage_job_posting_account","actionTarget":"https://www.linkedin.com/talent/job-management-redirect?trk=nav_app_launcher_manage_job_post_nept","text":{"textDirection":"USER_LOCALE","text":"Account voor plaatsen vacatures","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Beheren","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"}],"controlName":"nav_settings","text":"Ik","$recipeTypes":["com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"],"$type":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"navItem":null},{"appLauncher":{"showDivider":true,"productItems":[{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_LEARNING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_learning","showIconBorder":true,"actionTarget":"https://www.linkedin.com/learning/?trk=nav_neptune_learning","text":"Learning","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"LEARNING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_TALENT_INSIGHTS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_insights","showIconBorder":true,"actionTarget":"https://www.linkedin.com/insights?trk=nav_app_launcher_insights_nept&amp;src=li-nav","text":"Talent Insights","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"INSIGHTS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_JOBS_POSTING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_job_postings","showIconBorder":true,"actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_app_launcher_job_post_nept","text":"Vacature plaatsen","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOB_POSTINGS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_ADS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_ads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/campaignmanager/accounts","text":"Adverteren","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"ADVERTISE","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_SALES_NAVIGATOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_find_leads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/premium/products/?intentType=FIND_LEADS&amp;upsellOrderOrigin=premium_nav_more_products_panel&amp;utype=sales","text":"Leads zoeken","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"FIND_LEADS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_GROUPS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_groups","showIconBorder":true,"actionTarget":"https://www.linkedin.com/groups","text":"Groepen","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"GROUPS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_PROFINDER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_services_marketplace","showIconBorder":true,"actionTarget":"https://www.linkedin.com/services?trk=d_flagship3_nav","text":"Service\nhandels\nplaats","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"PROFINDER","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"}],"businessServices":[{"businessServiceAction":{"controlName":"nav_business_talent_solutions","actionTarget":"https://business.linkedin.com/talent-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lts-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Werven op LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Vind, interesseer en werf talent","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_sales_solutions","actionTarget":"https://business.linkedin.com/sales-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lss-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Verkopen met LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Bouw relaties op met kopers","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_post_a_job","actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_biz_serv_job_post_nept","title":{"textDirection":"USER_LOCALE","text":"Plaats gratis een vacature","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Zoek hoogwaardige kandidaten","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_advertise","actionTarget":"https://business.linkedin.com/nl-nl/marketing-solutions/ads?trk=n_nav_ads_rr_b&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Adverteer op LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Vind nieuwe klanten en stimuleer groei","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_learning_solutions","actionTarget":"https://learning.linkedin.com/?trk=d_flagship3_nav&amp;veh=learning_solutions&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Leren met LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Cursussen om uw medewerkers te ontwikkelen","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_admin_center","actionTarget":"https://business.linkedin.com/talent-solutions/customer/admin-center","title":{"textDirection":"USER_LOCALE","text":"Beheerderscentrum","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Facturerings- en accountgegevens beheren","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"}],"productItemsTitle":{"textDirection":"USER_LOCALE","text":"Meer LinkedIn-producten bezoeken","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"businessServicesTitle":"Ontdek meer voor bedrijven","businessServiceActions":[{"controlName":"nav_business_create_company","actionTarget":"https://www.linkedin.com/company/setup/new/","title":{"textDirection":"USER_LOCALE","text":"Bedrijfspagina maken  ","attributesV2":[{"start":21,"length":1,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":"IC_PLUS_16DP","epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.9d1c9f8db73cc268b525043481708082"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"}],"controlName":"nav_launcher","icon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_APP_SWITCHER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":"Voor bedrijven","$recipeTypes":["com.linkedin.f94aca2770a997f8a58dfb7d6154954a"],"$type":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"meMenu":null,"navItem":null}],"$type":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"$recipeTypes":["com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"],"$type":"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.6c29a299f640b6e802d15141c355bd92"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.1e797f68b521546886989869c5b44a2d":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.6cf7727c816c79567a0564982ee35ef4":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PremiumUpsellSlotUrn"},"upsellCard":{"type":"com.linkedin.8b2657e6d402c000382bd65eb94ba628"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.fb4511fb6c2df4a805ef662367e15026":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"quickHelp":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeItem"},"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.61dcef34ee68ce7a34919062297c038b":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.6c29a299f640b6e802d15141c355bd92":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.566376561ef86249fc89f5596eaf6fe1":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d":{"fields":{"feedDashGlobalNavs":{"type":"com.linkedin.eae2ed6e463973c8e0d55baecd814363"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c":{"fields":{"title":{"type":"string"},"meItemsUnions":{"type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"string"}}}},"meItems":{"resolvedFrom":"meItemsUnions","type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeGroup"},"com.linkedin.1112232e110fbaf9f6866335ae420a21":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.bda725ec0f4ddc7380fb44de84e29723":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.2b5530b88c12ba36048beee3aca27737":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.b2c03744140a6cd6a2e693c32790988c":{"fields":{"staticIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"activeIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hasCoachmark":{"type":"boolean"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"showIconBorder":{"type":"boolean"},"text":{"type":"string"},"actionTarget":{"type":"com.linkedin.common.Url"},"hasPaid":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavElement"},"com.linkedin.feccefa74b57bf711c74d0b09805c9f6":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.188858304ae14097e59518d245c7cc85":{"fields":{"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.1c710707068d7b33bcbee05036904a3b":{"fields":{"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.8b2657e6d402c000382bd65eb94ba628":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"socialProofInsight":{"type":"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273"},"subtext":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"ctaText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"upsellOrderOrigin":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionUrl":{"type":"com.linkedin.common.Url"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"cancelCta":{"type":"com.linkedin.513969f6493543c4ad7f04b46c3b67f6"},"globalLegoTrackingToken":{"type":"string"},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"com.linkedin.voyager.dash.premium.PremiumUpsellLayoutStyle"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.44f2985bd25b663c32c3c4d4ec070248":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.d66f0cb7963e0345d3ff893927e60c88":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.10f415928dc4647be1fe1eebc72fe277":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64":{"fields":{"type":{"type":"com.linkedin.voyager.dash.feed.nav.NavType"},"navContent":{"resolvedFrom":"navContentUnion","type":{"union":{"premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}},"navContentUnion":{"type":{"union":{"premiumUpsellSlotUrn":"string","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavItem"},"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"footerText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"insightText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.96db794fa6a744844b487cfa3e195632":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.1e797f68b521546886989869c5b44a2d"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.eae2ed6e463973c8e0d55baecd814363":{"fields":{"logo":{"type":"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"},"primaryItemsUnions":{"type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}},"spotlight":{"type":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"},"primaryItems":{"derivedFrom":"primaryItemsUnions","type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"com.linkedin.9d1c9f8db73cc268b525043481708082":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a","profileFamiliarName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.6c29a299f640b6e802d15141c355bd92","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","learningCourseName":"com.linkedin.44f2985bd25b663c32c3c4d4ec070248","profileMention":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.1e797f68b521546886989869c5b44a2d","hashtag":"com.linkedin.10f415928dc4647be1fe1eebc72fe277"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.513969f6493543c4ad7f04b46c3b67f6":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"appearance":{"type":"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf"},"openExternally":{"type":"boolean"},"accessibilityText":{"type":"string"},"disable":{"type":"boolean"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.f94aca2770a997f8a58dfb7d6154954a":{"fields":{"showDivider":{"type":"boolean"},"productItems":{"type":{"array":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"}},"productItemsTitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"businessServices":{"type":{"array":"com.linkedin.a94ff0919cddaa7056025e9491dc3794"}},"businessServicesTitle":{"type":"string"},"icon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"businessServiceActions":{"type":{"array":"com.linkedin.188858304ae14097e59518d245c7cc85"}},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"com.linkedin.a94ff0919cddaa7056025e9491dc3794":{"fields":{"businessServiceAction":{"type":"com.linkedin.188858304ae14097e59518d245c7cc85"},"description":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessService"},"com.linkedin.637f09655a21b1df5a03c68636fd2d18":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d":{"fields":{"viewProfileText":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"string"},"meGroups":{"type":{"array":"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"}},"profileUrn":{"type":"string"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"com.linkedin.f223abd9a48757f8395ac497a91593f3":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.c8b3cc209fc06180b17a4ffb72120344":{"fields":{"profilePicture":{"type":"com.linkedin.bda725ec0f4ddc7380fb44de84e29723"},"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b","isInjection":true},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","profilePictureWithRingStatus":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","companyLogo":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"com.linkedin.1e797f68b521546886989869c5b44a2d","groupLogo":"com.linkedin.6c29a299f640b6e802d15141c355bd92","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"string","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tapTargets":{"type":{"array":"com.linkedin.d66f0cb7963e0345d3ff893927e60c88"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.ca116024396bd29dd1345ad0998a57d5":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.72c1c1091a60c5a77aaadac057a84b54":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.2b5530b88c12ba36048beee3aca27737"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"logo":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hoverText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"}}}},"included":[{"firstName":"Fayyoz","lastName":"Usmon","profilePicture":{"displayImageReferenceResolutionResult":null,"displayImageWithFrameReference":null,"$recipeTypes":["com.linkedin.bda725ec0f4ddc7380fb44de84e29723"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"entityUrn":"urn:li:fsd_profile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","headline":"--","publicIdentifier":"fayyoz-usmon-a1410b267","$recipeTypes":["com.linkedin.c8b3cc209fc06180b17a4ffb72120344"],"$type":"com.linkedin.voyager.dash.identity.profile.Profile"},{"entityUrn":"urn:li:fsd_premiumUpsellSlot:NAV_SPOTLIGHT","$recipeTypes":["com.linkedin.6cf7727c816c79567a0564982ee35ef4"],"upsellCard":{"image":null,"socialProofInsight":null,"subtext":null,"upsellOrderOrigin":"premium_nav_upsell_text","ctaText":{"textDirection":"USER_LOCALE","text":"Probeer Premium gratis","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"actionUrl":"https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_nav_upsell_text&amp;referenceId=sH42Ay32QVy8LZYHybv6Dg%3D%3D","controlName":"premium_nav_upsell_text_click","legoTrackingToken":null,"title":null,"$recipeTypes":["com.linkedin.8b2657e6d402c000382bd65eb94ba628"],"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellCard","cancelCta":null,"globalLegoTrackingToken":null,"promotionLegoTrackingToken":null,"funnelCommonHeader":{"utype":null,"$recipeTypes":["com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"],"referenceId":"°~6\u0003-öA\\¼-\u0007É»ú\u000E","$type":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":false,"subtitle":null,"dismissible":false,"layoutStyle":"CUSTOM","ctaButtonSecondaryEmphasizedTheme":false},"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},{"entityUrn":"urn:li:fsd_premiumUpsellSlot:NAV_ME","$recipeTypes":["com.linkedin.6cf7727c816c79567a0564982ee35ef4"],"upsellCard":{"image":null,"socialProofInsight":null,"subtext":null,"upsellOrderOrigin":"premium_nav_me_upsell","ctaText":{"textDirection":"USER_LOCALE","text":"Probeer Premium gratis","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"actionUrl":"https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_nav_me_upsell&amp;referenceId=d5iXGYVQRkaImO94FoJ8Gw%3D%3D","controlName":"premium_nav_me_upsell_click","legoTrackingToken":null,"title":null,"$recipeTypes":["com.linkedin.8b2657e6d402c000382bd65eb94ba628"],"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellCard","cancelCta":null,"globalLegoTrackingToken":null,"promotionLegoTrackingToken":null,"funnelCommonHeader":{"utype":null,"$recipeTypes":["com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"],"referenceId":"w\u0019PFFïx\u0016|\u001B","$type":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":false,"subtitle":null,"dismissible":false,"layoutStyle":"TEXT_LINK","ctaButtonSecondaryEmphasizedTheme":false},"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231324">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D()\u0026\u0026queryId\u003DvoyagerFeedDashGlobalNavs.c70182517bc900aa0452b092c2e2ff39","status":200,"body":"bpr-guid-6231324","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231324"><code style="display: none" id="bpr-guid-6231331">
  {"data":{"data":{"premiumDashFeatureAccessByFeatureAccessTypes":{"*elements":["urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE"],"$recipeTypes":["com.linkedin.3fad2247165582aa95a2690797d7e0ab"],"$type":"com.linkedin.restli.common.CollectionResponse"},"$recipeTypes":["com.linkedin.6488012243094b4ef82d764d10e79d45"],"$type":"com.linkedin.6488012243094b4ef82d764d10e79d45"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.3fad2247165582aa95a2690797d7e0ab":{"fields":{"elements":{"type":{"array":"com.linkedin.9e452672002943dc9a1521bcc7ad23cb"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.9e452672002943dc9a1521bcc7ad23cb":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.FeatureAccessUrn"},"hasAccess":{"type":"boolean"},"featureAccessType":{"type":"com.linkedin.voyager.dash.premium.FeatureAccessType"}},"baseType":"com.linkedin.voyager.dash.premium.FeatureAccess"},"com.linkedin.6488012243094b4ef82d764d10e79d45":{"fields":{"premiumDashFeatureAccessByFeatureAccessTypes":{"type":"com.linkedin.3fad2247165582aa95a2690797d7e0ab"}},"baseType":"com.linkedin.graphql.Query"}}}},"included":[{"featureAccessType":"CAN_ACCESS_SALES_NAV_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"},{"featureAccessType":"CAN_ACCESS_ADVERTISE_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231331">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(featureAccessTypes:List(CAN_ACCESS_SALES_NAV_BADGE,CAN_ACCESS_ADVERTISE_BADGE))\u0026\u0026queryId\u003DvoyagerPremiumDashFeatureAccess.c87b20dac35795f9920f2a8072fd7af5","status":200,"body":"bpr-guid-6231331","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231331"><code style="display: none" id="bpr-guid-6231338">
  {"data":{"entityUrn":"urn:li:collectionResponse:7ZvAwvPwYldk7Tvl3jXgC4ymrtxthgc+3EdE7ncMd98=","elements":[{"count":11,"tab":"NOTIFICATIONS","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":0,"tab":"MESSAGING","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":2,"tab":"MY_NETWORK","$type":"com.linkedin.voyager.common.communications.TabBadge"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231338">
  {"request":"/voyager/api/voyagerCommunicationsTabBadges?q\u003DtabBadges\u0026countFrom\u003D0","status":200,"body":"bpr-guid-6231338","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231338"><code style="display: none" id="bpr-guid-6231354">
  {"data":{"data":{"$recipeTypes":["com.linkedin.6760ca9c00d2cb00ed53ee695927b17a"],"identityDashProfilesByMemberIdentity":{"*elements":["urn:li:fsd_profile:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ"],"$recipeTypes":["com.linkedin.c16481ff510ae7f0273d85f18aa15219"],"$type":"com.linkedin.restli.common.CollectionResponse"},"$type":"com.linkedin.6760ca9c00d2cb00ed53ee695927b17a"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.6ee28b2b65af6b1ea34ebc4526078982":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"accessibilityText":{"type":"string"},"appearance":{"type":"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeneratedSuggestionButton"},"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.8ee23bb31c0e8d4cb7e6a3e86e720799":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.5fba45f960b9273c3b1bc933082ed44c":{"fields":{"x":{"type":"double"},"y":{"type":"double"}},"baseType":"com.linkedin.common.Coordinate2D"},"com.linkedin.87507521b880a59f51dfe7916463212a":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.b724930290217f55fa1b0d6ec7cdf954"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.b724930290217f55fa1b0d6ec7cdf954"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.fe63f2527824313d8a7a69a542ee70c2":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.e96795656e0f04f0745c758d2fd67d79"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.a4c3cd660ad2e3b0173f3c1b7356f2c6":{"fields":{"t":{"type":"string"},"x":{"type":"string"},"u":{"type":"string"}},"baseType":"com.linkedin.common.LocaleExtensions"},"com.linkedin.4c7a6a878ae117485d617b8a09075c76":{"fields":{"bottomLeft":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"},"saturation":{"type":"double"},"vignette":{"type":"double"},"brightness":{"type":"double"},"photoFilterType":{"type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterType"},"bottomRight":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"},"topLeft":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"},"contrast":{"type":"double"},"topRight":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterEditInfo"},"com.linkedin.42bbd7c505b9890dbd35cd379a5ffd49":{"fields":{"name":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.aa58984e06b9abd3aa0ba432da210113":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.848cd7bd87249ea0e34a0778a20bef83":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"originalImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"originalImageUrn":{"type":"com.linkedin.common.Urn"},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"originalImageReferenceResolutionResult":{"derivedFrom":"originalImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"frameType":{"type":"com.linkedin.voyager.dash.identity.profile.PhotoFrameType"},"photoFilterEditInfo":{"type":"com.linkedin.4c7a6a878ae117485d617b8a09075c76"},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.8b38bd16089709d88ed7104608137baf":{"fields":{"memberDistance":{"type":"com.linkedin.voyager.dash.relationships.NoConnectionMemberDistance"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.a4c44f1ec152e8b1415e8ed859230f0c":{"fields":{"buttonPlacement":{"type":"com.linkedin.voyager.dash.identity.profile.tetris.ButtonPlacement"},"buttonAppearance":{"type":"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35"},"legoTrackingId":{"type":"string"},"icon":{"type":"com.linkedin.ff3f998e6eedb8e317a96fb99dd62775"},"actionControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.tetris.NavigationAction"},"com.linkedin.7959475b6283d014d8d529b6f2ef5b88":{"fields":{"badgeIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"badgeText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.TopVoiceBadge"},"com.linkedin.fb515f37046fc34f7a17064266659842":{"fields":{"elements":{"type":{"array":"com.linkedin.cff5540190d0fbce1a4cab42ae0afdd3"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.1d7f409d378b3c2595366d0625fbf68e":{"fields":{"ringStatus":{"type":"com.linkedin.848cd7bd87249ea0e34a0778a20bef83"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.2cdd21b0b3dad6ccee057d5d8ba4203a":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.d16b47c42001019f9bb43f9ce18412ef":{"fields":{"geo":{"resolvedFrom":"geoUrn","type":"com.linkedin.20bf1398649ed8da5a403515fbffb6bf"},"geoUrn":{"type":"string"},"postalCode":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"com.linkedin.f2c61005acb2fbe69cd38580d01cb3d5":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.8ee23bb31c0e8d4cb7e6a3e86e720799"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.5f3d321f9b6631058c0c944d77e41938"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.be5d51d61e628b01ab0d22e4b670bcd2":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.b724930290217f55fa1b0d6ec7cdf954":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.be5d51d61e628b01ab0d22e4b670bcd2","profileFamiliarName":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.aa58984e06b9abd3aa0ba432da210113","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.055a5fa30bbe60d94f41c14ebe173bbf","listItemStyle":"com.linkedin.2322b567653571139f2d983df4580732","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.d22a136895aa551adfab2b49ab2928a1","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.a3be506e3c2e76c76d0e9c0f867f1429","profileFullName":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","learningCourseName":"com.linkedin.6fdcc7e0d8a5276f82b704ee433b31cc","profileMention":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.e96795656e0f04f0745c758d2fd67d79","hashtag":"com.linkedin.c8ae18b1f64cb4a44b60cb752e911c5b"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.aa58984e06b9abd3aa0ba432da210113","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.055a5fa30bbe60d94f41c14ebe173bbf","listItemStyle":"com.linkedin.2322b567653571139f2d983df4580732","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.a3be506e3c2e76c76d0e9c0f867f1429","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.cff5540190d0fbce1a4cab42ae0afdd3":{"fields":{"dismissCta":{"type":"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35"},"headlineVersion":{"type":"com.linkedin.4a0a959f2280a25e8a09005334df5da9"},"dismissCtaButton":{"type":"com.linkedin.6ee28b2b65af6b1ea34ebc4526078982"},"description":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"legoTrackingToken":{"type":"string"},"primaryCtaButtonV2":{"type":"com.linkedin.a4c44f1ec152e8b1415e8ed859230f0c"},"headline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"dismissCtaAccessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeneratedSuggestionPromoCard"},"com.linkedin.055a5fa30bbe60d94f41c14ebe173bbf":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.6a0e2ca2b255b817da99cc43e27e5b44"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172":{"fields":{"birthDateOn":{"type":"com.linkedin.c6c63f18a3ab2fb1a340292406594762"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"objectUrn":{"type":"com.linkedin.common.Urn"},"multiLocaleLastName":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"showPremiumSubscriberBadge":{"type":"boolean"},"tempStatusEmoji":{"type":"string"},"pronounUnion":{"type":{"union":{"customPronoun":"string","standardizedPronoun":"com.linkedin.voyager.dash.identity.profile.StandardizedPronoun"}}},"tempStatusExpiredAtUnion":{"type":{"union":{"customizedExpiredAt":"com.linkedin.common.Time","standardizedExpiration":"com.linkedin.voyager.dash.identity.profile.StandardizedExpiration"}}},"multiLocaleFirstName":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"memberPostAnalytics":{"type":"com.linkedin.6f81a0561c6d67d56507fcaec5cf100b"},"premium":{"type":"boolean"},"influencer":{"type":"boolean"},"multiLocaleTempStatus":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"pronoun":{"derivedFrom":"pronounUnion","type":{"union":{"customPronoun":"string","standardizedPronoun":"com.linkedin.voyager.dash.identity.profile.StandardizedPronoun"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"ringStatus":{"type":"com.linkedin.848cd7bd87249ea0e34a0778a20bef83","isInjection":true},"headlineGeneratedSuggestionDelegateUrn":{"type":"com.linkedin.voyager.dash.common.ProfileActionDelegateUrn"},"headline":{"type":"string"},"publicIdentifier":{"type":"string"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"},"topVoiceBadge":{"type":"com.linkedin.7959475b6283d014d8d529b6f2ef5b88"},"creator":{"type":"boolean"},"supportedLocales":{"type":{"array":"com.linkedin.a4044e55c3fcc69e2de7519872763501"}},"creatorInfo":{"type":"com.linkedin.80e29e7323b2b17315871707a1231af9"},"versionTag":{"type":"string"},"privacySettings":{"type":"com.linkedin.eafc20924d49659eb7e0335979bc8d29","isInjection":true},"profilePicture":{"type":"com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"multiLocaleMaidenName":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"geoLocation":{"type":"com.linkedin.d16b47c42001019f9bb43f9ce18412ef"},"showVerificationBadge":{"type":"boolean"},"memberRelationship":{"type":"com.linkedin.de96d1e9b7a124ebb464644d217619fc","isInjection":true},"profileGeneratedSuggestionPromo":{"type":"com.linkedin.fb515f37046fc34f7a17064266659842","isInjection":true},"primaryLocale":{"type":"com.linkedin.a4044e55c3fcc69e2de7519872763501"},"multiLocaleHeadline":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"backgroundPicture":{"type":"com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"},"tempStatusExpiredAt":{"derivedFrom":"tempStatusExpiredAtUnion","type":{"union":{"customizedExpiredAt":"com.linkedin.common.Time","standardizedExpiration":"com.linkedin.voyager.dash.identity.profile.StandardizedExpiration"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.ff3f998e6eedb8e317a96fb99dd62775":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.c7ef039b7fe1b9f39c5d2dab33f7e026"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.b724930290217f55fa1b0d6ec7cdf954"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.cc70ada274dbeca435a87c70e5f65742":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConnectionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.e96795656e0f04f0745c758d2fd67d79":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.c7ef039b7fe1b9f39c5d2dab33f7e026":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","profilePictureWithRingStatus":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","companyLogo":"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.28872f0409786487d12514b9187690b3","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164","nonEntityProfessionalEventLogo":"com.linkedin.f2c61005acb2fbe69cd38580d01cb3d5","profilePicture":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","imageUrl":"com.linkedin.2cdd21b0b3dad6ccee057d5d8ba4203a","professionalEventLogo":"com.linkedin.8ee23bb31c0e8d4cb7e6a3e86e720799","nonEntitySchoolLogo":"com.linkedin.fe63f2527824313d8a7a69a542ee70c2","nonEntityCompanyLogo":"com.linkedin.4f05fb063efef8a581811a71ec2a3559","schoolLogo":"com.linkedin.e96795656e0f04f0745c758d2fd67d79","groupLogo":"com.linkedin.d22a136895aa551adfab2b49ab2928a1","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1d7f409d378b3c2595366d0625fbf68e"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.28872f0409786487d12514b9187690b3","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164","nonEntityProfessionalEventLogo":"com.linkedin.f2c61005acb2fbe69cd38580d01cb3d5","profilePicture":"string","imageUrl":"com.linkedin.2cdd21b0b3dad6ccee057d5d8ba4203a","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.fe63f2527824313d8a7a69a542ee70c2","nonEntityCompanyLogo":"com.linkedin.4f05fb063efef8a581811a71ec2a3559","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1d7f409d378b3c2595366d0625fbf68e"}}},"tapTargets":{"type":{"array":"com.linkedin.c046626a3323577b67c41e8693ea4802"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.80e29e7323b2b17315871707a1231af9":{"fields":{"creatorWebsite":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"associatedHashtag":{"resolvedFrom":"associatedHashtagUrns","type":{"array":"com.linkedin.c8ae18b1f64cb4a44b60cb752e911c5b"}},"associatedHashtagUrns":{"type":{"array":"string"}},"creatorPostAnalytics":{"type":"com.linkedin.6f81a0561c6d67d56507fcaec5cf100b"}},"baseType":"com.linkedin.voyager.dash.identity.profile.CreatorInfo"},"com.linkedin.6760ca9c00d2cb00ed53ee695927b17a":{"fields":{"identityDashProfilesByMemberIdentity":{"type":"com.linkedin.c16481ff510ae7f0273d85f18aa15219"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.a3be506e3c2e76c76d0e9c0f867f1429":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.63a835f468299bb310d28f506cabd1a6":{"fields":{"value":{"type":"string"},"key":{"type":"string"}},"baseType":"com.linkedin.data.StringMapEntry"},"com.linkedin.20bf1398649ed8da5a403515fbffb6bf":{"fields":{"countryUrn":{"type":"string"},"country":{"resolvedFrom":"countryUrn","type":"com.linkedin.20bf1398649ed8da5a403515fbffb6bf"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GeoUrn"},"defaultLocalizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.4a0a959f2280a25e8a09005334df5da9":{"fields":{"colorStyle":{"type":"com.linkedin.voyager.dash.common.ux.label.LabelColorStyle"},"text":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"}},"baseType":"com.linkedin.voyager.dash.common.ux.label.LabelViewModel"},"com.linkedin.4f05fb063efef8a581811a71ec2a3559":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.6f81a0561c6d67d56507fcaec5cf100b":{"fields":{"description":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"totalCount":{"type":"long"},"title":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"percentageChange":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.identity.profile.CreatorPostAnalytics"},"com.linkedin.2322b567653571139f2d983df4580732":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.6a0e2ca2b255b817da99cc43e27e5b44":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.b7f5bab2ab7602febe3216fc910827f5":{"fields":{},"baseType":"com.linkedin.restli.common.EmptyRecord"},"com.linkedin.cc4781d9200ac2c549e7bb9fe6f70e98":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FollowingStateUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"preDashFollowingInfoUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"following":{"type":"boolean"},"followingType":{"type":"com.linkedin.voyager.dash.feed.FollowingType"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.28872f0409786487d12514b9187690b3":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.d22a136895aa551adfab2b49ab2928a1"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.c8ae18b1f64cb4a44b60cb752e911c5b":{"fields":{"actionTarget":{"type":"com.linkedin.common.Url"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"followingState":{"type":"com.linkedin.cc4781d9200ac2c549e7bb9fe6f70e98","isInjection":true},"displayName":{"type":"string"},"trackingUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.d22a136895aa551adfab2b49ab2928a1":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.5f3d321f9b6631058c0c944d77e41938":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.c6c63f18a3ab2fb1a340292406594762":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.b5218fbabd3538967ac9576bb65b170d":{"fields":{"name":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"nameSupplementaryInfo":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"headline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"subHeadline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.a4044e55c3fcc69e2de7519872763501":{"fields":{"variant":{"type":"string"},"country":{"type":"string"},"language":{"type":"string"},"extensions":{"type":"com.linkedin.a4c3cd660ad2e3b0173f3c1b7356f2c6"},"script":{"type":"string"}},"baseType":"com.linkedin.common.Locale"},"com.linkedin.c16481ff510ae7f0273d85f18aa15219":{"fields":{"elements":{"type":{"array":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.c046626a3323577b67c41e8693ea4802":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.b5218fbabd3538967ac9576bb65b170d","largeTemplate":"com.linkedin.c1233c6614d75709cf8a4843c636852c","smallTemplate":"com.linkedin.42bbd7c505b9890dbd35cd379a5ffd49"}}},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.b5218fbabd3538967ac9576bb65b170d","largeTemplate":"com.linkedin.c1233c6614d75709cf8a4843c636852c","smallTemplate":"com.linkedin.42bbd7c505b9890dbd35cd379a5ffd49"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.de96d1e9b7a124ebb464644d217619fc":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.MemberRelationshipUrn"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.b7f5bab2ab7602febe3216fc910827f5","connection":"com.linkedin.cc70ada274dbeca435a87c70e5f65742","noConnection":"com.linkedin.8b38bd16089709d88ed7104608137baf"}}},"memberRelationship":{"derivedFrom":"memberRelationshipUnion","type":{"union":{"self":"com.linkedin.b7f5bab2ab7602febe3216fc910827f5","connection":"com.linkedin.cc70ada274dbeca435a87c70e5f65742","noConnection":"com.linkedin.8b38bd16089709d88ed7104608137baf"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.eafc20924d49659eb7e0335979bc8d29":{"fields":{"fullLastNameShown":{"type":"boolean"},"requireReferral":{"type":"boolean"},"showPublicProfile":{"type":"boolean"},"formerNameVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"messagingSeenReceipts":{"type":"com.linkedin.voyager.dash.identity.profile.InstantMessagingVisibilitySetting"},"discloseAsProfileViewer":{"type":"com.linkedin.voyager.dash.identity.profile.DiscloseAsProfileViewerInfo"},"allowProfileEditBroadcasts":{"type":"boolean"},"messagingTypingIndicators":{"type":"com.linkedin.voyager.dash.identity.profile.InstantMessagingVisibilitySetting"},"allowOpenProfile":{"type":"boolean"},"profilePictureVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.PrivacySettingsUrn"},"publicProfilePictureVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"namePronunciationVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"pronounVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PrivacySettings"},"com.linkedin.6fdcc7e0d8a5276f82b704ee433b31cc":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.c1233c6614d75709cf8a4843c636852c":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"nameSupplementaryInfo":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"footerText":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"subHeadline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"name":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"insightText":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"headline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"}}}},"included":[{"entityUrn":"urn:li:fsd_geo:102890719","$recipeTypes":["com.linkedin.20bf1398649ed8da5a403515fbffb6bf"],"defaultLocalizedName":"Nederland","$type":"com.linkedin.voyager.dash.common.Geo"},{"defaultLocalizedNameWithoutCountryName":"Eindhoven, Noord-Brabant","*country":"urn:li:fsd_geo:102890719","entityUrn":"urn:li:fsd_geo:102922143","$recipeTypes":["com.linkedin.20bf1398649ed8da5a403515fbffb6bf"],"defaultLocalizedName":"Eindhoven, Noord-Brabant, Nederland","$type":"com.linkedin.voyager.dash.common.Geo"},{"fullLastNameShown":true,"requireReferral":false,"showPublicProfile":true,"formerNameVisibilitySetting":"PUBLIC","discloseAsProfileViewer":"DISCLOSE_FULL","messagingSeenReceipts":"DISABLED","$recipeTypes":["com.linkedin.eafc20924d49659eb7e0335979bc8d29"],"allowProfileEditBroadcasts":false,"$type":"com.linkedin.voyager.dash.identity.profile.PrivacySettings","messagingTypingIndicators":"DISABLED","allowOpenProfile":false,"profilePictureVisibilitySetting":"PUBLIC","entityUrn":"urn:li:fsd_privacySettings:singleton","publicProfilePictureVisibilitySetting":"PUBLIC","namePronunciationVisibilitySetting":"MEMBERS","pronounVisibilitySetting":"MEMBERS"},{"birthDateOn":null,"lastName":"Madej","memorialized":false,"objectUrn":"urn:li:member:305921936","multiLocaleLastName":[{"value":"Madej","key":"en_US"},{"value":"Madej","key":"pl_PL"}],"showPremiumSubscriberBadge":false,"tempStatusEmoji":null,"multiLocaleFirstName":[{"value":"Krystian","key":"en_US"},{"value":"Krystian","key":"pl_PL"}],"memberPostAnalytics":null,"premium":false,"influencer":false,"entityUrn":"urn:li:fsd_profile:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ","multiLocaleTempStatus":[],"pronoun":null,"headlineGeneratedSuggestionDelegateUrn":"urn:li:fsu_profileActionDelegate:-1839964074","headline":"Senior Software Designer","publicIdentifier":"krystian-madej","trackingId":"J+wt7gHBTdStvrm9wx4FWw==","topVoiceBadge":null,"creator":false,"supportedLocales":[{"variant":null,"country":"US","language":"en","$recipeTypes":["com.linkedin.a4044e55c3fcc69e2de7519872763501"],"$type":"com.linkedin.common.Locale"},{"variant":null,"country":"PL","language":"pl","$recipeTypes":["com.linkedin.a4044e55c3fcc69e2de7519872763501"],"$type":"com.linkedin.common.Locale"}],"*memberRelationship":"urn:li:fsd_memberRelationship:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ","creatorInfo":null,"versionTag":"3131526026","$recipeTypes":["com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172"],"$type":"com.linkedin.voyager.dash.identity.profile.Profile","firstName":"Krystian","profilePicture":{"displayImageReferenceResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"],"artifacts":[{"width":100,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"100_100/0/1523376389588?e=1701302400&amp;v=beta&amp;t=Jfwd7yy-uVy1nU9eKVQ3rxspV8RLh8GoG1EagGIlMdk","expiresAt":1701302400000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":200,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"200_200/0/1523376389588?e=1701302400&amp;v=beta&amp;t=gqc3A89TEU-GPyV7bIC2Jv1IgejxVLDqjZvfBjjOSMg","expiresAt":1701302400000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":400,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"400_400/0/1523376389588?e=1701302400&amp;v=beta&amp;t=rYLx2_utszX-G0cbseRT2rRQqfHtne1y9nsFH3Nr6fM","expiresAt":1701302400000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":800,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"800_800/0/1523376389588?e=1701302400&amp;v=beta&amp;t=sVxr-zUQl9Tm14X8JCUdunZSI7qu5N6nBkxuz-ovc0M","expiresAt":1701302400000,"height":800,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C4D03AQErIf349st-lg/profile-displayphoto-shrink_","$type":"com.linkedin.common.VectorImage"}},"originalImageUrn":null,"displayImageWithFrameReference":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"],"artifacts":[{"width":200,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"200_200/0/1693843853826?e=1696248000&amp;v=beta&amp;t=Lc9kt7BO17I1NE492Y31keVc0v6k3jQ6du5XxbthGw0","expiresAt":1696248000000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":400,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"400_400/0/1693843853826?e=1696248000&amp;v=beta&amp;t=wyX6G_n4KLcsPsLe21DcUkzmkOqbdAYYbQFieWBI9wA","expiresAt":1696248000000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":800,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"800_800/0/1693843853826?e=1696248000&amp;v=beta&amp;t=k6o1iIRDQh-h_dHIVTvKern1hjS7cByiEuZrK_KbZy8","expiresAt":1696248000000,"height":800,"$type":"com.linkedin.common.VectorArtifact"},{"width":100,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"100_100/0/1693843853826?e=1696248000&amp;v=beta&amp;t=gDIVsZmhUGvNCoBmpOtjnFWLOLZtdz7qL2dGfj-XnJc","expiresAt":1696248000000,"height":100,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/D4E35AQEA7ib1HDsfCA/profile-framedphoto-shrink_","$type":"com.linkedin.common.VectorImage"}},"originalImageReferenceResolutionResult":null,"frameType":"OPEN_TO_WORK","$recipeTypes":["com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"],"photoFilterEditInfo":{"bottomLeft":{"x":-6.178632484870436E-17,"y":0.9989583333333333,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"saturation":0.0,"brightness":0.0,"vignette":0.0,"photoFilterType":"ORIGINAL","bottomRight":{"x":1.0,"y":0.9989583333333333,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"topLeft":{"x":-6.178632484870436E-17,"y":0.0010416666666664927,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"contrast":0.0,"topRight":{"x":1.0,"y":0.0010416666666664927,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"$recipeTypes":["com.linkedin.4c7a6a878ae117485d617b8a09075c76"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterEditInfo"},"displayImageUrn":"urn:li:digitalmediaAsset:C4D03AQErIf349st-lg","$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"geoLocation":{"*geo":"urn:li:fsd_geo:102922143","$recipeTypes":["com.linkedin.d16b47c42001019f9bb43f9ce18412ef"],"postalCode":null,"$type":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"multiLocaleMaidenName":[],"tempStatus":null,"*privacySettings":"urn:li:fsd_privacySettings:singleton","showVerificationBadge":true,"profileGeneratedSuggestionPromo":{"$recipeTypes":["com.linkedin.fb515f37046fc34f7a17064266659842"],"elements":[],"$type":"com.linkedin.restli.common.CollectionResponse"},"backgroundPicture":{"displayImageReferenceResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"],"artifacts":[{"width":800,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"200_800/0/1615934876702?e=1701302400&amp;v=beta&amp;t=MRGX9D_DIQukFYJ68i5evDERgtJBGyNIl5bnLwkbixQ","expiresAt":1701302400000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":1400,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"350_1400/0/1615934876702?e=1701302400&amp;v=beta&amp;t=7Rglcy3pMKVdMfQgTZf7vqjA2-jTHocI8pZOLqIZTyU","expiresAt":1701302400000,"height":350,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C4E16AQHG32g-LtiiGQ/profile-displaybackgroundimage-shrink_","$type":"com.linkedin.common.VectorImage"}},"originalImageUrn":null,"originalImageReferenceResolutionResult":null,"$recipeTypes":["com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"],"photoFilterEditInfo":{"bottomLeft":{"x":8.167157882300002E-17,"y":0.8773584905660375,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"saturation":0.0,"brightness":0.0,"vignette":0.0,"photoFilterType":"ORIGINAL","bottomRight":{"x":1.0,"y":0.8773584905660375,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"topLeft":{"x":8.167157882300002E-17,"y":0.22075471698113192,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"contrast":0.0,"topRight":{"x":1.0,"y":0.22075471698113192,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"$recipeTypes":["com.linkedin.4c7a6a878ae117485d617b8a09075c76"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterEditInfo"},"displayImageUrn":"urn:li:digitalmediaAsset:C4E16AQHG32g-LtiiGQ","$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"multiLocaleHeadline":[{"value":"Senior Software Designer","key":"en_US"},{"value":"Senior Software Designer","key":"pl_PL"}],"primaryLocale":{"country":"US","extensions":null,"variant":null,"language":"en","$recipeTypes":["com.linkedin.a4044e55c3fcc69e2de7519872763501"],"script":null,"$type":"com.linkedin.common.Locale"},"tempStatusExpiredAt":null},{"entityUrn":"urn:li:fsd_memberRelationship:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ","memberRelationship":{"self":null,"connection":null,"noConnection":{"memberDistance":"OUT_OF_NETWORK","$recipeTypes":["com.linkedin.8b38bd16089709d88ed7104608137baf"],"$type":"com.linkedin.voyager.dash.relationships.NoConnection"}},"$recipeTypes":["com.linkedin.de96d1e9b7a124ebb464644d217619fc"],"$type":"com.linkedin.voyager.dash.relationships.MemberRelationship"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231354">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(vanityName:krystian-madej)\u0026\u0026queryId\u003DvoyagerIdentityDashProfiles.9faa7ab476db884bf471a9361f87aaf2","status":200,"body":"bpr-guid-6231354","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231354"><code style="display: none" id="clientPageInstance">
  urn:li:page:d_flagship3_profile_view_base;3dfedaeb-ee6c-4ab5-bcfa-845675830e10
</code>

<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="terminatorlet">



    <div id="a11y-notification" class="visually-hidden" role="region" aria-live="polite"></div>
  





  <svg role="none" xmlns="http://www.w3.org/2000/svg" id="hue-web-icons-sprite" style="display: none;">
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="search-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M14.56 12.44L11.3 9.18a5.51 5.51 0 10-2.12 2.12l3.26 3.26a1.5 1.5 0 102.12-2.12zM3 6.5A3.5 3.5 0 116.5 10 3.5 3.5 0 013 6.5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="caret-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M8 11L3 6h10z" fill-rule="evenodd"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="overflow-web-ios-medium" data-supported-dps="24x24" fill="currentColor">
  <path d="M14 12a2 2 0 11-2-2 2 2 0 012 2zM4 10a2 2 0 102 2 2 2 0 00-2-2zm16 0a2 2 0 102 2 2 2 0 00-2-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="verified-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M11.99 22l-1.23-.44C6.11 19.81 2.99 16 2.99 11V5L12 2l9 3v6c0 5-3.11 8.81-7.74 10.56l-1.27.44zM5 6.44V11c0 4.11 2.6 7.35 6.46 8.8l.54.2.58-.2C16.41 18.35 19 15.1 19 11V6.44l-7-2.32-7 2.32zM17 8h-2.57l-4.02 5.01-2.18-2.18-1.41 1.41 3.75 3.75 6.43-8z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="rtl-flip" id="arrow-right-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="overflow-web-ios-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M3 9.5A1.5 1.5 0 114.5 8 1.5 1.5 0 013 9.5zM11.5 8A1.5 1.5 0 1013 6.5 1.5 1.5 0 0011.5 8zm-5 0A1.5 1.5 0 108 6.5 1.5 1.5 0 006.5 8z"></path>
</svg>

    <svg id="filter-small" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor">
  <path d="M15,4H6.72c-.35,.6-.98,1-1.72,1s-1.38-.4-1.72-1H1V2H3.28c.35-.6,.98-1,1.72-1s1.38,.4,1.72,1H15v2Zm0,8H6.72c-.35-.6-.98-1-1.72-1s-1.38,.4-1.72,1H1v2H3.28c.35,.6,.98,1,1.72,1s1.38-.4,1.72-1H15v-2Zm0-5h-2.28c-.35-.6-.98-1-1.72-1s-1.38,.4-1.72,1H1v2H9.28c.35,.6,.98,1,1.72,1s1.38-.4,1.72-1h2.28v-2Z"></path>
</svg>
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" class="rtl-flip" id="question-medium" data-supported-dps="24x24" fill="currentColor">
  <path d="M12 2a10 10 0 1010 10A10 10 0 0012 2zm0 16.25A1.25 1.25 0 1113.25 17 1.25 1.25 0 0112 18.25zm1.41-5.46L13 13v1h-2v-2.21l1.49-.79C13.82 10.34 14 9.77 14 9.3c0-.78-.92-1.3-2.3-1.3A7.12 7.12 0 008 9.24V7a8 8 0 013.7-1c3 0 4.3 1.55 4.3 3.3a3.91 3.91 0 01-2.59 3.49z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="settings-medium" data-supported-dps="24x24" fill="currentColor">
  <path d="M9.18 2l-4.3 2.52L6.22 8l-.52.91-3.7.55v5l3.64.54.54.93-1.34 3.53L9.14 22l2.29-2.9h1.09l2.3 2.9 4.32-2.52L17.79 16l.53-.93 3.68-.53v-5L18.32 9l-.51-.9 1.36-3.51L14.86 2l-2.33 3h-1zM12 9a3 3 0 11-3 3 3 3 0 013-3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="shield-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M12 2L3 5v6c0 5 3.12 8.81 7.77 10.56L12 22l1.27-.44C17.9 19.81 21.01 16 21.01 11V5L12 2zm-.53 17.8C7.6 18.35 5.01 15.1 5.01 11V6.44l7-2.32V20l-.54-.2z"></path>
</svg>

</svg>

  <!---->

  
<!---->  

<!---->
<div class="application-outlet">
<!---->
        
    <div class="global-nav__a11y-menu
        ">
      <div class="global-nav__a11y-menu-container">
          <button id="ember8" class="mr4 artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To search function</font></font>
</span></button>

        <button id="ember9" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Skip to main content</font></font>
</span></button>

<!---->
        <button id="ember10" class="global-nav__a11y-menu-close artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--2 artdeco-button--tertiary ember-view">  <li-icon aria-hidden="true" type="close" class="artdeco-button__icon"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M13.42 12L20 18.58 18.58 20 12 13.42 5.42 20 4 18.58 10.58 12 4 5.42 5.42 4 12 10.58 18.58 4 20 5.42z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Close menu</font></font>
</span></button>
      </div>
    </div>
  
      
    <header id="global-nav" class="global-nav global-alert-offset-top
        
        global-nav--visible">
      <div class="global-nav__content">
          
    <a class="app-aware-link " href="https://www.linkedin.com/feed/?nis=true" data-test-app-aware-link="">
      
    <div class="ivm-image-view-model    global-nav__branding-logo">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon type="app-linkedin-bug-color-icon" class=" " size="large" role="img" aria-label="LinkedIn"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M20.5 2h-17A1.5 1.5 0 002 3.5v17A1.5 1.5 0 003.5 22h17a1.5 1.5 0 001.5-1.5v-17A1.5 1.5 0 0020.5 2zM8 19H5v-9h3zM6.5 8.25A1.75 1.75 0 118.3 6.5a1.78 1.78 0 01-1.8 1.75zM19 19h-3v-4.74c0-1.42-.6-1.93-1.38-1.93A1.74 1.74 0 0013 14.19a.66.66 0 000 .14V19h-3v-9h2.9v1.3a3.11 3.11 0 012.7-1.4c1.55 0 3.36.86 3.36 3.66z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
    </a>
  

        <div id="global-nav-search" class="global-nav__search
            ">
          
          
    <div class="search-global-typeahead
        
         global-nav__search-typeahead">
        
    <div id="global-nav-typeahead" class="search-global-typeahead__typeahead">
<!---->        
          
    <input class="search-global-typeahead__input" placeholder="To search" role="combobox" aria-autocomplete="list" aria-label="To search" aria-activedescendant="" aria-expanded="false" type="text">
  
          <div aria-hidden="true" class="search-global-typeahead__search-icon-container">
            <svg role="none" aria-hidden="true" class="search-global-typeahead__search-icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="search-small">
<!---->    

    <use href="#search-small" width="16" height="16"></use>
</svg>

          </div>
<!---->          <div class="search-global-typeahead__overlay global-alert-offset-top
              "></div>
<!---->        
    </div>
  
      <button class="search-global-typeahead__collapsed-search-button" aria-label="Click to start a search" type="button">
        <li-icon aria-hidden="true" type="search" class="search-global-typeahead__collapsed-search-button-icon t-black--light" size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21.41 18.59l-5.27-5.28A6.83 6.83 0 0017 10a7 7 0 10-7 7 6.83 6.83 0 003.31-.86l5.28 5.27a2 2 0 002.82-2.82zM5 10a5 5 0 115 5 5 5 0 01-5-5z"></path>
    </svg></li-icon>
        <div class="search-global-typeahead__collapsed-search-button-text t-black--light t-12 t-normal">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To search</font></font>
        </div>
      </button>
      
  <div id="ember12" class="ember-view"><!----></div>

    </div>
  
      
        </div>

        <nav class="global-nav__nav" aria-label="Main navigation">
          <ul class="global-nav__primary-items
              ">
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/feed/?nis=true&amp;" data-test-app-aware-link="">
      

      <div id="ember13" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
        <span aria-hidden="true" class="notification-badge__no-count"></span>
        <span class="a11y-text" data-test-notification-a11y=""><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">new Feed updates notifications</font></font></span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="home" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 9v2h-2v7a3 3 0 01-3 3h-4v-6h-4v6H6a3 3 0 01-3-3v-7H1V9l11-7z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Home">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Home</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/mynetwork/?" data-test-app-aware-link="">
      

      <div id="ember14" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
          <span aria-hidden="true" class="notification-badge__count "><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2</font></font></span>
          <span class="a11y-text" data-test-notification-a11y=""><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2 new network updates notifications</font></font></span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="people" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M12 16v6H3v-6a3 3 0 013-3h3a3 3 0 013 3zm5.5-3A3.5 3.5 0 1014 9.5a3.5 3.5 0 003.5 3.5zm1 2h-2a2.5 2.5 0 00-2.5 2.5V22h7v-4.5a2.5 2.5 0 00-2.5-2.5zM7.5 2A4.5 4.5 0 1012 6.5 4.49 4.49 0 007.5 2z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="My network">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">My network</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/jobs/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="job" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17 6V5a3 3 0 00-3-3h-4a3 3 0 00-3 3v1H2v4a3 3 0 003 3h14a3 3 0 003-3V6zM9 5a1 1 0 011-1h4a1 1 0 011 1v1H9zm10 9a4 4 0 003-1.38V17a3 3 0 01-3 3H5a3 3 0 01-3-3v-4.38A4 4 0 005 14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Vacancies">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Vacancies</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/messaging/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="nav-small-messaging-icon" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M16 4H8a7 7 0 000 14h4v4l8.16-5.39A6.78 6.78 0 0023 11a7 7 0 00-7-7zm-8 8.25A1.25 1.25 0 119.25 11 1.25 1.25 0 018 12.25zm4 0A1.25 1.25 0 1113.25 11 1.25 1.25 0 0112 12.25zm4 0A1.25 1.25 0 1117.25 11 1.25 1.25 0 0116 12.25z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="To inform">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To inform</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/notifications/?" data-test-app-aware-link="">
      

      <div id="ember15" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
          <span aria-hidden="true" class="notification-badge__count "><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">11</font></font></span>
          <span class="a11y-text" data-test-notification-a11y=""><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">11 new notifications</font></font></span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="bell-fill" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M22 19h-8.28a2 2 0 11-3.44 0H2v-1a4.52 4.52 0 011.17-2.83l1-1.17h15.7l1 1.17A4.42 4.42 0 0122 18zM18.21 7.44A6.27 6.27 0 0012 2a6.27 6.27 0 00-6.21 5.44L5 13h14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="notifications">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">notifications</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!---->                  
    <div id="ember16" class="global-nav__me artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
      <button aria-expanded="false" id="ember17" class="global-nav__primary-link global-nav__primary-link-me-menu-trigger artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
        <img width="24" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" height="24" alt="Fayyoz Usmon" id="ember18" class="global-nav__me-photo evi-image ghost-person ember-view">

        <span class="global-nav__primary-link-text">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">I</font></font>
          <svg role="none" aria-hidden="true" class="global-nav__icon global-nav__icon--small" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
<!----></button>

      <div tabindex="-1" aria-hidden="true" id="ember19" class="global-nav__me-content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>

<!---->  
<!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!---->                  
    
    
      
  <button aria-expanded="false" class="global-nav__primary-link
          global-nav__primary-item--divider pl3
          global-nav__app-launcher-trigger" type="button">
    
          
    <div class="ivm-image-view-model    global-nav__icon-ivm">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="grid" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M3 3h4v4H3zm7 4h4V3h-4zm7-4v4h4V3zM3 14h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4zM3 21h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  

        <span class="global-nav__primary-link-text" title="For companies">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">For companies</font></font>
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
  </button>


      <!---->
    

    <div id="ember20" class="ember-view"><!----></div>
  
  
<!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!----><!---->                  
<!---->
      
        
          <div class="premium-upsell-link">
            <a href="https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_nav_upsell_text&amp;referenceId=sH42Ay32QVy8LZYHybv6Dg%3D%3D&amp;destRedirectURL=https%3A%2F%2Fwww.linkedin.com%2Fin%2Fkrystian-madej%2F" class="link-without-visited-state global-nav__primary-link global-nav__primary-link--premium global-nav__spotlight-upsell premium-upsell-link--truncate premium-upsell-link--long">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Try Premium for free</font></font>
            </a>
          </div>
      
        
  
              </li>

<!---->          </ul>
        </nav>
      </div>
    </header>
  

    
    <div class="videoinappalert-inapp-alerts-manager
        hidden">
      
<!---->    
    </div>
  

<!---->
<div class="authentication-outlet">
<!---->  <div id="profile-content" class="extended tetris pv-profile-body-wrapper">
  <div class="body">
    

      <div class="scaffold-layout__tracking-element"></div>

<div class="scaffold-layout
    scaffold-layout--breakpoint-xl
    scaffold-layout--main-aside
    
    scaffold-layout--reflow
    
     pv-profile">
    <section class="scaffold-layout-toolbar
    scaffold-layout-toolbar--is-fixed
    
    ">
  <div class="scaffold-layout-toolbar__content scaffold-layout-container
      scaffold-layout-container--reflow
      ">
    
      
        
    
    <div class="pv-profile-sticky-header-v2__container pv1">
      <div id="ember25" class="artdeco-entity-lockup artdeco-entity-lockup--size-1 ember-view pv-profile-sticky-header-v2__mini-profile-container">
        

    <div class="presence-entity presence-entity--size-1 m1">
      <img src="https://media.licdn.com/dms/image/C4D03AQErIf349st-lg/profile-displayphoto-shrink_100_100/0/1523376389588?e=1701302400&amp;v=beta&amp;t=Jfwd7yy-uVy1nU9eKVQ3rxspV8RLh8GoG1EagGIlMdk" loading="lazy" alt="Krystian Madej" id="ember26" class="presence-entity__image   EntityPhoto-circle-1 evi-image lazy-image ember-view">

      

    <div class="presence-entity__indicator
          
          presence-entity__indicator--size-1 presence-indicator
        hidden
        presence-indicator--size-1">
      <span class="visually-hidden">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Status offline</font></font>
              </span>
    </div>
  
    </div>
  
        <div id="ember27" class="artdeco-entity-lockup__content ember-view overflow-hidden ml1 align-self-flex-start">
          <div id="ember28" class="artdeco-entity-lockup__title ember-view">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Krystian Madej</font></font>
<!---->          
</div>

          <div id="ember29" class="artdeco-entity-lockup__subtitle ember-view truncate">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font>
          </div>
        </div>
      
</div>
      <div class="pv-profile-sticky-header-v2__actions-container">
        
      
  <div class="display-flex inline-flex">
        <div class="pvs-sticky-header-profile-actions inline-flex">
      <div class="pvs-sticky-header-profile-actions__action">
        <div id="ember124" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
  <button aria-expanded="false" aria-label="More actions" id="ember125" class="artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view pvs-profile-actions__action artdeco-button artdeco-button--secondary artdeco-button--muted
      artdeco-button--2
      
      " type="button" tabindex="0">
      <span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">More</font></font></span>
  
<!----></button>
  <div tabindex="-1" aria-hidden="true" id="ember126" class="pvs-overflow-actions-dropdown__content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-bottom ember-view"><div class="artdeco-dropdown__content-inner">
  
    <ul>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Send Krystian Madej's profile via message" role="button" id="ember127" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
    <li-icon aria-hidden="true" type="share-linkedin" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 12l-4.61 7H16l4-6H8a3.92 3.92 0 00-4 3.84V17a4 4 0 00.19 1.24L5.12 21H3l-.73-2.22A6.4 6.4 0 012 16.94 6 6 0 018 11h12l-4-6h2.39z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Send profile in message</font></font>
    </span>
  </div>




        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Save Krystian Madej's profile as PDF" role="button" id="ember128" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="download" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 14v5a3 3 0 01-3 3H6a3 3 0 01-3-3v-5h2v5a1 1 0 001 1h12a1 1 0 001-1v-5zm-4-.57V11l-4 2.85V3h-2v10.85L7 11v2.43L12 17z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Save as PDF</font></font></span>
  </div>



<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
    
    
    <div aria-label="Follow Krystian Madej" role="button" id="ember129" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="add" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 13h-8v8h-2v-8H3v-2h8V3h2v8h8z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font></span>
    </div>



<!---->  

<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Report or block Krystian Madej's profile" role="button" id="ember130" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="report" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17.9 4.38a5.82 5.82 0 01-1.9.31 6.1 6.1 0 01-1.9-.31l-3.2-1.07A6.41 6.41 0 009 3a5.57 5.57 0 00-1.9.31L6 3.67V2H4v20h2v-8l1.1-.37a6.41 6.41 0 011.9-.32 5.82 5.82 0 011.9.31l3.2 1.07a6 6 0 003.8 0L20 14V3.68z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Report/block</font></font></span>
  </div>


<!---->

<!---->
<!---->


        </li>

        <li>
          
    <div role="button" id="ember131" class="display-flex align-items-center full-width  artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="signal-notice" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M18 3H6a3 3 0 00-3 3v12a3 3 0 003 3h12a3 3 0 003-3V6a3 3 0 00-3-3zm-4 15h-1a3 3 0 01-3-3 3.22 3.22 0 01.1-.75L11.2 10h2.07L12 14.75A1 1 0 0013 16h1zm-1-9.75A1.25 1.25 0 1114.25 7 1.25 1.25 0 0113 8.25z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
      <span class="a11y-text" aria-live="off"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
    </div>

        </li>
    </ul>
  
</div>
</div>
</div>
      </div>
<!---->      <!---->
<!---->
<!---->
        

    <div class="entry-point profile-action-compose-option">
      
<!---->  

          
      <button aria-label="Send Krystian a message" id="ember181" class="artdeco-button artdeco-button--2 artdeco-button--secondary ember-view pvs-sticky-header-profile-actions__action"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Message</font></font>
</span></button>
    
    </div>
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


            
      
        
    
          
        
    <button aria-label="Send connection request to Krystian Madej" id="ember133" class="artdeco-button artdeco-button--2 artdeco-button--primary ember-view pvs-sticky-header-profile-actions__action" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


  </div>
  <!---->


<!---->
<!---->  </div>

    
      </div>
    </div>
  
  
      
    
  </div>
</section>

  <div class="scaffold-layout__inner scaffold-layout-container
      scaffold-layout-container--reflow
      ">
<!---->
      <div class="scaffold-layout__row scaffold-layout__content
          scaffold-layout__content--main-aside
          
          
          scaffold-layout__content--has-aside
          
          ">
<!---->
          <main class="scaffold-layout__main">
  
            
        
<!----><!---->    
    <section id="ember31" class="artdeco-card ember-view pv-top-card" data-member-id="305921936"><!---->

      
  <div class="live-video-hero-image
      " style="min-height: 201px; max-height: 201px;">
    <div class="live-video-hero-image__bg-image">
      
            
    <div tabindex="-1" class="profile-background-image">
        <div class="profile-background-image__image-container">
            <img class="profile-background-image__image relative
                full-width full-height" id="profile-background-image-target-image" alt="Background image" src="https://media.licdn.com/dms/image/C4E16AQHG32g-LtiiGQ/profile-displaybackgroundimage-shrink_350_1400/0/1615934876702?e=1701302400&amp;v=beta&amp;t=7Rglcy3pMKVdMfQgTZf7vqjA2-jTHocI8pZOLqIZTyU" style="left:0px;top:0px;">
        </div>
    </div>
  
        
    </div>
    <div class="live-video-hero-image__live-video">
<!---->    </div>
  </div>

      <div class="ph5 ">
        <div class="display-flex">
          <div class="pv-top-card--photo text-align-left
              pv-top-card--photo-resize">
                  
    <div class="pv-top-card__non-self-photo-wrapper ml0">
      
    

      <button class="pv-top-card-profile-picture pv-top-card-profile-picture__container display-block
          pv-top-card__photo presence-entity__image EntityPhoto-circle-9
          " type="button">
        
      
          
        <img width="200" title="Krystian Madej, #OPEN_TO_WORK" src="https://media.licdn.com/dms/image/D4E35AQEA7ib1HDsfCA/profile-framedphoto-shrink_200_200/0/1693843853826?e=1696248000&amp;v=beta&amp;t=Lc9kt7BO17I1NE492Y31keVc0v6k3jQ6du5XxbthGw0" height="200" alt="Krystian Madej, #OPEN_TO_WORK" id="ember32" class="pv-top-card-profile-picture__image pv-top-card-profile-picture__image--show evi-image ember-view">

<!---->
<!---->    
        
  
      </button>
  
  
<!---->    </div>
  

          </div>
          <div class="flex-1 flex-column">
            <div class="pv-top-card__badge-wrap">
<!----><!----><!---->            </div>
          </div>
        </div>

        
    <div class="mt2 relative">
      <div class="pv-text-details__left-panel
          ">
        <div>
            <a href="/in/krystian-madej/overlay/about-this-profile/" id="ember33" class="ember-view pv-text-details__about-this-profile-entrypoint">
              <h1 class="text-heading-xlarge inline t-24 v-align-middle break-words"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Krystian Madej</font></font></h1>
                <svg role="img" aria-hidden="false" class="v-align-middle t-black--light" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="verified-medium">
    <title>Krystian has verifications</title>
    

    <use href="#verified-medium" width="24" height="24"></use>
</svg>

<!---->            </a>

<!----><!----><!---->        </div>
<!---->        <div class="text-body-medium break-words" data-generated-suggestion-target="urn:li:fsu_profileActionDelegate:-1839964074">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font>
        </div>
<!----><!----><!---->      </div>
        <ul class="pv-text-details__right-panel">
            <li class="pv-text-details__right-panel-item">
              <button class="pv-text-details__right-panel-item-link text-align-left" aria-label="Current company: ASML.  Click to skip this and go to experience map" type="button">
                <img src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" alt="" id="ember134" class="evi-image ember-view pv-text-details__right-panel-item-text-image EntityPhoto-square-1">
                <span class="pv-text-details__right-panel-item-text hoverable-link-text break-words text-body-small t-black">
                  
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         inline" style="-webkit-line-clamp:2;" tabindex="-1">

        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML</font></font>
      
<!---->    </div>
  
                </span>
              </button>
            </li>
<!---->        </ul>
      <div class="pv-text-details__left-panel mt2">
          <span class="text-body-small inline t-black--light break-words">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Eindhoven, North Brabant, Netherlands</font></font>
          </span>
          <span class="pv-text-details__separator t-black--light">
            <a href="/in/krystian-madej/overlay/contact-info/" id="top-card-text-details-contact-info" class="ember-view link-without-visited-state cursor-pointer text-heading-small inline-block break-words">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Contact details</font></font>
            </a>
          </span>
      </div>
    </div>
  
<!---->
        
    <ul class="pv-top-card--list pv-top-card--list-bullet">
<!---->
        <li class="text-body-small">
            <span class="t-black--light">
              <span class="t-bold"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">456</font></font></span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;"> connections</font></font>
            </span>
        </li>
    </ul>
  
<!---->
        

  <div class="pv-top-card-v2-ctas
      ">
<!---->
<!---->
      <div class="pvs-profile-actions
    
    ">
          
      
        
    
          
        
    <button aria-label="Send connection request to Krystian Madej" id="ember135" class="artdeco-button artdeco-button--2 artdeco-button--primary ember-view pvs-profile-actions__action" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->



    <!---->
<!---->
<!---->
        

    <div class="entry-point profile-action-compose-option">
      
<!---->  

          
      <button aria-label="Send Krystian a message" id="ember198" class="artdeco-button artdeco-button--2 artdeco-button--secondary ember-view pvs-profile-actions__action"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Message</font></font>
</span></button>
    
    </div>
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->



<!---->
<!---->
<!---->
    <div id="ember137" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
  <button aria-expanded="false" aria-label="More actions" id="ember138" class="artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view pvs-profile-actions__action artdeco-button artdeco-button--secondary artdeco-button--muted
      artdeco-button--2
      
      " type="button" tabindex="0">
      <span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">More</font></font></span>
  
<!----></button>
  <div tabindex="-1" aria-hidden="true" id="ember139" class="pvs-overflow-actions-dropdown__content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-bottom ember-view"><div class="artdeco-dropdown__content-inner">
  
    <ul>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Send Krystian Madej's profile via message" role="button" id="ember140" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
    <li-icon aria-hidden="true" type="share-linkedin" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 12l-4.61 7H16l4-6H8a3.92 3.92 0 00-4 3.84V17a4 4 0 00.19 1.24L5.12 21H3l-.73-2.22A6.4 6.4 0 012 16.94 6 6 0 018 11h12l-4-6h2.39z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Send profile in message</font></font>
    </span>
  </div>




        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Save Krystian Madej's profile as PDF" role="button" id="ember141" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="download" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 14v5a3 3 0 01-3 3H6a3 3 0 01-3-3v-5h2v5a1 1 0 001 1h12a1 1 0 001-1v-5zm-4-.57V11l-4 2.85V3h-2v10.85L7 11v2.43L12 17z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Save as PDF</font></font></span>
  </div>



<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
    
    
    <div aria-label="Follow Krystian Madej" role="button" id="ember142" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="add" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 13h-8v8h-2v-8H3v-2h8V3h2v8h8z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font></span>
    </div>



<!---->  

<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Report or block Krystian Madej's profile" role="button" id="ember143" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="report" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17.9 4.38a5.82 5.82 0 01-1.9.31 6.1 6.1 0 01-1.9-.31l-3.2-1.07A6.41 6.41 0 009 3a5.57 5.57 0 00-1.9.31L6 3.67V2H4v20h2v-8l1.1-.37a6.41 6.41 0 011.9-.32 5.82 5.82 0 011.9.31l3.2 1.07a6 6 0 003.8 0L20 14V3.68z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Report/block</font></font></span>
  </div>


<!---->

<!---->
<!---->


        </li>

        <li>
          
    <div role="button" id="ember144" class="display-flex align-items-center full-width  artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="signal-notice" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M18 3H6a3 3 0 00-3 3v12a3 3 0 003 3h12a3 3 0 003-3V6a3 3 0 00-3-3zm-4 15h-1a3 3 0 01-3-3 3.22 3.22 0 01.1-.75L11.2 10h2.07L12 14.75A1 1 0 0013 16h1zm-1-9.75A1.25 1.25 0 1114.25 7 1.25 1.25 0 0113 8.25z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
      <span class="a11y-text" aria-live="off"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
    </div>

        </li>
    </ul>
  
</div>
</div>
</div>
</div>
<!----><!---->
<!---->
<!---->

  </div>

<!---->
      </div>
      <div id="profile-sticky-header-toggle"></div>
        
    <div class="pv-open-to-carousel ">
          
    <section id="ember61" class="artdeco-card ember-view pv-open-to-carousel-card
        pv-open-to-carousel-card--enrolled pv-open-to-carousel-card--single"><!---->

        <div class="display-flex justify-space-between pl3
            pr3">
          <a class="app-aware-link  pv-open-to-carousel-card__content link-without-hover-visited flex-grow-1 pv3" data-view-name="profile-opportunity-card" href="https://www.linkedin.com/in/krystian-madej/opportunities/job-opportunities/details?profileUrn=urn%3Ali%3Afs_normalized_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;trk=opento_nprofile_details" data-test-app-aware-link="">
            <h3 class="truncate text-body-small">
              <strong><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Available for work</font></font><!----></strong>
            </h3>
              <p class="truncate text-body-small">
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Positions for Architecture Specialist, Senior Software Engineer, Senior Software Engineer and Team Leader</font></font>
              </p>
            <p class="link-without-visited-state text-heading-small hoverable-link-text">
              <!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show details</font></font><!---->
            </p>
<!---->          </a>

<!---->        </div>
    </section>
  
      
<!---->
<!---->    </div>
  
<!---->    </section>
<!---->  
  
        
    
    
<!---->
  
  
        
<!---->    
<!---->  
  
        
        
<!---->
<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember105" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="about" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Info</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Info</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="display-flex ph5 pv3">
      
      <div class="
          display-flex full-width" data-generated-suggestion-target="urn:li:fsu_profileActionDelegate:-1603929916">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:4;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Producer with 10 years of professional experience in solving complex challenges in embedded systems. </font><font style="vertical-align: inherit;">Through the years I managed to acquire solid skills in majority of software development concepts, frameworks and methodologies. </font><font style="vertical-align: inherit;">Due to personal preferences my core language is C++ combined with Python and all libraries around those languages. </font><font style="vertical-align: inherit;">However I usually quickly adapt to the technology stack used in the project.</font></font><!----><br><br><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">I have started my journey in telecommunication industry at Nokia where I have been developing BTS firmware for global customers. </font><font style="vertical-align: inherit;">After that I moved into automotive systems where I could play around with ecodrive so gathering data from various car sensors to decide if the driver cares for the environment ;) Next adventure was Windows desktop CAD application which main purpose was assistance during the process of transformation into smart city by urban planners. </font><font style="vertical-align: inherit;">After playing a role in designing cities of the future I moved away from earth into air so into airport management systems. </font><font style="vertical-align: inherit;">In this project I was working on a system that synchronizes various subsystems which had many goals like improvement of life-critical safety, optimization of operations and many more. </font><font style="vertical-align: inherit;">Sadly I did not get free travels but experience gained was very nice ;-) My last and current project is global TOP 50 company ASML so embedded photolithography in which I am working in Optical Column group. </font><font style="vertical-align: inherit;">This group takes care of firmware for world's most complex and expensive chip manufacturing machine.</font></font><!----></span><span class="visually-hidden"><!---->Software Producer with 10 years of professional experience in solving complex challenges in embedded systems. Through the years I managed to acquire solid skills in majority of software development concepts, frameworks and methodologies. Due to personal preferences my core language is C++ combined with Python and all libraries around those languages. However I usually quickly adapt to the technology stack used in the project.

I have started my journey in telecommunication industry at Nokia where I have been developing BTS firmware for global customers. After that I moved into automotive systems where I could play around with ecodrive so gathering data from various car sensors to decide if the driver cares for the environment ;) Next adventure was Windows desktop CAD application which main purpose was assistance during the process of transformation into smart city by urban planners. After playing a role in designing cities of the future I moved away from earth into air so into airport management systems. In this project I was working on a system that synchronises various subsystems which had many goals like improvement of life-critical safety, optimisation of operations and many more. Sadly I did not get free travels but experience gained was very nice ;-) My last and current project is global TOP 50 company ASML so embedded photolithography in which I am working in Optical Column group. This group takes care of firmware for world's most complex and expensive chip manufacturing machine.<!----></span>

          <span class="inline-show-more-text__link-container-collapsed">
<!---->            <button class="inline-show-more-text__button
                inline-show-more-text__button--light
                link" aria-expanded="false" role="button" type="button">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">…Show more</font></font>
            </button>
          </span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
            <div>
                
          
    <div class="pvs-list__outer-container">
<!----><!----><!---->    </div>
  
  
            </div>
        </section>

<!---->  
        
<!---->
<!---->  
        
        <section id="ember67" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="content_collections" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Activity</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Activity</font></font><!----></span>
              </h2>
              <p class="pvs-header__subtitle pvs-header__optional-link text-body-small">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">456 followers</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">456 followers</font></font><!----></span>
              </p>
<!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
  
    
    <div class="mb3">
      
        
<!---->  
      
    </div>
    <div class="pv0 ph5">
      
          
      <div class="display-flex flex-column full-width pb3">
        <span class="text-body-medium-bold"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Krystian hasn't posted anything yet</font></font></span>
        <span class="text-body-small"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Recent posts shared by Krystian are listed here.</font></font></span>
      </div>
  
      
    </div>
      <footer class="artdeco-card__actions profile-creator-shared-content-view__footer-actions">
        <a class="app-aware-link  artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--3 artdeco-button--fluid artdeco-button--tertiary" href="https://www.linkedin.com/in/krystian-madej/recent-activity/all/" data-test-app-aware-link="">
          <svg role="none" aria-hidden="true" class="artdeco-button__icon rtl-flip" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="arrow-right-small">
<!---->    

    <use href="#arrow-right-small" width="16" height="16"></use>
</svg>

          <span class="artdeco-button__text">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">View all activity</font></font>
          </span>
        </a>
      </footer>
  
  

  
<!---->        </section>

<!---->  
        
        <section id="ember106" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="experience" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Experience</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Experience</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/3894/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" loading="lazy" height="48" alt="ASML logo" id="ember107" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">May 2022 - Present · 1 yr 5 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">May 2022 - Present · 1 yr 5 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Veldhoven, North Brabant, Holandia</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Veldhoven, North Brabant, Holandia</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/37854828/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQHTN6M1fk2LTg/company-logo_100_100/0/1662474475472?e=1703721600&amp;v=beta&amp;t=qAOcBzdk2iIW0lyZgVfqKakorgU7qu9Y7FOnbPGYRmo" loading="lazy" height="48" alt="Logo of NobleProg Polska" id="ember108" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Trainer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Trainer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">NobleProg Polska · Seasonal</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">NobleProg Polska · Seasonal</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">July </font><font style="vertical-align: inherit;">2021 - Nov. </font><font style="vertical-align: inherit;">2022 · 1 yr 5 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">July </font><font style="vertical-align: inherit;">2021 - Nov. </font><font style="vertical-align: inherit;">2022 · 1 yr 5 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław i okolice</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław i okolice</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/10843534/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C510BAQGDMGLyhRKDxQ/company-logo_100_100/0/1519879047653?e=1703721600&amp;v=beta&amp;t=FeFiLA8S62aDJ5hfFqFHkIGqYe94o2yOxGP3Ib1PVNY" loading="lazy" height="48" alt="Logo of GlobalLogic Poland" id="ember109" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Engineer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Engineer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">GlobalLogic Poland · Full-time</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">GlobalLogic Poland · Full-time</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2022 - Apr. </font><font style="vertical-align: inherit;">2022 · 2 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2022 - Apr. </font><font style="vertical-align: inherit;">2022 · 2 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Woj. </font><font style="vertical-align: inherit;">Dolnośląskie, Polska</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Woj. </font><font style="vertical-align: inherit;">Dolnośląskie, Polska</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/11387324/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E0BAQE2DrxnvQOxfw/company-logo_100_100/0/1654501549855?e=1703721600&amp;v=beta&amp;t=CAIG89ziKTyz0BbYd1Ju4QkrcG29gyNmBOJNEvZvw5k" loading="lazy" height="48" alt="Logo of xBerry - R&amp;D House" id="ember110" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ Software Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ Software Developer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">xBerry - Software Development · Full-time</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">xBerry - Software Development · Full-time</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Feb. </font><font style="vertical-align: inherit;">2020 - Jan. </font><font style="vertical-align: inherit;">2022 · 2 yrs</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Feb. </font><font style="vertical-align: inherit;">2020 - Jan. </font><font style="vertical-align: inherit;">2022 · 2 yrs</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Wed. </font><font style="vertical-align: inherit;">dolnośląskie, Polska</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Wed. </font><font style="vertical-align: inherit;">dolnośląskie, Polska</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/3894/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" loading="lazy" height="48" alt="ASML logo" id="ember111" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Software Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Software Developer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML · Full-time</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML · Full-time</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2020 - Mar. </font><font style="vertical-align: inherit;">2021 · 1 yr 1 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2020 - Mar. </font><font style="vertical-align: inherit;">2021 · 1 yr 1 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wilton, Connecticut, Stany Zjednoczone</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wilton, Connecticut, Stany Zjednoczone</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="pvs-list__item--with-top-padding pvs-list__item--one-column">
              
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Embedded project for lithography machine. </font><font style="vertical-align: inherit;">EUV department.</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Embedded project for lithography machine. </font><font style="vertical-align: inherit;">EUV department.</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-see-all-experiences" href="https://www.linkedin.com/in/krystian-madej/details/experience?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ">
<!---->            <span class="pvs-navigation__text">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">View all 8 experiences</font></font>
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember112" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="education" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Course</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Course</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/6908713/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGgO7Z7D24ppA/company-logo_100_100/0/1519935717598?e=1703721600&amp;v=beta&amp;t=C3AfugR1ASlwkRFpXT_DbHKYT83DXTFPqrgZjchVfhk" loading="lazy" height="48" alt="Logo of Politechnika Wrocławska" id="ember113" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/6908713/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Politechnika Wrocławska</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Politechnika Wrocławska</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Inżynier (Inż.), Informatyka</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Inżynier (Inż.), Informatyka</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->2010 - 2013<!----></span><span class="visually-hidden"><!---->2010 - 2013<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember114" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="licenses_and_certifications" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Licenties en certificaten<!----></span><span class="visually-hidden"><!---->Licenties en certificaten<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="entity_image_licenses_and_certifications" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/162868/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQHzAFzG6umH9g/company-logo_100_100/0/1627841333886?e=1703721600&amp;v=beta&amp;t=lz4c73FARTWvDl-mHYvw1Wl952Xe92EjUDt6vJdvLvk" loading="lazy" height="48" alt="Logo of Cambridge University Press &amp; Assessment" id="ember115" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->CAE<!----></span><span class="visually-hidden"><!---->CAE<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!---->Cambridge University Press &amp; Assessment<!----></span><span class="visually-hidden"><!---->Cambridge University Press &amp; Assessment<!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->Afgegeven op jun. 2014<!----></span><span class="visually-hidden"><!---->Afgegeven op jun. 2014<!----></span>
              </span>
<!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember74" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="skills" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Vaardigheden<!----></span><span class="visually-hidden"><!---->Vaardigheden<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="skill_card_skill_topic" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/search/results/all/?keywords=C%2B%2B&amp;origin=PROFILE_PAGE_SKILL_NAVIGATION">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->C++<!----></span><span class="visually-hidden"><!---->C++<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!----><!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
      <div class="display-flex link-without-hover-visited">
        
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="clipboard-check" class="ivm-view-attr__icon--brand  " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M14.73 10H17l-5.5 8L8 14.5l1.34-1.34L11.21 15zM20 3v16a3 3 0 01-3 3H7a3 3 0 01-3-3V3h5.69l.52-1A2 2 0 0112 1a2 2 0 011.76 1l.52 1zm-2 2h-2.6l.6 1.1V7H8v-.9L8.6 5H6v14a1 1 0 001 1h10a1 1 0 001-1z"></path>
    </svg></li-icon>
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->Geslaagd voor LinkedIn-vaardigheidsevaluatie<!----></span><span class="visually-hidden"><!---->Geslaagd voor LinkedIn-vaardigheidsevaluatie<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
      
      </div>
  
<!---->    </div>
  
  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
        <a class="optional-action-target-wrapper display-flex link-without-hover-visited" target="_self" href="https://www.linkedin.com/in/krystian-madej/overlay/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,12)/endorsers?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=1&amp;modalTabIndex=1">
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="24" src="https://media.licdn.com/dms/image/C4E03AQGcXE4RmaO9uA/profile-displayphoto-shrink_100_100/0/1635332489775?e=1701302400&amp;v=beta&amp;t=dry8SjPi4xIojGu-eOebg4yENFwHvSXTFWCIXQ-UFuM" loading="lazy" height="24" alt="" id="ember75" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-0 EntityPhoto-circle-0-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black hoverable-link-text display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->Onderschreven door<span class="white-space-pre"> </span><!---->Wojciech 👤 Razik<!----><!---->, die hier erg goed in is<!----></span><span class="visually-hidden"><!---->Onderschreven door<span class="white-space-pre"> </span><!---->Wojciech 👤 Razik<!----><!---->, die hier erg goed in is<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
      </a>
  
<!---->    </div>
  
  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
        <a class="optional-action-target-wrapper display-flex link-without-hover-visited" target="_self" href="https://www.linkedin.com/in/krystian-madej/overlay/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,12)/endorsers?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=2&amp;modalTabIndex=2">
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="24" src="https://media.licdn.com/dms/image/C4E0BAQGL8hpduEqGKQ/company-logo_100_100/0/1677420438777?e=1703721600&amp;v=beta&amp;t=6Havf9e8JQQcReBL0yF3K-c4bd2AMMTNq_15Ulm9YaY" loading="lazy" height="24" alt="" id="ember76" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-square-size-0 EntityPhoto-square-0-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black hoverable-link-text display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->Onderschreven door 4 collega’s bij<span class="white-space-pre"> </span><!---->Nokia<!----></span><span class="visually-hidden"><!---->Onderschreven door 4 collega’s bij<span class="white-space-pre"> </span><!---->Nokia<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
      </a>
  
<!---->    </div>
  
  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center align-self-flex-start
            
            " target="_self" id="navigation-overlay-section-with-parent-Alle-4-details-weergeven" href="https://www.linkedin.com/in/krystian-madej/overlay/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,12)/skill-insights?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=0&amp;modalTabIndex=0">
<!---->            <span class="pvs-navigation__text">
              Alle 4 details weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="skill_card_skill_topic" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/search/results/all/?keywords=Boost+Library+C%2B%2B&amp;origin=PROFILE_PAGE_SKILL_NAVIGATION">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->Boost Library C++<!----></span><span class="visually-hidden"><!---->Boost Library C++<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!----><!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
        <a class="optional-action-target-wrapper display-flex link-without-hover-visited" target="_self" href="https://www.linkedin.com/in/krystian-madej/details/skills/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,14)/endorsers?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=0&amp;modalTabIndex=0&amp;detailScreenTabIndex=0">
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="people" class="ivm-view-attr__icon--icon  " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M12 16v6H3v-6a3 3 0 013-3h3a3 3 0 013 3zm5.5-3A3.5 3.5 0 1014 9.5a3.5 3.5 0 003.5 3.5zm1 2h-2a2.5 2.5 0 00-2.5 2.5V22h7v-4.5a2.5 2.5 0 00-2.5-2.5zM7.5 2A4.5 4.5 0 1012 6.5 4.49 4.49 0 007.5 2z"></path>
    </svg></li-icon>
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="hoverable-link-text display-flex align-items-center
              t-14 t-normal t-black">
            <span aria-hidden="true"><!---->4 onderschrijvingen<!----></span><span class="visually-hidden"><!---->4 onderschrijvingen<!----></span>
          </div>
      
      </div>
  
    </div>
  
      </a>
  
<!---->    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-Alle-33-vaardigheden-weergeven" href="https://www.linkedin.com/in/krystian-madej/details/skills?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ">
<!---->            <span class="pvs-navigation__text">
              Alle 33 vaardigheden weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember116" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="courses" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Cursussen<!----></span><span class="visually-hidden"><!---->Cursussen<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->C++ Clean Code<!----></span><span class="visually-hidden"><!---->C++ Clean Code<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <!----><!----><!---->
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->C++ Memory Management<!----></span><span class="visually-hidden"><!---->C++ Memory Management<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <!----><!----><!---->
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-see-all-courses" href="https://www.linkedin.com/in/krystian-madej/details/courses?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ">
<!---->            <span class="pvs-navigation__text">
              Alle 5 cursussen weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember117" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="languages" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Talen<!----></span><span class="visually-hidden"><!---->Talen<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->angielski<!----></span><span class="visually-hidden"><!---->angielski<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->Professionele werkvaardigheid<!----></span><span class="visually-hidden"><!---->Professionele werkvaardigheid<!----></span>
              </span>
<!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->niemiecki<!----></span><span class="visually-hidden"><!---->niemiecki<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->Basisvaardigheid<!----></span><span class="visually-hidden"><!---->Basisvaardigheid<!----></span>
              </span>
<!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember118" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="organizations" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Organisaties<!----></span><span class="visually-hidden"><!---->Organisaties<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->United Music<!----></span><span class="visually-hidden"><!---->United Music<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!---->Ambassador · nov. 2013 - nov. 2018<!----></span><span class="visually-hidden"><!---->Ambassador · nov. 2013 - nov. 2018<!----></span>
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="pvs-list__item--with-top-padding pvs-list__item--one-column">
              
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->https://www.facebook.com/UnitedMusic<!----></span><span class="visually-hidden"><!---->https://www.facebook.com/UnitedMusic<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->Google Developers Group Wrocław<!----></span><span class="visually-hidden"><!---->Google Developers Group Wrocław<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!---->Events organiser · jan. 2017 - jan. 2018<!----></span><span class="visually-hidden"><!---->Events organiser · jan. 2017 - jan. 2018<!----></span>
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember84" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="interests" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Interesses<!----></span><span class="visually-hidden"><!---->Interesses<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
      <div id="ember85" class="artdeco-tabs artdeco-tabs--size-t-48 ember-view">
          <div aria-multiselectable="false" id="ember86" class="artdeco-tablist ember-view" role="tablist">
              <button tabindex="0" aria-selected="true" id="ember87" class="artdeco-tab active artdeco-tab--selected ember-view" role="tab" aria-controls="ember90">
                <span aria-hidden="true"><!---->Bedrijven<!----></span><span class="visually-hidden"><!---->Bedrijven<!----></span>
              </button>

              <button tabindex="-1" aria-selected="false" id="ember88" class="artdeco-tab ember-view" role="tab" aria-controls="ember95">
                <span aria-hidden="true"><!---->Groepen<!----></span><span class="visually-hidden"><!---->Groepen<!----></span>
              </button>

              <button tabindex="-1" aria-selected="false" id="ember89" class="artdeco-tab ember-view" role="tab" aria-controls="ember98">
                <span aria-hidden="true"><!---->Onderwijsinstellingen<!----></span><span class="visually-hidden"><!---->Onderwijsinstellingen<!----></span>
              </button>

          
</div>
          <div tabindex="0" id="ember90" class="artdeco-tabpanel active ember-view" role="tabpanel" aria-labelledby="ember87">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            ph5 display-flex flex-row flex-wrap
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/3894/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" loading="lazy" height="48" alt="ASML logo" id="ember91" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/3894/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->ASML<!----></span><span class="visually-hidden"><!---->ASML<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->552.746 volgers<!----></span><span class="visually-hidden"><!---->552.746 volgers<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember92" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    Volgen
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/6908713/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGgO7Z7D24ppA/company-logo_100_100/0/1519935717598?e=1703721600&amp;v=beta&amp;t=C3AfugR1ASlwkRFpXT_DbHKYT83DXTFPqrgZjchVfhk" loading="lazy" height="48" alt="Logo of Politechnika Wrocławska" id="ember93" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/6908713/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->Politechnika Wrocławska<!----></span><span class="visually-hidden"><!---->Politechnika Wrocławska<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->98.015 volgers<!----></span><span class="visually-hidden"><!---->98.015 volgers<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember94" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    Volgen
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-see-all-companies" href="https://www.linkedin.com/in/krystian-madej/details/interests?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=0&amp;detailScreenTabIndex=0">
<!---->            <span class="pvs-navigation__text">
              Alle bedrijven weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
          </div>
          <div tabindex="0" hidden="" id="ember95" class="artdeco-tabpanel artdeco-tabpanel--hidden ember-view" role="tabpanel" aria-labelledby="ember88">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            ph5 display-flex flex-row flex-wrap
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_groups_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/groups/12484134">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E07AQH9G3-K6F5-Lw/group-logo_image-shrink_48x48/0/1630999330146?e=1696248000&amp;v=beta&amp;t=fEWn987p0PtXCJqxZAHiGOa1Z5P6HUfDqURYe7yLXt8" loading="lazy" height="48" alt="Logo van Jest praca! Polska: Embedded Software " id="ember96" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_groups_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/groups/12484134">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jest praca! </font><font style="vertical-align: inherit;">Polska: Embedded Software</font></font><span class="white-space-pre"> </span></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jest praca! </font><font style="vertical-align: inherit;">Polska: Embedded Software</font></font><span class="white-space-pre"> </span></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->31 leden<!----></span><span class="visually-hidden"><!---->31 leden<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
      
      <button id="ember97" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button"><!---->
<span class="artdeco-button__text">
    Deelnemen
</span></button>
  
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
          </div>
          <div tabindex="0" hidden="" id="ember98" class="artdeco-tabpanel artdeco-tabpanel--hidden ember-view" role="tabpanel" aria-labelledby="ember89">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            ph5 display-flex flex-row flex-wrap
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_schools_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/6908713/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGgO7Z7D24ppA/company-logo_100_100/0/1519935717598?e=1703721600&amp;v=beta&amp;t=C3AfugR1ASlwkRFpXT_DbHKYT83DXTFPqrgZjchVfhk" loading="lazy" height="48" alt="Logo van Politechnika Wrocławska" id="ember99" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_schools_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/school/6908713/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->Politechnika Wrocławska<!----></span><span class="visually-hidden"><!---->Politechnika Wrocławska<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->98.015 volgers<!----></span><span class="visually-hidden"><!---->98.015 volgers<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember100" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    Volgen
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
          </div>
      </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember119" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="volunteer_causes" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Goede doelen<!----></span><span class="visually-hidden"><!---->Goede doelen<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="display-flex ph5 pv3">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:4;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Animal welfare • Art and culture • Education • Environment • Health • Human rights • Science and technology</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Animal welfare • Art and culture • Education • Environment • Health • Human rights • Science and technology</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
<!---->        </section>

<!---->  
  
      
          
</main>

          <aside class="scaffold-layout__aside
    
    ">
  
            
        
    
<!---->
      
    <div class="relative artdeco-card m0 mb2 p4">
      <div id="ember40" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view update-supported-locales-section-dropDown">
<!---->          <div class="display-flex">
            
    <div class=" display-flex flex-1 flex-wrap">
      <div class="pv-button-group__choice-pill-group display-flex full-width">
          <button aria-checked="true" role="radio" id="ember59" class="pv-button-group__choice-pill-item full-width artdeco-pill artdeco-pill--slate artdeco-pill--3 artdeco-pill--choice artdeco-pill--selected ember-view" lang="en-us" type="button">  <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">English</font></font>
</button>
          <button aria-checked="false" role="radio" id="ember60" class="pv-button-group__choice-pill-item full-width artdeco-pill artdeco-pill--slate artdeco-pill--3 artdeco-pill--choice ember-view" lang="pl-pl" type="button">  <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Polski</font></font>
</button>
      </div>
    </div>
  
          </div>
        <div tabindex="-1" aria-hidden="true" id="ember43" class="pv-update-supported-locales-section__locales-dropdown-content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
      </div>
    </div>
  
    
          <section class="ad-banner-container
    
    artdeco-card overflow-hidden xl
     mb2">
  <iframe data-ad-banner="" class="ad-banner" width="300" height="250" src="about:blank" scrolling="no" title="advertisement"></iframe>

<!---->
  
<!---->  

<!---->
<!----></section>

          
        <section id="ember148" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="browsemap_recommendation" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-medium">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Also viewed</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Also viewed</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/alexander-korenyak-2a198319">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D03AQGGRlFjjmKK2A/profile-displayphoto-shrink_100_100/0/1516821033988?e=1701302400&amp;v=beta&amp;t=tG1y8rC1MIksMf0p-_meTdAvtGK2a_m3mDTG3Luq1HM" loading="lazy" height="48" alt="Profile photo of Alexander Korenyak" id="ember149" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/alexander-korenyak-2a198319?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAPhmdkBNiP1PAws1rCmlTDjMlPA9r9yVBw">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Alexander Korenyak</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Alexander Korenyak</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!---->                
    <div class="ivm-image-view-model  mr1 ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="linkedin-bug" class="ivm-view-attr__icon--premium-inbug  " size="14dp" color="premium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 14 14" data-supported-dps="14x14" fill="currentColor" class="mercado-match" width="14" height="14" focusable="false">
      <g>
        <path class="background-mercado" d="M14 1v12a1 1 0 01-1 1H1a1 1 0 01-1-1V1a1 1 0 011-1h12a1 1 0 011 1zM4 5H2v7h2zm.25-2A1.27 1.27 0 003 1.8 1.27 1.27 0 001.75 3 1.27 1.27 0 003 4.2 1.27 1.27 0 004.25 3zM12 8.29c0-2.2-.73-3.49-2.86-3.49A2.71 2.71 0 006.89 6V5H5v7h2V8.73A1.74 1.74 0 018.66 6.8C9.82 6.8 10 7.94 10 8.73V12h2z"></path>
      </g>
    </svg></li-icon>
    </div>
  
          </div>
  
                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Alexander Korenyak" id="ember150" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/bas-bastiaansen-637a43171">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E03AQE7uN1LxcxsQg/profile-displayphoto-shrink_100_100/0/1624475515434?e=1701302400&amp;v=beta&amp;t=HdHGBq6bVxkYocSLXdBURmKavzpBxgp_W2Yt4p8vink" loading="lazy" height="48" alt="Profile photo of Bas Bastiaansen" id="ember151" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/bas-bastiaansen-637a43171?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACjpI8sB_1W0Nu7jcIyVMEoz4O37oD4k3ao">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bas Bastiaansen</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bas Bastiaansen</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Bas Bastiaansen" id="ember152" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/florisbob">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <div class="EntityPhoto-circle-3-ghost-person ivm-view-attr__ghost-entity  ">
            <div class="visually-hidden"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Profile photo of Floris Bob van Elzelingen</font></font></div>
        </div>
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/florisbob?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABT1GT8B1bH8JnagTTR374FcYarPW_GsAEM">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Floris Bob van Elzelingen</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Floris Bob van Elzelingen</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Floris Bob van Elzelingen" id="ember153" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/kishorkumar1990">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E03AQE6fb3TKvTvuQ/profile-displayphoto-shrink_100_100/0/1653645994332?e=1701302400&amp;v=beta&amp;t=u5PgfO9DzidlJXdy1FmfeWYz0VpungApARz9gVpR8iU" loading="lazy" height="48" alt="Profile photo of Kishor Kumar" id="ember154" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/kishorkumar1990?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAZ8D8wBt3948dcqQ-cqfALU1B6BMJlOyjk">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Kishor Kumar</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Kishor Kumar</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer @ ASML | </font><font style="vertical-align: inherit;">Ex-Philips | </font><font style="vertical-align: inherit;">C# C++</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer @ ASML | </font><font style="vertical-align: inherit;">Ex-Philips | </font><font style="vertical-align: inherit;">C# C++</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Kishor Kumar" id="ember155" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/jjpawillems">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E03AQFVBOy2pNrb6w/profile-displayphoto-shrink_100_100/0/1517758604845?e=1701302400&amp;v=beta&amp;t=LMOWq5Fg_Tl-B11N8--qBi23pcJXO6cKPNP8c2ODiIo" loading="lazy" height="48" alt="Profile photo of Jeroen Willems" id="ember156" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/jjpawillems?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAEBje0BeCdTFXzfQpFCJf2jlR5Ye_EDpj4">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jeroen Willems</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jeroen Willems</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Product Owner at ASML</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Product Owner at ASML</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Jeroen Willems" id="ember157" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
    <button id="ember158" class="pv3 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--fluid artdeco-button--tertiary ember-view" type="button">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show more</font></font>
</span></button>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  

        
        <section id="ember164" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="pymk_recommendation_from_company" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-medium">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">People you may know</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">People you may know</font></font><!----></span>
              </h2>
              <p class="pvs-header__subtitle pvs-header__optional-link text-body-small">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">From Krystian's company</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">From Krystian's company</font></font><!----></span>
              </p>
<!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/raja-n-1b847b284">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4E03AQFNT8NK7dHL_A/profile-displayphoto-shrink_100_100/0/1690352648841?e=1701302400&amp;v=beta&amp;t=Mrnm92SVQjY8GLKyeiHTSCL0p49BT7a8WA7_mgfCZJw" loading="lazy" height="48" alt="Profile photo of RAJA N" id="ember165" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/raja-n-1b847b284?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEUf8lwBR2gag5gHBTKQLwq6lO9P9PfN__A">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">RAJA N</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">RAJA N</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!---->                
    <div class="ivm-image-view-model  mr1 ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="linkedin-bug" class="ivm-view-attr__icon--premium-inbug  " size="14dp" color="premium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 14 14" data-supported-dps="14x14" fill="currentColor" class="mercado-match" width="14" height="14" focusable="false">
      <g>
        <path class="background-mercado" d="M14 1v12a1 1 0 01-1 1H1a1 1 0 01-1-1V1a1 1 0 011-1h12a1 1 0 011 1zM4 5H2v7h2zm.25-2A1.27 1.27 0 003 1.8 1.27 1.27 0 001.75 3 1.27 1.27 0 003 4.2 1.27 1.27 0 004.25 3zM12 8.29c0-2.2-.73-3.49-2.86-3.49A2.71 2.71 0 006.89 6V5H5v7h2V8.73A1.74 1.74 0 018.66 6.8C9.82 6.8 10 7.94 10 8.73V12h2z"></path>
      </g>
    </svg></li-icon>
    </div>
  
          </div>
  
<!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Ui ux at ASML</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Ui ux at ASML</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to RAJA N" id="ember166" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/hamad-jamil-420357285">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4D03AQGSo8fWDOh6dA/profile-displayphoto-shrink_100_100/0/1691453327080?e=1701302400&amp;v=beta&amp;t=0nXgU77X1YQmoVQ9NyocXsLEWE150ZtguXBYCqLQ6Ro" loading="lazy" height="48" alt="Profile photo of Hamad Jamil" id="ember167" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/hamad-jamil-420357285?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEVI-pgBmUI-gdW6FKLUCIu9lQfbc9kuINc">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Hamad Jamil</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Hamad Jamil</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Amazon Virtual Assistant | </font><font style="vertical-align: inherit;">Website Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Amazon Virtual Assistant | </font><font style="vertical-align: inherit;">Website Developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Hamad Jamil" id="ember168" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/gpkanade">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4E03AQF5FvIrgfo_QQ/profile-displayphoto-shrink_100_100/0/1694146996263?e=1701302400&amp;v=beta&amp;t=4fnUOu4eMH05IA4nOgpLZPwQlcmY9b8OzCEsNfuE5kY" loading="lazy" height="48" alt="Profile photo of Goutham Kanade" id="ember169" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/gpkanade?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEUkyLcBJURSMeO9yHYiBSEF-jU7R3g16Yk">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Goutham Kanade</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Goutham Kanade</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Test Architect/Manager | </font><font style="vertical-align: inherit;">Filmmaker | </font><font style="vertical-align: inherit;">Entrepreneur</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Test Architect/Manager | </font><font style="vertical-align: inherit;">Filmmaker | </font><font style="vertical-align: inherit;">Entrepreneur</font></font><!----></span>

<!---->
<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Goutham Kanade" id="ember170" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/mudit-kelwa-297693274">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4D03AQF9aMlzB2pMkQ/profile-displayphoto-shrink_100_100/0/1691510822085?e=1701302400&amp;v=beta&amp;t=H7nstWT4T4LlNy6Bw0RhOMGICnzQZnRxVr3Ulag0qgU" loading="lazy" height="48" alt="Profile photo of Mudit Kelwa" id="ember171" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/mudit-kelwa-297693274?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEMFNtMBVR5px0ojlayBQoev6s9ZpRbMroc">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mudit Kelwa</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mudit Kelwa</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Mudit Kelwa" id="ember172" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/raymond-jumaoas-b35a3a277">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D5635AQEMEav3snuwKQ/profile-framedphoto-shrink_100_100/0/1695472538668?e=1696248000&amp;v=beta&amp;t=Am5VLiQdZYJlEDgq1kVOccgeJIZQtpcJWdZJsR5Au48" loading="lazy" height="48" alt="Profile photo of Raymond Jumaoas" id="ember173" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/raymond-jumaoas-b35a3a277?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEObZNkBsQ9etujCUHKra_KmTCBY4pdhY6M">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Raymond Jumaoas</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Raymond Jumaoas</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Raymond Jumaoas" id="ember174" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
    <button id="ember175" class="pv3 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--fluid artdeco-button--tertiary ember-view" type="button">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show more</font></font>
</span></button>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  

        
        <section id="ember51" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="company_recommendation" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-medium">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You might find this interesting:</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You might find this interesting:</font></font><!----></span>
              </h2>
              <p class="pvs-header__subtitle pvs-header__optional-link text-body-small">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Pages for you</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Pages for you</font></font><!----></span>
              </p>
<!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="company_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/1441/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQHiNSL4Or29cg/company-logo_100_100/0/1519856215226?e=1703721600&amp;v=beta&amp;t=ABquqvWu81IO9c3y4U_4GAFiT7aaZ2OX0o6WrjgKU18" loading="lazy" height="48" alt="Google logo" id="ember52" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="company_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/1441/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Google</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Google</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software development</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software development</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">31,654,272 followers</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">31,654,272 followers</font></font><!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
<!---->  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember53" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font>
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="company_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/42821781/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGW6Sn_g6-xZA/company-logo_100_100/0/1647120980872?e=1703721600&amp;v=beta&amp;t=X_CgVds4WIui4vMtLSg-5s_UZdE3uGGh2FmX_jFgmcs" loading="lazy" height="48" alt="Dubai Vacancy logo" id="ember54" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="company_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/42821781/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dubai Vacancy</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dubai Vacancy</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">catering industry</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">catering industry</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">434,984 followers</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">434,984 followers</font></font><!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
<!---->  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember55" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font>
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
    <button id="ember56" class="pv3 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--fluid artdeco-button--tertiary ember-view" type="button">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show more</font></font>
</span></button>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  

        <div class="mt2">
          
    <div class="sticky
        
        " style="height: 250px; width: 300px; margin: 0px auto;">
      
            <div class="pv-right-rail__sticky-ad-banner">
              <section class="ad-banner-container
    
    artdeco-card overflow-hidden
    ">
  <iframe data-ad-banner="" class="ad-banner" width="300" height="250" src="about:blank" scrolling="no" title="advertisement"></iframe>

<!---->
  
<!---->  

<!---->
<!----></section>
            </div>
          
    </div>
  
        </div>
  
  
      
          
</aside>
      </div>

<!---->  </div>
</div>
  
<!---->

    <footer class="global-footer
        global-footer--static">
      
        <div id="ember49" class="ember-view   ">
            
        <div id="expanded-footer" class="global-footer__container">
<!---->
          <div class="grid grid--is-nested grid--no-gutters mv3">
            <nav class="grid__col
                grid__col--12">
              <ul class="grid grid--no-gutters grid--is-nested">
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://about.linkedin.com/nl-nl" id="globalfooter-about" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About LinkedIn</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://www.linkedin.com/accessibility" id="globalfooter-accessibility" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Accessibility</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/talent-solutions?trk=flagship_nav&amp;veh=li-footer-lts-control&amp;src=li-footer" id="globalfooter-talent_solutions" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Talent Solutions</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://www.linkedin.com/legal/professional-community-policies" id="globalfooter-community_guidelines" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Community Guidelines</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://careers.linkedin.com/" id="globalfooter-careers" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Careers</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/nl-nl/marketing-solutions?trk=n_nav_lms_f&amp;src=li-footer" id="globalfooter-marketing_solutions" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Marketing Solutions</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      
    
    
    <div id="ember215" class="artdeco-dropdown artdeco-dropdown--placement-top artdeco-dropdown--justification-left ember-view global-footer-dropdown">
      <button aria-expanded="false" id="globalfooter-privacy_dropdown-trigger" class="artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-top ember-view global-footer-dropdown__trigger" type="button" tabindex="0">
        <span class="global-footer__link global-footer__link--static t-12 t-bold">
          <span class="text-align-left">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Privacy and terms</font></font>
            <svg role="none" aria-hidden="true" class="global-footer-dropdown__trigger-icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

          </span>
        </span>
      
<!----></button>
      <div tabindex="-1" aria-hidden="true" id="ember217" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-top ember-view global-footer-dropdown__options"><!----></div>
    </div>
  
  
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://www.linkedin.com/help/linkedin/answer/62931?lang=nl" id="globalfooter-ad_choices" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Advertising choice</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/nl-nl/marketing-solutions/ads?trk=n_nav_ads_f" id="globalfooter-advertising" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Advertisement</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/sales-solutions?trk=flagship_nav&amp;veh=li-footer-lss-control&amp;src=li-footer" id="globalfooter-sales_solutions" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">SalesSolutions</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://mobile.linkedin.com/nl-nl" id="globalfooter-mobile" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mobile</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://smallbusiness.linkedin.com?&amp;src=li-footer" id="globalfooter-small_business" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Small company</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://safety.linkedin.com" id="globalfooter-safety_center" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Safety center</font></font>
                      </a>
                  </li>
              </ul>
            </nav>

            <div class="grid__col
                grid__col--12">
              <div class="grid grid--no-gutters grid--is-nested">
                <ul id="footer-action-list" class="global-footer__action-list grid__col
                    grid__col--12">
                    <li class="display-flex list-style-none mb4">
                      <svg role="none" aria-hidden="true" class="global-footer__action-icon rtl-flip" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="question-medium">
<!---->    

    <use href="#question-medium" width="24" height="24"></use>
</svg>

                      <span>
                          <a tabindex="0" target="_blank" href="https://www.linkedin.com/help/linkedin?trk=d_flagship3_profile_view_base" id="questions-help-center" class="ember-view global-footer__link global-footer__link--static t-14 t-bold">
                            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To ask?</font></font>
                          </a>

                        <p class="global-footer__label t-12">
                          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Visit our Help Center.</font></font>
                        </p>
                      </span>
                    </li>
                    <li class="display-flex list-style-none mb4">
                      <svg role="none" aria-hidden="true" class="global-footer__action-icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="settings-medium">
<!---->    

    <use href="#settings-medium" width="24" height="24"></use>
</svg>

                      <span>
                          <a tabindex="0" target="_blank" href="https://www.linkedin.com/psettings/" id="privacy-settings" class="ember-view global-footer__link global-footer__link--static t-14 t-bold">
                            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Manage your account and privacy</font></font>
                          </a>

                        <p class="global-footer__label t-12">
                          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Go to Settings.</font></font>
                        </p>
                      </span>
                    </li>
                    <li class="display-flex list-style-none mb4">
                      <svg role="none" aria-hidden="true" class="global-footer__action-icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="shield-medium">
<!---->    

    <use href="#shield-medium" width="24" height="24"></use>
</svg>

                      <span>
                          <a tabindex="0" target="_blank" href="https://www.linkedin.com/help/linkedin/answer/a1339724" id="recommendation-transparency" class="ember-view global-footer__link global-footer__link--static t-14 t-bold">
                            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Transparency about recommendations</font></font>
                          </a>

                        <p class="global-footer__label t-12">
                          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Read more about recommended content.</font></font>
                        </p>
                      </span>
                    </li>
                </ul>

                <div class="grid__col
                    grid__col--12">
                  <label for="globalfooter-select_language" class="global-footer__label t-12 t-normal mb1 mt0">
                    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Select language</font></font>
                  </label>

                  
    <select id="globalfooter-select_language" class="global-footer__language-selection-dropdown t-12 t-black--light t-bold">
        <option value="ar_AE" lang="ar-ae">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">العربية (Arabic)</font></font>
        </option>
        <option value="cs_CZ" lang="cs-cz">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Čeština (Czech)</font></font>
        </option>
        <option value="da_DK" lang="da-dk">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dansk (Danish)</font></font>
        </option>
        <option value="de_DE" lang="de-de">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Deutsch (German)</font></font>
        </option>
        <option value="en_US" lang="en-us">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">English (English)</font></font>
        </option>
        <option value="es_ES" lang="es-es">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Español (Spanish)</font></font>
        </option>
        <option value="fr_FR" lang="fr-fr">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Français (French)</font></font>
        </option>
        <option value="hi_IN" lang="hi-in">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">हिंदी (Hindi)</font></font>
        </option>
        <option value="in_ID" lang="in-id">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bahasa Indonesia (Indonesian)</font></font>
        </option>
        <option value="it_IT" lang="it-it">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Italiano (Italian)</font></font>
        </option>
        <option value="ja_JP" lang="ja-jp">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">日本語 (Japanese)</font></font>
        </option>
        <option value="ko_KR" lang="ko-kr">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">한국어 (Korean)</font></font>
        </option>
        <option value="ms_MY" lang="ms-my">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bahasa Malaysia (Malay)</font></font>
        </option>
        <option value="nl_NL" lang="nl-nl">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dutch</font></font>
        </option>
        <option value="no_NO" lang="no-no">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Norsk (Norwegian)</font></font>
        </option>
        <option value="pl_PL" lang="pl-pl">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Polski (Polish)</font></font>
        </option>
        <option value="pt_BR" lang="pt-br">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Português (Portuguese)</font></font>
        </option>
        <option value="ro_RO" lang="ro-ro">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Română (Romanian)</font></font>
        </option>
        <option value="ru_RU" lang="ru-ru">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Русский (Russian)</font></font>
        </option>
        <option value="sv_SE" lang="sv-se">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Svenska (Swedish)</font></font>
        </option>
        <option value="th_TH" lang="th-th">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ภาษาไทย (Thai)</font></font>
        </option>
        <option value="tl_PH" lang="tl-ph">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Tagalog (Tagalog)</font></font>
        </option>
        <option value="tr_TR" lang="tr-tr">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Türkçe (Turkish)</font></font>
        </option>
        <option value="uk_UA" lang="uk-ua">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Українська (Ukrainian)</font></font>
        </option>
        <option value="zh_CN" lang="zh-cn">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">简体中文 (Chinese - Simplified)</font></font>
        </option>
        <option value="zh_TW" lang="zh-tw">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">正體中文 (Chinese - Traditional)</font></font>
        </option>
    </select>
  
                </div>
              </div>
            </div>
          </div>

          <p id="globalfooter-copyright" class="t-12 t-black--light t-normal text-align-left clear-both">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">© 2023 LinkedIn Corporation</font></font>
          </p>
        </div>
      
        </div>
  
    </footer>
  
<!----><!---->
  </div>
</div>
</div>

<div class="application-outlet__overlay-container
    ">
<!---->
    <!----><aside id="msg-overlay" class="msg-overlay-container msg-overlay-container-reflow
    ">
  <div tabindex="-1" class="msg-overlay-list-bubble
    
    
    ml4">
  <header class="msg-overlay-bubble-header">
    <div class="msg-overlay-bubble-header__badge-container
        "></div>
    <!---->
    <div class="msg-overlay-bubble-header__details flex-row align-items-center ml1">
      
    <div class="presence-entity presence-entity--size-1">
      <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" loading="lazy" alt="Fayyoz Usmon" id="ember184" class="presence-entity__image EntityPhoto-circle-1  evi-image lazy-image ghost-person ember-view">

        

    <div class="presence-entity__indicator presence-entity__indicator--size-1
             presence-indicator
        presence-indicator--is-online
        presence-indicator--size-1">
      <span class="visually-hidden">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Status is online</font></font>
      </span>
    </div>
  
    </div>
  
      <button class="msg-overlay-bubble-header__button truncate ml2" type="button">
        <span class="truncate t-14 t-bold
            t-black">
          <span aria-hidden="true">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To inform</font></font>
          </span>
          <span class="visually-hidden">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You are on the message overlay. </font><font style="vertical-align: inherit;">Press enter to minimize it.</font></font>
          </span>
        </span>
      </button>

<!----><!---->    </div>
    <div class="msg-overlay-bubble-header__controls display-flex">
      <div id="ember185" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
        <button aria-expanded="false" id="ember186" class="msg-overlay-bubble-header__dropdown-trigger artdeco-button artdeco-button--1 artdeco-button--circle artdeco-button--muted artdeco-button--tertiary artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
          <svg role="img" aria-hidden="false" aria-label="Open selection menu" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="overflow-web-ios-small">
<!---->    

    <use href="#overflow-web-ios-small" width="16" height="16"></use>
</svg>

        
<!----></button>
        <div class="msg-overlay-bubble-header__dropdown-container">
          <div tabindex="-1" aria-hidden="true" id="ember187" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
        </div>
      </div>

      <button id="ember188" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">  <li-icon aria-hidden="true" type="compose-icon" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M15 2.53a1.51 1.51 0 01-.44 1L9.15 9 6 10l1-3.12 5.44-5.44A1.5 1.5 0 0115 2.53zM12 11a1 1 0 01-1 1H5a1 1 0 01-1-1V5a1 1 0 011-1h3V2H5a3 3 0 00-3 3v6a3 3 0 003 3h6a3 3 0 003-3V8h-2z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Compose message</font></font>
</span></button>

      <button id="ember189" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You are on the message overlay. </font><font style="vertical-align: inherit;">Press enter to minimize it.</font></font>
</span></button>
    </div>
  </header>

  <!---->

<!---->
    <div class="msg-overlay-list-bubble-search
    ">
    <div class="msg-overlay-list-bubble-search__input-container">
      <label class="a11y-text" for="msg-overlay-list-bubble-search__search-typeahead-input">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Type to search for connections and conversations.</font></font>
      </label>
      <span class="msg-overlay-list-search__search-icon display-flex">
        <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="search-small">
<!---->    

    <use href="#search-small" width="16" height="16"></use>
</svg>

      </span>
      <input id="msg-overlay-list-bubble-search__search-typeahead-input" class="ember-text-field ember-view msg-overlay-list-bubble-search__search-typeahead-input" placeholder="Search messages" autocomplete="off" type="text">
      <div id="ember190" class="msg-overlay-list-bubble__filters-dropdown artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
        <button aria-expanded="false" tabindex="0" id="ember191" class="msg-overlay-list-bubble__filters-btn artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button">
          <svg role="img" aria-hidden="false" aria-label="Filter messages by" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="filter-small">
<!---->    

    <use href="#filter-small" width="16" height="16"></use>
</svg>

        
<!----></button>
        <div tabindex="-1" aria-hidden="true" id="ember192" class="artdeco-dropdown__content msg-overlay-list-bubble__filters-dropdown-content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
      </div>
    </div>

<!---->
<!---->  <div class="relative display-flex justify-center flex-column overflow-hidden
       hidden">
    <div class="msg-overlay-list-bubble-search-content">
        <div class="msg-overlay-list-bubble-search__history-placeholder-container text-align-center">
          <div class="msg-overlay-list-bubble__illustration msg-overlay-list-bubble-search__history-placeholder-illustration"></div>
          <p class="t-14 t-black t-normal ph2">
            Zoek in uw berichten op ontvanger, inhoud of gespreksnaam
          </p>
        </div>
    </div>
  </div>
</div>

    <div class="msg-overlay-list-bubble__top-static-area">
        <div id="ember193" class="artdeco-tabs artdeco-tabs--size-t-40 artdeco-tabs--centered ember-view msg-focused-inbox-tabs">
  <div aria-multiselectable="false" id="ember194" class="artdeco-tablist artdeco-tablist--no-wrap ember-view" role="tablist">
    <button tabindex="0" aria-selected="true" id="ember195" class="artdeco-tab active artdeco-tab--selected ember-view" role="tab">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Targeted</font></font>
    </button>
    <button tabindex="-1" aria-selected="false" id="ember196" class="artdeco-tab ember-view" role="tab">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Other</font></font>
<!---->    </button>
  
</div>
</div>

<!---->
      <!---->
<!---->

<!---->    </div>

<!---->
<!---->
<!---->
        <section class="scrollable msg-overlay-list-bubble__content msg-overlay-list-bubble__content--scrollable">
          <div></div>
          <div class="msg-overlay-list-bubble__default-conversation-container">
<!---->
                <!---->
              <!---->
              <span class="visually-hidden">
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Attention screen reader users: messages are constantly updated. </font><font style="vertical-align: inherit;">Use the tab and shift+tab keys instead of the up and down arrows to navigate between different messages.</font></font>
              </span>
<!----><!---->
            
      
              <!---->
            
  

            <div class="msg-overlay-list-bubble__conversations-list">
        <div class="msg-conversation-listitem__link msg-overlay-list-bubble__convo-item" tabindex="0" id="overlay-conversation-card-ember199" data-feedback-redacted="">
  <div class="msg-overlay-list-bubble__convo-card-container">
    <div class="msg-conversation-card msg-overlay-list-bubble__convo-card display-flex">
      <div class="msg-selectable-entity
    
    msg-selectable-entity--3">
      <div class="msg-facepile-grid--no-facepile msg-facepile-grid
    msg-facepile-grid--3 msg-selectable-entity__entity">
<!---->      <img src="https://media.licdn.com/dms/image/C4D03AQFjtPx2a0NPeQ/profile-displayphoto-shrink_100_100/0/1516553080018?e=1701302400&amp;v=beta&amp;t=f4IpP56HJwztWkwW0jYIaOvRaQFnxmZajpsQAmijDl4" loading="lazy" alt="Diane Chiang" id="ember201" class="evi-image lazy-image msg-facepile-grid__img msg-facepile-grid__img--person ember-view">
</div>
      <div class="msg-selectable-entity__checkbox-container">
    <input id="checkbox-msg-selectable-entity__checkbox-ember200" class="ember-checkbox ember-view msg-selectable-entity__input simple-form" type="checkbox">
    <label class="msg-selectable-entity__checkbox-label ml2" aria-label="Select conversation" for="checkbox-msg-selectable-entity__checkbox-ember200">
    </label>

    <div class="msg-selectable-entity__checkbox-circle-container">
      <div class="msg-selectable-entity__checkbox-circle">
      </div>
    </div>
  </div>
</div>

      <div class="overflow-hidden pl2 msg-overlay-list-bubble__convo-card-content">
        <div class="msg-overlay-list-bubble__convo-card-content-wrapper fl">
          <div class="msg-conversation-card__row align-items-center display-flex">
            <h3 class="msg-conversation-listitem__participant-names msg-conversation-card__participant-names truncate t-14 t-black
                t-normal">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Diane Chiang</font></font>
            </h3>
            <div class="msg-conversation-card__mute-icon-holder
                msg-conversation-card__mute-icon-holder--inbox-shortcuts">
<!---->            </div>
              <time class="msg-overlay-list-bubble-item__time-stamp t-12
                  msg-overlay-list-bubble-item__time-stamp--inbox-shortcuts
                  t-normal t-black--light">
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Aug 4</font></font>
              </time>
              <div class="msg-overlay-list-bubble__inbox-shortcuts-container">
                <div class="msg-overlay-list-bubble__inbox-shortcuts">
                  


    <div id="ember202" class="artdeco-dropdown msg-thread-actions__dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
      <button aria-expanded="false" id="ember203" class="msg-thread-actions__control artdeco-button artdeco-button--circle artdeco-button--1 artdeco-button--muted artdeco-button--tertiary artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
        <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="overflow-web-ios-small">
<!---->    

    <use href="#overflow-web-ios-small" width="16" height="16"></use>
</svg>

        <span class="visually-hidden">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Open options list in your conversation with Diane Chiang</font></font>
        </span>
      
<!----></button>
      <div class="msg-thread-actions__dropdown-container">
        <div tabindex="-1" aria-hidden="true" id="ember204" class="msg-thread-actions__dropdown-options--inbox-shortcuts artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
      </div>
    </div>

    
<!---->  

    
    <div>
<!---->    </div>
  

<!---->  
                </div>
              </div>
          </div>

          <div class="msg-conversation-card__row justify-space-between">
            <div class="msg-overlay-list-bubble__message-snippet-container--narrow-two-line">
<!---->              <p class="msg-overlay-list-bubble__message-snippet--v2
                  
                  m0 t-12
                  t-black--light">
<!---->                  <span class="msg-conversation-card__pill t-14
                      
                      t-bold pr1">
                    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">LinkedIn Offer</font></font>
                  </span>
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;"> Hello Fayyoz, I'm Diane from the LinkedIn Premium team. </font><font style="vertical-align: inherit;">Thank you for joining the LinkedIn community. </font><font style="vertical-align: inherit;">We appreciate this and would like to offer you an upgrade to LinkedIn Premium with the first month free.
</font></font>
              </p>
<!---->            </div>
            <div class="display-flex">
              <div class="msg-conversation-card__conversation-status msg-conversation-card__star-icon
                  ">
<!---->              </div>
<!---->            </div>
          </div>
        </div>
      </div>
    </div>
  </div>
</div>
</div>

<!---->
<!---->
<!---->          </div>
        </section>

<!----><!----></div>

<!---->
<!---->
  <div id="msg-overlay__emoji-hoverable-outlet"></div>

  <div id="msg-overlay__reactor-list-outlet"></div>
</aside>
</div>

<!---->
  
      
        
    <div id="ember50" class="ember-view"><!----></div>
  
    
  

  
      
    <img src="https://px.ads.linkedin.com/collect/?pid=6883&amp;fmt=gif&amp;_t=1695640000462" alt="" role="none" class="third-party-tracking-pixel hidden" data-test-third-party-tracking-pixel="">
  
  

</div>

<div id="artdeco-hoverable-outlet"></div>

<!---->
<div id="type-ahead-wormhole" class="type-ahead type-ahead-wormhole">
</div>

<div id="toast-wormhole" class="toast-wormhole">
</div>

<div id="profile-inline-modal-outlet">
</div>

<!---->

  <div><!----></div><div id="goog-gt-tt" class="VIpgJd-yAWNEb-L7lbkb skiptranslate" style="border-radius: 12px; margin: 0 0 0 -23px; padding: 0; font-family: 'Google Sans', Arial, sans-serif;" data-id=""><div id="goog-gt-vt" class="VIpgJd-yAWNEb-hvhgNd"><div class=" VIpgJd-yAWNEb-hvhgNd-l4eHX-i3jM8c"><img src="https://fonts.gstatic.com/s/i/productlogos/translate/v14/24px.svg" width="24" height="24" alt=""></div><div class=" VIpgJd-yAWNEb-hvhgNd-k77Iif-i3jM8c"><div class="VIpgJd-yAWNEb-hvhgNd-IuizWc" dir="ltr">Original text</div><div id="goog-gt-original-text" class="VIpgJd-yAWNEb-nVMfcd-fmcmS VIpgJd-yAWNEb-hvhgNd-axAV1"></div></div><div class="VIpgJd-yAWNEb-hvhgNd-N7Eqid ltr"><div class="VIpgJd-yAWNEb-hvhgNd-N7Eqid-B7I4Od ltr" dir="ltr"><div class="VIpgJd-yAWNEb-hvhgNd-UTujCb">Rate this translation</div><div class="VIpgJd-yAWNEb-hvhgNd-eO9mKe">Your feedback will be used to help improve Google Translate</div></div><div class="VIpgJd-yAWNEb-hvhgNd-xgov5 ltr"><button id="goog-gt-thumbUpButton" type="button" class="VIpgJd-yAWNEb-hvhgNd-bgm6sf" title="Good translation" aria-label="Good translation" aria-pressed="false"><span id="goog-gt-thumbUpIcon"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M21 7h-6.31l.95-4.57.03-.32c0-.41-.17-.79-.44-1.06L14.17 0S7.08 6.85 7 7H2v13h16c.83 0 1.54-.5 1.84-1.22l3.02-7.05c.09-.23.14-.47.14-.73V9c0-1.1-.9-2-2-2zM7 18H4V9h3v9zm14-7l-3 7H9V8l4.34-4.34L12 9h9v2z"></path></svg></span><span id="goog-gt-thumbUpIconFilled"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M21 7h-6.31l.95-4.57.03-.32c0-.41-.17-.79-.44-1.06L14.17 0S7.08 6.85 7 7v13h11c.83 0 1.54-.5 1.84-1.22l3.02-7.05c.09-.23.14-.47.14-.73V9c0-1.1-.9-2-2-2zM5 7H1v13h4V7z"></path></svg></span></button><button id="goog-gt-thumbDownButton" type="button" class="VIpgJd-yAWNEb-hvhgNd-bgm6sf" title="Poor translation" aria-label="Poor translation" aria-pressed="false"><span id="goog-gt-thumbDownIcon"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M3 17h6.31l-.95 4.57-.03.32c0 .41.17.79.44 1.06L9.83 24s7.09-6.85 7.17-7h5V4H6c-.83 0-1.54.5-1.84 1.22l-3.02 7.05c-.09.23-.14.47-.14.73v2c0 1.1.9 2 2 2zM17 6h3v9h-3V6zM3 13l3-7h9v10l-4.34 4.34L12 15H3v-2z"></path></svg></span><span id="goog-gt-thumbDownIconFilled"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M3 17h6.31l-.95 4.57-.03.32c0 .41.17.79.44 1.06L9.83 24s7.09-6.85 7.17-7V4H6c-.83 0-1.54.5-1.84 1.22l-3.02 7.05c-.09.23-.14.47-.14.73v2c0 1.1.9 2 2 2zm16 0h4V4h-4v13z"></path></svg></span></button></div></div><div id="goog-gt-votingHiddenPane" class="VIpgJd-yAWNEb-hvhgNd-aXYTce"><form id="goog-gt-votingForm" action="//translate.googleapis.com/translate_voting?client=te_lib" method="post" target="votingFrame" class="VIpgJd-yAWNEb-hvhgNd-aXYTce"><input type="text" name="sl" id="goog-gt-votingInputSrcLang"><input type="text" name="tl" id="goog-gt-votingInputTrgLang"><input type="text" name="query" id="goog-gt-votingInputSrcText"><input type="text" name="gtrans" id="goog-gt-votingInputTrgText"><input type="text" name="vote" id="goog-gt-votingInputVote"></form><iframe name="votingFrame" frameborder="0"></iframe></div></div></div>
</body><body dir="ltr" class="render-mode-BIGPIPE nav-v2 ember-application boot-complete icons-loaded" data-t-link-to-event-attached="true">
  

  
    <!-- HUED-11420 -->
    <div id="artdeco-toasts__wormhole" role="region">    <section id="artdeco-toasts" class="artdeco-toasts" aria-label="Toast message">
      <header class="artdeco-toasts__header">
        <h2 class="artdeco-toasts__title">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">0 reports in total</font></font>
        </h2>
      </header>

<!---->
      <div class="artdeco-toasts_toasts">
<!---->      </div>
    </section>
</div>
    <!-- EMBER_CLI_FASTBOOT_BODY -->
<div id="app-boot-bg-loader" class="app-boot-bg-skeleton">
  <div class="top-bar"></div>
  <div class="content">
    <div class="initial-load-animation fade-load">
      <div class="linkedin-image display-flex justify-center">
        <svg xmlns="http://www.w3.org/2000/svg" class="loader__linkedin-logo" width="190" height="48" viewBox="0 0 190 48">
          <g>
            <g>
              <path d="M58,27.22V41H51V28.88c0-3.7-2.07-5.24-4-5.24a5,5,0,0,0-5.14,4.85,4.34,4.34,0,0,0,0,.51V41H35V18h6.6v3.25h.09c.69-1.41,3.64-3.75,7.66-3.75S58,20.22,58,27.22ZM24,41h7V18H24ZM27.5,6.45a4.05,4.05,0,1,0,4.1,4.05,4,4,0,0,0-4-4ZM190,3.5v41a3.5,3.5,0,0,1-3.5,3.5h-41a3.5,3.5,0,0,1-3.5-3.5V3.5A3.5,3.5,0,0,1,145.5,0h41A3.5,3.5,0,0,1,190,3.5ZM156,18h-7V41h7Zm.6-7.5a4.1,4.1,0,1,0-4.15,4.05h.05a4,4,0,0,0,4.1-3.9ZM183,27.22c0-7-4.63-9.72-8.65-9.72s-7,2.34-7.66,3.75h-.09V18H160V41h7V29c0-3.69,2.51-5.33,4.95-5.33,2,0,4.05,1.54,4.05,5.24V41h7ZM8,7H0V41H21V34H8ZM108,29.77v2H91a1.33,1.33,0,0,0,.11.43c.58,1.93,2.67,3.56,5.75,3.56A6.3,6.3,0,0,0,102,33.52l5.1,3.18a12.72,12.72,0,0,1-10.45,4.8C89.94,41.5,84,37.42,84,29.59S90,17.5,96.44,17.5,108,21.81,108,29.77ZM101,27c0-2.4-1.56-4.38-4.75-4.38-3,0-5.09,2-5.25,4.38ZM85.26,18H76.68l-7.54,9.37H69V7H62V41h7V30h.14l7.72,11h8.77L76.2,28.43ZM128,7h7V41h-6.6V38h-.09c-.88,1.52-3.24,3.49-7.4,3.49-5,0-10.91-3.63-10.91-12,0-7.53,5.1-11.95,10.82-11.95a9.55,9.55,0,0,1,7.09,3H128Zm.3,22.49a5.74,5.74,0,0,0-5.59-5.89h-.15A5.54,5.54,0,0,0,116.89,29c0,.15,0,.29,0,.44a5.61,5.61,0,0,0,5.26,5.94h.4A5.83,5.83,0,0,0,128.3,29.49Z" fill="currentColor"></path>
            </g>
          </g>
        </svg>
      </div>
      <div class="loading-bar">
        <div class="blue-bar"></div>
      </div>
    </div>
  </div>
</div>

    <script src="https://static.licdn.com/sc/h/9vghoawmkb5fvzav7oabel24l" data-fastboot-src="/assets/vendor-static.js" crossorigin=""></script>
    <script src="https://static.licdn.com/sc/h/cfg1twijroe2fxyw06xj1tl2b" data-fastboot-src="/assets/vendor.js" crossorigin=""></script>
    <!-- graphql-script-injection -->
    <script src="https://static.licdn.com/sc/h/a8i9w57wqoyi8fjxs9ulzk14e" data-fastboot-src="/assets/pdsc/register-metadata.js" crossorigin=""></script>
    <script src="https://static.licdn.com/sc/h/8wptp0fxde5h0i9gff7kb1m4x" data-fastboot-src="/assets/chunk.493.5b8b62fabf1d1503e941.js" crossorigin=""></script>
<script src="https://static.licdn.com/sc/h/g05vqk1gm77fzj80yree9bjy" data-fastboot-src="/assets/chunk.143.a1eb44166acaecd73a8c.js" crossorigin=""></script>
    <script data-embroider-ignore="" src="https://static.licdn.com/sc/h/gse0tl7twp7z0fxtw0sr0txg" data-fastboot-src="/assets/i18n/support_nl_NL.js" crossorigin=""></script>
    <script src="https://static.licdn.com/sc/h/a3j8ctylze6rj0llcay1oty82"></script><script src="https://static.licdn.com/sc/h/509jd8gtpgo1wyjjko55vkyn1"></script><script src="https://static.licdn.com/sc/h/9q2lwmy4bj3o3bnsnvdbbxr0s"></script><script src="https://static.licdn.com/sc/h/ax2u4g85ovs8qdwk52xeima18"></script><script src="https://static.licdn.com/sc/h/c61zqgindr3pmda3ezj15ozm7"></script>
    <script src="https://static.licdn.com/sc/h/524t2ryyko20rgy5udl7f30am" data-fastboot-src="/assets/voyager-web.js" crossorigin=""></script>
    <div id="artdeco-modal-outlet"></div>

<code style="display: none" id="bpr-guid-6231299">
  {"data":{"entityUrn":"urn:li:collectionResponse:4HlijJjSiEXmgxSNTaZbyKnZ88db1eTefgOnimBx6wM=","elements":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719291,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719291,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48919_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48919","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48918_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48919","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"Het plaatsen van een gratis vacature is nu nog eenvoudiger","locale":"nl_NL","key":"i18n_form_header_jdai_v2_free_eligible","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.parent","displayName":"Child Test For WOW Page - Form - Header","description":"Child Test Test different headers on our form in the WOW page. Test efficiency messaging above the fold.\nFree Eligible Control: Post a job for free. Variants: 1) Posting a free job just became easier 2) Post a free job in minutes\nFree Ineligible Control: Post a job now. Variants: 1) Posting a job just became easier 2) Post a job in minutes","lixTreatment":"VAR_t48919_PR_1","parentLixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48919","creatorDisplayName":"cferraro","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4710852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4710852,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"mynetwork/templates/network-manager/people-follow/following","message":"","locale":"nl_NL","key":"i18n_following_empty_state_description","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"SH Positioning Experiment: Following List","description":"Our primary goal is to learn whether adding the secondary value props of engagement, social proof, creator similarity, and LinkedIn guidance to the headline in the Following List module for those following 0 members will increase the number of follow edges made by Career Accelerators.","lixTreatment":"control","lixKey":"chameleon.GROWTH_global.web-copy-definition.31854","creatorDisplayName":"mswoopes","status":"INITIAL_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718003,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718003,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32055","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_subheader_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_subheader_copy","description":"GTA subheader copy experiment.","lixTreatment":"control","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32055","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533282,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533282,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26210","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/templates/components/widgets/people-you-may-know","message":"","locale":"nl_NL","key":"i18n_onboarding_pymk_page_header_phase_3","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Onboarding PYMK CA and Concise Copy Test","description":"Test career accelerator confidence and concise related copy on PYMK onboarding widget on web.","lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26210","creatorDisplayName":"whchan","status":"CUSTOM_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4562276,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-details-engine/components/detail/job-view-layout","message":"Meer vacatures","locale":"nl_NL","key":"i18n_simjob_header","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"similar-jobs-to-more-jobs-job-details-engine-copy","description":"Ramp prior copy change for \"Similar Jobs\" to \"More jobs\"","lixTreatment":"VAR_t33653_PR_1","lixKey":"chameleon.CAREER-SEARCH-EVAL:GLOBAL.web-copy-definition.27904","creatorDisplayName":"rorodrig","status":"PERMANENT_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719292,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719292,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48920_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48920","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48918_PR_1","lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48920","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"job-posting/templates/index","message":"Het plaatsen van een vacature is nu nog eenvoudiger","locale":"nl_NL","key":"i18n_form_header_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"parentLixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.parent","displayName":"Child Test For WOW Page - Form - Header","description":"Child Test Test different headers on our form in the WOW page. Test efficiency messaging above the fold.\nFree Eligible Control: Post a job for free. Variants: 1) Posting a free job just became easier 2) Post a free job in minutes\nFree Ineligible Control: Post a job now. Variants: 1) Posting a job just became easier 2) Post a job in minutes","lixTreatment":"VAR_t48920_PR_1","parentLixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4719293,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixKey":"chameleon.OJ_GLOBAL.web-copy-definition.32199.child.48920","creatorDisplayName":"cferraro","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718002,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4718002,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48706_PR_1","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32054","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Maak connecties en blijf op de hoogte van uw professionele netwerk in de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_subheader_profile","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_subheader_copy_profile","description":"Customize GTA subheader copy.","lixTreatment":"VAR_t48706_PR_1","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32054","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718001,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718001,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Bekijk het volledige profiel van {:firstName} via de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_profile","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy_profile","description":"Customize the desktop GTA profile header copy.","lixTreatment":"VAR_t48704_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32052","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717859,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717859,"treatmentIndex":3,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48697_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32045","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Maak een connectie met recruiters in de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_jobs","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy_jobs","description":"Customize the GTA page header on desktop.","lixTreatment":"VAR_t48697_PR_3","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32045","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":2,"experimentId":4697659,"treatmentIndex":1,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"profile-creator-mode-shared/components/creator-tools-list/item","message":"Meer informatie","locale":"nl_NL","key":"i18n_available","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Profile Page - Creator Mode - Learn More - 8/14/2023","description":"Profile Page - Creator Mode - Learn More - 8/14/2023 ","lixTreatment":"VAR_t47452_PR_1","lixKey":"chameleon.PROFILE_GLOBAL.web-copy-definition.31506","creatorDisplayName":"jenli","status":"PERMANENT_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717999,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4717999,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48700_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32048","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Ontdek onderweg nieuwe bedrijven","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_company","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy_company","description":"Customize the GTA header copy for company joins.","lixTreatment":"VAR_t48700_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32048","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718000,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4718000,"treatmentIndex":2,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"VAR_t48701_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32049","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/components/widgets/get-the-app","message":"Geef uw zoektocht naar een baan een boost met de app","locale":"nl_NL","key":"i18n_onboarding_get_the_app_page_header_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"desktop_gta_header_copy","description":"Customize the GTA header copy.","lixTreatment":"VAR_t48701_PR_2","lixKey":"chameleon.ONBOARDING_global.web-copy-definition.32049","creatorDisplayName":"jesturgi","status":"MAX_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":3,"experimentId":4510388,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin-pages-entrypoint/components/card","message":"","locale":"nl_NL","key":"i18n_org_page_notifications","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Pages-left-rail-activity-copy","description":"Testing copy for the left rail landing page from page notifications to page activity (to mirror activity tab)","lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26519","creatorDisplayName":"nalberts","status":"INITIAL_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":5,"experimentId":4510389,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"organization-admin/components/admin/admin-nav","message":"","locale":"nl_NL","key":"i18n_org_activity_tab","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Pages-Activity-Tab-Copy-Test","description":"Testing different copy for the activity tab in Pages admin view ","lixTreatment":"control","lixKey":"chameleon.PAGES:Global.web-copy-definition.26520","creatorDisplayName":"nalberts","status":"INITIAL_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"},{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533279,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"chameleonConfigTrackingItem":{"configLixTrackingInfoListV2":[{"lixTracking":{"urn":"urn:li:member:1096125424","segmentIndex":4,"experimentId":4533279,"treatmentIndex":0,"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfo"},"lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26106","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigLixTrackingInfoWrapper"}],"$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigTrackingItem"},"data":{"namespace":"onboarding/templates/components/widgets/photo-upload","message":"","locale":"nl_NL","key":"i18n_onboarding_photo_page_header_v2","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigDataI18n"},"displayName":"Onboarding Add Photo Social Proof Copy Test","description":"Test social proof in onboarding photo upload on web.","lixTreatment":"control","lixKey":"chameleon.GROWTH:global.web-copy-definition.26106","creatorDisplayName":"whchan","status":"CUSTOM_RAMP","$type":"com.linkedin.voyager.dash.segments.chameleon.ChameleonConfigItem"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231299">
  {"request":"/voyager/api/voyagerSegmentsDashChameleonConfig","status":200,"body":"bpr-guid-6231299","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231299"><code style="display: none" id="bpr-guid-6231303">
  {"data":{"canBrowseProfiles":false,"reactivationFeaturesEligible":false,"canViewJobAnalytics":false,"canViewWVMP":false,"premiumFreeTrialEligible":true,"canAccessLearningVideos":false,"premiumLearningUpgradeEligible":false,"canViewCompanyInsights":false,"$type":"com.linkedin.voyager.premium.FeatureAccess"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231303">
  {"request":"/voyager/api/premium/featureAccess?name\u003DreactivationFeaturesEligible","status":200,"body":"bpr-guid-6231303","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231303"><code style="display: none" id="bpr-guid-6231304">
  {"data":{"plainId":1096125424,"publicContactInfo":{"$type":"com.linkedin.voyager.identity.shared.PublicContactInfo"},"premiumSubscriber":false,"*miniProfile":"urn:li:fs_miniProfile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","$type":"com.linkedin.voyager.common.Me"},"included":[{"customPronoun":null,"memorialized":false,"lastName":"Usmon","dashEntityUrn":"urn:li:fsd_profile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","standardizedPronoun":null,"occupation":"--","objectUrn":"urn:li:member:1096125424","backgroundImage":null,"picture":null,"$type":"com.linkedin.voyager.identity.shared.MiniProfile","firstName":"Fayyoz","entityUrn":"urn:li:fs_miniProfile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","publicIdentifier":"fayyoz-usmon-a1410b267","trackingId":"L+RhQkkbRHWM2PJg5WP/lg=="}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231304">
  {"request":"/voyager/api/me","status":200,"body":"bpr-guid-6231304","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231304"><code style="display: none" id="bpr-guid-6231320">
  {"data":{"entityUrn":"urn:li:collectionResponse:QQzXT8CXeExC567LZS8tvEQXw4LEVc6X26d/Bet8ySE=","elements":[],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"meta":{"microSchema":{"version":"2","types":{"com.linkedin.deco.recipe.anonymous.Anon1080314395":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon489758292":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"contentSeriesUrn":{"type":"string"},"contentSeries":{"type":"com.linkedin.deco.recipe.anonymous.Anon1789236903","resolvedFrom":"contentSeriesUrn"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToContentSeriesSubscribeMetadata"},"com.linkedin.deco.recipe.anonymous.Anon377984030":{"fields":{"professionalEvent":{"type":"com.linkedin.deco.recipe.anonymous.Anon1213723597","resolvedFrom":"professionalEventUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.voyager.dash.deco.common.image.Company":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction":{"fields":{"subscribed":{"type":"boolean"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"subscriberCount":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.publishing.SubscribeAction"},"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason":{"fields":{"sourceType":{"type":"string"},"reasonObjects":{"type":{"array":"string"}},"reasonContext":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.EdgeBuildingCohortReason"},"com.linkedin.deco.recipe.anonymous.Anon1444411496":{"fields":{"countryUrn":{"type":"string"},"country":{"type":"com.linkedin.deco.recipe.anonymous.Anon1918694303","resolvedFrom":"countryUrn"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon270389304":{"fields":{"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"productsAccessible":{"type":"boolean"},"croppedCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"lcpStaffingOrganization":{"type":"boolean"},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"lcpTreatment":{"type":"boolean"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"relevantFollowersForViewerAndOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon2053503342","isInjection":true},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"originalCoverImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"name":{"type":"string"},"lcpCustomer":{"type":"boolean"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1226238669":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon798864187":{"fields":{"influencer":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel":{"fields":{"actionDataModel":{"type":{"union":{"follow":"com.linkedin.voyager.dash.deco.feed.FollowingState","connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"actionDataModelUnion"},"actionDataModelUnion":{"type":{"union":{"connectionOrInvitation":"string","relationshipActionData":"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData","memberToEntityRelationship":"string","follow":"string"}}},"emphasisStyle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.StatefulButtonModel"},"com.linkedin.deco.recipe.anonymous.Anon1636615691":{"fields":{"transcripts":{"type":{"array":"string"}},"pagesPerResolution":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1525160909"}}},"baseType":"com.linkedin.documentcontent.DocumentPages"},"com.linkedin.deco.recipe.anonymous.Anon934606613":{"fields":{"type":{"type":"string"},"name":{"type":"string"},"controlName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipOption"},"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.deco.recipe.anonymous.Anon739258202","largeTemplate":"com.linkedin.deco.recipe.anonymous.Anon2003298771","smallTemplate":"com.linkedin.deco.recipe.anonymous.Anon548743587"}}},"stickerLinkTemplateSize":{"type":"string"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"string"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"string"},"urn":{"type":"string"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"text":{"type":"string"},"fourthCornerYOffsetPercentage":{"type":"float"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.deco.recipe.anonymous.Anon1956213100":{"fields":{"productCategories":{"type":{"array":"string"}},"signatureProduct":{"type":"boolean"},"localizedName":{"type":"string"},"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon570745354"},"resolvedFrom":"organizationsUsingProductUrns"},"entityUrn":{"type":"string"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1122581856","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"universalName":{"type":"string"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.deco.recipe.anonymous.Anon1312950162":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1792705474"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.OrganizationPermissions":{"fields":{"canCreateBroadcast":{"type":"boolean"},"canReadContentSuggestions":{"type":"boolean"},"canSeeProducts":{"type":"boolean"},"canUpdateOrganizationProfile":{"type":"boolean"},"canSponsorShare":{"type":"boolean"},"canEditPendingDirectSponsoredContentPosters":{"type":"boolean"},"canReadAdminFeedFollowingPill":{"type":"boolean"},"canUntagFromMention":{"type":"boolean"},"canReadOrganizationActivity":{"type":"boolean"},"canManageCareerPages":{"type":"boolean"},"canAssociateHashtag":{"type":"boolean"},"canDeleteShare":{"type":"boolean"},"canInviteMemberToFollow":{"type":"boolean"},"canEditPipelineBuilderAdministrators":{"type":"boolean"},"canReadBroadcastAnalytics":{"type":"boolean"},"canManageMessagingAccess":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canReadPipelineBuilderAdminPage":{"type":"boolean"},"canDeactivateOrganization":{"type":"boolean"},"canCreateOrganicShare":{"type":"boolean"},"canNotifyEmployeesOfShare":{"type":"boolean"},"canEditDarkShare":{"type":"boolean"},"canSeePostJobButton":{"type":"boolean"},"canEnableCommentsShare":{"type":"boolean"},"canReadOrganizationLeadsAnalytics":{"type":"boolean"},"canDeleteDarkShare":{"type":"boolean"},"canCreateJobPosting":{"type":"boolean"},"canEditLeadGenerationFormManagers":{"type":"boolean"},"canReadMessages":{"type":"boolean"},"canReadOrganizationTalentBrandAnalytics":{"type":"boolean"},"canEditDirectSponsoredContentPosters":{"type":"boolean"},"canPinShare":{"type":"boolean"},"canReadPipelineBuilderAdministrators":{"type":"boolean"},"canCreateLinkedInPagesProductFeedBack":{"type":"boolean"},"canReadEvents":{"type":"boolean"},"organizationRoles":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType"}},"canEditEvents":{"type":"boolean"},"canManageOrganizationalPageFollow":{"type":"boolean"},"canReadOrganizationUpdateAnalytics":{"type":"boolean"},"canCreateComment":{"type":"boolean"},"canRequestAdminAccess":{"type":"boolean"},"canEditAdministrators":{"type":"boolean"},"canCreateDarkShare":{"type":"boolean"},"canEmployeesInviteToFollow":{"type":"boolean"},"canSeeEmployeeExperienceAsMember":{"type":"boolean"},"canReadAdministrators":{"type":"boolean"},"canExportLeads":{"type":"boolean"},"canReadDirectSponsoredContentPosters":{"type":"boolean"},"canCreateShowcase":{"type":"boolean"},"canManageEmployeeExperienceSettings":{"type":"boolean"},"canReadLeadGenerationFormManagers":{"type":"boolean"},"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canEditCurators":{"type":"boolean"},"canEditProducts":{"type":"boolean"},"canManageVerifiedEmailDomains":{"type":"boolean"},"canReadPendingAdministrators":{"type":"boolean"},"canReadPendingDirectSponsoredContentPosters":{"type":"boolean"},"canEditPipelineBuilderAdminPage":{"type":"boolean"},"canReadTermsAndAgreements":{"type":"boolean"},"canReadOrganizationFollowerAnalytics":{"type":"boolean"},"canManageOrganizationCompetitors":{"type":"boolean"},"canSendMessages":{"type":"boolean"},"canCreateReaction":{"type":"boolean"},"canDisableCommentsShare":{"type":"boolean"},"canEditPendingAdministrators":{"type":"boolean"},"canNotifyEmployees":{"type":"boolean"},"canReadOrganizationPipelineBuilderAnalytics":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.voyager.dash.deco.common.video.Transcript":{"fields":{"captionFormat":{"type":"string"},"locale":{"type":"com.linkedin.voyager.dash.deco.common.Locale"},"lines":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.TranscriptLine"}},"isAutogenerated":{"type":"boolean"},"captionFile":{"type":"string"}},"baseType":"com.linkedin.videocontent.Transcript"},"com.linkedin.deco.recipe.anonymous.Anon1286825962":{"fields":{"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"pageType":{"type":"string"},"logoV2":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.deco.recipe.anonymous.Anon1358424483":{"fields":{"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextFieldComponent"},"com.linkedin.deco.recipe.anonymous.Anon632521550":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon523036483":{"fields":{"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.FollowersMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1949606970":{"fields":{"followerResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1313960689","resolvedFrom":"follower"},"followedAt":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"follower":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationFollower"},"com.linkedin.voyager.dash.deco.common.image.Group":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.deco.recipe.anonymous.Anon1918694303":{"fields":{"entityUrn":{"type":"string"},"defaultLocalizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow":{"fields":{"socialProofInsight":{"type":{"union":{"standard":"com.linkedin.deco.recipe.anonymous.Anon1350307109","compact":"com.linkedin.deco.recipe.anonymous.Anon853977300"}},"resolvedFrom":"socialProofInsightUnion"},"viewerPermissions":{"type":"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions"},"salesNavigatorCompanyUrl":{"type":"string"},"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryUrns":{"type":{"array":"string"}},"industryV2Urns":{"type":{"array":"string"}},"socialProofInsightUnion":{"type":{"union":{"standard":"string","compact":"string"}}},"employeeCount":{"type":"long"},"associatedSignatureProductUrn":{"type":"string"},"url":{"type":"string"},"callToAction":{"type":"com.linkedin.voyager.dash.deco.launchpad.CallToAction"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"associatedSignatureProduct":{"type":"com.linkedin.voyager.dash.deco.organization.FullMemberProduct","resolvedFrom":"associatedSignatureProductUrn"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1213723597":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.deco.recipe.anonymous.Anon184130218":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon487759616":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"schoolName":{"type":"string"},"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon1621960601","resolvedFrom":"schoolUrn"},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Education"},"com.linkedin.deco.recipe.anonymous.Anon1251747613":{"fields":{"urn":{"type":"string"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.voyager.dash.deco.organization.FullMemberProduct":{"fields":{"organizationsUsingProduct":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1432799903"},"resolvedFrom":"organizationsUsingProductUrns"},"productUserTitle":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon1369245809"},"resolvedFrom":"productUserTitleUrns"},"companyUrn":{"type":"string"},"productUserTitleUrns":{"type":{"array":"string"}},"similarProducts":{"type":"com.linkedin.deco.recipe.anonymous.Anon975174582","isInjection":true},"entityUrn":{"type":"string"},"organizationalPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1196575529","resolvedFrom":"organizationalPageUrn"},"logo":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"similarProductsFromSameOrganization":{"type":"com.linkedin.deco.recipe.anonymous.Anon1914510571","isInjection":true},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon270389304","resolvedFrom":"companyUrn"},"productCardCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon205248910","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"state":{"type":"string"},"universalName":{"type":"string"},"groupUrn":{"type":"string"},"localizedDescription":{"type":"string"},"localizedWebsite":{"type":"string"},"customizableCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon205248910","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon690575082","resolvedFrom":"groupUrn"},"hashtag":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon2070591469"},"resolvedFrom":"hashtagUrns"},"localizedName":{"type":"string"},"connectionsUsingProductProfiles":{"type":"com.linkedin.deco.recipe.anonymous.Anon1517647575","isInjection":true},"mediaSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon805140200"}},"productCategoriesResolutionResults":{"type":{"map":"com.linkedin.voyager.dash.deco.organization.FullProductCategory"},"resolvedFrom":"productCategories"},"hashtagUrns":{"type":{"array":"string"}},"productCategories":{"type":{"array":"string"}},"memberFacingCallToActionUnion":{"type":{"union":{"organizationProductLeadGenFormCallToAction":"com.linkedin.deco.recipe.anonymous.Anon205248910","organizationProductCallToAction":"com.linkedin.deco.recipe.anonymous.Anon625674184","organizationProductAddSkillCallToAction":"com.linkedin.deco.recipe.anonymous.Anon2115336743"}}},"standardizedSkillUrns":{"type":{"array":"string"}},"organizationalPageUrn":{"type":"string"},"autoPublished":{"type":"boolean"},"organizationsUsingProductUrns":{"type":{"array":"string"}},"standardizedSkill":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon503562015"},"resolvedFrom":"standardizedSkillUrns"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProduct"},"com.linkedin.voyager.dash.deco.launchpad.OrganizationPermissions":{"fields":{"canReadOrganizationVisitorAnalytics":{"type":"boolean"},"canCreateBroadcast":{"type":"boolean"},"canMembersInviteToFollow":{"type":"boolean"},"canSeeOrganizationAdministrativePage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationPermissions"},"com.linkedin.deco.recipe.anonymous.Anon1869367056":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection":{"fields":{"secondaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"secondaryAction"},"primaryActionResolutionResult":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"primaryAction"},"trackingIdentifier":{"type":"bytes"},"primaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"secondaryAction":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"overflowActionsV2":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"overflowActionsResolutionResults":{"type":{"map":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}}},"resolvedFrom":"overflowActions"},"overflowActions":{"type":{"array":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}}},"secondaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"},"trackingId":{"type":"string"},"primaryActionV2":{"type":"com.linkedin.deco.recipe.anonymous.Anon956797500"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActions"},"com.linkedin.deco.recipe.anonymous.Anon2127563472":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon514683669":{"fields":{"currentProgress":{"type":"int"},"threshold":{"type":"int"},"progressText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadProgressMeter"},"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.organization.FullProductCategory":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"entityUrn":{"type":"string"},"productCategoryPageUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.ProductCategory"},"com.linkedin.deco.recipe.anonymous.Anon1428941918":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon2116040434"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon4539795":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon2058972220"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.text.ProfileForMention":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution":{"fields":{"attribution":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon831664802":{"fields":{"entityUrn":{"type":"string"},"invitationId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon2070591469":{"fields":{"actionTarget":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon2127563472","isInjection":true},"displayName":{"type":"string"},"trackingUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.voyager.dash.deco.common.Industry":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Industry"},"com.linkedin.deco.recipe.anonymous.Anon912817537":{"fields":{"memberDistance":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1154789573":{"fields":{"maxResponseLength":{"type":"int"},"response":{"type":"string"},"validationType":{"type":"string"},"minResponseLength":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.TextInputComponent"},"com.linkedin.deco.recipe.anonymous.Anon8381687":{"fields":{"multipleChoiceQuestionOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1507465610"}},"placeholderText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.DropdownSelectComponent"},"com.linkedin.voyager.dash.deco.relationships.InvitationRaw":{"fields":{"genericInvitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon739258202":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream":{"fields":{"initialBitRate":{"type":"int"},"protocol":{"type":"string"},"mediaType":{"type":"string"},"masterPlaylists":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}}},"baseType":"com.linkedin.videocontent.AdaptiveStream"},"com.linkedin.deco.recipe.anonymous.Anon1529112441":{"fields":{"originalImageUrl":{"type":"string"},"originalHeight":{"type":"int"},"url":{"type":"string"},"originalWidth":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.voyager.dash.deco.common.video.StreamingLocation":{"fields":{"url":{"type":"string"},"expiresAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.StreamingLocation"},"com.linkedin.deco.recipe.anonymous.Anon1683511457":{"fields":{"requiredFieldMissingErrorText":{"type":"string"},"formComponentUnion":{"type":{"union":{"textFieldComponent":"com.linkedin.deco.recipe.anonymous.Anon1358424483","dropdownSelectComponent":"com.linkedin.deco.recipe.anonymous.Anon8381687","textInputComponent":"com.linkedin.deco.recipe.anonymous.Anon1154789573","checkboxComponent":"com.linkedin.deco.recipe.anonymous.Anon554507693"}}},"question":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"required":{"type":"boolean"},"editable":{"type":"boolean"},"leadGenFormQuestionUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormQuestion"},"com.linkedin.voyager.dash.deco.launchpad.LaunchpadView":{"fields":{"launchpadCards":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1121002957"}},"dismissDialog":{"type":"com.linkedin.deco.recipe.anonymous.Anon1670704461"},"headerImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"focusedCardIndex":{"type":"int"},"progressMeter":{"type":"com.linkedin.deco.recipe.anonymous.Anon514683669"},"branding":{"type":"com.linkedin.deco.recipe.anonymous.Anon1963883771"},"dismissible":{"type":"boolean"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"theme":{"type":"string"},"legoTrackingToken":{"type":"string"},"title":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadView"},"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata":{"fields":{"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl"},"progressiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata"}},"liveStreamCreatedAt":{"type":"long"},"prevMedia":{"type":"string"},"transcripts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Transcript"}},"aspectRatio":{"type":"float"},"media":{"type":"string"},"adaptiveStreams":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.AdaptiveStream"}},"liveStreamEndedAt":{"type":"long"},"duration":{"type":"long"},"provider":{"type":"string"},"entityUrn":{"type":"string"},"nextMedia":{"type":"string"},"thumbnails":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.Thumbnail"}},"trackingId":{"type":"string"}},"baseType":"com.linkedin.videocontent.VideoPlayMetadata"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground":{"fields":{"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon2058972220":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon412783727","invitation":"com.linkedin.deco.recipe.anonymous.Anon831664802"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon561948584":{"fields":{"ctaText":{"type":"string"},"landingPage":{"type":"com.linkedin.deco.recipe.anonymous.Anon1586568086"},"document":{"type":"com.linkedin.deco.recipe.anonymous.Anon856218111"},"confirmationTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"urlViewingBehavior":{"type":"string"},"sponsoredUrlAttributes":{"type":"com.linkedin.voyager.dash.deco.common.sponsored.SponsoredUrlAttributes"},"confirmationDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenGatedContent"},"com.linkedin.deco.recipe.anonymous.Anon783596108":{"fields":{"ctaColor":{"type":"string"},"ctaType":{"type":"string"},"ctaStyle":{"type":"string"},"presentationStyle":{"type":"string"},"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctaTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"deeplinkUrl":{"type":"string"},"ctaActionUnion":{"type":{"union":{"redirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","launchpadApiRequest":"com.linkedin.restli.common.EmptyRecord","launchpadApiRequestAndRedirect":"com.linkedin.voyager.dash.deco.launchpad.RedirectAction","shareBox":"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction"}}},"legoActionCategory":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCta"},"com.linkedin.voyager.dash.deco.relationships.Invitation":{"fields":{"inviteeMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"inviteeMember"},"invitationType":{"type":"string"},"inviteeMember":{"type":"string"},"entityUrn":{"type":"string"},"invitationState":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.voyager.dash.deco.common.IndustryV2":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.IndustryV2"},"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow":{"fields":{"lastName":{"type":"string"},"profileTopEducation":{"type":"com.linkedin.deco.recipe.anonymous.Anon964860152","isInjection":true},"profilePositions":{"type":"com.linkedin.deco.recipe.anonymous.Anon1861880920","isInjection":true},"educationOnProfileTopCardShown":{"type":"boolean"},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1449981882","isInjection":true},"associatedHashtagsCopy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"firstName":{"type":"string"},"profileInsight":{"type":"com.linkedin.deco.recipe.anonymous.Anon1439603308","isInjection":true},"geoLocation":{"type":"com.linkedin.deco.recipe.anonymous.Anon708807917"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.deco.recipe.anonymous.Anon89639392","isInjection":true},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"companyNameOnProfileTopCardShown":{"type":"boolean"},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1069349607":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon912817537"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.deco.recipe.anonymous.Anon205248910":{"fields":{"dashLeadGenFormUrn":{"type":"string"},"privacyPolicyUrl":{"type":"string"},"dashLeadGenForm":{"type":"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm","resolvedFrom":"dashLeadGenFormUrn"},"organizationProductCallToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon625674184"},"leadGenFormUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductLeadGenFormCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon554507693":{"fields":{"checked":{"type":"boolean"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.CheckboxComponent"},"com.linkedin.deco.recipe.anonymous.Anon1128782547":{"fields":{"entityUrn":{"type":"string"},"joinedAt":{"type":"long"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon437558093","resolvedFrom":"profileUrn"},"groupUrn":{"type":"string"},"preDashEntityUrn":{"type":"string"},"profileUrn":{"type":"string"},"status":{"type":"string"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.GroupMembership"},"com.linkedin.deco.recipe.anonymous.Anon1509374374":{"fields":{"displayText":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.GenericCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon503562015":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.StandardizedSkill"},"com.linkedin.deco.recipe.anonymous.Anon708807917":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1444411496","resolvedFrom":"geoUrn"},"geoUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"frameType":{"type":"string"},"a11yText":{"type":"string"},"displayImageUrn":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.deco.recipe.anonymous.Anon858976209":{"fields":{"type":{"type":"string"},"epochAt":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.deco.recipe.anonymous.Anon1517647575":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1703899306"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction":{"fields":{"deeplinkUrl":{"type":"string"},"recruiterContext":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ViewProfileInExternalContextAction"},"com.linkedin.deco.recipe.anonymous.Anon1843097075":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon1286825962"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon89639392":{"fields":{"followerCount":{"type":"long"},"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.voyager.dash.deco.common.video.TranscriptLine":{"fields":{"lineStartAt":{"type":"long"},"caption":{"type":"string"},"lineEndAt":{"type":"long"}},"baseType":"com.linkedin.videocontent.TranscriptLine"},"com.linkedin.voyager.dash.deco.launchpad.CallToAction":{"fields":{"displayText":{"type":"string"},"visible":{"type":"boolean"},"type":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.CallToAction"},"com.linkedin.deco.recipe.anonymous.Anon625674184":{"fields":{"type":{"type":"string"},"callToAction":{"type":"com.linkedin.deco.recipe.anonymous.Anon1509374374"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductCallToAction"},"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage":{"fields":{"entityUrn":{"type":"string"},"localizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.deco.recipe.anonymous.Anon1432799903":{"fields":{"industryV2Taxonomy":{"type":{"map":"com.linkedin.voyager.dash.deco.common.IndustryV2"},"resolvedFrom":"industryV2TaxonomyUrns"},"industry":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryUrns"},"industryV2Urns":{"type":{"array":"string"}},"industryUrns":{"type":{"array":"string"}},"url":{"type":"string"},"relevanceReason":{"type":"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel"},"industryV2":{"type":{"map":"com.linkedin.voyager.dash.deco.common.Industry"},"resolvedFrom":"industryV2Urns"},"industryV2TaxonomyUrns":{"type":{"array":"string"}},"followingStateUrn":{"type":"string"},"entityUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"},"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"universalName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon1031101697":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"invitationRelationshipForm":{"type":"com.linkedin.deco.recipe.anonymous.Anon918376989"},"inviterResolutionResult":{"type":"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon1271139837":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.deco.recipe.anonymous.Anon1313960689":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"insight":{"type":"com.linkedin.deco.recipe.anonymous.Anon137428918","isInjection":true},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"profileTopPosition":{"type":"com.linkedin.deco.recipe.anonymous.Anon1428941918","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"backgroundPicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyBackground"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon163061530":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.relationships.profile":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"creator":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon378816379":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon660720507","resolvedFrom":"companyUrn"},"companyUrn":{"type":"string"},"viewerUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToCompanyFollowMetadata"},"com.linkedin.deco.recipe.anonymous.Anon743120773":{"fields":{"authorProfileId":{"type":"string"},"targetUrn":{"type":"string"},"contentSource":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.semaphore.SemaphoreContext"},"com.linkedin.deco.recipe.anonymous.Anon805140200":{"fields":{"mediaContentUnion":{"type":{"union":{"embeddedVideo":"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec","url":"string","videoPlayMetadata":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"description":{"type":"string"},"mediaEntityUnion":{"type":{"union":{"digitalMediaAssetUrn":"string","articleUrn":"string"}}},"title":{"type":"string"},"mediaSource":{"type":"string"},"primary":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.MediaSection"},"com.linkedin.deco.recipe.anonymous.Anon964860152":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon487759616"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.NoConnection":{"fields":{"memberDistance":{"type":"string"},"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1575907873":{"fields":{"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeNavigationContext":{"type":"com.linkedin.deco.recipe.anonymous.Anon199392175"},"displayText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"textStartIcon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"composeOptionType":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeOption"},"com.linkedin.deco.recipe.anonymous.Anon40833127":{"fields":{"metadataUnion":{"type":{"union":{"memberToContentSeriesSubscribe":"com.linkedin.deco.recipe.anonymous.Anon489758292"}}},"subscribeActionUrn":{"type":"string"},"subscribeAction":{"type":"com.linkedin.voyager.dash.deco.feed.publishing.SubscribeAction","resolvedFrom":"subscribeActionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.SubscribeRelationship"},"com.linkedin.deco.recipe.anonymous.Anon2115336743":{"fields":{"displayText":{"type":"string"},"toggledDisplayText":{"type":"string"},"enabled":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationProductAddSkillCallToAction"},"com.linkedin.deco.recipe.anonymous.Anon336652209":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"tempStatusExpiredAtUnion":{"type":{"union":{"customizedExpiredAt":"long","standardizedExpiration":"string"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.VectorArtifact":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"long"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.voyager.dash.deco.common.text.School":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon2066892616":{"fields":{"invitationType":{"type":"string"},"invitationId":{"type":"long"},"sharedSecret":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.Invitation"},"com.linkedin.deco.recipe.anonymous.Anon1350307109":{"fields":{"insightViewee":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"insightVieweeUrn"},"insightVieweeUrn":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon798864187","resolvedFrom":"sharedConnectionDetailTarget"},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.launchpad.PageKey":{"fields":{"pageKey":{"type":"string"},"anchorPage":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.tracking.PageKey"},"com.linkedin.deco.recipe.anonymous.Anon430310092":{"fields":{"entityUrn":{"type":"string"},"creator":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1586568086":{"fields":{"url":{"type":"string"},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Link"},"com.linkedin.deco.recipe.anonymous.Anon1976702492":{"fields":{"iweWarned":{"type":"boolean"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.launchpad.ShareBoxAction":{"fields":{"shareMessage":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.ShareBoxAction"},"com.linkedin.deco.recipe.anonymous.Anon1439603308":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1122468175"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon437558093":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.deco.recipe.anonymous.Anon1069349607","isInjection":true},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1507465610":{"fields":{"type":{"type":"string"},"textOption":{"type":"string"},"selected":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.MultipleChoiceQuestionOptions"},"com.linkedin.deco.recipe.anonymous.Anon409680083":{"fields":{"type":{"type":"string"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.deco.recipe.anonymous.Anon1792705474":{"fields":{"invitationUnion":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon881636240","invitation":"com.linkedin.deco.recipe.anonymous.Anon2066892616"}}}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.deco.recipe.anonymous.Anon1369245809":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.StandardizedTitle"},"com.linkedin.deco.recipe.anonymous.Anon412783727":{"fields":{"targetInviteeResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon352881287","resolvedFrom":"targetInvitee"},"inviter":{"type":"string"},"targetInvitee":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon1976702492","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.deco.recipe.anonymous.Anon533924488":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"sharedConnectionDetailTarget":{"type":"string"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"},"sharedConnectionDetailTargetResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"sharedConnectionDetailTarget"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.deco.recipe.anonymous.Anon918376989":{"fields":{"title":{"type":"string"},"invitationRelationshipOptions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon934606613"}},"subtitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.InvitationRelationshipForm"},"com.linkedin.voyager.dash.deco.common.Locale":{"fields":{"variant":{"type":"string"},"country":{"type":"string"},"language":{"type":"string"}},"baseType":"com.linkedin.common.Locale"},"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrl":{"fields":{"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.voyager.dash.deco.common.video.Resolution":{"fields":{"width":{"type":"int"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.Resolution"},"com.linkedin.deco.recipe.anonymous.Anon853977300":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName":{"fields":{"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.organization.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1525160909":{"fields":{"width":{"type":"int"},"imageUrls":{"type":{"array":"string"}},"height":{"type":"int"}},"baseType":"com.linkedin.documentcontent.DocumentResolutionPages"},"com.linkedin.voyager.dash.deco.common.Link":{"fields":{"type":{"type":"string"},"rel":{"type":"string"},"href":{"type":"string"}},"baseType":"com.linkedin.restli.common.Link"},"com.linkedin.voyager.dash.deco.common.ux.baseInsightViewModel":{"fields":{"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.voyager.dash.deco.common.image.School":{"fields":{"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.voyager.dash.deco.common.VectorImage":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"string"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.voyager.dash.deco.common.VectorArtifact"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.deco.recipe.anonymous.Anon881636240":{"fields":{"inviter":{"type":"string"},"inviterResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"inviter"}},"baseType":"com.linkedin.voyager.dash.relationships.invitation.NoInvitation"},"com.linkedin.voyager.dash.deco.relationships.Connection":{"fields":{"createdAt":{"type":"long"},"connectedMemberResolutionResult":{"type":"com.linkedin.deco.recipe.anonymous.Anon336652209","resolvedFrom":"connectedMember"},"connectedMember":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.voyager.dash.deco.organization.MiniCompany":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon198476569":{"fields":{"viewer":{"type":"com.linkedin.deco.recipe.anonymous.Anon430310092","resolvedFrom":"viewerUrn"},"vieweeFollowingViewer":{"type":"boolean"},"viewerUrn":{"type":"string"},"viewee":{"type":"com.linkedin.voyager.dash.deco.relationships.profile","resolvedFrom":"vieweeUrn"},"vieweeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberToMemberFollowMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1703899306":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfilePhotoDecoSpecReadOnlyProfilePicture"},"lastName":{"type":"string"},"firstName":{"type":"string"},"profileProfileActions":{"type":"com.linkedin.voyager.dash.deco.identity.profile.ProfileActionsInjection","isInjection":true},"entityUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2ForInjection","isInjection":true},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon2003298771":{"fields":{"nameSupplementaryInfo":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"footerText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"subHeadline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"insightText":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"headline":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"backgroundImageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.voyager.dash.deco.feed.FollowingState":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"},"followingType":{"type":"string"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon1914510571":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship":{"fields":{"invitationUrn":{"type":"string"},"entityUrn":{"type":"string"},"invitation":{"type":"com.linkedin.voyager.dash.deco.relationships.InvitationRaw","resolvedFrom":"invitationUrn"},"relationshipDataUnion":{"type":{"union":{"follow":"com.linkedin.deco.recipe.anonymous.Anon191738356","subscribe":"com.linkedin.deco.recipe.anonymous.Anon40833127"}}}},"baseType":"com.linkedin.voyager.dash.relationships.DirectionalEntityRelationship"},"com.linkedin.deco.recipe.anonymous.Anon1568806612":{"fields":{"school":{"type":"com.linkedin.deco.recipe.anonymous.Anon929195650","resolvedFrom":"schoolUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.deco.recipe.anonymous.Anon1449981882":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1566936928"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon660720507":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithIweWarned":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"iweWarned":{"type":"boolean"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.image.ImageViewModel":{"fields":{"attributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.image.ImageAttribute"}},"actionTarget":{"type":"string"},"totalCount":{"type":"int"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.deco.recipe.anonymous.Anon648914460":{"fields":{"companyUrn":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1927941263","resolvedFrom":"companyUrn"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.deco.recipe.anonymous.Anon1042323916":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.Hashtag":{"fields":{"entityUrn":{"type":"string"},"trackingUrn":{"type":"string"},"actionTarget":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.voyager.dash.deco.common.FullPaging":{"fields":{"start":{"type":"int"},"count":{"type":"int"},"total":{"type":"int"},"links":{"type":{"array":"com.linkedin.voyager.dash.deco.common.Link"}}},"baseType":"com.linkedin.restli.common.CollectionMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1122468175":{"fields":{"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"text":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"navigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Insight"},"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery":{"fields":{"geo":{"type":"com.linkedin.deco.recipe.anonymous.Anon1271139837","resolvedFrom":"geoUrn"},"keywords":{"type":"string"},"geoUrn":{"type":"string"},"savedSearchId":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.launchpad.attribute.JobSearchQuery"},"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent":{"fields":{"logoImage":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.voyager.dash.deco.common.video.Thumbnail":{"fields":{"resolution":{"type":"com.linkedin.voyager.dash.deco.common.video.Resolution"},"url":{"type":"string"}},"baseType":"com.linkedin.videocontent.Thumbnail"},"com.linkedin.voyager.dash.deco.common.DateRange":{"fields":{"start":{"type":"com.linkedin.voyager.dash.deco.common.Date"},"end":{"type":"com.linkedin.voyager.dash.deco.common.Date"}},"baseType":"com.linkedin.common.DateRange"},"com.linkedin.deco.recipe.anonymous.Anon301818032":{"fields":{"entityImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"subtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"subDescription":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"description":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"contentHorizontallyCentered":{"type":"boolean"},"title":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenBannerComponent"},"com.linkedin.voyager.dash.deco.organization.EmbeddedVideoDecoSpec":{"fields":{"url":{"type":"string"},"thumbnail":{"type":"com.linkedin.voyager.dash.deco.common.VectorImage"}},"baseType":"com.linkedin.voyager.dash.organization.EmbeddedVideo"},"com.linkedin.deco.recipe.anonymous.Anon1670704461":{"fields":{"title":{"type":"string"},"subTitle":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadDismissDialog"},"com.linkedin.voyager.dash.deco.common.Date":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.voyager.dash.deco.common.video.ProgressiveDownloadMetadata":{"fields":{"streamingLocations":{"type":{"array":"com.linkedin.voyager.dash.deco.common.video.StreamingLocation"}},"size":{"type":"long"},"bitRate":{"type":"int"},"width":{"type":"int"},"mediaType":{"type":"string"},"mimeType":{"type":"string"},"height":{"type":"int"}},"baseType":"com.linkedin.videocontent.ProgressiveDownloadMetadata"},"com.linkedin.deco.recipe.anonymous.Anon1196575529":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationalPage"},"com.linkedin.voyager.dash.deco.organization.OrganizationRoleType":{"fields":{"localizedName":{"type":"string"},"role":{"type":"string"},"paidMediaRole":{"type":"boolean"},"localizedDescription":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.OrganizationRoleType"},"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2":{"fields":{"textDirection":{"type":"string"},"text":{"type":"string"},"attributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.deco.recipe.anonymous.Anon1789236903":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.publishing.ContentSeries"},"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2":{"fields":{"memberRelationshipData":{"type":{"union":{"noInvitation":"com.linkedin.deco.recipe.anonymous.Anon1031101697","invitation":"com.linkedin.voyager.dash.deco.relationships.Invitation","connection":"com.linkedin.voyager.dash.deco.relationships.Connection"}}},"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.voyager.dash.deco.relationships.Connection","noConnection":"com.linkedin.voyager.dash.deco.relationships.NoConnection"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.sponsoredcontent.LeadGenForm":{"fields":{"formSections":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1025867760"}},"submitted":{"type":"boolean"},"entityUrn":{"type":"string"},"privacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"postSubmissionContent":{"type":"com.linkedin.deco.recipe.anonymous.Anon561948584"},"lastSubmittedAt":{"type":"long"},"trackingMetadata":{"type":"string"},"header":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"banner":{"type":"com.linkedin.deco.recipe.anonymous.Anon301818032"},"submitButtonText":{"type":"string"},"customPrivacyPolicy":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"consentSection":{"type":"com.linkedin.deco.recipe.anonymous.Anon1025867760"}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenForm"},"com.linkedin.voyager.dash.deco.relationships.ProfileWithEmailRequired":{"fields":{"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"entityUrn":{"type":"string"},"tempStatusEmoji":{"type":"string"},"publicIdentifier":{"type":"string"},"headline":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1975396536":{"fields":{"saved":{"type":"boolean"},"salesListNavigationUrl":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.SalesNavigatorSaveState"},"com.linkedin.deco.recipe.anonymous.Anon929195650":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon1299693827":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon352881287":{"fields":{"entityUrn":{"type":"string"},"emailRequired":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1760283115":{"fields":{"entityUrn":{"type":"string"},"preDashFollowingInfoUrn":{"type":"string"},"following":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.deco.recipe.anonymous.Anon690575082":{"fields":{"viewerGroupMembership":{"type":"com.linkedin.deco.recipe.anonymous.Anon632521550","resolvedFrom":"viewerGroupMembershipUrn"},"displayNotificationSubscriptionLevelOptions":{"type":"boolean"},"groupPostNotificationsEdgeSetting":{"type":"com.linkedin.deco.recipe.anonymous.Anon735489647","resolvedFrom":"groupPostNotificationsEdgeSettingUrn"},"groupPostNotificationsEdgeSettingUrn":{"type":"string"},"postApprovalEnabled":{"type":"boolean"},"backendGroupUrn":{"type":"string"},"memberCount":{"type":"int"},"owners":{"type":"com.linkedin.deco.recipe.anonymous.Anon1299693827","isInjection":true},"memberConnectionsFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"memberConnections":{"type":"com.linkedin.deco.recipe.anonymous.Anon721197842","isInjection":true},"membersFacePile":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"directJoinEnabled":{"type":"boolean"},"entityUrn":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"globalNewPostNotificationSettingOn":{"type":"boolean"},"viewerGroupMembershipUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.voyager.dash.deco.common.text.LearningCourse":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.deco.recipe.anonymous.Anon50101142":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1320789737"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"profile":{"type":"com.linkedin.deco.recipe.anonymous.Anon587401631","resolvedFrom":"profileUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.deco.recipe.anonymous.Anon1566936928":{"fields":{"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon1226238669","resolvedFrom":"companyUrn"},"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1122581856":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon1585981083"},"elements":{"type":{"array":"com.linkedin.voyager.dash.deco.organization.Profile"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.image.ImageAttribute":{"fields":{"detailData":{"type":{"union":{"profilePictureWithoutFrame":"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame","profilePicture":"com.linkedin.voyager.dash.deco.common.image.Profile","profilePictureWithRingStatus":"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus","companyLogo":"com.linkedin.voyager.dash.deco.common.image.Company","professionalEventLogo":"com.linkedin.voyager.dash.deco.common.image.ProfessionalEvent","organizationalPageLogo":"com.linkedin.voyager.dash.deco.common.image.OrganizationalPage","schoolLogo":"com.linkedin.voyager.dash.deco.common.image.School","groupLogo":"com.linkedin.voyager.dash.deco.common.image.Group"}},"resolvedFrom":"detailDataUnion"},"tintColor":{"type":"string"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"string","systemImage":"string","nonEntityGroupLogo":"com.linkedin.deco.recipe.anonymous.Anon1436383648","organizationalPageLogo":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution","nonEntityProfessionalEventLogo":"com.linkedin.deco.recipe.anonymous.Anon377984030","profilePicture":"string","imageUrl":"com.linkedin.deco.recipe.anonymous.Anon1529112441","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.deco.recipe.anonymous.Anon1568806612","nonEntityCompanyLogo":"com.linkedin.deco.recipe.anonymous.Anon648914460","schoolLogo":"string","groupLogo":"string","ghostImage":"string","nonEntityProfilePicture":"com.linkedin.deco.recipe.anonymous.Anon50101142"}}},"tapTargets":{"type":{"array":"com.linkedin.voyager.dash.deco.common.TapTargetWithoutEntity"}},"scalingType":{"type":"string"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.deco.recipe.anonymous.Anon2053503342":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"metadata":{"type":"com.linkedin.deco.recipe.anonymous.Anon523036483"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1949606970"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithRingStatus":{"fields":{"ringStatus":{"type":"com.linkedin.deco.recipe.anonymous.Anon1080314395","isInjection":true},"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"a11yText":{"type":"string"},"displayImageReference":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.voyager.dash.deco.common.text.Company":{"fields":{"name":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.deco.recipe.anonymous.Anon462345189":{"fields":{"controlName":{"type":"string"},"viewingBehavior":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.deco.recipe.anonymous.Anon956797500":{"fields":{"profileActionUnion":{"type":{"union":{"personalizedConnect":"string","saveToPdf":"string","shareProfileUrlViaMessage":"string","saveToPdfUrl":"string","composeOption":"string","saveInSalesNavigatorState":"string","statefulAction":"com.linkedin.voyager.dash.deco.common.ux.StatefulButtonModel","viewProfileInExternalContext":"com.linkedin.voyager.dash.deco.identity.profile.ViewProfileInExternalContextAction","statelessAction":"string","followingState":"string","report":"com.linkedin.deco.recipe.anonymous.Anon743120773","ignore":"string","shareProfileUrl":"string","connection":"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper","withdraw":"string"}}},"profileAction":{"type":{"union":{"personalizedConnect":"com.linkedin.deco.recipe.anonymous.Anon4539795","saveToPdf":"com.linkedin.deco.recipe.anonymous.Anon587401631","followingState":"com.linkedin.deco.recipe.anonymous.Anon1760283115","ignore":"com.linkedin.deco.recipe.anonymous.Anon1312950162","composeOption":"com.linkedin.deco.recipe.anonymous.Anon1575907873","withdraw":"com.linkedin.deco.recipe.anonymous.Anon1213132949","saveInSalesNavigatorState":"com.linkedin.deco.recipe.anonymous.Anon1975396536"}},"resolvedFrom":"profileActionUnion"},"trackingId":{"type":"bytes"}},"baseType":"com.linkedin.voyager.dash.identity.profile.actions.ProfileActionInfo"},"com.linkedin.deco.recipe.anonymous.Anon1121002957":{"fields":{"backgroundImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"cardType":{"type":"string"},"icon":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"pageKey":{"type":"com.linkedin.voyager.dash.deco.launchpad.PageKey"},"completed":{"type":"boolean"},"legoTrackingToken":{"type":"string"},"cardTitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"ctas":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon783596108"}},"animate":{"type":"boolean"},"videoMetadata":{"type":"com.linkedin.voyager.dash.deco.common.video.VideoPlayMetadata"},"cardLabel":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"attributeUnion":{"type":{"union":{"followOrganization":"string","launchpad":"string","jobSearchQuery":"com.linkedin.voyager.dash.deco.launchpad.JobSearchQuery","companyJYMBII":"string","generalJYMBII":{"array":"string"},"viewExperienceEntity":"string","followPostCreator":"string","fifWidgetGroupId":"string","connectWithEntity":"string","entityActivityFeed":"string","edgeBuilding":{"array":"com.linkedin.voyager.dash.deco.launchpad.EdgeBuildingCohortReason"},"jobAlertJYMBII":{"array":"string"},"organizationActivityFeed":"string"}}},"insightImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"overlayImage":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"},"attribute":{"type":{"union":{"followOrganization":"com.linkedin.voyager.dash.deco.launchpad.OrganizationTopCardForEntityFollow","companyJYMBII":"com.linkedin.deco.recipe.anonymous.Anon184130218","generalJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"connectWithEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","viewExperienceEntity":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","entityActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon587401631","followPostCreator":"com.linkedin.voyager.dash.deco.launchpad.ProfileTopCardForEntityFollow","jobAlertJYMBII":{"map":"com.linkedin.deco.recipe.anonymous.Anon1042323916"},"organizationActivityFeed":"com.linkedin.deco.recipe.anonymous.Anon1927941263"}},"resolvedFrom":"attributeUnion"},"cardSubtitle":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"cardStyle":{"type":"string"},"entitySize":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadCard"},"com.linkedin.deco.recipe.anonymous.Anon735489647":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.notifications.edgesetting.EdgeSetting"},"com.linkedin.deco.recipe.anonymous.Anon1927941263":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.image.ProfileWithoutFrame":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon587401631":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.deco.recipe.anonymous.Anon1621960601":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}},"entityUrn":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.deco.recipe.anonymous.Anon199392175":{"fields":{"paidInMail":{"type":"boolean"},"recipient":{"type":{"map":"com.linkedin.deco.recipe.anonymous.Anon587401631"},"resolvedFrom":"recipientUrns"},"messageRequestContextUrn":{"type":"string"},"recipientUrns":{"type":{"array":"string"}},"targetUrl":{"type":"string"},"existingConversation":{"type":"com.linkedin.deco.recipe.anonymous.Anon807183080","resolvedFrom":"existingConversationUrn"},"existingConversationUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.compose.ComposeNavigationContext"},"com.linkedin.deco.recipe.anonymous.Anon1320789737":{"fields":{"ringContentType":{"type":"string"},"actionTarget":{"type":"string"},"preDashEntityUrn":{"type":"string"},"entityUrn":{"type":"string"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.deco.recipe.anonymous.Anon721197842":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1128782547"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1085906135":{"fields":{"multiLocaleCompanyName":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"dateRange":{"type":"com.linkedin.voyager.dash.deco.common.DateRange"},"companyName":{"type":"string"},"company":{"type":"com.linkedin.deco.recipe.anonymous.Anon184130218","resolvedFrom":"companyUrn"},"title":{"type":"string"},"companyUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon1963883771":{"fields":{"accentType":{"type":"string"},"image":{"type":"com.linkedin.voyager.dash.deco.common.image.ImageViewModel"}},"baseType":"com.linkedin.voyager.dash.launchpad.LaunchpadBranding"},"com.linkedin.deco.recipe.anonymous.Anon807183080":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.messaging.Conversation"},"com.linkedin.deco.recipe.anonymous.Anon570745354":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImage"}}},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.voyager.dash.deco.common.image.Profile":{"fields":{"profilePicture":{"type":"com.linkedin.voyager.dash.deco.common.image.PhotoFilterPicture"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.voyager.dash.deco.common.text.JobPosting":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.voyager.dash.deco.common.text.TextAttributeV2":{"fields":{"start":{"type":"int"},"length":{"type":"int"},"detailData":{"type":{"union":{"jobPostingName":"com.linkedin.voyager.dash.deco.common.text.JobPosting","profileFamiliarName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFamiliarName","groupName":"com.linkedin.deco.recipe.anonymous.Anon163061530","profileFullName":"com.linkedin.voyager.dash.deco.common.text.ProfileForFullName","learningCourseName":"com.linkedin.voyager.dash.deco.common.text.LearningCourse","companyName":"com.linkedin.voyager.dash.deco.common.text.Company","profileMention":"com.linkedin.voyager.dash.deco.common.text.ProfileForMention","schoolName":"com.linkedin.voyager.dash.deco.common.text.School","organizationalPageName":"com.linkedin.voyager.dash.deco.common.text.OrganizationalPage","hashtag":"com.linkedin.voyager.dash.deco.common.text.Hashtag"}},"resolvedFrom":"detailDataUnion"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"string","color":"string","companyName":"string","icon":"string","systemImage":"string","epoch":"com.linkedin.deco.recipe.anonymous.Anon858976209","organizationalPageName":"string","textLink":"com.linkedin.deco.recipe.anonymous.Anon462345189","listItemStyle":"com.linkedin.deco.recipe.anonymous.Anon409680083","hyperlinkOpenExternally":"string","listStyle":"string","groupName":"string","profileFullName":"string","stringFieldReference":"com.linkedin.deco.recipe.anonymous.Anon1251747613","learningCourseName":"string","profileMention":"string","style":"string","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.deco.recipe.anonymous.Anon975174582":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1956213100"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1213132949":{"fields":{"entityUrn":{"type":"string"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.restli.common.EmptyRecord","connection":"com.linkedin.deco.recipe.anonymous.Anon2134644859","noConnection":"com.linkedin.deco.recipe.anonymous.Anon1843097075"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.voyager.dash.deco.common.ux.RelationshipActionData":{"fields":{"relationshipsTrackingId":{"type":"bytes"},"relationshipData":{"type":{"union":{"connectionOrInvitation":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","memberToEntityRelationship":"com.linkedin.voyager.dash.deco.relationships.DirectionalEntityRelationship"}},"resolvedFrom":"relationshipDataUnion"},"relationshipDataUnion":{"type":{"union":{"connectionOrInvitation":"string","memberToEntityRelationship":"string"}}},"moduleKey":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.relationships.RelationshipActionData"},"com.linkedin.voyager.dash.deco.identity.profile.MemberRelationshipWrapper":{"fields":{"memberRelationshipUrn":{"type":"string"},"memberRelationship":{"type":"com.linkedin.voyager.dash.deco.relationships.MemberRelationshipV2","resolvedFrom":"memberRelationshipUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationshipWrapper"},"com.linkedin.deco.recipe.anonymous.Anon137428918":{"fields":{"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon533924488"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon191738356":{"fields":{"metadataUnion":{"type":{"union":{"memberToMemberFollow":"com.linkedin.deco.recipe.anonymous.Anon198476569","memberToCompanyFollow":"com.linkedin.deco.recipe.anonymous.Anon378816379"}}},"followingStateUrn":{"type":"string"},"followingState":{"type":"com.linkedin.voyager.dash.deco.feed.FollowingState","resolvedFrom":"followingStateUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.FollowRelationship"},"com.linkedin.voyager.dash.deco.launchpad.RedirectAction":{"fields":{"presentationStyle":{"type":"string"},"url":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.launchpad.actionrecommendation.RedirectAction"},"com.linkedin.deco.recipe.anonymous.Anon2116040434":{"fields":{"company":{"type":"com.linkedin.voyager.dash.deco.organization.MiniCompany","resolvedFrom":"companyUrn"},"title":{"type":"string"},"multiLocaleTitle":{"type":{"map":"string"}},"entityUrn":{"type":"string"},"companyUrn":{"type":"string"},"companyName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Position"},"com.linkedin.deco.recipe.anonymous.Anon856218111":{"fields":{"transcribedDocumentUrl":{"type":"string"},"urn":{"type":"string"},"manifestUrl":{"type":"string"},"scanRequiredForDownload":{"type":"boolean"},"totalPageCount":{"type":"int"},"manifestUrlExpiresAt":{"type":"long"},"title":{"type":"string"},"coverPages":{"type":"com.linkedin.deco.recipe.anonymous.Anon1636615691"},"transcribedDocumentUrlExpiresAt":{"type":"long"}},"baseType":"com.linkedin.documentcontent.Document"},"com.linkedin.deco.recipe.anonymous.Anon1025867760":{"fields":{"questions":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1683511457"}}},"baseType":"com.linkedin.voyager.dash.sponsoredcontent.leadgen.LeadGenFormSection"},"com.linkedin.deco.recipe.anonymous.Anon1436383648":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"},"group":{"type":"com.linkedin.deco.recipe.anonymous.Anon1869367056","resolvedFrom":"groupUrn"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.deco.recipe.anonymous.Anon548743587":{"fields":{"name":{"type":"com.linkedin.voyager.dash.deco.common.text.TextViewModelV2"},"imageUnion":{"type":{"union":{"url":"string","vectorImage":"com.linkedin.voyager.dash.deco.common.VectorImageOnlyRootUrlAndAttribution"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.deco.recipe.anonymous.Anon1861880920":{"fields":{"paging":{"type":"com.linkedin.voyager.dash.deco.common.FullPaging"},"elements":{"type":{"array":"com.linkedin.deco.recipe.anonymous.Anon1085906135"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.deco.recipe.anonymous.Anon1585981083":{"fields":{"title":{"type":"string"},"description":{"type":"string"},"connectionsUsingProductTypeUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ConnectionsUsingProductMetadata"},"com.linkedin.deco.recipe.anonymous.Anon2134644859":{"fields":{"entityUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"}}}},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231320">
  {"request":"/voyager/api/voyagerLaunchpadDashLaunchpadViews?decorationId\u003Dcom.linkedin.voyager.dash.deco.launchpad.LaunchpadView-69\u0026launchpadContext\u003DTAKEOVER\u0026q\u003Dcontext","status":200,"body":"bpr-guid-6231320","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231320"><code style="display: none" id="bpr-guid-6231324">
  {"data":{"data":{"feedDashGlobalNavs":{"logo":{"controlName":"nav_inbug","logo":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_LINKEDIN_BUG_COLOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":"LinkedIn","$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"$recipeTypes":["com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"],"hoverText":null,"$type":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"$recipeTypes":["com.linkedin.eae2ed6e463973c8e0d55baecd814363"],"spotlight":{"navContent":{"*premiumUpsellSlotUrn":"urn:li:fsd_premiumUpsellSlot:NAV_SPOTLIGHT","navElement":null},"type":"PREMIUM_UPSELL","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"primaryItems":[{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_HOME_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_homepage","showIconBorder":false,"actionTarget":"https://www.linkedin.com/feed/?doFeedRefresh=true&amp;nis=true","text":"Home","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"HOME","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_PEOPLE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_mynetwork","showIconBorder":false,"actionTarget":"https://www.linkedin.com/mynetwork/","text":"Mijn netwerk","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MY_NETWORK","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_JOBS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_jobs","showIconBorder":false,"actionTarget":"https://www.linkedin.com/jobs/","text":"Vacatures","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOBS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_MESSAGING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_messaging","showIconBorder":false,"actionTarget":"https://www.linkedin.com/messaging/","text":"Berichten","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"MESSAGING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"navItem":{"navContent":{"navElement":{"activeIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_ACTIVE_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_NOTIFICATIONS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"hasCoachmark":false,"controlName":"nav_notifications","showIconBorder":false,"actionTarget":"https://www.linkedin.com/notifications/","text":"Meldingen","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"NOTIFICATIONS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},"meMenu":null},{"appLauncher":null,"meMenu":{"viewProfileText":"Profiel weergeven","*profile":"urn:li:fsd_profile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","meGroups":[{"meItems":[{"*premiumUpsellSlotUrn":"urn:li:fsd_premiumUpsellSlot:NAV_ME","meItem":null},{"meItem":{"controlName":"nav_settings_account_manage_account","actionTarget":"https://www.linkedin.com/mypreferences/d/","text":{"textDirection":"USER_LOCALE","text":"Instellingen en privacy","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_quick_help","actionTarget":"https://www.linkedin.com/help/linkedin/","text":{"textDirection":"USER_LOCALE","text":"Hulp","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":true,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_account_language","actionTarget":"https://www.linkedin.com/mypreferences/d/language/","text":{"textDirection":"USER_LOCALE","text":"Taal","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Account","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"},{"meItems":[{"meItem":{"controlName":"recent_activity_nav_all","actionTarget":"https://www.linkedin.com/in/fayyoz-usmon-a1410b267/recent-activity/","text":{"textDirection":"USER_LOCALE","text":"Bijdragen en activiteit","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null},{"meItem":{"controlName":"nav_settings_manage_job_posting_account","actionTarget":"https://www.linkedin.com/talent/job-management-redirect?trk=nav_app_launcher_manage_job_post_nept","text":{"textDirection":"USER_LOCALE","text":"Account voor plaatsen vacatures","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"quickHelp":false,"$recipeTypes":["com.linkedin.fb4511fb6c2df4a805ef662367e15026"],"$type":"com.linkedin.voyager.dash.feed.nav.MeItem"},"premiumUpsellSlotUrn":null}],"title":"Beheren","$recipeTypes":["com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"],"$type":"com.linkedin.voyager.dash.feed.nav.MeGroup"}],"controlName":"nav_settings","text":"Ik","$recipeTypes":["com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"],"$type":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"navItem":null},{"appLauncher":{"showDivider":true,"productItems":[{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_LEARNING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_learning","showIconBorder":true,"actionTarget":"https://www.linkedin.com/learning/?trk=nav_neptune_learning","text":"Learning","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"LEARNING","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_TALENT_INSIGHTS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_insights","showIconBorder":true,"actionTarget":"https://www.linkedin.com/insights?trk=nav_app_launcher_insights_nept&amp;src=li-nav","text":"Talent Insights","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"INSIGHTS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_JOBS_POSTING_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_job_postings","showIconBorder":true,"actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_app_launcher_job_post_nept","text":"Vacature plaatsen","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"JOB_POSTINGS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_ADS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_ads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/campaignmanager/accounts","text":"Adverteren","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":true,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"ADVERTISE","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_SALES_NAVIGATOR_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_find_leads","showIconBorder":true,"actionTarget":"https://www.linkedin.com/premium/products/?intentType=FIND_LEADS&amp;upsellOrderOrigin=premium_nav_more_products_panel&amp;utype=sales","text":"Leads zoeken","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"FIND_LEADS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_GROUPS_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_groups","showIconBorder":true,"actionTarget":"https://www.linkedin.com/groups","text":"Groepen","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"GROUPS","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"},{"navContent":{"navElement":{"staticIcon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_APP_PROFINDER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"activeIcon":null,"hasCoachmark":false,"controlName":"nav_launcher_services_marketplace","showIconBorder":true,"actionTarget":"https://www.linkedin.com/services?trk=d_flagship3_nav","text":"Service\nhandels\nplaats","$recipeTypes":["com.linkedin.b2c03744140a6cd6a2e693c32790988c"],"hasPaid":false,"$type":"com.linkedin.voyager.dash.feed.nav.NavElement"},"premiumUpsellSlotUrn":null},"type":"PROFINDER","$recipeTypes":["com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"],"$type":"com.linkedin.voyager.dash.feed.nav.NavItem"}],"businessServices":[{"businessServiceAction":{"controlName":"nav_business_talent_solutions","actionTarget":"https://business.linkedin.com/talent-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lts-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Werven op LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Vind, interesseer en werf talent","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_sales_solutions","actionTarget":"https://business.linkedin.com/sales-solutions?trk=flagship_nav&amp;veh=li-header-dropdown-lss-control&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Verkopen met LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Bouw relaties op met kopers","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_post_a_job","actionTarget":"https://www.linkedin.com/talent/job-posting-redirect?trk=nav_biz_serv_job_post_nept","title":{"textDirection":"USER_LOCALE","text":"Plaats gratis een vacature","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Zoek hoogwaardige kandidaten","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_advertise","actionTarget":"https://business.linkedin.com/nl-nl/marketing-solutions/ads?trk=n_nav_ads_rr_b&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Adverteer op LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Vind nieuwe klanten en stimuleer groei","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_learning_solutions","actionTarget":"https://learning.linkedin.com/?trk=d_flagship3_nav&amp;veh=learning_solutions&amp;src=li-nav","title":{"textDirection":"USER_LOCALE","text":"Leren met LinkedIn","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Cursussen om uw medewerkers te ontwikkelen","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"},{"businessServiceAction":{"controlName":"nav_business_admin_center","actionTarget":"https://business.linkedin.com/talent-solutions/customer/admin-center","title":{"textDirection":"USER_LOCALE","text":"Beheerderscentrum","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"description":"Facturerings- en accountgegevens beheren","$recipeTypes":["com.linkedin.a94ff0919cddaa7056025e9491dc3794"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessService"}],"productItemsTitle":{"textDirection":"USER_LOCALE","text":"Meer LinkedIn-producten bezoeken","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"businessServicesTitle":"Ontdek meer voor bedrijven","businessServiceActions":[{"controlName":"nav_business_create_company","actionTarget":"https://www.linkedin.com/company/setup/new/","title":{"textDirection":"USER_LOCALE","text":"Bedrijfspagina maken  ","attributesV2":[{"start":21,"length":1,"detailData":{"jobPostingName":null,"hyperlink":null,"profileFamiliarName":null,"color":null,"companyName":null,"icon":"IC_PLUS_16DP","epoch":null,"systemImage":null,"textLink":null,"listItemStyle":null,"groupName":null,"hyperlinkOpenExternally":null,"listStyle":null,"profileFullName":null,"stringFieldReference":null,"learningCourseName":null,"profileMention":null,"style":null,"schoolName":null,"hashtag":null},"$recipeTypes":["com.linkedin.9d1c9f8db73cc268b525043481708082"],"$type":"com.linkedin.voyager.dash.common.text.TextAttribute"}],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"$recipeTypes":["com.linkedin.188858304ae14097e59518d245c7cc85"],"$type":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"}],"controlName":"nav_launcher","icon":{"attributes":[{"scalingType":null,"detailData":{"profilePictureWithoutFrame":null,"profilePictureWithRingStatus":null,"companyLogo":null,"icon":"IC_NAV_SMALL_APP_SWITCHER_24DP","systemImage":null,"nonEntityGroupLogo":null,"vectorImage":null,"nonEntityProfessionalEventLogo":null,"profilePicture":null,"imageUrl":null,"professionalEventLogo":null,"nonEntityCompanyLogo":null,"nonEntitySchoolLogo":null,"groupLogo":null,"schoolLogo":null,"ghostImage":null,"nonEntityProfilePicture":null},"tintColor":null,"$recipeTypes":["com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"],"tapTargets":[],"displayAspectRatio":null,"$type":"com.linkedin.voyager.dash.common.image.ImageAttribute"}],"editableAccessibilityText":false,"actionTarget":null,"accessibilityTextAttributes":[],"totalCount":null,"accessibilityText":null,"$recipeTypes":["com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"],"$type":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"text":"Voor bedrijven","$recipeTypes":["com.linkedin.f94aca2770a997f8a58dfb7d6154954a"],"$type":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"meMenu":null,"navItem":null}],"$type":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"$recipeTypes":["com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"],"$type":"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.6c29a299f640b6e802d15141c355bd92"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.1e797f68b521546886989869c5b44a2d":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.6cf7727c816c79567a0564982ee35ef4":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.PremiumUpsellSlotUrn"},"upsellCard":{"type":"com.linkedin.8b2657e6d402c000382bd65eb94ba628"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},"com.linkedin.fb4511fb6c2df4a805ef662367e15026":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"quickHelp":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeItem"},"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.61dcef34ee68ce7a34919062297c038b":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.6c29a299f640b6e802d15141c355bd92":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.566376561ef86249fc89f5596eaf6fe1":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.9d1c9f8db73cc268b525043481708082"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.1f89c60d56117f36dc02eb3d95a6504d":{"fields":{"feedDashGlobalNavs":{"type":"com.linkedin.eae2ed6e463973c8e0d55baecd814363"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c":{"fields":{"title":{"type":"string"},"meItemsUnions":{"type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"string"}}}},"meItems":{"resolvedFrom":"meItemsUnions","type":{"array":{"union":{"meItem":"com.linkedin.fb4511fb6c2df4a805ef662367e15026","premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeGroup"},"com.linkedin.1112232e110fbaf9f6866335ae420a21":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.bda725ec0f4ddc7380fb44de84e29723":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.2b5530b88c12ba36048beee3aca27737":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.b2c03744140a6cd6a2e693c32790988c":{"fields":{"staticIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"activeIcon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hasCoachmark":{"type":"boolean"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"showIconBorder":{"type":"boolean"},"text":{"type":"string"},"actionTarget":{"type":"com.linkedin.common.Url"},"hasPaid":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavElement"},"com.linkedin.feccefa74b57bf711c74d0b09805c9f6":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.188858304ae14097e59518d245c7cc85":{"fields":{"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessServiceAction"},"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.1c710707068d7b33bcbee05036904a3b":{"fields":{"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.8b2657e6d402c000382bd65eb94ba628":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"socialProofInsight":{"type":"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273"},"subtext":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"ctaText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"upsellOrderOrigin":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"actionUrl":{"type":"com.linkedin.common.Url"},"legoTrackingToken":{"type":"string"},"title":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"cancelCta":{"type":"com.linkedin.513969f6493543c4ad7f04b46c3b67f6"},"globalLegoTrackingToken":{"type":"string"},"promotionLegoTrackingToken":{"type":"string"},"funnelCommonHeader":{"type":"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"},"fireDiscoveryImpressionEvent":{"type":"boolean"},"subtitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"dismissible":{"type":"boolean"},"layoutStyle":{"type":"com.linkedin.voyager.dash.premium.PremiumUpsellLayoutStyle"},"ctaButtonSecondaryEmphasizedTheme":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumUpsellCard"},"com.linkedin.44f2985bd25b663c32c3c4d4ec070248":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.ae72a2e8f27d029e18d8e6d4350b0273":{"fields":{"image":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"text":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.ux.InsightViewModel"},"com.linkedin.d66f0cb7963e0345d3ff893927e60c88":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.637f09655a21b1df5a03c68636fd2d18","largeTemplate":"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18","smallTemplate":"com.linkedin.f223abd9a48757f8395ac497a91593f3"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.10f415928dc4647be1fe1eebc72fe277":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"actionTarget":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64":{"fields":{"type":{"type":"com.linkedin.voyager.dash.feed.nav.NavType"},"navContent":{"resolvedFrom":"navContentUnion","type":{"union":{"premiumUpsellSlotUrn":"com.linkedin.6cf7727c816c79567a0564982ee35ef4","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}},"navContentUnion":{"type":{"union":{"premiumUpsellSlotUrn":"string","navElement":"com.linkedin.b2c03744140a6cd6a2e693c32790988c"}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.NavItem"},"com.linkedin.1bcf1aba50ed23a46d11fe327f319b18":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"footerText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"insightText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"},"com.linkedin.96db794fa6a744844b487cfa3e195632":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.1e797f68b521546886989869c5b44a2d"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.eae2ed6e463973c8e0d55baecd814363":{"fields":{"logo":{"type":"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b"},"primaryItemsUnions":{"type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}},"spotlight":{"type":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"},"primaryItems":{"derivedFrom":"primaryItemsUnions","type":{"array":{"union":{"appLauncher":"com.linkedin.f94aca2770a997f8a58dfb7d6154954a","navItem":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64","meMenu":"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d"}}}}},"baseType":"com.linkedin.voyager.dash.feed.nav.GlobalNav"},"com.linkedin.9d1c9f8db73cc268b525043481708082":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.1febda95b00bfc8c540a7af0dc75a88a","profileFamiliarName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.6c29a299f640b6e802d15141c355bd92","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","learningCourseName":"com.linkedin.44f2985bd25b663c32c3c4d4ec070248","profileMention":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.1e797f68b521546886989869c5b44a2d","hashtag":"com.linkedin.10f415928dc4647be1fe1eebc72fe277"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.feccefa74b57bf711c74d0b09805c9f6","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.72c1c1091a60c5a77aaadac057a84b54","listItemStyle":"com.linkedin.1112232e110fbaf9f6866335ae420a21","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.65deee184b56ce21c8d7d30fec6acfe4","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.513969f6493543c4ad7f04b46c3b67f6":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"appearance":{"type":"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf"},"openExternally":{"type":"boolean"},"accessibilityText":{"type":"string"},"disable":{"type":"boolean"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumButton"},"com.linkedin.f94aca2770a997f8a58dfb7d6154954a":{"fields":{"showDivider":{"type":"boolean"},"productItems":{"type":{"array":"com.linkedin.c7f876e8e9f8a631b2896f62a656ce64"}},"productItemsTitle":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"businessServices":{"type":{"array":"com.linkedin.a94ff0919cddaa7056025e9491dc3794"}},"businessServicesTitle":{"type":"string"},"icon":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"businessServiceActions":{"type":{"array":"com.linkedin.188858304ae14097e59518d245c7cc85"}},"text":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.AppLauncher"},"com.linkedin.a94ff0919cddaa7056025e9491dc3794":{"fields":{"businessServiceAction":{"type":"com.linkedin.188858304ae14097e59518d245c7cc85"},"description":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BusinessService"},"com.linkedin.637f09655a21b1df5a03c68636fd2d18":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"nameSupplementaryInfo":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"headline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"subHeadline":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.73113dd58d976ff22a5e4aa2ca35d21d":{"fields":{"viewProfileText":{"type":"string"},"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"string"},"meGroups":{"type":{"array":"com.linkedin.bb35e914e880d9f9ad79d52bf644c44c"}},"profileUrn":{"type":"string"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344"}},"baseType":"com.linkedin.voyager.dash.feed.nav.MeMenu"},"com.linkedin.f223abd9a48757f8395ac497a91593f3":{"fields":{"name":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.c8b3cc209fc06180b17a4ffb72120344":{"fields":{"profilePicture":{"type":"com.linkedin.bda725ec0f4ddc7380fb44de84e29723"},"lastName":{"type":"string"},"firstName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"ringStatus":{"type":"com.linkedin.61dcef34ee68ce7a34919062297c038b","isInjection":true},"headline":{"type":"string"},"publicIdentifier":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.4ef51bc3dc729207a4f87aa1ac1b37cf":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.e6506ba5f5cc1a9b5124781a3661a7db":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","profilePictureWithRingStatus":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","companyLogo":"com.linkedin.2b5530b88c12ba36048beee3aca27737","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"com.linkedin.c8b3cc209fc06180b17a4ffb72120344","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"com.linkedin.0d2a5c07095246fc3e756f3e922d8aa0","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"com.linkedin.1e797f68b521546886989869c5b44a2d","groupLogo":"com.linkedin.6c29a299f640b6e802d15141c355bd92","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.dcb85e1ca353361cb2b5f6fa24a1c9af","vectorImage":"com.linkedin.ca116024396bd29dd1345ad0998a57d5","nonEntityProfessionalEventLogo":"com.linkedin.371c4584dc45b7919c4eb6ed6697e8cf","profilePicture":"string","imageUrl":"com.linkedin.550a9af095fffdd7fc0f6ceb8743a141","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.96db794fa6a744844b487cfa3e195632","nonEntityCompanyLogo":"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1c710707068d7b33bcbee05036904a3b"}}},"tapTargets":{"type":{"array":"com.linkedin.d66f0cb7963e0345d3ff893927e60c88"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.ca116024396bd29dd1345ad0998a57d5":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.f1332bd87cf4dccab8aa4b4583ad44be"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.72c1c1091a60c5a77aaadac057a84b54":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.72e4cd594b8572e0ec9ddb034eafc088":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.2b5530b88c12ba36048beee3aca27737"},"vectorImage":{"type":"com.linkedin.ca116024396bd29dd1345ad0998a57d5"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b":{"fields":{"utype":{"type":"string"},"referenceId":{"type":"com.linkedin.common.TrackingId"}},"baseType":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"com.linkedin.4973670c8e07d9bf8572c4c7f8658e2b":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"logo":{"type":"com.linkedin.f89edd0cdc11632a12f86e550f2f5e46"},"hoverText":{"type":"com.linkedin.566376561ef86249fc89f5596eaf6fe1"}},"baseType":"com.linkedin.voyager.dash.feed.nav.BrandingLogo"},"com.linkedin.8fbc11c3b77aca2f1a34ab7a34600da7":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"}}}},"included":[{"firstName":"Fayyoz","lastName":"Usmon","profilePicture":{"displayImageReferenceResolutionResult":null,"displayImageWithFrameReference":null,"$recipeTypes":["com.linkedin.bda725ec0f4ddc7380fb44de84e29723"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"entityUrn":"urn:li:fsd_profile:ACoAAEFVi_ABd7oZ-rr-hlNa6438NMFd5_fzQj0","headline":"--","publicIdentifier":"fayyoz-usmon-a1410b267","$recipeTypes":["com.linkedin.c8b3cc209fc06180b17a4ffb72120344"],"$type":"com.linkedin.voyager.dash.identity.profile.Profile"},{"entityUrn":"urn:li:fsd_premiumUpsellSlot:NAV_SPOTLIGHT","$recipeTypes":["com.linkedin.6cf7727c816c79567a0564982ee35ef4"],"upsellCard":{"image":null,"socialProofInsight":null,"subtext":null,"upsellOrderOrigin":"premium_nav_upsell_text","ctaText":{"textDirection":"USER_LOCALE","text":"Probeer Premium gratis","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"actionUrl":"https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_nav_upsell_text&amp;referenceId=sH42Ay32QVy8LZYHybv6Dg%3D%3D","controlName":"premium_nav_upsell_text_click","legoTrackingToken":null,"title":null,"$recipeTypes":["com.linkedin.8b2657e6d402c000382bd65eb94ba628"],"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellCard","cancelCta":null,"globalLegoTrackingToken":null,"promotionLegoTrackingToken":null,"funnelCommonHeader":{"utype":null,"$recipeTypes":["com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"],"referenceId":"°~6\u0003-öA\\¼-\u0007É»ú\u000E","$type":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":false,"subtitle":null,"dismissible":false,"layoutStyle":"CUSTOM","ctaButtonSecondaryEmphasizedTheme":false},"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"},{"entityUrn":"urn:li:fsd_premiumUpsellSlot:NAV_ME","$recipeTypes":["com.linkedin.6cf7727c816c79567a0564982ee35ef4"],"upsellCard":{"image":null,"socialProofInsight":null,"subtext":null,"upsellOrderOrigin":"premium_nav_me_upsell","ctaText":{"textDirection":"USER_LOCALE","text":"Probeer Premium gratis","attributesV2":[],"accessibilityTextAttributesV2":[],"accessibilityText":null,"$recipeTypes":["com.linkedin.566376561ef86249fc89f5596eaf6fe1"],"$type":"com.linkedin.voyager.dash.common.text.TextViewModel"},"actionUrl":"https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_nav_me_upsell&amp;referenceId=d5iXGYVQRkaImO94FoJ8Gw%3D%3D","controlName":"premium_nav_me_upsell_click","legoTrackingToken":null,"title":null,"$recipeTypes":["com.linkedin.8b2657e6d402c000382bd65eb94ba628"],"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellCard","cancelCta":null,"globalLegoTrackingToken":null,"promotionLegoTrackingToken":null,"funnelCommonHeader":{"utype":null,"$recipeTypes":["com.linkedin.f58e91c23e63f52ceb12bd0fab3eb07b"],"referenceId":"w\u0019PFFïx\u0016|\u001B","$type":"com.linkedin.voyager.dash.premium.PremiumFunnelCommonHeader"},"fireDiscoveryImpressionEvent":false,"subtitle":null,"dismissible":false,"layoutStyle":"TEXT_LINK","ctaButtonSecondaryEmphasizedTheme":false},"$type":"com.linkedin.voyager.dash.premium.PremiumUpsellSlotContent"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231324">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D()\u0026\u0026queryId\u003DvoyagerFeedDashGlobalNavs.c70182517bc900aa0452b092c2e2ff39","status":200,"body":"bpr-guid-6231324","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231324"><code style="display: none" id="bpr-guid-6231331">
  {"data":{"data":{"premiumDashFeatureAccessByFeatureAccessTypes":{"*elements":["urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE"],"$recipeTypes":["com.linkedin.3fad2247165582aa95a2690797d7e0ab"],"$type":"com.linkedin.restli.common.CollectionResponse"},"$recipeTypes":["com.linkedin.6488012243094b4ef82d764d10e79d45"],"$type":"com.linkedin.6488012243094b4ef82d764d10e79d45"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.3fad2247165582aa95a2690797d7e0ab":{"fields":{"elements":{"type":{"array":"com.linkedin.9e452672002943dc9a1521bcc7ad23cb"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.9e452672002943dc9a1521bcc7ad23cb":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.FeatureAccessUrn"},"hasAccess":{"type":"boolean"},"featureAccessType":{"type":"com.linkedin.voyager.dash.premium.FeatureAccessType"}},"baseType":"com.linkedin.voyager.dash.premium.FeatureAccess"},"com.linkedin.6488012243094b4ef82d764d10e79d45":{"fields":{"premiumDashFeatureAccessByFeatureAccessTypes":{"type":"com.linkedin.3fad2247165582aa95a2690797d7e0ab"}},"baseType":"com.linkedin.graphql.Query"}}}},"included":[{"featureAccessType":"CAN_ACCESS_SALES_NAV_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_SALES_NAV_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"},{"featureAccessType":"CAN_ACCESS_ADVERTISE_BADGE","entityUrn":"urn:li:fsd_featureAccess:CAN_ACCESS_ADVERTISE_BADGE","hasAccess":false,"$recipeTypes":["com.linkedin.9e452672002943dc9a1521bcc7ad23cb"],"$type":"com.linkedin.voyager.dash.premium.FeatureAccess"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231331">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(featureAccessTypes:List(CAN_ACCESS_SALES_NAV_BADGE,CAN_ACCESS_ADVERTISE_BADGE))\u0026\u0026queryId\u003DvoyagerPremiumDashFeatureAccess.c87b20dac35795f9920f2a8072fd7af5","status":200,"body":"bpr-guid-6231331","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231331"><code style="display: none" id="bpr-guid-6231338">
  {"data":{"entityUrn":"urn:li:collectionResponse:7ZvAwvPwYldk7Tvl3jXgC4ymrtxthgc+3EdE7ncMd98=","elements":[{"count":11,"tab":"NOTIFICATIONS","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":0,"tab":"MESSAGING","$type":"com.linkedin.voyager.common.communications.TabBadge"},{"count":2,"tab":"MY_NETWORK","$type":"com.linkedin.voyager.common.communications.TabBadge"}],"paging":{"count":10,"start":0,"links":[]},"$type":"com.linkedin.restli.common.CollectionResponse"},"included":[]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231338">
  {"request":"/voyager/api/voyagerCommunicationsTabBadges?q\u003DtabBadges\u0026countFrom\u003D0","status":200,"body":"bpr-guid-6231338","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231338"><code style="display: none" id="bpr-guid-6231354">
  {"data":{"data":{"$recipeTypes":["com.linkedin.6760ca9c00d2cb00ed53ee695927b17a"],"identityDashProfilesByMemberIdentity":{"*elements":["urn:li:fsd_profile:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ"],"$recipeTypes":["com.linkedin.c16481ff510ae7f0273d85f18aa15219"],"$type":"com.linkedin.restli.common.CollectionResponse"},"$type":"com.linkedin.6760ca9c00d2cb00ed53ee695927b17a"},"extensions":{"webMetadata":{}}},"meta":{"microSchema":{"isGraphQL":true,"version":"2.1","types":{"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35":{"fields":{"premiumStyle":{"type":"boolean"},"size":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonSize"},"icon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"text":{"type":"string"},"iconAfterText":{"type":"boolean"},"emphasize":{"type":"boolean"},"category":{"type":"com.linkedin.voyager.dash.common.ux.button.ButtonCategory"}},"baseType":"com.linkedin.voyager.dash.common.ux.button.ButtonAppearance"},"com.linkedin.6ee28b2b65af6b1ea34ebc4526078982":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"accessibilityText":{"type":"string"},"appearance":{"type":"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeneratedSuggestionButton"},"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.CompanyUrn"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.organization.Company"},"com.linkedin.8ee23bb31c0e8d4cb7e6a3e86e720799":{"fields":{"logoImage":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoImageResolutionResult":{"derivedFrom":"logoImage","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfessionalEventUrn"}},"baseType":"com.linkedin.voyager.dash.events.ProfessionalEvent"},"com.linkedin.5fba45f960b9273c3b1bc933082ed44c":{"fields":{"x":{"type":"double"},"y":{"type":"double"}},"baseType":"com.linkedin.common.Coordinate2D"},"com.linkedin.87507521b880a59f51dfe7916463212a":{"fields":{"textDirection":{"type":"com.linkedin.voyager.dash.common.text.TextDirection"},"attributesV2":{"type":{"array":"com.linkedin.b724930290217f55fa1b0d6ec7cdf954"}},"text":{"type":"string"},"accessibilityTextAttributesV2":{"type":{"array":"com.linkedin.b724930290217f55fa1b0d6ec7cdf954"}},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.text.TextViewModel"},"com.linkedin.fe63f2527824313d8a7a69a542ee70c2":{"fields":{"school":{"resolvedFrom":"schoolUrn","type":"com.linkedin.e96795656e0f04f0745c758d2fd67d79"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"schoolUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntitySchoolLogo"},"com.linkedin.a4c3cd660ad2e3b0173f3c1b7356f2c6":{"fields":{"t":{"type":"string"},"x":{"type":"string"},"u":{"type":"string"}},"baseType":"com.linkedin.common.LocaleExtensions"},"com.linkedin.4c7a6a878ae117485d617b8a09075c76":{"fields":{"bottomLeft":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"},"saturation":{"type":"double"},"vignette":{"type":"double"},"brightness":{"type":"double"},"photoFilterType":{"type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterType"},"bottomRight":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"},"topLeft":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"},"contrast":{"type":"double"},"topRight":{"type":"com.linkedin.5fba45f960b9273c3b1bc933082ed44c"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterEditInfo"},"com.linkedin.42bbd7c505b9890dbd35cd379a5ffd49":{"fields":{"name":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkSmallTemplateView"},"com.linkedin.aa58984e06b9abd3aa0ba432da210113":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.EpochFormat"},"epochAt":{"type":"com.linkedin.common.Time"}},"baseType":"com.linkedin.voyager.dash.common.text.EpochTime"},"com.linkedin.848cd7bd87249ea0e34a0778a20bef83":{"fields":{"ringContentType":{"type":"com.linkedin.voyager.dash.common.image.RingContentType"},"actionTarget":{"type":"com.linkedin.common.Url"},"preDashEntityUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.RingStatusUrn"},"emphasized":{"type":"boolean"}},"baseType":"com.linkedin.voyager.dash.common.image.RingStatus"},"com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85":{"fields":{"displayImageWithFrameReferenceUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"originalImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"displayImageReferenceResolutionResult":{"derivedFrom":"displayImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"originalImageUrn":{"type":"com.linkedin.common.Urn"},"displayImageWithFrameReference":{"derivedFrom":"displayImageWithFrameReferenceUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"originalImageReferenceResolutionResult":{"derivedFrom":"originalImageReference","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"displayImageReference":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"frameType":{"type":"com.linkedin.voyager.dash.identity.profile.PhotoFrameType"},"photoFilterEditInfo":{"type":"com.linkedin.4c7a6a878ae117485d617b8a09075c76"},"displayImageUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"com.linkedin.8b38bd16089709d88ed7104608137baf":{"fields":{"memberDistance":{"type":"com.linkedin.voyager.dash.relationships.NoConnectionMemberDistance"}},"baseType":"com.linkedin.voyager.dash.relationships.NoConnection"},"com.linkedin.a4c44f1ec152e8b1415e8ed859230f0c":{"fields":{"buttonPlacement":{"type":"com.linkedin.voyager.dash.identity.profile.tetris.ButtonPlacement"},"buttonAppearance":{"type":"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35"},"legoTrackingId":{"type":"string"},"icon":{"type":"com.linkedin.ff3f998e6eedb8e317a96fb99dd62775"},"actionControlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"text":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.tetris.NavigationAction"},"com.linkedin.7959475b6283d014d8d529b6f2ef5b88":{"fields":{"badgeIcon":{"type":"com.linkedin.voyager.dash.common.SystemImageName"},"badgeText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.TopVoiceBadge"},"com.linkedin.fb515f37046fc34f7a17064266659842":{"fields":{"elements":{"type":{"array":"com.linkedin.cff5540190d0fbce1a4cab42ae0afdd3"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.1d7f409d378b3c2595366d0625fbf68e":{"fields":{"ringStatus":{"type":"com.linkedin.848cd7bd87249ea0e34a0778a20bef83"},"profileUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"profile":{"resolvedFrom":"profileUrn","type":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfilePicture"},"com.linkedin.2cdd21b0b3dad6ccee057d5d8ba4203a":{"fields":{"originalImageUrl":{"type":"com.linkedin.common.Url"},"originalHeight":{"type":"int"},"originalWidth":{"type":"int"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.ImageUrl"},"com.linkedin.d16b47c42001019f9bb43f9ce18412ef":{"fields":{"geo":{"resolvedFrom":"geoUrn","type":"com.linkedin.20bf1398649ed8da5a403515fbffb6bf"},"geoUrn":{"type":"string"},"postalCode":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"com.linkedin.f2c61005acb2fbe69cd38580d01cb3d5":{"fields":{"professionalEvent":{"resolvedFrom":"professionalEventUrn","type":"com.linkedin.8ee23bb31c0e8d4cb7e6a3e86e720799"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"professionalEventUrn":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityProfessionalEventLogo"},"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164":{"fields":{"attribution":{"type":"string"},"digitalmediaAsset":{"type":"com.linkedin.common.DigitalmediaAssetUrn"},"rootUrl":{"type":"string"},"artifacts":{"type":{"array":"com.linkedin.5f3d321f9b6631058c0c944d77e41938"}}},"baseType":"com.linkedin.common.VectorImage"},"com.linkedin.be5d51d61e628b01ab0d22e4b670bcd2":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.JobPostingUrn"}},"baseType":"com.linkedin.voyager.dash.jobs.JobPosting"},"com.linkedin.b724930290217f55fa1b0d6ec7cdf954":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"jobPostingName":"com.linkedin.be5d51d61e628b01ab0d22e4b670bcd2","profileFamiliarName":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.aa58984e06b9abd3aa0ba432da210113","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.055a5fa30bbe60d94f41c14ebe173bbf","listItemStyle":"com.linkedin.2322b567653571139f2d983df4580732","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"com.linkedin.d22a136895aa551adfab2b49ab2928a1","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.a3be506e3c2e76c76d0e9c0f867f1429","profileFullName":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","learningCourseName":"com.linkedin.6fdcc7e0d8a5276f82b704ee433b31cc","profileMention":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"com.linkedin.e96795656e0f04f0745c758d2fd67d79","hashtag":"com.linkedin.c8ae18b1f64cb4a44b60cb752e911c5b"}}},"length":{"type":"int"},"start":{"type":"int"},"detailDataUnion":{"type":{"union":{"jobPostingName":"string","profileFamiliarName":"string","hyperlink":"com.linkedin.common.Url","color":"com.linkedin.voyager.dash.common.text.TextColor","companyName":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","epoch":"com.linkedin.aa58984e06b9abd3aa0ba432da210113","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","textLink":"com.linkedin.055a5fa30bbe60d94f41c14ebe173bbf","listItemStyle":"com.linkedin.2322b567653571139f2d983df4580732","listStyle":"com.linkedin.voyager.dash.common.text.ListStyleType","groupName":"string","hyperlinkOpenExternally":"com.linkedin.common.Url","stringFieldReference":"com.linkedin.a3be506e3c2e76c76d0e9c0f867f1429","profileFullName":"string","learningCourseName":"string","profileMention":"string","style":"com.linkedin.voyager.dash.common.text.TextStyle","schoolName":"string","hashtag":"string"}}}},"baseType":"com.linkedin.voyager.dash.common.text.TextAttribute"},"com.linkedin.cff5540190d0fbce1a4cab42ae0afdd3":{"fields":{"dismissCta":{"type":"com.linkedin.ca9d072a605c2b0ba43fa1d988920a35"},"headlineVersion":{"type":"com.linkedin.4a0a959f2280a25e8a09005334df5da9"},"dismissCtaButton":{"type":"com.linkedin.6ee28b2b65af6b1ea34ebc4526078982"},"description":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"legoTrackingToken":{"type":"string"},"primaryCtaButtonV2":{"type":"com.linkedin.a4c44f1ec152e8b1415e8ed859230f0c"},"headline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"dismissCtaAccessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.identity.profile.ProfileGeneratedSuggestionPromoCard"},"com.linkedin.055a5fa30bbe60d94f41c14ebe173bbf":{"fields":{"controlName":{"type":"com.linkedin.voyager.dash.common.tracking.ControlName"},"viewingBehavior":{"type":"com.linkedin.voyager.dash.common.ux.UrlViewingBehavior"},"sponsoredUrlAttributes":{"type":"com.linkedin.6a0e2ca2b255b817da99cc43e27e5b44"},"url":{"type":"com.linkedin.common.Url"}},"baseType":"com.linkedin.voyager.dash.common.text.TextLink"},"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172":{"fields":{"birthDateOn":{"type":"com.linkedin.c6c63f18a3ab2fb1a340292406594762"},"memorialized":{"type":"boolean"},"lastName":{"type":"string"},"objectUrn":{"type":"com.linkedin.common.Urn"},"multiLocaleLastName":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"showPremiumSubscriberBadge":{"type":"boolean"},"tempStatusEmoji":{"type":"string"},"pronounUnion":{"type":{"union":{"customPronoun":"string","standardizedPronoun":"com.linkedin.voyager.dash.identity.profile.StandardizedPronoun"}}},"tempStatusExpiredAtUnion":{"type":{"union":{"customizedExpiredAt":"com.linkedin.common.Time","standardizedExpiration":"com.linkedin.voyager.dash.identity.profile.StandardizedExpiration"}}},"multiLocaleFirstName":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"memberPostAnalytics":{"type":"com.linkedin.6f81a0561c6d67d56507fcaec5cf100b"},"premium":{"type":"boolean"},"influencer":{"type":"boolean"},"multiLocaleTempStatus":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"pronoun":{"derivedFrom":"pronounUnion","type":{"union":{"customPronoun":"string","standardizedPronoun":"com.linkedin.voyager.dash.identity.profile.StandardizedPronoun"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.ProfileUrn"},"ringStatus":{"type":"com.linkedin.848cd7bd87249ea0e34a0778a20bef83","isInjection":true},"headlineGeneratedSuggestionDelegateUrn":{"type":"com.linkedin.voyager.dash.common.ProfileActionDelegateUrn"},"headline":{"type":"string"},"publicIdentifier":{"type":"string"},"trackingId":{"type":"com.linkedin.voyager.dash.common.Base64String"},"topVoiceBadge":{"type":"com.linkedin.7959475b6283d014d8d529b6f2ef5b88"},"creator":{"type":"boolean"},"supportedLocales":{"type":{"array":"com.linkedin.a4044e55c3fcc69e2de7519872763501"}},"creatorInfo":{"type":"com.linkedin.80e29e7323b2b17315871707a1231af9"},"versionTag":{"type":"string"},"privacySettings":{"type":"com.linkedin.eafc20924d49659eb7e0335979bc8d29","isInjection":true},"profilePicture":{"type":"com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"},"firstName":{"type":"string"},"tempStatus":{"type":"string"},"multiLocaleMaidenName":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"geoLocation":{"type":"com.linkedin.d16b47c42001019f9bb43f9ce18412ef"},"showVerificationBadge":{"type":"boolean"},"memberRelationship":{"type":"com.linkedin.de96d1e9b7a124ebb464644d217619fc","isInjection":true},"profileGeneratedSuggestionPromo":{"type":"com.linkedin.fb515f37046fc34f7a17064266659842","isInjection":true},"primaryLocale":{"type":"com.linkedin.a4044e55c3fcc69e2de7519872763501"},"multiLocaleHeadline":{"type":{"array":"com.linkedin.63a835f468299bb310d28f506cabd1a6"},"isMap":true},"backgroundPicture":{"type":"com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"},"tempStatusExpiredAt":{"derivedFrom":"tempStatusExpiredAtUnion","type":{"union":{"customizedExpiredAt":"com.linkedin.common.Time","standardizedExpiration":"com.linkedin.voyager.dash.identity.profile.StandardizedExpiration"}}}},"baseType":"com.linkedin.voyager.dash.identity.profile.Profile"},"com.linkedin.ff3f998e6eedb8e317a96fb99dd62775":{"fields":{"editableAccessibilityText":{"type":"boolean"},"attributes":{"type":{"array":"com.linkedin.c7ef039b7fe1b9f39c5d2dab33f7e026"}},"actionTarget":{"type":"com.linkedin.common.Url"},"accessibilityTextAttributes":{"type":{"array":"com.linkedin.b724930290217f55fa1b0d6ec7cdf954"}},"totalCount":{"type":"int"},"accessibilityText":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageViewModel"},"com.linkedin.cc70ada274dbeca435a87c70e5f65742":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.ConnectionUrn"}},"baseType":"com.linkedin.voyager.dash.relationships.Connection"},"com.linkedin.e96795656e0f04f0745c758d2fd67d79":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.SchoolUrn"}},"baseType":"com.linkedin.voyager.dash.organization.School"},"com.linkedin.c7ef039b7fe1b9f39c5d2dab33f7e026":{"fields":{"detailData":{"resolvedFrom":"detailDataUnion","type":{"union":{"profilePictureWithoutFrame":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","profilePictureWithRingStatus":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","companyLogo":"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.28872f0409786487d12514b9187690b3","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164","nonEntityProfessionalEventLogo":"com.linkedin.f2c61005acb2fbe69cd38580d01cb3d5","profilePicture":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172","imageUrl":"com.linkedin.2cdd21b0b3dad6ccee057d5d8ba4203a","professionalEventLogo":"com.linkedin.8ee23bb31c0e8d4cb7e6a3e86e720799","nonEntitySchoolLogo":"com.linkedin.fe63f2527824313d8a7a69a542ee70c2","nonEntityCompanyLogo":"com.linkedin.4f05fb063efef8a581811a71ec2a3559","schoolLogo":"com.linkedin.e96795656e0f04f0745c758d2fd67d79","groupLogo":"com.linkedin.d22a136895aa551adfab2b49ab2928a1","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1d7f409d378b3c2595366d0625fbf68e"}}},"tintColor":{"type":"com.linkedin.voyager.dash.common.SystemImageTintColor"},"detailDataUnion":{"type":{"union":{"profilePictureWithoutFrame":"string","profilePictureWithRingStatus":"string","companyLogo":"string","icon":"com.linkedin.voyager.dash.common.ArtDecoIconName","systemImage":"com.linkedin.voyager.dash.common.SystemImageName","nonEntityGroupLogo":"com.linkedin.28872f0409786487d12514b9187690b3","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164","nonEntityProfessionalEventLogo":"com.linkedin.f2c61005acb2fbe69cd38580d01cb3d5","profilePicture":"string","imageUrl":"com.linkedin.2cdd21b0b3dad6ccee057d5d8ba4203a","professionalEventLogo":"string","nonEntitySchoolLogo":"com.linkedin.fe63f2527824313d8a7a69a542ee70c2","nonEntityCompanyLogo":"com.linkedin.4f05fb063efef8a581811a71ec2a3559","schoolLogo":"string","groupLogo":"string","ghostImage":"com.linkedin.voyager.dash.common.image.GhostImageType","nonEntityProfilePicture":"com.linkedin.1d7f409d378b3c2595366d0625fbf68e"}}},"tapTargets":{"type":{"array":"com.linkedin.c046626a3323577b67c41e8693ea4802"}},"scalingType":{"type":"com.linkedin.voyager.dash.common.image.ScalingType"},"displayAspectRatio":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.common.image.ImageAttribute"},"com.linkedin.80e29e7323b2b17315871707a1231af9":{"fields":{"creatorWebsite":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"associatedHashtag":{"resolvedFrom":"associatedHashtagUrns","type":{"array":"com.linkedin.c8ae18b1f64cb4a44b60cb752e911c5b"}},"associatedHashtagUrns":{"type":{"array":"string"}},"creatorPostAnalytics":{"type":"com.linkedin.6f81a0561c6d67d56507fcaec5cf100b"}},"baseType":"com.linkedin.voyager.dash.identity.profile.CreatorInfo"},"com.linkedin.6760ca9c00d2cb00ed53ee695927b17a":{"fields":{"identityDashProfilesByMemberIdentity":{"type":"com.linkedin.c16481ff510ae7f0273d85f18aa15219"}},"baseType":"com.linkedin.graphql.Query"},"com.linkedin.a3be506e3c2e76c76d0e9c0f867f1429":{"fields":{"urn":{"type":"com.linkedin.common.Urn"},"modelName":{"type":"string"},"fieldName":{"type":"string"},"value":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.StringFieldReference"},"com.linkedin.63a835f468299bb310d28f506cabd1a6":{"fields":{"value":{"type":"string"},"key":{"type":"string"}},"baseType":"com.linkedin.data.StringMapEntry"},"com.linkedin.20bf1398649ed8da5a403515fbffb6bf":{"fields":{"countryUrn":{"type":"string"},"country":{"resolvedFrom":"countryUrn","type":"com.linkedin.20bf1398649ed8da5a403515fbffb6bf"},"defaultLocalizedNameWithoutCountryName":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GeoUrn"},"defaultLocalizedName":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.Geo"},"com.linkedin.4a0a959f2280a25e8a09005334df5da9":{"fields":{"colorStyle":{"type":"com.linkedin.voyager.dash.common.ux.label.LabelColorStyle"},"text":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"}},"baseType":"com.linkedin.voyager.dash.common.ux.label.LabelViewModel"},"com.linkedin.4f05fb063efef8a581811a71ec2a3559":{"fields":{"companyUrn":{"type":"string"},"company":{"resolvedFrom":"companyUrn","type":"com.linkedin.9a7d3af38e7a038a1c54004a4524f7d2"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityCompanyLogo"},"com.linkedin.6f81a0561c6d67d56507fcaec5cf100b":{"fields":{"description":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"totalCount":{"type":"long"},"title":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"percentageChange":{"type":"double"}},"baseType":"com.linkedin.voyager.dash.identity.profile.CreatorPostAnalytics"},"com.linkedin.2322b567653571139f2d983df4580732":{"fields":{"type":{"type":"com.linkedin.voyager.dash.common.text.ListStyleType"},"index":{"type":"int"}},"baseType":"com.linkedin.voyager.dash.common.text.ListItemStyle"},"com.linkedin.6a0e2ca2b255b817da99cc43e27e5b44":{"fields":{"impressionId":{"type":"string"},"unwrappedUrlDomain":{"type":"string"},"creativeId":{"type":"string"},"campaignId":{"type":"string"},"memberId":{"type":"string"}},"baseType":"com.linkedin.voyager.dash.common.sponsored.SponsoredUrlAttributes"},"com.linkedin.b7f5bab2ab7602febe3216fc910827f5":{"fields":{},"baseType":"com.linkedin.restli.common.EmptyRecord"},"com.linkedin.cc4781d9200ac2c549e7bb9fe6f70e98":{"fields":{"followeeCount":{"type":"long"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.FollowingStateUrn"},"trackingUrn":{"type":"com.linkedin.common.Urn"},"preDashFollowingInfoUrn":{"type":"com.linkedin.voyager.dash.common.PreDashUrn"},"following":{"type":"boolean"},"followingType":{"type":"com.linkedin.voyager.dash.feed.FollowingType"},"followerCount":{"type":"long"}},"baseType":"com.linkedin.voyager.dash.feed.FollowingState"},"com.linkedin.28872f0409786487d12514b9187690b3":{"fields":{"groupUrn":{"type":"string"},"vectorImage":{"type":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"},"group":{"resolvedFrom":"groupUrn","type":"com.linkedin.d22a136895aa551adfab2b49ab2928a1"}},"baseType":"com.linkedin.voyager.dash.common.image.NonEntityGroupLogo"},"com.linkedin.c8ae18b1f64cb4a44b60cb752e911c5b":{"fields":{"actionTarget":{"type":"com.linkedin.common.Url"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.HashtagUrn"},"followingState":{"type":"com.linkedin.cc4781d9200ac2c549e7bb9fe6f70e98","isInjection":true},"displayName":{"type":"string"},"trackingUrn":{"type":"com.linkedin.common.Urn"}},"baseType":"com.linkedin.voyager.dash.feed.Hashtag"},"com.linkedin.d22a136895aa551adfab2b49ab2928a1":{"fields":{"name":{"type":"string"},"logo":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"logoResolutionResult":{"derivedFrom":"logo","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"entityUrn":{"type":"com.linkedin.voyager.dash.common.GroupUrn"}},"baseType":"com.linkedin.voyager.dash.groups.Group"},"com.linkedin.5f3d321f9b6631058c0c944d77e41938":{"fields":{"width":{"type":"int"},"fileIdentifyingUrlPathSegment":{"type":"string"},"expiresAt":{"type":"com.linkedin.common.Time"},"height":{"type":"int"}},"baseType":"com.linkedin.common.VectorArtifact"},"com.linkedin.c6c63f18a3ab2fb1a340292406594762":{"fields":{"month":{"type":"int"},"year":{"type":"int"},"day":{"type":"int"}},"baseType":"com.linkedin.common.Date"},"com.linkedin.b5218fbabd3538967ac9576bb65b170d":{"fields":{"name":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"nameSupplementaryInfo":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"headline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"subHeadline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkMediumTemplateView"},"com.linkedin.a4044e55c3fcc69e2de7519872763501":{"fields":{"variant":{"type":"string"},"country":{"type":"string"},"language":{"type":"string"},"extensions":{"type":"com.linkedin.a4c3cd660ad2e3b0173f3c1b7356f2c6"},"script":{"type":"string"}},"baseType":"com.linkedin.common.Locale"},"com.linkedin.c16481ff510ae7f0273d85f18aa15219":{"fields":{"elements":{"type":{"array":"com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172"}}},"baseType":"com.linkedin.restli.common.CollectionResponse"},"com.linkedin.c046626a3323577b67c41e8693ea4802":{"fields":{"stickerLinkViewUnion":{"type":{"union":{"mediumTemplate":"com.linkedin.b5218fbabd3538967ac9576bb65b170d","largeTemplate":"com.linkedin.c1233c6614d75709cf8a4843c636852c","smallTemplate":"com.linkedin.42bbd7c505b9890dbd35cd379a5ffd49"}}},"stickerLinkTemplateSize":{"type":"com.linkedin.voyager.dash.common.media.StickerLinkTemplateSize"},"firstCornerXOffsetPercentage":{"type":"float"},"type":{"type":"com.linkedin.voyager.dash.common.TapTargetAttributeType"},"thirdCornerYOffsetPercentage":{"type":"float"},"url":{"type":"com.linkedin.common.Url"},"stickerLinkView":{"derivedFrom":"stickerLinkViewUnion","type":{"union":{"mediumTemplate":"com.linkedin.b5218fbabd3538967ac9576bb65b170d","largeTemplate":"com.linkedin.c1233c6614d75709cf8a4843c636852c","smallTemplate":"com.linkedin.42bbd7c505b9890dbd35cd379a5ffd49"}}},"urn":{"type":"com.linkedin.common.Urn"},"thirdCornerXOffsetPercentage":{"type":"float"},"secondCornerYOffsetPercentage":{"type":"float"},"fourthCornerXOffsetPercentage":{"type":"float"},"firstCornerYOffsetPercentage":{"type":"float"},"untaggable":{"type":"boolean"},"rank":{"type":"int"},"fourthCornerYOffsetPercentage":{"type":"float"},"text":{"type":"string"},"secondCornerXOffsetPercentage":{"type":"float"}},"baseType":"com.linkedin.voyager.dash.common.TapTarget"},"com.linkedin.de96d1e9b7a124ebb464644d217619fc":{"fields":{"entityUrn":{"type":"com.linkedin.voyager.dash.common.MemberRelationshipUrn"},"memberRelationshipUnion":{"type":{"union":{"self":"com.linkedin.b7f5bab2ab7602febe3216fc910827f5","connection":"com.linkedin.cc70ada274dbeca435a87c70e5f65742","noConnection":"com.linkedin.8b38bd16089709d88ed7104608137baf"}}},"memberRelationship":{"derivedFrom":"memberRelationshipUnion","type":{"union":{"self":"com.linkedin.b7f5bab2ab7602febe3216fc910827f5","connection":"com.linkedin.cc70ada274dbeca435a87c70e5f65742","noConnection":"com.linkedin.8b38bd16089709d88ed7104608137baf"}}}},"baseType":"com.linkedin.voyager.dash.relationships.MemberRelationship"},"com.linkedin.eafc20924d49659eb7e0335979bc8d29":{"fields":{"fullLastNameShown":{"type":"boolean"},"requireReferral":{"type":"boolean"},"showPublicProfile":{"type":"boolean"},"formerNameVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"messagingSeenReceipts":{"type":"com.linkedin.voyager.dash.identity.profile.InstantMessagingVisibilitySetting"},"discloseAsProfileViewer":{"type":"com.linkedin.voyager.dash.identity.profile.DiscloseAsProfileViewerInfo"},"allowProfileEditBroadcasts":{"type":"boolean"},"messagingTypingIndicators":{"type":"com.linkedin.voyager.dash.identity.profile.InstantMessagingVisibilitySetting"},"allowOpenProfile":{"type":"boolean"},"profilePictureVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.PrivacySettingsUrn"},"publicProfilePictureVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"namePronunciationVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"},"pronounVisibilitySetting":{"type":"com.linkedin.voyager.dash.identity.profile.NetworkVisibilitySetting"}},"baseType":"com.linkedin.voyager.dash.identity.profile.PrivacySettings"},"com.linkedin.6fdcc7e0d8a5276f82b704ee433b31cc":{"fields":{"title":{"type":"string"},"entityUrn":{"type":"com.linkedin.voyager.dash.common.LearningCourseUrn"}},"baseType":"com.linkedin.voyager.dash.learning.LearningCourse"},"com.linkedin.c1233c6614d75709cf8a4843c636852c":{"fields":{"image":{"derivedFrom":"imageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"nameSupplementaryInfo":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"footerText":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"imageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"backgroundImage":{"derivedFrom":"backgroundImageUnion","type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}},"subHeadline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"name":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"insightText":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"headline":{"type":"com.linkedin.87507521b880a59f51dfe7916463212a"},"backgroundImageUnion":{"type":{"union":{"url":"com.linkedin.common.Url","vectorImage":"com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"}}}},"baseType":"com.linkedin.voyager.dash.common.media.StickerLinkLargeTemplateView"}}}},"included":[{"entityUrn":"urn:li:fsd_geo:102890719","$recipeTypes":["com.linkedin.20bf1398649ed8da5a403515fbffb6bf"],"defaultLocalizedName":"Nederland","$type":"com.linkedin.voyager.dash.common.Geo"},{"defaultLocalizedNameWithoutCountryName":"Eindhoven, Noord-Brabant","*country":"urn:li:fsd_geo:102890719","entityUrn":"urn:li:fsd_geo:102922143","$recipeTypes":["com.linkedin.20bf1398649ed8da5a403515fbffb6bf"],"defaultLocalizedName":"Eindhoven, Noord-Brabant, Nederland","$type":"com.linkedin.voyager.dash.common.Geo"},{"fullLastNameShown":true,"requireReferral":false,"showPublicProfile":true,"formerNameVisibilitySetting":"PUBLIC","discloseAsProfileViewer":"DISCLOSE_FULL","messagingSeenReceipts":"DISABLED","$recipeTypes":["com.linkedin.eafc20924d49659eb7e0335979bc8d29"],"allowProfileEditBroadcasts":false,"$type":"com.linkedin.voyager.dash.identity.profile.PrivacySettings","messagingTypingIndicators":"DISABLED","allowOpenProfile":false,"profilePictureVisibilitySetting":"PUBLIC","entityUrn":"urn:li:fsd_privacySettings:singleton","publicProfilePictureVisibilitySetting":"PUBLIC","namePronunciationVisibilitySetting":"MEMBERS","pronounVisibilitySetting":"MEMBERS"},{"birthDateOn":null,"lastName":"Madej","memorialized":false,"objectUrn":"urn:li:member:305921936","multiLocaleLastName":[{"value":"Madej","key":"en_US"},{"value":"Madej","key":"pl_PL"}],"showPremiumSubscriberBadge":false,"tempStatusEmoji":null,"multiLocaleFirstName":[{"value":"Krystian","key":"en_US"},{"value":"Krystian","key":"pl_PL"}],"memberPostAnalytics":null,"premium":false,"influencer":false,"entityUrn":"urn:li:fsd_profile:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ","multiLocaleTempStatus":[],"pronoun":null,"headlineGeneratedSuggestionDelegateUrn":"urn:li:fsu_profileActionDelegate:-1839964074","headline":"Senior Software Designer","publicIdentifier":"krystian-madej","trackingId":"J+wt7gHBTdStvrm9wx4FWw==","topVoiceBadge":null,"creator":false,"supportedLocales":[{"variant":null,"country":"US","language":"en","$recipeTypes":["com.linkedin.a4044e55c3fcc69e2de7519872763501"],"$type":"com.linkedin.common.Locale"},{"variant":null,"country":"PL","language":"pl","$recipeTypes":["com.linkedin.a4044e55c3fcc69e2de7519872763501"],"$type":"com.linkedin.common.Locale"}],"*memberRelationship":"urn:li:fsd_memberRelationship:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ","creatorInfo":null,"versionTag":"3131526026","$recipeTypes":["com.linkedin.eb60c5c4f2ae9910a2e0643e3628b172"],"$type":"com.linkedin.voyager.dash.identity.profile.Profile","firstName":"Krystian","profilePicture":{"displayImageReferenceResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"],"artifacts":[{"width":100,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"100_100/0/1523376389588?e=1701302400&amp;v=beta&amp;t=Jfwd7yy-uVy1nU9eKVQ3rxspV8RLh8GoG1EagGIlMdk","expiresAt":1701302400000,"height":100,"$type":"com.linkedin.common.VectorArtifact"},{"width":200,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"200_200/0/1523376389588?e=1701302400&amp;v=beta&amp;t=gqc3A89TEU-GPyV7bIC2Jv1IgejxVLDqjZvfBjjOSMg","expiresAt":1701302400000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":400,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"400_400/0/1523376389588?e=1701302400&amp;v=beta&amp;t=rYLx2_utszX-G0cbseRT2rRQqfHtne1y9nsFH3Nr6fM","expiresAt":1701302400000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":800,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"800_800/0/1523376389588?e=1701302400&amp;v=beta&amp;t=sVxr-zUQl9Tm14X8JCUdunZSI7qu5N6nBkxuz-ovc0M","expiresAt":1701302400000,"height":800,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C4D03AQErIf349st-lg/profile-displayphoto-shrink_","$type":"com.linkedin.common.VectorImage"}},"originalImageUrn":null,"displayImageWithFrameReference":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"],"artifacts":[{"width":200,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"200_200/0/1693843853826?e=1696248000&amp;v=beta&amp;t=Lc9kt7BO17I1NE492Y31keVc0v6k3jQ6du5XxbthGw0","expiresAt":1696248000000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":400,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"400_400/0/1693843853826?e=1696248000&amp;v=beta&amp;t=wyX6G_n4KLcsPsLe21DcUkzmkOqbdAYYbQFieWBI9wA","expiresAt":1696248000000,"height":400,"$type":"com.linkedin.common.VectorArtifact"},{"width":800,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"800_800/0/1693843853826?e=1696248000&amp;v=beta&amp;t=k6o1iIRDQh-h_dHIVTvKern1hjS7cByiEuZrK_KbZy8","expiresAt":1696248000000,"height":800,"$type":"com.linkedin.common.VectorArtifact"},{"width":100,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"100_100/0/1693843853826?e=1696248000&amp;v=beta&amp;t=gDIVsZmhUGvNCoBmpOtjnFWLOLZtdz7qL2dGfj-XnJc","expiresAt":1696248000000,"height":100,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/D4E35AQEA7ib1HDsfCA/profile-framedphoto-shrink_","$type":"com.linkedin.common.VectorImage"}},"originalImageReferenceResolutionResult":null,"frameType":"OPEN_TO_WORK","$recipeTypes":["com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"],"photoFilterEditInfo":{"bottomLeft":{"x":-6.178632484870436E-17,"y":0.9989583333333333,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"saturation":0.0,"brightness":0.0,"vignette":0.0,"photoFilterType":"ORIGINAL","bottomRight":{"x":1.0,"y":0.9989583333333333,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"topLeft":{"x":-6.178632484870436E-17,"y":0.0010416666666664927,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"contrast":0.0,"topRight":{"x":1.0,"y":0.0010416666666664927,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"$recipeTypes":["com.linkedin.4c7a6a878ae117485d617b8a09075c76"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterEditInfo"},"displayImageUrn":"urn:li:digitalmediaAsset:C4D03AQErIf349st-lg","$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"geoLocation":{"*geo":"urn:li:fsd_geo:102922143","$recipeTypes":["com.linkedin.d16b47c42001019f9bb43f9ce18412ef"],"postalCode":null,"$type":"com.linkedin.voyager.dash.identity.profile.ProfileGeoLocation"},"multiLocaleMaidenName":[],"tempStatus":null,"*privacySettings":"urn:li:fsd_privacySettings:singleton","showVerificationBadge":true,"profileGeneratedSuggestionPromo":{"$recipeTypes":["com.linkedin.fb515f37046fc34f7a17064266659842"],"elements":[],"$type":"com.linkedin.restli.common.CollectionResponse"},"backgroundPicture":{"displayImageReferenceResolutionResult":{"url":null,"vectorImage":{"digitalmediaAsset":null,"attribution":null,"$recipeTypes":["com.linkedin.db6ef1d2cb1c6ae1f3db4d47412bb164"],"artifacts":[{"width":800,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"200_800/0/1615934876702?e=1701302400&amp;v=beta&amp;t=MRGX9D_DIQukFYJ68i5evDERgtJBGyNIl5bnLwkbixQ","expiresAt":1701302400000,"height":200,"$type":"com.linkedin.common.VectorArtifact"},{"width":1400,"$recipeTypes":["com.linkedin.5f3d321f9b6631058c0c944d77e41938"],"fileIdentifyingUrlPathSegment":"350_1400/0/1615934876702?e=1701302400&amp;v=beta&amp;t=7Rglcy3pMKVdMfQgTZf7vqjA2-jTHocI8pZOLqIZTyU","expiresAt":1701302400000,"height":350,"$type":"com.linkedin.common.VectorArtifact"}],"rootUrl":"https://media.licdn.com/dms/image/C4E16AQHG32g-LtiiGQ/profile-displaybackgroundimage-shrink_","$type":"com.linkedin.common.VectorImage"}},"originalImageUrn":null,"originalImageReferenceResolutionResult":null,"$recipeTypes":["com.linkedin.9b6c35b7f98abda686c8bb17ab3dfb85"],"photoFilterEditInfo":{"bottomLeft":{"x":8.167157882300002E-17,"y":0.8773584905660375,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"saturation":0.0,"brightness":0.0,"vignette":0.0,"photoFilterType":"ORIGINAL","bottomRight":{"x":1.0,"y":0.8773584905660375,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"topLeft":{"x":8.167157882300002E-17,"y":0.22075471698113192,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"contrast":0.0,"topRight":{"x":1.0,"y":0.22075471698113192,"$recipeTypes":["com.linkedin.5fba45f960b9273c3b1bc933082ed44c"],"$type":"com.linkedin.common.Coordinate2D"},"$recipeTypes":["com.linkedin.4c7a6a878ae117485d617b8a09075c76"],"$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterEditInfo"},"displayImageUrn":"urn:li:digitalmediaAsset:C4E16AQHG32g-LtiiGQ","$type":"com.linkedin.voyager.dash.identity.profile.PhotoFilterPicture"},"multiLocaleHeadline":[{"value":"Senior Software Designer","key":"en_US"},{"value":"Senior Software Designer","key":"pl_PL"}],"primaryLocale":{"country":"US","extensions":null,"variant":null,"language":"en","$recipeTypes":["com.linkedin.a4044e55c3fcc69e2de7519872763501"],"script":null,"$type":"com.linkedin.common.Locale"},"tempStatusExpiredAt":null},{"entityUrn":"urn:li:fsd_memberRelationship:ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ","memberRelationship":{"self":null,"connection":null,"noConnection":{"memberDistance":"OUT_OF_NETWORK","$recipeTypes":["com.linkedin.8b38bd16089709d88ed7104608137baf"],"$type":"com.linkedin.voyager.dash.relationships.NoConnection"}},"$recipeTypes":["com.linkedin.de96d1e9b7a124ebb464644d217619fc"],"$type":"com.linkedin.voyager.dash.relationships.MemberRelationship"}]}
</code>
<code style="display: none" id="datalet-bpr-guid-6231354">
  {"request":"/voyager/api/graphql?includeWebMetadata\u003Dtrue\u0026variables\u003D(vanityName:krystian-madej)\u0026\u0026queryId\u003DvoyagerIdentityDashProfiles.9faa7ab476db884bf471a9361f87aaf2","status":200,"body":"bpr-guid-6231354","method":"GET","headers":{"x-li-uuid":"AAYGLPO3crcczrAKuxHpzw\u003D\u003D"}}
</code>
<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="datalet-bpr-guid-6231354"><code style="display: none" id="clientPageInstance">
  urn:li:page:d_flagship3_profile_view_base;3dfedaeb-ee6c-4ab5-bcfa-845675830e10
</code>

<img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" style="display: none" class="terminatorlet">



    <div id="a11y-notification" class="visually-hidden" role="region" aria-live="polite"></div>
  





  <svg role="none" xmlns="http://www.w3.org/2000/svg" id="hue-web-icons-sprite" style="display: none;">
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="search-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M14.56 12.44L11.3 9.18a5.51 5.51 0 10-2.12 2.12l3.26 3.26a1.5 1.5 0 102.12-2.12zM3 6.5A3.5 3.5 0 116.5 10 3.5 3.5 0 013 6.5z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="caret-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M8 11L3 6h10z" fill-rule="evenodd"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="overflow-web-ios-medium" data-supported-dps="24x24" fill="currentColor">
  <path d="M14 12a2 2 0 11-2-2 2 2 0 012 2zM4 10a2 2 0 102 2 2 2 0 00-2-2zm16 0a2 2 0 102 2 2 2 0 00-2-2z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="verified-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M11.99 22l-1.23-.44C6.11 19.81 2.99 16 2.99 11V5L12 2l9 3v6c0 5-3.11 8.81-7.74 10.56l-1.27.44zM5 6.44V11c0 4.11 2.6 7.35 6.46 8.8l.54.2.58-.2C16.41 18.35 19 15.1 19 11V6.44l-7-2.32-7 2.32zM17 8h-2.57l-4.02 5.01-2.18-2.18-1.41 1.41 3.75 3.75 6.43-8z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="rtl-flip" id="arrow-right-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" id="overflow-web-ios-small" data-supported-dps="16x16" fill="currentColor">
  <path d="M3 9.5A1.5 1.5 0 114.5 8 1.5 1.5 0 013 9.5zM11.5 8A1.5 1.5 0 1013 6.5 1.5 1.5 0 0011.5 8zm-5 0A1.5 1.5 0 108 6.5 1.5 1.5 0 006.5 8z"></path>
</svg>

    <svg id="filter-small" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor">
  <path d="M15,4H6.72c-.35,.6-.98,1-1.72,1s-1.38-.4-1.72-1H1V2H3.28c.35-.6,.98-1,1.72-1s1.38,.4,1.72,1H15v2Zm0,8H6.72c-.35-.6-.98-1-1.72-1s-1.38,.4-1.72,1H1v2H3.28c.35,.6,.98,1,1.72,1s1.38-.4,1.72-1H15v-2Zm0-5h-2.28c-.35-.6-.98-1-1.72-1s-1.38,.4-1.72,1H1v2H9.28c.35,.6,.98,1,1.72,1s1.38-.4,1.72-1h2.28v-2Z"></path>
</svg>
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" class="rtl-flip" id="question-medium" data-supported-dps="24x24" fill="currentColor">
  <path d="M12 2a10 10 0 1010 10A10 10 0 0012 2zm0 16.25A1.25 1.25 0 1113.25 17 1.25 1.25 0 0112 18.25zm1.41-5.46L13 13v1h-2v-2.21l1.49-.79C13.82 10.34 14 9.77 14 9.3c0-.78-.92-1.3-2.3-1.3A7.12 7.12 0 008 9.24V7a8 8 0 013.7-1c3 0 4.3 1.55 4.3 3.3a3.91 3.91 0 01-2.59 3.49z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="settings-medium" data-supported-dps="24x24" fill="currentColor">
  <path d="M9.18 2l-4.3 2.52L6.22 8l-.52.91-3.7.55v5l3.64.54.54.93-1.34 3.53L9.14 22l2.29-2.9h1.09l2.3 2.9 4.32-2.52L17.79 16l.53-.93 3.68-.53v-5L18.32 9l-.51-.9 1.36-3.51L14.86 2l-2.33 3h-1zM12 9a3 3 0 11-3 3 3 3 0 013-3z"></path>
</svg>

    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" id="shield-medium" aria-hidden="true" role="none" data-supported-dps="24x24" fill="currentColor">
  <path d="M12 2L3 5v6c0 5 3.12 8.81 7.77 10.56L12 22l1.27-.44C17.9 19.81 21.01 16 21.01 11V5L12 2zm-.53 17.8C7.6 18.35 5.01 15.1 5.01 11V6.44l7-2.32V20l-.54-.2z"></path>
</svg>

</svg>

  <!---->

  
<!---->  

<!---->
<div class="application-outlet">
<!---->
        
    <div class="global-nav__a11y-menu
        ">
      <div class="global-nav__a11y-menu-container">
          <button id="ember8" class="mr4 artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To search function</font></font>
</span></button>

        <button id="ember9" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Skip to main content</font></font>
</span></button>

<!---->
        <button id="ember10" class="global-nav__a11y-menu-close artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--2 artdeco-button--tertiary ember-view">  <li-icon aria-hidden="true" type="close" class="artdeco-button__icon"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M13.42 12L20 18.58 18.58 20 12 13.42 5.42 20 4 18.58 10.58 12 4 5.42 5.42 4 12 10.58 18.58 4 20 5.42z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Close menu</font></font>
</span></button>
      </div>
    </div>
  
      
    <header id="global-nav" class="global-nav global-alert-offset-top
        
        global-nav--visible">
      <div class="global-nav__content">
          
    <a class="app-aware-link " href="https://www.linkedin.com/feed/?nis=true" data-test-app-aware-link="">
      
    <div class="ivm-image-view-model    global-nav__branding-logo">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon type="app-linkedin-bug-color-icon" class=" " size="large" role="img" aria-label="LinkedIn"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M20.5 2h-17A1.5 1.5 0 002 3.5v17A1.5 1.5 0 003.5 22h17a1.5 1.5 0 001.5-1.5v-17A1.5 1.5 0 0020.5 2zM8 19H5v-9h3zM6.5 8.25A1.75 1.75 0 118.3 6.5a1.78 1.78 0 01-1.8 1.75zM19 19h-3v-4.74c0-1.42-.6-1.93-1.38-1.93A1.74 1.74 0 0013 14.19a.66.66 0 000 .14V19h-3v-9h2.9v1.3a3.11 3.11 0 012.7-1.4c1.55 0 3.36.86 3.36 3.66z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
    </a>
  

        <div id="global-nav-search" class="global-nav__search
            ">
          
          
    <div class="search-global-typeahead
        
         global-nav__search-typeahead">
        
    <div id="global-nav-typeahead" class="search-global-typeahead__typeahead">
<!---->        
          
    <input class="search-global-typeahead__input" placeholder="To search" role="combobox" aria-autocomplete="list" aria-label="To search" aria-activedescendant="" aria-expanded="false" type="text">
  
          <div aria-hidden="true" class="search-global-typeahead__search-icon-container">
            <svg role="none" aria-hidden="true" class="search-global-typeahead__search-icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="search-small">
<!---->    

    <use href="#search-small" width="16" height="16"></use>
</svg>

          </div>
<!---->          <div class="search-global-typeahead__overlay global-alert-offset-top
              "></div>
<!---->        
    </div>
  
      <button class="search-global-typeahead__collapsed-search-button" aria-label="Click to start a search" type="button">
        <li-icon aria-hidden="true" type="search" class="search-global-typeahead__collapsed-search-button-icon t-black--light" size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21.41 18.59l-5.27-5.28A6.83 6.83 0 0017 10a7 7 0 10-7 7 6.83 6.83 0 003.31-.86l5.28 5.27a2 2 0 002.82-2.82zM5 10a5 5 0 115 5 5 5 0 01-5-5z"></path>
    </svg></li-icon>
        <div class="search-global-typeahead__collapsed-search-button-text t-black--light t-12 t-normal">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To search</font></font>
        </div>
      </button>
      
  <div id="ember12" class="ember-view"><!----></div>

    </div>
  
      
        </div>

        <nav class="global-nav__nav" aria-label="Main navigation">
          <ul class="global-nav__primary-items
              ">
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/feed/?nis=true&amp;" data-test-app-aware-link="">
      

      <div id="ember13" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
        <span aria-hidden="true" class="notification-badge__no-count"></span>
        <span class="a11y-text" data-test-notification-a11y=""><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">new Feed updates notifications</font></font></span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="home" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 9v2h-2v7a3 3 0 01-3 3h-4v-6h-4v6H6a3 3 0 01-3-3v-7H1V9l11-7z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Home">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Home</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/mynetwork/?" data-test-app-aware-link="">
      

      <div id="ember14" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
          <span aria-hidden="true" class="notification-badge__count "><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2</font></font></span>
          <span class="a11y-text" data-test-notification-a11y=""><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2 new network updates notifications</font></font></span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="people" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M12 16v6H3v-6a3 3 0 013-3h3a3 3 0 013 3zm5.5-3A3.5 3.5 0 1014 9.5a3.5 3.5 0 003.5 3.5zm1 2h-2a2.5 2.5 0 00-2.5 2.5V22h7v-4.5a2.5 2.5 0 00-2.5-2.5zM7.5 2A4.5 4.5 0 1012 6.5 4.49 4.49 0 007.5 2z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="My network">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">My network</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/jobs/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="job" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17 6V5a3 3 0 00-3-3h-4a3 3 0 00-3 3v1H2v4a3 3 0 003 3h14a3 3 0 003-3V6zM9 5a1 1 0 011-1h4a1 1 0 011 1v1H9zm10 9a4 4 0 003-1.38V17a3 3 0 01-3 3H5a3 3 0 01-3-3v-4.38A4 4 0 005 14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="Vacancies">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Vacancies</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/messaging/?" data-test-app-aware-link="">
      

        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="nav-small-messaging-icon" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M16 4H8a7 7 0 000 14h4v4l8.16-5.39A6.78 6.78 0 0023 11a7 7 0 00-7-7zm-8 8.25A1.25 1.25 0 119.25 11 1.25 1.25 0 018 12.25zm4 0A1.25 1.25 0 1113.25 11 1.25 1.25 0 0112 12.25zm4 0A1.25 1.25 0 1117.25 11 1.25 1.25 0 0116 12.25z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="To inform">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To inform</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
                  
      
    <a class="app-aware-link  global-nav__primary-link" target="_self" href="https://www.linkedin.com/notifications/?" data-test-app-aware-link="">
      

      <div id="ember15" class="global-nav__primary-link-notif artdeco-notification-badge ember-view">  <span class="notification-badge notification-badge--show ">
          <span aria-hidden="true" class="notification-badge__count "><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">11</font></font></span>
          <span class="a11y-text" data-test-notification-a11y=""><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">11 new notifications</font></font></span>
  </span>
  
        
    <div class="ivm-image-view-model    global-nav__icon-ivm
            ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="bell-fill" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M22 19h-8.28a2 2 0 11-3.44 0H2v-1a4.52 4.52 0 011.17-2.83l1-1.17h15.7l1 1.17A4.42 4.42 0 0122 18zM18.21 7.44A6.27 6.27 0 0012 2a6.27 6.27 0 00-6.21 5.44L5 13h14z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  
      
</div>
  

      <span class="t-12 break-words block t-black--light t-normal
          global-nav__primary-link-text" title="notifications">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">notifications</font></font>
      </span>
    </a>
  

<!---->  
<!----><!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!---->                  
    <div id="ember16" class="global-nav__me artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
      <button aria-expanded="false" id="ember17" class="global-nav__primary-link global-nav__primary-link-me-menu-trigger artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
        <img width="24" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" height="24" alt="Fayyoz Usmon" id="ember18" class="global-nav__me-photo evi-image ghost-person ember-view">

        <span class="global-nav__primary-link-text">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">I</font></font>
          <svg role="none" aria-hidden="true" class="global-nav__icon global-nav__icon--small" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
<!----></button>

      <div tabindex="-1" aria-hidden="true" id="ember19" class="global-nav__me-content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
    </div>

<!---->  
<!----><!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!---->                  
    
    
      
  <button aria-expanded="false" class="global-nav__primary-link
          global-nav__primary-item--divider pl3
          global-nav__app-launcher-trigger" type="button">
    
          
    <div class="ivm-image-view-model    global-nav__icon-ivm">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="grid" class=" " size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M3 3h4v4H3zm7 4h4V3h-4zm7-4v4h4V3zM3 14h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4zM3 21h4v-4H3zm7 0h4v-4h-4zm7 0h4v-4h-4z"></path>
    </svg></li-icon>
    </div>
  
          </div>
  

        <span class="global-nav__primary-link-text" title="For companies">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">For companies</font></font>
          <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

        </span>
      
  </button>


      <!---->
    

    <div id="ember20" class="ember-view"><!----></div>
  
  
<!---->              </li>
              <li class="global-nav__primary-item
                  ">
<!----><!----><!---->                  
<!---->
      
        
          <div class="premium-upsell-link">
            <a href="https://www.linkedin.com/premium/products/?upsellOrderOrigin=premium_nav_upsell_text&amp;referenceId=sH42Ay32QVy8LZYHybv6Dg%3D%3D&amp;destRedirectURL=https%3A%2F%2Fwww.linkedin.com%2Fin%2Fkrystian-madej%2F" class="link-without-visited-state global-nav__primary-link global-nav__primary-link--premium global-nav__spotlight-upsell premium-upsell-link--truncate premium-upsell-link--long">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Try Premium for free</font></font>
            </a>
          </div>
      
        
  
              </li>

<!---->          </ul>
        </nav>
      </div>
    </header>
  

    
    <div class="videoinappalert-inapp-alerts-manager
        hidden">
      
<!---->    
    </div>
  

<!---->
<div class="authentication-outlet">
<!---->  <div id="profile-content" class="extended tetris pv-profile-body-wrapper">
  <div class="body">
    

      <div class="scaffold-layout__tracking-element"></div>

<div class="scaffold-layout
    scaffold-layout--breakpoint-xl
    scaffold-layout--main-aside
    
    scaffold-layout--reflow
    
     pv-profile">
    <section class="scaffold-layout-toolbar
    scaffold-layout-toolbar--is-fixed
    
    ">
  <div class="scaffold-layout-toolbar__content scaffold-layout-container
      scaffold-layout-container--reflow
      ">
    
      
        
    
    <div class="pv-profile-sticky-header-v2__container pv1">
      <div id="ember25" class="artdeco-entity-lockup artdeco-entity-lockup--size-1 ember-view pv-profile-sticky-header-v2__mini-profile-container">
        

    <div class="presence-entity presence-entity--size-1 m1">
      <img src="https://media.licdn.com/dms/image/C4D03AQErIf349st-lg/profile-displayphoto-shrink_100_100/0/1523376389588?e=1701302400&amp;v=beta&amp;t=Jfwd7yy-uVy1nU9eKVQ3rxspV8RLh8GoG1EagGIlMdk" loading="lazy" alt="Krystian Madej" id="ember26" class="presence-entity__image   EntityPhoto-circle-1 evi-image lazy-image ember-view">

      

    <div class="presence-entity__indicator
          
          presence-entity__indicator--size-1 presence-indicator
        hidden
        presence-indicator--size-1">
      <span class="visually-hidden">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Status offline</font></font>
              </span>
    </div>
  
    </div>
  
        <div id="ember27" class="artdeco-entity-lockup__content ember-view overflow-hidden ml1 align-self-flex-start">
          <div id="ember28" class="artdeco-entity-lockup__title ember-view">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Krystian Madej</font></font>
<!---->          
</div>

          <div id="ember29" class="artdeco-entity-lockup__subtitle ember-view truncate">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font>
          </div>
        </div>
      
</div>
      <div class="pv-profile-sticky-header-v2__actions-container">
        
      
  <div class="display-flex inline-flex">
        <div class="pvs-sticky-header-profile-actions inline-flex">
      <div class="pvs-sticky-header-profile-actions__action">
        <div id="ember124" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
  <button aria-expanded="false" aria-label="More actions" id="ember125" class="artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view pvs-profile-actions__action artdeco-button artdeco-button--secondary artdeco-button--muted
      artdeco-button--2
      
      " type="button" tabindex="0">
      <span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">More</font></font></span>
  
<!----></button>
  <div tabindex="-1" aria-hidden="true" id="ember126" class="pvs-overflow-actions-dropdown__content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-bottom ember-view"><div class="artdeco-dropdown__content-inner">
  
    <ul>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Send Krystian Madej's profile via message" role="button" id="ember127" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
    <li-icon aria-hidden="true" type="share-linkedin" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 12l-4.61 7H16l4-6H8a3.92 3.92 0 00-4 3.84V17a4 4 0 00.19 1.24L5.12 21H3l-.73-2.22A6.4 6.4 0 012 16.94 6 6 0 018 11h12l-4-6h2.39z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Send profile in message</font></font>
    </span>
  </div>




        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Save Krystian Madej's profile as PDF" role="button" id="ember128" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="download" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 14v5a3 3 0 01-3 3H6a3 3 0 01-3-3v-5h2v5a1 1 0 001 1h12a1 1 0 001-1v-5zm-4-.57V11l-4 2.85V3h-2v10.85L7 11v2.43L12 17z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Save as PDF</font></font></span>
  </div>



<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
    
    
    <div aria-label="Follow Krystian Madej" role="button" id="ember129" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="add" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 13h-8v8h-2v-8H3v-2h8V3h2v8h8z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font></span>
    </div>



<!---->  

<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Report or block Krystian Madej's profile" role="button" id="ember130" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="report" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17.9 4.38a5.82 5.82 0 01-1.9.31 6.1 6.1 0 01-1.9-.31l-3.2-1.07A6.41 6.41 0 009 3a5.57 5.57 0 00-1.9.31L6 3.67V2H4v20h2v-8l1.1-.37a6.41 6.41 0 011.9-.32 5.82 5.82 0 011.9.31l3.2 1.07a6 6 0 003.8 0L20 14V3.68z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Report/block</font></font></span>
  </div>


<!---->

<!---->
<!---->


        </li>

        <li>
          
    <div role="button" id="ember131" class="display-flex align-items-center full-width  artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="signal-notice" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M18 3H6a3 3 0 00-3 3v12a3 3 0 003 3h12a3 3 0 003-3V6a3 3 0 00-3-3zm-4 15h-1a3 3 0 01-3-3 3.22 3.22 0 01.1-.75L11.2 10h2.07L12 14.75A1 1 0 0013 16h1zm-1-9.75A1.25 1.25 0 1114.25 7 1.25 1.25 0 0113 8.25z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
      <span class="a11y-text" aria-live="off"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
    </div>

        </li>
    </ul>
  
</div>
</div>
</div>
      </div>
<!---->      <!---->
<!---->
<!---->
        

    <div class="entry-point profile-action-compose-option">
      
<!---->  

          
      <button aria-label="Send Krystian a message" id="ember181" class="artdeco-button artdeco-button--2 artdeco-button--secondary ember-view pvs-sticky-header-profile-actions__action"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Message</font></font>
</span></button>
    
    </div>
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


            
      
        
    
          
        
    <button aria-label="Send connection request to Krystian Madej" id="ember133" class="artdeco-button artdeco-button--2 artdeco-button--primary ember-view pvs-sticky-header-profile-actions__action" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


  </div>
  <!---->


<!---->
<!---->  </div>

    
      </div>
    </div>
  
  
      
    
  </div>
</section>

  <div class="scaffold-layout__inner scaffold-layout-container
      scaffold-layout-container--reflow
      ">
<!---->
      <div class="scaffold-layout__row scaffold-layout__content
          scaffold-layout__content--main-aside
          
          
          scaffold-layout__content--has-aside
          
          ">
<!---->
          <main class="scaffold-layout__main">
  
            
        
<!----><!---->    
    <section id="ember31" class="artdeco-card ember-view pv-top-card" data-member-id="305921936"><!---->

      
  <div class="live-video-hero-image
      " style="min-height: 201px; max-height: 201px;">
    <div class="live-video-hero-image__bg-image">
      
            
    <div tabindex="-1" class="profile-background-image">
        <div class="profile-background-image__image-container">
            <img class="profile-background-image__image relative
                full-width full-height" id="profile-background-image-target-image" alt="Background image" src="https://media.licdn.com/dms/image/C4E16AQHG32g-LtiiGQ/profile-displaybackgroundimage-shrink_350_1400/0/1615934876702?e=1701302400&amp;v=beta&amp;t=7Rglcy3pMKVdMfQgTZf7vqjA2-jTHocI8pZOLqIZTyU" style="left:0px;top:0px;">
        </div>
    </div>
  
        
    </div>
    <div class="live-video-hero-image__live-video">
<!---->    </div>
  </div>

      <div class="ph5 ">
        <div class="display-flex">
          <div class="pv-top-card--photo text-align-left
              pv-top-card--photo-resize">
                  
    <div class="pv-top-card__non-self-photo-wrapper ml0">
      
    

      <button class="pv-top-card-profile-picture pv-top-card-profile-picture__container display-block
          pv-top-card__photo presence-entity__image EntityPhoto-circle-9
          " type="button">
        
      
          
        <img width="200" title="Krystian Madej, #OPEN_TO_WORK" src="https://media.licdn.com/dms/image/D4E35AQEA7ib1HDsfCA/profile-framedphoto-shrink_200_200/0/1693843853826?e=1696248000&amp;v=beta&amp;t=Lc9kt7BO17I1NE492Y31keVc0v6k3jQ6du5XxbthGw0" height="200" alt="Krystian Madej, #OPEN_TO_WORK" id="ember32" class="pv-top-card-profile-picture__image pv-top-card-profile-picture__image--show evi-image ember-view">

<!---->
<!---->    
        
  
      </button>
  
  
<!---->    </div>
  

          </div>
          <div class="flex-1 flex-column">
            <div class="pv-top-card__badge-wrap">
<!----><!----><!---->            </div>
          </div>
        </div>

        
    <div class="mt2 relative">
      <div class="pv-text-details__left-panel
          ">
        <div>
            <a href="/in/krystian-madej/overlay/about-this-profile/" id="ember33" class="ember-view pv-text-details__about-this-profile-entrypoint">
              <h1 class="text-heading-xlarge inline t-24 v-align-middle break-words"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Krystian Madej</font></font></h1>
                <svg role="img" aria-hidden="false" class="v-align-middle t-black--light" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="verified-medium">
    <title>Krystian has verifications</title>
    

    <use href="#verified-medium" width="24" height="24"></use>
</svg>

<!---->            </a>

<!----><!----><!---->        </div>
<!---->        <div class="text-body-medium break-words" data-generated-suggestion-target="urn:li:fsu_profileActionDelegate:-1839964074">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font>
        </div>
<!----><!----><!---->      </div>
        <ul class="pv-text-details__right-panel">
            <li class="pv-text-details__right-panel-item">
              <button class="pv-text-details__right-panel-item-link text-align-left" aria-label="Current company: ASML.  Click to skip this and go to experience map" type="button">
                <img src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" alt="" id="ember134" class="evi-image ember-view pv-text-details__right-panel-item-text-image EntityPhoto-square-1">
                <span class="pv-text-details__right-panel-item-text hoverable-link-text break-words text-body-small t-black">
                  
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         inline" style="-webkit-line-clamp:2;" tabindex="-1">

        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML</font></font>
      
<!---->    </div>
  
                </span>
              </button>
            </li>
<!---->        </ul>
      <div class="pv-text-details__left-panel mt2">
          <span class="text-body-small inline t-black--light break-words">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Eindhoven, North Brabant, Netherlands</font></font>
          </span>
          <span class="pv-text-details__separator t-black--light">
            <a href="/in/krystian-madej/overlay/contact-info/" id="top-card-text-details-contact-info" class="ember-view link-without-visited-state cursor-pointer text-heading-small inline-block break-words">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Contact details</font></font>
            </a>
          </span>
      </div>
    </div>
  
<!---->
        
    <ul class="pv-top-card--list pv-top-card--list-bullet">
<!---->
        <li class="text-body-small">
            <span class="t-black--light">
              <span class="t-bold"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">456</font></font></span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;"> connections</font></font>
            </span>
        </li>
    </ul>
  
<!---->
        

  <div class="pv-top-card-v2-ctas
      ">
<!---->
<!---->
      <div class="pvs-profile-actions
    
    ">
          
      
        
    
          
        
    <button aria-label="Send connection request to Krystian Madej" id="ember135" class="artdeco-button artdeco-button--2 artdeco-button--primary ember-view pvs-profile-actions__action" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->



    <!---->
<!---->
<!---->
        

    <div class="entry-point profile-action-compose-option">
      
<!---->  

          
      <button aria-label="Send Krystian a message" id="ember198" class="artdeco-button artdeco-button--2 artdeco-button--secondary ember-view pvs-profile-actions__action"><!---->
<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Message</font></font>
</span></button>
    
    </div>
  


<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->



<!---->
<!---->
<!---->
    <div id="ember137" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view">
  <button aria-expanded="false" aria-label="More actions" id="ember138" class="artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view pvs-profile-actions__action artdeco-button artdeco-button--secondary artdeco-button--muted
      artdeco-button--2
      
      " type="button" tabindex="0">
      <span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">More</font></font></span>
  
<!----></button>
  <div tabindex="-1" aria-hidden="true" id="ember139" class="pvs-overflow-actions-dropdown__content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-bottom ember-view"><div class="artdeco-dropdown__content-inner">
  
    <ul>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Send Krystian Madej's profile via message" role="button" id="ember140" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
    <li-icon aria-hidden="true" type="share-linkedin" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M23 12l-4.61 7H16l4-6H8a3.92 3.92 0 00-4 3.84V17a4 4 0 00.19 1.24L5.12 21H3l-.73-2.22A6.4 6.4 0 012 16.94 6 6 0 018 11h12l-4-6h2.39z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Send profile in message</font></font>
    </span>
  </div>




        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Save Krystian Madej's profile as PDF" role="button" id="ember141" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="download" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 14v5a3 3 0 01-3 3H6a3 3 0 01-3-3v-5h2v5a1 1 0 001 1h12a1 1 0 001-1v-5zm-4-.57V11l-4 2.85V3h-2v10.85L7 11v2.43L12 17z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Save as PDF</font></font></span>
  </div>



<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
    
    
    <div aria-label="Follow Krystian Madej" role="button" id="ember142" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="add" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M21 13h-8v8h-2v-8H3v-2h8V3h2v8h8z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font></span>
    </div>



<!---->  

<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->


        </li>
        <li>
          <!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
<!---->
      <div aria-label="Report or block Krystian Madej's profile" role="button" id="ember143" class="artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view full-width display-flex align-items-center" tabindex="0"><!---->
      <li-icon aria-hidden="true" type="report" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M17.9 4.38a5.82 5.82 0 01-1.9.31 6.1 6.1 0 01-1.9-.31l-3.2-1.07A6.41 6.41 0 009 3a5.57 5.57 0 00-1.9.31L6 3.67V2H4v20h2v-8l1.1-.37a6.41 6.41 0 011.9-.32 5.82 5.82 0 011.9.31l3.2 1.07a6 6 0 003.8 0L20 14V3.68z"></path>
    </svg></li-icon>
    <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Report/block</font></font></span>
  </div>


<!---->

<!---->
<!---->


        </li>

        <li>
          
    <div role="button" id="ember144" class="display-flex align-items-center full-width  artdeco-dropdown__item artdeco-dropdown__item--is-dropdown ember-view" tabindex="0"><!---->
        <li-icon aria-hidden="true" type="signal-notice" class="mr3 flex-grow-0" size="large"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M18 3H6a3 3 0 00-3 3v12a3 3 0 003 3h12a3 3 0 003-3V6a3 3 0 00-3-3zm-4 15h-1a3 3 0 01-3-3 3.22 3.22 0 01.1-.75L11.2 10h2.07L12 14.75A1 1 0 0013 16h1zm-1-9.75A1.25 1.25 0 1114.25 7 1.25 1.25 0 0113 8.25z"></path>
    </svg></li-icon>
      <span class="display-flex t-normal flex-1" aria-hidden="true"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
      <span class="a11y-text" aria-live="off"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About this profile</font></font></span>
    </div>

        </li>
    </ul>
  
</div>
</div>
</div>
</div>
<!----><!---->
<!---->
<!---->

  </div>

<!---->
      </div>
      <div id="profile-sticky-header-toggle"></div>
        
    <div class="pv-open-to-carousel ">
          
    <section id="ember61" class="artdeco-card ember-view pv-open-to-carousel-card
        pv-open-to-carousel-card--enrolled pv-open-to-carousel-card--single"><!---->

        <div class="display-flex justify-space-between pl3
            pr3">
          <a class="app-aware-link  pv-open-to-carousel-card__content link-without-hover-visited flex-grow-1 pv3" data-view-name="profile-opportunity-card" href="https://www.linkedin.com/in/krystian-madej/opportunities/job-opportunities/details?profileUrn=urn%3Ali%3Afs_normalized_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;trk=opento_nprofile_details" data-test-app-aware-link="">
            <h3 class="truncate text-body-small">
              <strong><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Available for work</font></font><!----></strong>
            </h3>
              <p class="truncate text-body-small">
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Positions for Architecture Specialist, Senior Software Engineer, Senior Software Engineer and Team Leader</font></font>
              </p>
            <p class="link-without-visited-state text-heading-small hoverable-link-text">
              <!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show details</font></font><!---->
            </p>
<!---->          </a>

<!---->        </div>
    </section>
  
      
<!---->
<!---->    </div>
  
<!---->    </section>
<!---->  
  
        
    
    
<!---->
  
  
        
<!---->    
<!---->  
  
        
        
<!---->
<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember105" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="about" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Info</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Info</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="display-flex ph5 pv3">
      
      <div class="
          display-flex full-width" data-generated-suggestion-target="urn:li:fsu_profileActionDelegate:-1603929916">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:4;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Producer with 10 years of professional experience in solving complex challenges in embedded systems. </font><font style="vertical-align: inherit;">Through the years I managed to acquire solid skills in majority of software development concepts, frameworks and methodologies. </font><font style="vertical-align: inherit;">Due to personal preferences my core language is C++ combined with Python and all libraries around those languages. </font><font style="vertical-align: inherit;">However I usually quickly adapt to the technology stack used in the project.</font></font><!----><br><br><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">I have started my journey in telecommunication industry at Nokia where I have been developing BTS firmware for global customers. </font><font style="vertical-align: inherit;">After that I moved into automotive systems where I could play around with ecodrive so gathering data from various car sensors to decide if the driver cares for the environment ;) Next adventure was Windows desktop CAD application which main purpose was assistance during the process of transformation into smart city by urban planners. </font><font style="vertical-align: inherit;">After playing a role in designing cities of the future I moved away from earth into air so into airport management systems. </font><font style="vertical-align: inherit;">In this project I was working on a system that synchronizes various subsystems which had many goals like improvement of life-critical safety, optimization of operations and many more. </font><font style="vertical-align: inherit;">Sadly I did not get free travels but experience gained was very nice ;-) My last and current project is global TOP 50 company ASML so embedded photolithography in which I am working in Optical Column group. </font><font style="vertical-align: inherit;">This group takes care of firmware for world's most complex and expensive chip manufacturing machine.</font></font><!----></span><span class="visually-hidden"><!---->Software Producer with 10 years of professional experience in solving complex challenges in embedded systems. Through the years I managed to acquire solid skills in majority of software development concepts, frameworks and methodologies. Due to personal preferences my core language is C++ combined with Python and all libraries around those languages. However I usually quickly adapt to the technology stack used in the project.

I have started my journey in telecommunication industry at Nokia where I have been developing BTS firmware for global customers. After that I moved into automotive systems where I could play around with ecodrive so gathering data from various car sensors to decide if the driver cares for the environment ;) Next adventure was Windows desktop CAD application which main purpose was assistance during the process of transformation into smart city by urban planners. After playing a role in designing cities of the future I moved away from earth into air so into airport management systems. In this project I was working on a system that synchronises various subsystems which had many goals like improvement of life-critical safety, optimisation of operations and many more. Sadly I did not get free travels but experience gained was very nice ;-) My last and current project is global TOP 50 company ASML so embedded photolithography in which I am working in Optical Column group. This group takes care of firmware for world's most complex and expensive chip manufacturing machine.<!----></span>

          <span class="inline-show-more-text__link-container-collapsed">
<!---->            <button class="inline-show-more-text__button
                inline-show-more-text__button--light
                link" aria-expanded="false" role="button" type="button">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">…Show more</font></font>
            </button>
          </span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
            <div>
                
          
    <div class="pvs-list__outer-container">
<!----><!----><!---->    </div>
  
  
            </div>
        </section>

<!---->  
        
<!---->
<!---->  
        
        <section id="ember67" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="content_collections" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Activity</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Activity</font></font><!----></span>
              </h2>
              <p class="pvs-header__subtitle pvs-header__optional-link text-body-small">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">456 followers</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">456 followers</font></font><!----></span>
              </p>
<!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
  
    
    <div class="mb3">
      
        
<!---->  
      
    </div>
    <div class="pv0 ph5">
      
          
      <div class="display-flex flex-column full-width pb3">
        <span class="text-body-medium-bold"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Krystian hasn't posted anything yet</font></font></span>
        <span class="text-body-small"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Recent posts shared by Krystian are listed here.</font></font></span>
      </div>
  
      
    </div>
      <footer class="artdeco-card__actions profile-creator-shared-content-view__footer-actions">
        <a class="app-aware-link  artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--3 artdeco-button--fluid artdeco-button--tertiary" href="https://www.linkedin.com/in/krystian-madej/recent-activity/all/" data-test-app-aware-link="">
          <svg role="none" aria-hidden="true" class="artdeco-button__icon rtl-flip" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="arrow-right-small">
<!---->    

    <use href="#arrow-right-small" width="16" height="16"></use>
</svg>

          <span class="artdeco-button__text">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">View all activity</font></font>
          </span>
        </a>
      </footer>
  
  

  
<!---->        </section>

<!---->  
        
        <section id="ember106" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="experience" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Experience</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Experience</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/3894/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" loading="lazy" height="48" alt="ASML logo" id="ember107" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Designer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">May 2022 - Present · 1 yr 5 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">May 2022 - Present · 1 yr 5 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Veldhoven, North Brabant, Holandia</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Veldhoven, North Brabant, Holandia</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/37854828/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQHTN6M1fk2LTg/company-logo_100_100/0/1662474475472?e=1703721600&amp;v=beta&amp;t=qAOcBzdk2iIW0lyZgVfqKakorgU7qu9Y7FOnbPGYRmo" loading="lazy" height="48" alt="Logo of NobleProg Polska" id="ember108" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Trainer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Trainer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">NobleProg Polska · Seasonal</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">NobleProg Polska · Seasonal</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">July </font><font style="vertical-align: inherit;">2021 - Nov. </font><font style="vertical-align: inherit;">2022 · 1 yr 5 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">July </font><font style="vertical-align: inherit;">2021 - Nov. </font><font style="vertical-align: inherit;">2022 · 1 yr 5 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław i okolice</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław i okolice</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/10843534/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C510BAQGDMGLyhRKDxQ/company-logo_100_100/0/1519879047653?e=1703721600&amp;v=beta&amp;t=FeFiLA8S62aDJ5hfFqFHkIGqYe94o2yOxGP3Ib1PVNY" loading="lazy" height="48" alt="Logo of GlobalLogic Poland" id="ember109" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Engineer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior Software Engineer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">GlobalLogic Poland · Full-time</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">GlobalLogic Poland · Full-time</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2022 - Apr. </font><font style="vertical-align: inherit;">2022 · 2 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2022 - Apr. </font><font style="vertical-align: inherit;">2022 · 2 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Woj. </font><font style="vertical-align: inherit;">Dolnośląskie, Polska</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Woj. </font><font style="vertical-align: inherit;">Dolnośląskie, Polska</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/11387324/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E0BAQE2DrxnvQOxfw/company-logo_100_100/0/1654501549855?e=1703721600&amp;v=beta&amp;t=CAIG89ziKTyz0BbYd1Ju4QkrcG29gyNmBOJNEvZvw5k" loading="lazy" height="48" alt="Logo of xBerry - R&amp;D House" id="ember110" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ Software Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ Software Developer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">xBerry - Software Development · Full-time</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">xBerry - Software Development · Full-time</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Feb. </font><font style="vertical-align: inherit;">2020 - Jan. </font><font style="vertical-align: inherit;">2022 · 2 yrs</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Feb. </font><font style="vertical-align: inherit;">2020 - Jan. </font><font style="vertical-align: inherit;">2022 · 2 yrs</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Wed. </font><font style="vertical-align: inherit;">dolnośląskie, Polska</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wrocław, Wed. </font><font style="vertical-align: inherit;">dolnośląskie, Polska</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="experience_company_logo" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/3894/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" loading="lazy" height="48" alt="ASML logo" id="ember111" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Software Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Software Developer</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML · Full-time</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ASML · Full-time</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2020 - Mar. </font><font style="vertical-align: inherit;">2021 · 1 yr 1 mo</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mar. </font><font style="vertical-align: inherit;">2020 - Mar. </font><font style="vertical-align: inherit;">2021 · 1 yr 1 mo</font></font><!----></span>
              </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wilton, Connecticut, Stany Zjednoczone</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Wilton, Connecticut, Stany Zjednoczone</font></font><!----></span>
              </span>
          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="pvs-list__item--with-top-padding pvs-list__item--one-column">
              
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Embedded project for lithography machine. </font><font style="vertical-align: inherit;">EUV department.</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Embedded project for lithography machine. </font><font style="vertical-align: inherit;">EUV department.</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-see-all-experiences" href="https://www.linkedin.com/in/krystian-madej/details/experience?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ">
<!---->            <span class="pvs-navigation__text">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">View all 8 experiences</font></font>
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember112" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="education" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Course</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Course</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/6908713/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGgO7Z7D24ppA/company-logo_100_100/0/1519935717598?e=1703721600&amp;v=beta&amp;t=C3AfugR1ASlwkRFpXT_DbHKYT83DXTFPqrgZjchVfhk" loading="lazy" height="48" alt="Logo of Politechnika Wrocławska" id="ember113" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/6908713/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Politechnika Wrocławska</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Politechnika Wrocławska</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Inżynier (Inż.), Informatyka</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Inżynier (Inż.), Informatyka</font></font><!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->2010 - 2013<!----></span><span class="visually-hidden"><!---->2010 - 2013<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember114" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="licenses_and_certifications" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Licenties en certificaten<!----></span><span class="visually-hidden"><!---->Licenties en certificaten<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="entity_image_licenses_and_certifications" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/162868/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQHzAFzG6umH9g/company-logo_100_100/0/1627841333886?e=1703721600&amp;v=beta&amp;t=lz4c73FARTWvDl-mHYvw1Wl952Xe92EjUDt6vJdvLvk" loading="lazy" height="48" alt="Logo of Cambridge University Press &amp; Assessment" id="ember115" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->CAE<!----></span><span class="visually-hidden"><!---->CAE<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!---->Cambridge University Press &amp; Assessment<!----></span><span class="visually-hidden"><!---->Cambridge University Press &amp; Assessment<!----></span>
                </span>
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->Afgegeven op jun. 2014<!----></span><span class="visually-hidden"><!---->Afgegeven op jun. 2014<!----></span>
              </span>
<!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember74" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="skills" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Vaardigheden<!----></span><span class="visually-hidden"><!---->Vaardigheden<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="skill_card_skill_topic" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/search/results/all/?keywords=C%2B%2B&amp;origin=PROFILE_PAGE_SKILL_NAVIGATION">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->C++<!----></span><span class="visually-hidden"><!---->C++<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!----><!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
      <div class="display-flex link-without-hover-visited">
        
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="clipboard-check" class="ivm-view-attr__icon--brand  " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M14.73 10H17l-5.5 8L8 14.5l1.34-1.34L11.21 15zM20 3v16a3 3 0 01-3 3H7a3 3 0 01-3-3V3h5.69l.52-1A2 2 0 0112 1a2 2 0 011.76 1l.52 1zm-2 2h-2.6l.6 1.1V7H8v-.9L8.6 5H6v14a1 1 0 001 1h10a1 1 0 001-1z"></path>
    </svg></li-icon>
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->Geslaagd voor LinkedIn-vaardigheidsevaluatie<!----></span><span class="visually-hidden"><!---->Geslaagd voor LinkedIn-vaardigheidsevaluatie<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
      
      </div>
  
<!---->    </div>
  
  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
        <a class="optional-action-target-wrapper display-flex link-without-hover-visited" target="_self" href="https://www.linkedin.com/in/krystian-madej/overlay/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,12)/endorsers?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=1&amp;modalTabIndex=1">
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="24" src="https://media.licdn.com/dms/image/C4E03AQGcXE4RmaO9uA/profile-displayphoto-shrink_100_100/0/1635332489775?e=1701302400&amp;v=beta&amp;t=dry8SjPi4xIojGu-eOebg4yENFwHvSXTFWCIXQ-UFuM" loading="lazy" height="24" alt="" id="ember75" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-circle-size-0 EntityPhoto-circle-0-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black hoverable-link-text display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->Onderschreven door<span class="white-space-pre"> </span><!---->Wojciech 👤 Razik<!----><!---->, die hier erg goed in is<!----></span><span class="visually-hidden"><!---->Onderschreven door<span class="white-space-pre"> </span><!---->Wojciech 👤 Razik<!----><!---->, die hier erg goed in is<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
      </a>
  
<!---->    </div>
  
  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
        <a class="optional-action-target-wrapper display-flex link-without-hover-visited" target="_self" href="https://www.linkedin.com/in/krystian-madej/overlay/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,12)/endorsers?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=2&amp;modalTabIndex=2">
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="24" src="https://media.licdn.com/dms/image/C4E0BAQGL8hpduEqGKQ/company-logo_100_100/0/1677420438777?e=1703721600&amp;v=beta&amp;t=6Havf9e8JQQcReBL0yF3K-c4bd2AMMTNq_15Ulm9YaY" loading="lazy" height="24" alt="" id="ember76" class="ivm-view-attr__img--centered ivm-view-attr__img--stacked ivm-view-attr__img--stacked-square-size-0 EntityPhoto-square-0-stackedFacepile   evi-image lazy-image ember-view">
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black hoverable-link-text display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->Onderschreven door 4 collega’s bij<span class="white-space-pre"> </span><!---->Nokia<!----></span><span class="visually-hidden"><!---->Onderschreven door 4 collega’s bij<span class="white-space-pre"> </span><!---->Nokia<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
      </a>
  
<!---->    </div>
  
  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center align-self-flex-start
            
            " target="_self" id="navigation-overlay-section-with-parent-Alle-4-details-weergeven" href="https://www.linkedin.com/in/krystian-madej/overlay/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,12)/skill-insights?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=0&amp;modalTabIndex=0">
<!---->            <span class="pvs-navigation__text">
              Alle 4 details weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="skill_card_skill_topic" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/search/results/all/?keywords=Boost+Library+C%2B%2B&amp;origin=PROFILE_PAGE_SKILL_NAVIGATION">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->Boost Library C++<!----></span><span class="visually-hidden"><!---->Boost Library C++<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!----><!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="
        mv1
        display-flex align-items-center">
      
        <a class="optional-action-target-wrapper display-flex link-without-hover-visited" target="_self" href="https://www.linkedin.com/in/krystian-madej/details/skills/urn:li:fsd_skill:(ACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ,14)/endorsers?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=0&amp;modalTabIndex=0&amp;detailScreenTabIndex=0">
          <div class="mr2 mv1">
            
    <ul class="ivm-entity-pile
        display-flex align-items-center t-black">
        <li class="ivm-entity-pile__img-item--stacked">
            
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="people" class="ivm-view-attr__icon--icon  " size="medium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" data-supported-dps="24x24" fill="currentColor" class="mercado-match" width="24" height="24" focusable="false">
      <path d="M12 16v6H3v-6a3 3 0 013-3h3a3 3 0 013 3zm5.5-3A3.5 3.5 0 1014 9.5a3.5 3.5 0 003.5 3.5zm1 2h-2a2.5 2.5 0 00-2.5 2.5V22h7v-4.5a2.5 2.5 0 00-2.5-2.5zM7.5 2A4.5 4.5 0 1012 6.5 4.49 4.49 0 007.5 2z"></path>
    </svg></li-icon>
    </div>
  
        </li>
    </ul>
  
          </div>
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="hoverable-link-text display-flex align-items-center
              t-14 t-normal t-black">
            <span aria-hidden="true"><!---->4 onderschrijvingen<!----></span><span class="visually-hidden"><!---->4 onderschrijvingen<!----></span>
          </div>
      
      </div>
  
    </div>
  
      </a>
  
<!---->    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-Alle-33-vaardigheden-weergeven" href="https://www.linkedin.com/in/krystian-madej/details/skills?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ">
<!---->            <span class="pvs-navigation__text">
              Alle 33 vaardigheden weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember116" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="courses" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Cursussen<!----></span><span class="visually-hidden"><!---->Cursussen<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->C++ Clean Code<!----></span><span class="visually-hidden"><!---->C++ Clean Code<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <!----><!----><!---->
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->C++ Memory Management<!----></span><span class="visually-hidden"><!---->C++ Memory Management<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <!----><!----><!---->
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-see-all-courses" href="https://www.linkedin.com/in/krystian-madej/details/courses?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ">
<!---->            <span class="pvs-navigation__text">
              Alle 5 cursussen weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  
        
<!---->
<!---->  
        
<!---->
<!---->  
        
        <section id="ember117" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="languages" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Talen<!----></span><span class="visually-hidden"><!---->Talen<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->angielski<!----></span><span class="visually-hidden"><!---->angielski<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->Professionele werkvaardigheid<!----></span><span class="visually-hidden"><!---->Professionele werkvaardigheid<!----></span>
              </span>
<!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->niemiecki<!----></span><span class="visually-hidden"><!---->niemiecki<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->Basisvaardigheid<!----></span><span class="visually-hidden"><!---->Basisvaardigheid<!----></span>
              </span>
<!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember118" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="organizations" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Organisaties<!----></span><span class="visually-hidden"><!---->Organisaties<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->United Music<!----></span><span class="visually-hidden"><!---->United Music<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!---->Ambassador · nov. 2013 - nov. 2018<!----></span><span class="visually-hidden"><!---->Ambassador · nov. 2013 - nov. 2018<!----></span>
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="pvs-list__item--with-top-padding pvs-list__item--one-column">
              
          
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!---->https://www.facebook.com/UnitedMusic<!----></span><span class="visually-hidden"><!---->https://www.facebook.com/UnitedMusic<!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
      <div class="
            display-flex" tabindex="-1" aria-hidden="true">
        
<!---->        
      </div>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
      <div class="
              display-flex flex-column full-width">
        
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 t-bold">
            <span aria-hidden="true"><!---->Google Developers Group Wrocław<!----></span><span class="visually-hidden"><!---->Google Developers Group Wrocław<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                <span class="t-14 t-normal">
                  <span aria-hidden="true"><!---->Events organiser · jan. 2017 - jan. 2018<!----></span><span class="visually-hidden"><!---->Events organiser · jan. 2017 - jan. 2018<!----></span>
                </span>
<!----><!---->          
      </div>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

<!---->      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember84" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="interests" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Interesses<!----></span><span class="visually-hidden"><!---->Interesses<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
      <div id="ember85" class="artdeco-tabs artdeco-tabs--size-t-48 ember-view">
          <div aria-multiselectable="false" id="ember86" class="artdeco-tablist ember-view" role="tablist">
              <button tabindex="0" aria-selected="true" id="ember87" class="artdeco-tab active artdeco-tab--selected ember-view" role="tab" aria-controls="ember90">
                <span aria-hidden="true"><!---->Bedrijven<!----></span><span class="visually-hidden"><!---->Bedrijven<!----></span>
              </button>

              <button tabindex="-1" aria-selected="false" id="ember88" class="artdeco-tab ember-view" role="tab" aria-controls="ember95">
                <span aria-hidden="true"><!---->Groepen<!----></span><span class="visually-hidden"><!---->Groepen<!----></span>
              </button>

              <button tabindex="-1" aria-selected="false" id="ember89" class="artdeco-tab ember-view" role="tab" aria-controls="ember98">
                <span aria-hidden="true"><!---->Onderwijsinstellingen<!----></span><span class="visually-hidden"><!---->Onderwijsinstellingen<!----></span>
              </button>

          
</div>
          <div tabindex="0" id="ember90" class="artdeco-tabpanel active ember-view" role="tabpanel" aria-labelledby="ember87">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            ph5 display-flex flex-row flex-wrap
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/3894/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGssQfw50QsaQ/company-logo_100_100/0/1519855871260?e=1703721600&amp;v=beta&amp;t=58cfT9LARw3px2ugHjWGd6Qm3bFW2XPrQGjG8NHf4Dg" loading="lazy" height="48" alt="ASML logo" id="ember91" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/3894/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->ASML<!----></span><span class="visually-hidden"><!---->ASML<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->552.746 volgers<!----></span><span class="visually-hidden"><!---->552.746 volgers<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember92" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    Volgen
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/6908713/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGgO7Z7D24ppA/company-logo_100_100/0/1519935717598?e=1703721600&amp;v=beta&amp;t=C3AfugR1ASlwkRFpXT_DbHKYT83DXTFPqrgZjchVfhk" loading="lazy" height="48" alt="Logo of Politechnika Wrocławska" id="ember93" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_companies_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/6908713/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->Politechnika Wrocławska<!----></span><span class="visually-hidden"><!---->Politechnika Wrocławska<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->98.015 volgers<!----></span><span class="visually-hidden"><!---->98.015 volgers<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember94" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    Volgen
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
      <div>
        
        <a class="optional-action-target-wrapper artdeco-button artdeco-button--tertiary artdeco-button--standard artdeco-button--2 artdeco-button--muted 
            inline-flex justify-center full-width align-items-center artdeco-button--fluid
            
            " target="_self" id="navigation-index-see-all-companies" href="https://www.linkedin.com/in/krystian-madej/details/interests?profileUrn=urn%3Ali%3Afsd_profile%3AACoAABI7_5AB-fvkQZLHZ-rv1LIUwmUUYzKmbFQ&amp;tabIndex=0&amp;detailScreenTabIndex=0">
<!---->            <span class="pvs-navigation__text">
              Alle bedrijven weergeven
            </span>
            <div class="pvs-navigation__icon">
              <li-icon aria-hidden="true" type="arrow-right" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" class="mercado-match" data-supported-dps="16x16" fill="currentColor" width="16" height="16" focusable="false">
      <path d="M11.45 3L15 8l-3.55 5H9l2.84-4H2V7h9.84L9 3z"></path>
    </svg></li-icon>
            </div>
        </a>
  
      </div>
  
        </div>
  
        </div>
    </div>
  
  
          </div>
          <div tabindex="0" hidden="" id="ember95" class="artdeco-tabpanel artdeco-tabpanel--hidden ember-view" role="tabpanel" aria-labelledby="ember88">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            ph5 display-flex flex-row flex-wrap
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_groups_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/groups/12484134">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E07AQH9G3-K6F5-Lw/group-logo_image-shrink_48x48/0/1630999330146?e=1696248000&amp;v=beta&amp;t=fEWn987p0PtXCJqxZAHiGOa1Z5P6HUfDqURYe7yLXt8" loading="lazy" height="48" alt="Logo van Jest praca! Polska: Embedded Software " id="ember96" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_groups_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/groups/12484134">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jest praca! </font><font style="vertical-align: inherit;">Polska: Embedded Software</font></font><span class="white-space-pre"> </span></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jest praca! </font><font style="vertical-align: inherit;">Polska: Embedded Software</font></font><span class="white-space-pre"> </span></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->31 leden<!----></span><span class="visually-hidden"><!---->31 leden<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
      
      <button id="ember97" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button"><!---->
<span class="artdeco-button__text">
    Deelnemen
</span></button>
  
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
          </div>
          <div tabindex="0" hidden="" id="ember98" class="artdeco-tabpanel artdeco-tabpanel--hidden ember-view" role="tabpanel" aria-labelledby="ember89">
              
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            ph5 display-flex flex-row flex-wrap
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--two-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="active_tab_schools_interests" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/6908713/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQGgO7Z7D24ppA/company-logo_100_100/0/1519935717598?e=1703721600&amp;v=beta&amp;t=C3AfugR1ASlwkRFpXT_DbHKYT83DXTFPqrgZjchVfhk" loading="lazy" height="48" alt="Logo van Politechnika Wrocławska" id="ember99" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="active_tab_schools_interests" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/school/6908713/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!---->Politechnika Wrocławska<!----></span><span class="visually-hidden"><!---->Politechnika Wrocławska<!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
<!---->              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!---->98.015 volgers<!----></span><span class="visually-hidden"><!---->98.015 volgers<!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember100" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    Volgen
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
  
          </div>
      </div>
  
  
<!---->        </section>

<!---->  
        
        <section id="ember119" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="volunteer_causes" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-large">
                <span aria-hidden="true"><!---->Goede doelen<!----></span><span class="visually-hidden"><!---->Goede doelen<!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="display-flex ph5 pv3">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:4;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Animal welfare • Art and culture • Education • Environment • Health • Human rights • Science and technology</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Animal welfare • Art and culture • Education • Environment • Health • Human rights • Science and technology</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
  
<!---->        </section>

<!---->  
  
      
          
</main>

          <aside class="scaffold-layout__aside
    
    ">
  
            
        
    
<!---->
      
    <div class="relative artdeco-card m0 mb2 p4">
      <div id="ember40" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-left ember-view update-supported-locales-section-dropDown">
<!---->          <div class="display-flex">
            
    <div class=" display-flex flex-1 flex-wrap">
      <div class="pv-button-group__choice-pill-group display-flex full-width">
          <button aria-checked="true" role="radio" id="ember59" class="pv-button-group__choice-pill-item full-width artdeco-pill artdeco-pill--slate artdeco-pill--3 artdeco-pill--choice artdeco-pill--selected ember-view" lang="en-us" type="button">  <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">English</font></font>
</button>
          <button aria-checked="false" role="radio" id="ember60" class="pv-button-group__choice-pill-item full-width artdeco-pill artdeco-pill--slate artdeco-pill--3 artdeco-pill--choice ember-view" lang="pl-pl" type="button">  <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Polski</font></font>
</button>
      </div>
    </div>
  
          </div>
        <div tabindex="-1" aria-hidden="true" id="ember43" class="pv-update-supported-locales-section__locales-dropdown-content artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
      </div>
    </div>
  
    
          <section class="ad-banner-container
    
    artdeco-card overflow-hidden xl
     mb2">
  <iframe data-ad-banner="" class="ad-banner" width="300" height="250" src="about:blank" scrolling="no" title="advertisement"></iframe>

<!---->
  
<!---->  

<!---->
<!----></section>

          
        <section id="ember148" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="browsemap_recommendation" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-medium">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Also viewed</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Also viewed</font></font><!----></span>
              </h2>
<!----><!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/alexander-korenyak-2a198319">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D03AQGGRlFjjmKK2A/profile-displayphoto-shrink_100_100/0/1516821033988?e=1701302400&amp;v=beta&amp;t=tG1y8rC1MIksMf0p-_meTdAvtGK2a_m3mDTG3Luq1HM" loading="lazy" height="48" alt="Profile photo of Alexander Korenyak" id="ember149" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/alexander-korenyak-2a198319?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAPhmdkBNiP1PAws1rCmlTDjMlPA9r9yVBw">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Alexander Korenyak</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Alexander Korenyak</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!---->                
    <div class="ivm-image-view-model  mr1 ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="linkedin-bug" class="ivm-view-attr__icon--premium-inbug  " size="14dp" color="premium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 14 14" data-supported-dps="14x14" fill="currentColor" class="mercado-match" width="14" height="14" focusable="false">
      <g>
        <path class="background-mercado" d="M14 1v12a1 1 0 01-1 1H1a1 1 0 01-1-1V1a1 1 0 011-1h12a1 1 0 011 1zM4 5H2v7h2zm.25-2A1.27 1.27 0 003 1.8 1.27 1.27 0 001.75 3 1.27 1.27 0 003 4.2 1.27 1.27 0 004.25 3zM12 8.29c0-2.2-.73-3.49-2.86-3.49A2.71 2.71 0 006.89 6V5H5v7h2V8.73A1.74 1.74 0 018.66 6.8C9.82 6.8 10 7.94 10 8.73V12h2z"></path>
      </g>
    </svg></li-icon>
    </div>
  
          </div>
  
                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Senior C++ developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Alexander Korenyak" id="ember150" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/bas-bastiaansen-637a43171">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E03AQE7uN1LxcxsQg/profile-displayphoto-shrink_100_100/0/1624475515434?e=1701302400&amp;v=beta&amp;t=HdHGBq6bVxkYocSLXdBURmKavzpBxgp_W2Yt4p8vink" loading="lazy" height="48" alt="Profile photo of Bas Bastiaansen" id="ember151" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/bas-bastiaansen-637a43171?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAACjpI8sB_1W0Nu7jcIyVMEoz4O37oD4k3ao">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bas Bastiaansen</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bas Bastiaansen</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Bas Bastiaansen" id="ember152" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/florisbob">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <div class="EntityPhoto-circle-3-ghost-person ivm-view-attr__ghost-entity  ">
            <div class="visually-hidden"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Profile photo of Floris Bob van Elzelingen</font></font></div>
        </div>
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/florisbob?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABT1GT8B1bH8JnagTTR374FcYarPW_GsAEM">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Floris Bob van Elzelingen</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Floris Bob van Elzelingen</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">C++ Developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Floris Bob van Elzelingen" id="ember153" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/kishorkumar1990">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E03AQE6fb3TKvTvuQ/profile-displayphoto-shrink_100_100/0/1653645994332?e=1701302400&amp;v=beta&amp;t=u5PgfO9DzidlJXdy1FmfeWYz0VpungApARz9gVpR8iU" loading="lazy" height="48" alt="Profile photo of Kishor Kumar" id="ember154" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/kishorkumar1990?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAZ8D8wBt3948dcqQ-cqfALU1B6BMJlOyjk">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Kishor Kumar</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Kishor Kumar</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer @ ASML | </font><font style="vertical-align: inherit;">Ex-Philips | </font><font style="vertical-align: inherit;">C# C++</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Developer @ ASML | </font><font style="vertical-align: inherit;">Ex-Philips | </font><font style="vertical-align: inherit;">C# C++</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Kishor Kumar" id="ember155" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/jjpawillems">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4E03AQFVBOy2pNrb6w/profile-displayphoto-shrink_100_100/0/1517758604845?e=1701302400&amp;v=beta&amp;t=LMOWq5Fg_Tl-B11N8--qBi23pcJXO6cKPNP8c2ODiIo" loading="lazy" height="48" alt="Profile photo of Jeroen Willems" id="ember156" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="browsemap_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/jjpawillems?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAAEBje0BeCdTFXzfQpFCJf2jlR5Ye_EDpj4">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jeroen Willems</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Jeroen Willems</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!---->                <span class="pvs-entity__supplementary-info t-14 t-black--light t-normal mr1">
                  <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">· 3rd+</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">No network connection</font></font><!----></span>
                </span>
            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Product Owner at ASML</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Product Owner at ASML</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Jeroen Willems" id="ember157" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
    <button id="ember158" class="pv3 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--fluid artdeco-button--tertiary ember-view" type="button">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show more</font></font>
</span></button>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  

        
        <section id="ember164" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="pymk_recommendation_from_company" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-medium">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">People you may know</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">People you may know</font></font><!----></span>
              </h2>
              <p class="pvs-header__subtitle pvs-header__optional-link text-body-small">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">From Krystian's company</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">From Krystian's company</font></font><!----></span>
              </p>
<!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/raja-n-1b847b284">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4E03AQFNT8NK7dHL_A/profile-displayphoto-shrink_100_100/0/1690352648841?e=1701302400&amp;v=beta&amp;t=Mrnm92SVQjY8GLKyeiHTSCL0p49BT7a8WA7_mgfCZJw" loading="lazy" height="48" alt="Profile photo of RAJA N" id="ember165" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/raja-n-1b847b284?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEUf8lwBR2gag5gHBTKQLwq6lO9P9PfN__A">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">RAJA N</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">RAJA N</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!---->                
    <div class="ivm-image-view-model  mr1 ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
        <li-icon aria-hidden="true" type="linkedin-bug" class="ivm-view-attr__icon--premium-inbug  " size="14dp" color="premium"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 14 14" data-supported-dps="14x14" fill="currentColor" class="mercado-match" width="14" height="14" focusable="false">
      <g>
        <path class="background-mercado" d="M14 1v12a1 1 0 01-1 1H1a1 1 0 01-1-1V1a1 1 0 011-1h12a1 1 0 011 1zM4 5H2v7h2zm.25-2A1.27 1.27 0 003 1.8 1.27 1.27 0 001.75 3 1.27 1.27 0 003 4.2 1.27 1.27 0 004.25 3zM12 8.29c0-2.2-.73-3.49-2.86-3.49A2.71 2.71 0 006.89 6V5H5v7h2V8.73A1.74 1.74 0 018.66 6.8C9.82 6.8 10 7.94 10 8.73V12h2z"></path>
      </g>
    </svg></li-icon>
    </div>
  
          </div>
  
<!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Ui ux at ASML</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Ui ux at ASML</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to RAJA N" id="ember166" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/hamad-jamil-420357285">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4D03AQGSo8fWDOh6dA/profile-displayphoto-shrink_100_100/0/1691453327080?e=1701302400&amp;v=beta&amp;t=0nXgU77X1YQmoVQ9NyocXsLEWE150ZtguXBYCqLQ6Ro" loading="lazy" height="48" alt="Profile photo of Hamad Jamil" id="ember167" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/hamad-jamil-420357285?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEVI-pgBmUI-gdW6FKLUCIu9lQfbc9kuINc">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Hamad Jamil</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Hamad Jamil</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Amazon Virtual Assistant | </font><font style="vertical-align: inherit;">Website Developer</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Amazon Virtual Assistant | </font><font style="vertical-align: inherit;">Website Developer</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Hamad Jamil" id="ember168" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/gpkanade">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4E03AQF5FvIrgfo_QQ/profile-displayphoto-shrink_100_100/0/1694146996263?e=1701302400&amp;v=beta&amp;t=4fnUOu4eMH05IA4nOgpLZPwQlcmY9b8OzCEsNfuE5kY" loading="lazy" height="48" alt="Profile photo of Goutham Kanade" id="ember169" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/gpkanade?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEUkyLcBJURSMeO9yHYiBSEF-jU7R3g16Yk">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Goutham Kanade</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Goutham Kanade</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Test Architect/Manager | </font><font style="vertical-align: inherit;">Filmmaker | </font><font style="vertical-align: inherit;">Entrepreneur</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software Test Architect/Manager | </font><font style="vertical-align: inherit;">Filmmaker | </font><font style="vertical-align: inherit;">Entrepreneur</font></font><!----></span>

<!---->
<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Goutham Kanade" id="ember170" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/mudit-kelwa-297693274">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D4D03AQF9aMlzB2pMkQ/profile-displayphoto-shrink_100_100/0/1691510822085?e=1701302400&amp;v=beta&amp;t=H7nstWT4T4LlNy6Bw0RhOMGICnzQZnRxVr3Ulag0qgU" loading="lazy" height="48" alt="Profile photo of Mudit Kelwa" id="ember171" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/mudit-kelwa-297693274?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEMFNtMBVR5px0ojlayBQoev6s9ZpRbMroc">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mudit Kelwa</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mudit Kelwa</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Mudit Kelwa" id="ember172" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/in/raymond-jumaoas-b35a3a277">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/D5635AQEMEav3snuwKQ/profile-framedphoto-shrink_100_100/0/1695472538668?e=1696248000&amp;v=beta&amp;t=Am5VLiQdZYJlEDgq1kVOccgeJIZQtpcJWdZJsR5Au48" loading="lazy" height="48" alt="Profile photo of Raymond Jumaoas" id="ember173" class="ivm-view-attr__img--centered EntityPhoto-circle-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="pymk_position_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/in/raymond-jumaoas-b35a3a277?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAEObZNkBsQ9etujCUHKra_KmTCBY4pdhY6M">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Raymond Jumaoas</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Raymond Jumaoas</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">--</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
<!----><!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
        
      
        
    
          
        
    <button aria-label="Send connection request to Raymond Jumaoas" id="ember174" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="connect" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M9 4a3 3 0 11-3-3 3 3 0 013 3zM6.75 8h-1.5A2.25 2.25 0 003 10.25V15h6v-4.75A2.25 2.25 0 006.75 8zM13 8V6h-1v2h-2v1h2v2h1V9h2V8z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Make a connection</font></font>
</span></button>
  
      
        

<!---->  

<!---->
<!---->
<!---->  
  

  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
    <button id="ember175" class="pv3 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--fluid artdeco-button--tertiary ember-view" type="button">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show more</font></font>
</span></button>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  

        
        <section id="ember51" class="artdeco-card ember-view relative break-words pb3
            
            
            mt2" tabindex="-1" data-view-name="profile-card"><!---->

          <div id="company_recommendation" class="pv-profile-card__anchor">
          </div>
            
          
<!---->
    <div class="pvs-header__container">
      <div class="pvs-header__top-container--no-stack">
        <div class="pvs-header__left-container--stack">
          <div class="pvs-header__title-container">
              <h2 class="pvs-header__title
                  text-heading-medium">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You might find this interesting:</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You might find this interesting:</font></font><!----></span>
              </h2>
              <p class="pvs-header__subtitle pvs-header__optional-link text-body-small">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Pages for you</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Pages for you</font></font><!----></span>
              </p>
<!---->          </div>

<!---->        </div>

<!---->      </div>
    </div>
  
  
            
          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="company_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/1441/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C4D0BAQHiNSL4Or29cg/company-logo_100_100/0/1519856215226?e=1703721600&amp;v=beta&amp;t=ABquqvWu81IO9c3y4U_4GAFiT7aaZ2OX0o6WrjgKU18" loading="lazy" height="48" alt="Google logo" id="ember52" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="company_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/1441/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Google</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Google</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software development</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Software development</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">31,654,272 followers</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">31,654,272 followers</font></font><!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
<!---->  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember53" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font>
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
            <li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column">
              
          
<!---->    <div class="pvs-entity
        pvs-entity--padded pvs-list__item--no-padding-in-columns
        
        ">
      <div>
        
        <a data-field="company_card_click" class="optional-action-target-wrapper 
            display-flex" target="_self" href="https://www.linkedin.com/company/42821781/">
            
    <div class="ivm-image-view-model  pvs-entity__image ">
        
    <div class="ivm-view-attr__img-wrapper display-flex">
<!---->          <img width="48" src="https://media.licdn.com/dms/image/C560BAQGW6Sn_g6-xZA/company-logo_100_100/0/1647120980872?e=1703721600&amp;v=beta&amp;t=X_CgVds4WIui4vMtLSg-5s_UZdE3uGGh2FmX_jFgmcs" loading="lazy" height="48" alt="Dubai Vacancy logo" id="ember54" class="ivm-view-attr__img--centered EntityPhoto-square-3   evi-image lazy-image ember-view">
    </div>
  
          </div>
  
        </a>
  
      </div>

      <div class="display-flex flex-column full-width align-self-center">
        <div class="display-flex flex-row justify-space-between">
          
        <a data-field="company_card_click" class="optional-action-target-wrapper 
              display-flex flex-column full-width" target="_self" href="https://www.linkedin.com/company/42821781/">
            <div class="display-flex flex-wrap align-items-center full-height">
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          <div class="display-flex align-items-center
              mr1 hoverable-link-text t-bold">
            <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dubai Vacancy</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dubai Vacancy</font></font><!----></span>
          </div>
      
      </div>
  
    </div>
  
<!----><!----><!---->            </div>
                
    <div class="display-flex ">
      
      <div class="
          display-flex full-width">
        
          

      <div class="pv-shared-text-with-see-more full-width t-14 t-normal display-flex align-items-center">
        
    <div class="inline-show-more-text
        inline-show-more-text--is-collapsed
        inline-show-more-text--is-collapsed-with-line-clamp
        
        
         full-width" style="-webkit-line-clamp:2;" tabindex="-1">

        <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">catering industry</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">catering industry</font></font><!----></span>

<!---->    </div>
  
      </div>
  
      
      </div>
  
    </div>
  
              <span class="t-14 t-normal t-black--light">
                <span aria-hidden="true"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">434,984 followers</font></font><!----></span><span class="visually-hidden"><!----><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">434,984 followers</font></font><!----></span>
              </span>
<!---->          </a>
  

<!---->
          <div class="pvs-entity__action-container">
<!---->          </div>
        </div>

          
    <div class="pvs-list__outer-container">
<!---->        <ul class="pvs-list
            
            
            ">
            <li class=" pvs-list__item--one-column">
              
<!---->  
            </li>
            <li class=" pvs-list__item--one-column">
              
          
        <div class="pv2">
          
    <button id="ember55" class="artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view" type="button">  <li-icon aria-hidden="true" type="add" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M14 9H9v5H7V9H2V7h5V2h2v5h5z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To follow</font></font>
</span></button>
  
        </div>
  
  
            </li>
        </ul>
<!---->    </div>
  
      </div>
    </div>
  
  
            </li>
        </ul>
        <div class="pvs-list__footer-wrapper">
          
        <div class="">
          
    <button id="ember56" class="pv3 artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--2 artdeco-button--fluid artdeco-button--tertiary ember-view" type="button">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Show more</font></font>
</span></button>
  
        </div>
  
        </div>
    </div>
  
  
<!---->        </section>

<!---->  

        <div class="mt2">
          
    <div class="sticky
        
        " style="height: 250px; width: 300px; margin: 0px auto;">
      
            <div class="pv-right-rail__sticky-ad-banner">
              <section class="ad-banner-container
    
    artdeco-card overflow-hidden
    ">
  <iframe data-ad-banner="" class="ad-banner" width="300" height="250" src="about:blank" scrolling="no" title="advertisement"></iframe>

<!---->
  
<!---->  

<!---->
<!----></section>
            </div>
          
    </div>
  
        </div>
  
  
      
          
</aside>
      </div>

<!---->  </div>
</div>
  
<!---->

    <footer class="global-footer
        global-footer--static">
      
        <div id="ember49" class="ember-view   ">
            
        <div id="expanded-footer" class="global-footer__container">
<!---->
          <div class="grid grid--is-nested grid--no-gutters mv3">
            <nav class="grid__col
                grid__col--12">
              <ul class="grid grid--no-gutters grid--is-nested">
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://about.linkedin.com/nl-nl" id="globalfooter-about" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">About LinkedIn</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://www.linkedin.com/accessibility" id="globalfooter-accessibility" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Accessibility</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/talent-solutions?trk=flagship_nav&amp;veh=li-footer-lts-control&amp;src=li-footer" id="globalfooter-talent_solutions" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Talent Solutions</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://www.linkedin.com/legal/professional-community-policies" id="globalfooter-community_guidelines" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Community Guidelines</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://careers.linkedin.com/" id="globalfooter-careers" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Careers</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/nl-nl/marketing-solutions?trk=n_nav_lms_f&amp;src=li-footer" id="globalfooter-marketing_solutions" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Marketing Solutions</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      
    
    
    <div id="ember215" class="artdeco-dropdown artdeco-dropdown--placement-top artdeco-dropdown--justification-left ember-view global-footer-dropdown">
      <button aria-expanded="false" id="globalfooter-privacy_dropdown-trigger" class="artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-top ember-view global-footer-dropdown__trigger" type="button" tabindex="0">
        <span class="global-footer__link global-footer__link--static t-12 t-bold">
          <span class="text-align-left">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Privacy and terms</font></font>
            <svg role="none" aria-hidden="true" class="global-footer-dropdown__trigger-icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="caret-small">
<!---->    

    <use href="#caret-small" width="16" height="16"></use>
</svg>

          </span>
        </span>
      
<!----></button>
      <div tabindex="-1" aria-hidden="true" id="ember217" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--justification-left artdeco-dropdown__content--placement-top ember-view global-footer-dropdown__options"><!----></div>
    </div>
  
  
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://www.linkedin.com/help/linkedin/answer/62931?lang=nl" id="globalfooter-ad_choices" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Advertising choice</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/nl-nl/marketing-solutions/ads?trk=n_nav_ads_f" id="globalfooter-advertising" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Advertisement</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://business.linkedin.com/sales-solutions?trk=flagship_nav&amp;veh=li-footer-lss-control&amp;src=li-footer" id="globalfooter-sales_solutions" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">SalesSolutions</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://mobile.linkedin.com/nl-nl" id="globalfooter-mobile" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Mobile</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://smallbusiness.linkedin.com?&amp;src=li-footer" id="globalfooter-small_business" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Small company</font></font>
                      </a>
                  </li>
                  <li class="global-footer__link-container grid__col
                      grid__col--8">
                      <a tabindex="0" target="_blank" href="https://safety.linkedin.com" id="globalfooter-safety_center" class="ember-view global-footer__link global-footer__link--static t-12 t-bold">
                        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Safety center</font></font>
                      </a>
                  </li>
              </ul>
            </nav>

            <div class="grid__col
                grid__col--12">
              <div class="grid grid--no-gutters grid--is-nested">
                <ul id="footer-action-list" class="global-footer__action-list grid__col
                    grid__col--12">
                    <li class="display-flex list-style-none mb4">
                      <svg role="none" aria-hidden="true" class="global-footer__action-icon rtl-flip" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="question-medium">
<!---->    

    <use href="#question-medium" width="24" height="24"></use>
</svg>

                      <span>
                          <a tabindex="0" target="_blank" href="https://www.linkedin.com/help/linkedin?trk=d_flagship3_profile_view_base" id="questions-help-center" class="ember-view global-footer__link global-footer__link--static t-14 t-bold">
                            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To ask?</font></font>
                          </a>

                        <p class="global-footer__label t-12">
                          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Visit our Help Center.</font></font>
                        </p>
                      </span>
                    </li>
                    <li class="display-flex list-style-none mb4">
                      <svg role="none" aria-hidden="true" class="global-footer__action-icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="settings-medium">
<!---->    

    <use href="#settings-medium" width="24" height="24"></use>
</svg>

                      <span>
                          <a tabindex="0" target="_blank" href="https://www.linkedin.com/psettings/" id="privacy-settings" class="ember-view global-footer__link global-footer__link--static t-14 t-bold">
                            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Manage your account and privacy</font></font>
                          </a>

                        <p class="global-footer__label t-12">
                          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Go to Settings.</font></font>
                        </p>
                      </span>
                    </li>
                    <li class="display-flex list-style-none mb4">
                      <svg role="none" aria-hidden="true" class="global-footer__action-icon" xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" data-supported-dps="24x24" data-test-icon="shield-medium">
<!---->    

    <use href="#shield-medium" width="24" height="24"></use>
</svg>

                      <span>
                          <a tabindex="0" target="_blank" href="https://www.linkedin.com/help/linkedin/answer/a1339724" id="recommendation-transparency" class="ember-view global-footer__link global-footer__link--static t-14 t-bold">
                            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Transparency about recommendations</font></font>
                          </a>

                        <p class="global-footer__label t-12">
                          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Read more about recommended content.</font></font>
                        </p>
                      </span>
                    </li>
                </ul>

                <div class="grid__col
                    grid__col--12">
                  <label for="globalfooter-select_language" class="global-footer__label t-12 t-normal mb1 mt0">
                    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Select language</font></font>
                  </label>

                  
    <select id="globalfooter-select_language" class="global-footer__language-selection-dropdown t-12 t-black--light t-bold">
        <option value="ar_AE" lang="ar-ae">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">العربية (Arabic)</font></font>
        </option>
        <option value="cs_CZ" lang="cs-cz">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Čeština (Czech)</font></font>
        </option>
        <option value="da_DK" lang="da-dk">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dansk (Danish)</font></font>
        </option>
        <option value="de_DE" lang="de-de">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Deutsch (German)</font></font>
        </option>
        <option value="en_US" lang="en-us">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">English (English)</font></font>
        </option>
        <option value="es_ES" lang="es-es">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Español (Spanish)</font></font>
        </option>
        <option value="fr_FR" lang="fr-fr">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Français (French)</font></font>
        </option>
        <option value="hi_IN" lang="hi-in">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">हिंदी (Hindi)</font></font>
        </option>
        <option value="in_ID" lang="in-id">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bahasa Indonesia (Indonesian)</font></font>
        </option>
        <option value="it_IT" lang="it-it">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Italiano (Italian)</font></font>
        </option>
        <option value="ja_JP" lang="ja-jp">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">日本語 (Japanese)</font></font>
        </option>
        <option value="ko_KR" lang="ko-kr">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">한국어 (Korean)</font></font>
        </option>
        <option value="ms_MY" lang="ms-my">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Bahasa Malaysia (Malay)</font></font>
        </option>
        <option value="nl_NL" lang="nl-nl">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Dutch</font></font>
        </option>
        <option value="no_NO" lang="no-no">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Norsk (Norwegian)</font></font>
        </option>
        <option value="pl_PL" lang="pl-pl">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Polski (Polish)</font></font>
        </option>
        <option value="pt_BR" lang="pt-br">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Português (Portuguese)</font></font>
        </option>
        <option value="ro_RO" lang="ro-ro">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Română (Romanian)</font></font>
        </option>
        <option value="ru_RU" lang="ru-ru">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Русский (Russian)</font></font>
        </option>
        <option value="sv_SE" lang="sv-se">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Svenska (Swedish)</font></font>
        </option>
        <option value="th_TH" lang="th-th">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">ภาษาไทย (Thai)</font></font>
        </option>
        <option value="tl_PH" lang="tl-ph">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Tagalog (Tagalog)</font></font>
        </option>
        <option value="tr_TR" lang="tr-tr">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Türkçe (Turkish)</font></font>
        </option>
        <option value="uk_UA" lang="uk-ua">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Українська (Ukrainian)</font></font>
        </option>
        <option value="zh_CN" lang="zh-cn">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">简体中文 (Chinese - Simplified)</font></font>
        </option>
        <option value="zh_TW" lang="zh-tw">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">正體中文 (Chinese - Traditional)</font></font>
        </option>
    </select>
  
                </div>
              </div>
            </div>
          </div>

          <p id="globalfooter-copyright" class="t-12 t-black--light t-normal text-align-left clear-both">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">© 2023 LinkedIn Corporation</font></font>
          </p>
        </div>
      
        </div>
  
    </footer>
  
<!----><!---->
  </div>
</div>
</div>

<div class="application-outlet__overlay-container
    ">
<!---->
    <!----><aside id="msg-overlay" class="msg-overlay-container msg-overlay-container-reflow
    ">
  <div tabindex="-1" class="msg-overlay-list-bubble
    
    
    ml4">
  <header class="msg-overlay-bubble-header">
    <div class="msg-overlay-bubble-header__badge-container
        "></div>
    <!---->
    <div class="msg-overlay-bubble-header__details flex-row align-items-center ml1">
      
    <div class="presence-entity presence-entity--size-1">
      <img src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" loading="lazy" alt="Fayyoz Usmon" id="ember184" class="presence-entity__image EntityPhoto-circle-1  evi-image lazy-image ghost-person ember-view">

        

    <div class="presence-entity__indicator presence-entity__indicator--size-1
             presence-indicator
        presence-indicator--is-online
        presence-indicator--size-1">
      <span class="visually-hidden">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Status is online</font></font>
      </span>
    </div>
  
    </div>
  
      <button class="msg-overlay-bubble-header__button truncate ml2" type="button">
        <span class="truncate t-14 t-bold
            t-black">
          <span aria-hidden="true">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">To inform</font></font>
          </span>
          <span class="visually-hidden">
            <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You are on the message overlay. </font><font style="vertical-align: inherit;">Press enter to minimize it.</font></font>
          </span>
        </span>
      </button>

<!----><!---->    </div>
    <div class="msg-overlay-bubble-header__controls display-flex">
      <div id="ember185" class="artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
        <button aria-expanded="false" id="ember186" class="msg-overlay-bubble-header__dropdown-trigger artdeco-button artdeco-button--1 artdeco-button--circle artdeco-button--muted artdeco-button--tertiary artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
          <svg role="img" aria-hidden="false" aria-label="Open selection menu" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="overflow-web-ios-small">
<!---->    

    <use href="#overflow-web-ios-small" width="16" height="16"></use>
</svg>

        
<!----></button>
        <div class="msg-overlay-bubble-header__dropdown-container">
          <div tabindex="-1" aria-hidden="true" id="ember187" class="artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
        </div>
      </div>

      <button id="ember188" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">  <li-icon aria-hidden="true" type="compose-icon" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M15 2.53a1.51 1.51 0 01-.44 1L9.15 9 6 10l1-3.12 5.44-5.44A1.5 1.5 0 0115 2.53zM12 11a1 1 0 01-1 1H5a1 1 0 01-1-1V5a1 1 0 011-1h3V2H5a3 3 0 00-3 3v6a3 3 0 003 3h6a3 3 0 003-3V8h-2z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Compose message</font></font>
</span></button>

      <button id="ember189" class="msg-overlay-bubble-header__control msg-overlay-bubble-header__control--new-convo-btn artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view">  <li-icon aria-hidden="true" type="chevron-down" class="artdeco-button__icon" size="small"><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" data-supported-dps="16x16" fill="currentColor" class="mercado-match" width="16" height="16" focusable="false">
      <path d="M1 5l7 4.61L15 5v2.39L8 12 1 7.39z"></path>
    </svg></li-icon>

<span class="artdeco-button__text">
    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">You are on the message overlay. </font><font style="vertical-align: inherit;">Press enter to minimize it.</font></font>
</span></button>
    </div>
  </header>

  <!---->

<!---->
    <div class="msg-overlay-list-bubble-search
    ">
    <div class="msg-overlay-list-bubble-search__input-container">
      <label class="a11y-text" for="msg-overlay-list-bubble-search__search-typeahead-input">
        <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Type to search for connections and conversations.</font></font>
      </label>
      <span class="msg-overlay-list-search__search-icon display-flex">
        <svg role="none" aria-hidden="true" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="search-small">
<!---->    

    <use href="#search-small" width="16" height="16"></use>
</svg>

      </span>
      <input id="msg-overlay-list-bubble-search__search-typeahead-input" class="ember-text-field ember-view msg-overlay-list-bubble-search__search-typeahead-input" placeholder="Search messages" autocomplete="off" type="text">
      <div id="ember190" class="msg-overlay-list-bubble__filters-dropdown artdeco-dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
        <button aria-expanded="false" tabindex="0" id="ember191" class="msg-overlay-list-bubble__filters-btn artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button">
          <svg role="img" aria-hidden="false" aria-label="Filter messages by" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="filter-small">
<!---->    

    <use href="#filter-small" width="16" height="16"></use>
</svg>

        
<!----></button>
        <div tabindex="-1" aria-hidden="true" id="ember192" class="artdeco-dropdown__content msg-overlay-list-bubble__filters-dropdown-content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
      </div>
    </div>

<!---->
<!---->  <div class="relative display-flex justify-center flex-column overflow-hidden
       hidden">
    <div class="msg-overlay-list-bubble-search-content">
        <div class="msg-overlay-list-bubble-search__history-placeholder-container text-align-center">
          <div class="msg-overlay-list-bubble__illustration msg-overlay-list-bubble-search__history-placeholder-illustration"></div>
          <p class="t-14 t-black t-normal ph2">
            Zoek in uw berichten op ontvanger, inhoud of gespreksnaam
          </p>
        </div>
    </div>
  </div>
</div>

    <div class="msg-overlay-list-bubble__top-static-area">
        <div id="ember193" class="artdeco-tabs artdeco-tabs--size-t-40 artdeco-tabs--centered ember-view msg-focused-inbox-tabs">
  <div aria-multiselectable="false" id="ember194" class="artdeco-tablist artdeco-tablist--no-wrap ember-view" role="tablist">
    <button tabindex="0" aria-selected="true" id="ember195" class="artdeco-tab active artdeco-tab--selected ember-view" role="tab">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Targeted</font></font>
    </button>
    <button tabindex="-1" aria-selected="false" id="ember196" class="artdeco-tab ember-view" role="tab">
      <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Other</font></font>
<!---->    </button>
  
</div>
</div>

<!---->
      <!---->
<!---->

<!---->    </div>

<!---->
<!---->
<!---->
        <section class="scrollable msg-overlay-list-bubble__content msg-overlay-list-bubble__content--scrollable">
          <div></div>
          <div class="msg-overlay-list-bubble__default-conversation-container">
<!---->
                <!---->
              <!---->
              <span class="visually-hidden">
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Attention screen reader users: messages are constantly updated. </font><font style="vertical-align: inherit;">Use the tab and shift+tab keys instead of the up and down arrows to navigate between different messages.</font></font>
              </span>
<!----><!---->
            
      
              <!---->
            
  

            <div class="msg-overlay-list-bubble__conversations-list">
        <div class="msg-conversation-listitem__link msg-overlay-list-bubble__convo-item" tabindex="0" id="overlay-conversation-card-ember199" data-feedback-redacted="">
  <div class="msg-overlay-list-bubble__convo-card-container">
    <div class="msg-conversation-card msg-overlay-list-bubble__convo-card display-flex">
      <div class="msg-selectable-entity
    
    msg-selectable-entity--3">
      <div class="msg-facepile-grid--no-facepile msg-facepile-grid
    msg-facepile-grid--3 msg-selectable-entity__entity">
<!---->      <img src="https://media.licdn.com/dms/image/C4D03AQFjtPx2a0NPeQ/profile-displayphoto-shrink_100_100/0/1516553080018?e=1701302400&amp;v=beta&amp;t=f4IpP56HJwztWkwW0jYIaOvRaQFnxmZajpsQAmijDl4" loading="lazy" alt="Diane Chiang" id="ember201" class="evi-image lazy-image msg-facepile-grid__img msg-facepile-grid__img--person ember-view">
</div>
      <div class="msg-selectable-entity__checkbox-container">
    <input id="checkbox-msg-selectable-entity__checkbox-ember200" class="ember-checkbox ember-view msg-selectable-entity__input simple-form" type="checkbox">
    <label class="msg-selectable-entity__checkbox-label ml2" aria-label="Select conversation" for="checkbox-msg-selectable-entity__checkbox-ember200">
    </label>

    <div class="msg-selectable-entity__checkbox-circle-container">
      <div class="msg-selectable-entity__checkbox-circle">
      </div>
    </div>
  </div>
</div>

      <div class="overflow-hidden pl2 msg-overlay-list-bubble__convo-card-content">
        <div class="msg-overlay-list-bubble__convo-card-content-wrapper fl">
          <div class="msg-conversation-card__row align-items-center display-flex">
            <h3 class="msg-conversation-listitem__participant-names msg-conversation-card__participant-names truncate t-14 t-black
                t-normal">
              <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Diane Chiang</font></font>
            </h3>
            <div class="msg-conversation-card__mute-icon-holder
                msg-conversation-card__mute-icon-holder--inbox-shortcuts">
<!---->            </div>
              <time class="msg-overlay-list-bubble-item__time-stamp t-12
                  msg-overlay-list-bubble-item__time-stamp--inbox-shortcuts
                  t-normal t-black--light">
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Aug 4</font></font>
              </time>
              <div class="msg-overlay-list-bubble__inbox-shortcuts-container">
                <div class="msg-overlay-list-bubble__inbox-shortcuts">
                  


    <div id="ember202" class="artdeco-dropdown msg-thread-actions__dropdown artdeco-dropdown--placement-bottom artdeco-dropdown--justification-right ember-view">
      <button aria-expanded="false" id="ember203" class="msg-thread-actions__control artdeco-button artdeco-button--circle artdeco-button--1 artdeco-button--muted artdeco-button--tertiary artdeco-dropdown__trigger artdeco-dropdown__trigger--placement-bottom ember-view" type="button" tabindex="0">
        <svg role="none" aria-hidden="true" class="artdeco-button__icon" xmlns="http://www.w3.org/2000/svg" width="16" height="16" viewBox="0 0 16 16" data-supported-dps="16x16" data-test-icon="overflow-web-ios-small">
<!---->    

    <use href="#overflow-web-ios-small" width="16" height="16"></use>
</svg>

        <span class="visually-hidden">
          <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Open options list in your conversation with Diane Chiang</font></font>
        </span>
      
<!----></button>
      <div class="msg-thread-actions__dropdown-container">
        <div tabindex="-1" aria-hidden="true" id="ember204" class="msg-thread-actions__dropdown-options--inbox-shortcuts artdeco-dropdown__content artdeco-dropdown--is-dropdown-element artdeco-dropdown__content--has-arrow artdeco-dropdown__content--arrow-right artdeco-dropdown__content--justification-right artdeco-dropdown__content--placement-bottom ember-view"><!----></div>
      </div>
    </div>

    
<!---->  

    
    <div>
<!---->    </div>
  

<!---->  
                </div>
              </div>
          </div>

          <div class="msg-conversation-card__row justify-space-between">
            <div class="msg-overlay-list-bubble__message-snippet-container--narrow-two-line">
<!---->              <p class="msg-overlay-list-bubble__message-snippet--v2
                  
                  m0 t-12
                  t-black--light">
<!---->                  <span class="msg-conversation-card__pill t-14
                      
                      t-bold pr1">
                    <font style="vertical-align: inherit;"><font style="vertical-align: inherit;">LinkedIn Offer</font></font>
                  </span>
                <font style="vertical-align: inherit;"><font style="vertical-align: inherit;"> Hello Fayyoz, I'm Diane from the LinkedIn Premium team. </font><font style="vertical-align: inherit;">Thank you for joining the LinkedIn community. </font><font style="vertical-align: inherit;">We appreciate this and would like to offer you an upgrade to LinkedIn Premium with the first month free.
</font></font>
              </p>
<!---->            </div>
            <div class="display-flex">
              <div class="msg-conversation-card__conversation-status msg-conversation-card__star-icon
                  ">
<!---->              </div>
<!---->            </div>
          </div>
        </div>
      </div>
    </div>
  </div>
</div>
</div>

<!---->
<!---->
<!---->          </div>
        </section>

<!----><!----></div>

<!---->
<!---->
  <div id="msg-overlay__emoji-hoverable-outlet"></div>

  <div id="msg-overlay__reactor-list-outlet"></div>
</aside>
</div>

<!---->
  
      
        
    <div id="ember50" class="ember-view"><!----></div>
  
    
  

  
      
    <img src="https://px.ads.linkedin.com/collect/?pid=6883&amp;fmt=gif&amp;_t=1695640000462" alt="" role="none" class="third-party-tracking-pixel hidden" data-test-third-party-tracking-pixel="">
  
  

</div>

<div id="artdeco-hoverable-outlet"></div>

<!---->
<div id="type-ahead-wormhole" class="type-ahead type-ahead-wormhole">
</div>

<div id="toast-wormhole" class="toast-wormhole">
</div>

<div id="profile-inline-modal-outlet">
</div>

<!---->

  <div><!----></div><div id="goog-gt-tt" class="VIpgJd-yAWNEb-L7lbkb skiptranslate" style="border-radius: 12px; margin: 0 0 0 -23px; padding: 0; font-family: 'Google Sans', Arial, sans-serif;" data-id=""><div id="goog-gt-vt" class="VIpgJd-yAWNEb-hvhgNd"><div class=" VIpgJd-yAWNEb-hvhgNd-l4eHX-i3jM8c"><img src="https://fonts.gstatic.com/s/i/productlogos/translate/v14/24px.svg" width="24" height="24" alt=""></div><div class=" VIpgJd-yAWNEb-hvhgNd-k77Iif-i3jM8c"><div class="VIpgJd-yAWNEb-hvhgNd-IuizWc" dir="ltr">Original text</div><div id="goog-gt-original-text" class="VIpgJd-yAWNEb-nVMfcd-fmcmS VIpgJd-yAWNEb-hvhgNd-axAV1"></div></div><div class="VIpgJd-yAWNEb-hvhgNd-N7Eqid ltr"><div class="VIpgJd-yAWNEb-hvhgNd-N7Eqid-B7I4Od ltr" dir="ltr"><div class="VIpgJd-yAWNEb-hvhgNd-UTujCb">Rate this translation</div><div class="VIpgJd-yAWNEb-hvhgNd-eO9mKe">Your feedback will be used to help improve Google Translate</div></div><div class="VIpgJd-yAWNEb-hvhgNd-xgov5 ltr"><button id="goog-gt-thumbUpButton" type="button" class="VIpgJd-yAWNEb-hvhgNd-bgm6sf" title="Good translation" aria-label="Good translation" aria-pressed="false"><span id="goog-gt-thumbUpIcon"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M21 7h-6.31l.95-4.57.03-.32c0-.41-.17-.79-.44-1.06L14.17 0S7.08 6.85 7 7H2v13h16c.83 0 1.54-.5 1.84-1.22l3.02-7.05c.09-.23.14-.47.14-.73V9c0-1.1-.9-2-2-2zM7 18H4V9h3v9zm14-7l-3 7H9V8l4.34-4.34L12 9h9v2z"></path></svg></span><span id="goog-gt-thumbUpIconFilled"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M21 7h-6.31l.95-4.57.03-.32c0-.41-.17-.79-.44-1.06L14.17 0S7.08 6.85 7 7v13h11c.83 0 1.54-.5 1.84-1.22l3.02-7.05c.09-.23.14-.47.14-.73V9c0-1.1-.9-2-2-2zM5 7H1v13h4V7z"></path></svg></span></button><button id="goog-gt-thumbDownButton" type="button" class="VIpgJd-yAWNEb-hvhgNd-bgm6sf" title="Poor translation" aria-label="Poor translation" aria-pressed="false"><span id="goog-gt-thumbDownIcon"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M3 17h6.31l-.95 4.57-.03.32c0 .41.17.79.44 1.06L9.83 24s7.09-6.85 7.17-7h5V4H6c-.83 0-1.54.5-1.84 1.22l-3.02 7.05c-.09.23-.14.47-.14.73v2c0 1.1.9 2 2 2zM17 6h3v9h-3V6zM3 13l3-7h9v10l-4.34 4.34L12 15H3v-2z"></path></svg></span><span id="goog-gt-thumbDownIconFilled"><svg width="24" height="24" viewBox="0 0 24 24" focusable="false" class="VIpgJd-yAWNEb-hvhgNd-THI6Vb NMm5M"><path d="M3 17h6.31l-.95 4.57-.03.32c0 .41.17.79.44 1.06L9.83 24s7.09-6.85 7.17-7V4H6c-.83 0-1.54.5-1.84 1.22l-3.02 7.05c-.09.23-.14.47-.14.73v2c0 1.1.9 2 2 2zm16 0h4V4h-4v13z"></path></svg></span></button></div></div><div id="goog-gt-votingHiddenPane" class="VIpgJd-yAWNEb-hvhgNd-aXYTce"><form id="goog-gt-votingForm" action="//translate.googleapis.com/translate_voting?client=te_lib" method="post" target="votingFrame" class="VIpgJd-yAWNEb-hvhgNd-aXYTce"><input type="text" name="sl" id="goog-gt-votingInputSrcLang"><input type="text" name="tl" id="goog-gt-votingInputTrgLang"><input type="text" name="query" id="goog-gt-votingInputSrcText"><input type="text" name="gtrans" id="goog-gt-votingInputTrgText"><input type="text" name="vote" id="goog-gt-votingInputVote"></form><iframe name="votingFrame" frameborder="0"></iframe></div></div></div>
</body>